****Сode execution takes just over 1.5 hours. TODO: experiments with grouping data within categories****

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import gc
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn import metrics
# Plotly library
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected=True)
pd.set_option('display.max_columns', 500)

In [ ]:
#https://www.kaggle.com/theoviel/load-the-totality-of-the-data
dtypes = {
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_columns = [c for c,v in dtypes.items() if v in numerics]
categorical_columns = [c for c,v in dtypes.items() if v not in numerics]
retained_columns = numerical_columns + categorical_columns

***This code will load categorical columns from both datasets, and print unique values in the form of dictionaries.***

In [ ]:
'''%%time
retained_columns = numerical_columns + categorical_columns
train = pd.read_csv('../input/train.csv', dtype=dtypes,nrows=4000000, usecols=retained_columns)
for col in tqdm(categorical_columns): 
    print('Started {}'.format(col)) 
    tran = pd.read_csv('../input/train.csv', dtype=dtypes,usecols=[col]) 
    test = pd.read_csv('../input/test.csv', dtype=dtypes,usecols=[col]) 
    data = pd.concat([tran, test])
    unique = data[col].unique() 
    l_unique = list(x for x in range(len(unique))) 
    dct = {} dct = dict(zip(unique, l_unique))
    print('Done {}'.format(col))
    print('{} /n'.format(col), dct)
    del data, tran, test, dct, unique, l_unique
    gc.collect()'''

****This function prints the difference in categorical data columns between the test and the train****

In [ ]:
'''for col in tqdm(categorical_columns):
    print('Started {}'.format(col))
    tran = pd.read_csv('../input/train.csv', dtype=dtypes,usecols=[col])
    test = pd.read_csv('../input/test.csv', dtype=dtypes,usecols=[col])
    train_unique = train[col].nunique()
    test_unique = test[col].nunique()
    dif = test_unique - train_unique
    print('Diff between {} in test/train is: {}'.format(col, dif))
    del  tran, test
    gc.collect()'''

****Here the function does the encoding by the label. Caution, the AvSigVersion column has almost 10,000 values. Dictionaries are taken from the results of the previous function****

In [ ]:
def encode(train):
    print("Started")
    train['ProductName'] = train['ProductName'].map({'win8defender': 0, 'mse': 1, 'mseprerelease': 2, 'windowsintune': 3, 'fep': 4, 'scep': 5})
    train['EngineVersion'] = train['EngineVersion'].map({'1.1.15100.1': 0, '1.1.14600.4': 1, '1.1.15200.1': 2, '1.1.15300.6': 3, '1.1.13504.0': 4, '1.1.14800.3': 5, '1.1.15000.2': 6, '1.1.14104.0': 7, '1.1.13804.0': 8, '1.1.14306.0': 9, '1.1.15000.1': 10, '1.1.13407.0': 11, '1.1.14405.2': 12, '1.1.14901.4': 13, '1.1.14700.3': 14, '1.1.14202.0': 15, '1.1.13303.0': 16, '1.1.13701.0': 17, '1.1.13000.0': 18, '1.1.15300.5': 19, '1.1.14500.5': 20, '1.1.14700.5': 21, '1.1.14003.0': 22, '1.1.13601.0': 23, '1.1.13903.0': 24, '1.1.12805.0': 25, '1.1.13202.0': 26, '1.1.12902.0': 27, '1.1.14305.0': 28, '1.1.13704.0': 29, '1.1.14901.3': 30, '1.1.14002.0': 31, '1.1.14103.0': 32, '1.1.14700.4': 33, '1.1.14800.1': 34, '1.1.13103.0': 35, '1.1.14303.0': 36, '1.1.14201.0': 37, '1.1.13102.0': 38, '1.1.11701.0': 39, '1.1.12400.0': 40, '1.1.14500.2': 41, '1.1.14001.0': 42, '1.1.11804.0': 43, '1.1.10401.0': 44, '1.1.13902.0': 45, '1.1.14102.0': 46, '1.1.12603.0': 47, '1.1.12706.0': 48, '1.1.12002.0': 49, '1.1.13803.0': 50, '1.1.12101.0': 51, '1.1.13406.0': 52, '1.1.13802.0': 53, '1.1.10302.0': 54, '1.1.11400.0': 55, '1.1.12804.0': 56, '1.1.12300.0': 57, '1.1.12205.0': 58, '1.1.11903.0': 59, '1.1.12505.0': 60, '1.1.11502.0': 61, '1.1.9700.0': 62, '1.1.11602.0': 63, '1.1.13503.0': 64, '1.1.13301.0': 65, '1.1.11202.0': 66, '1.1.11104.0': 67, '1.1.12802.0': 68, '1.1.10701.0': 69, '1.1.15400.5': 70, '1.1.15400.4': 71, '1.1.15400.3': 72, '1.1.11302.0': 73})
    train['AppVersion'] = train['AppVersion'].map({'4.18.1807.18075': 0, '4.13.17134.1': 1, '4.9.218.0': 2, '4.18.1806.18062': 3, '4.18.1809.2': 4, '4.10.209.0': 5, '4.8.10240.17914': 6, '4.13.17134.228': 7, '4.12.16299.15': 8, '4.14.17639.18041': 9, '4.9.10586.494': 10, '4.10.14393.1593': 11, '4.9.10586.0': 12, '4.11.15063.0': 13, '4.9.10586.589': 14, '4.8.10240.16384': 15, '4.16.17656.18052': 16, '4.11.15063.447': 17, '4.8.10240.17443': 18, '4.9.10586.1106': 19, '4.10.14393.0': 20, '4.12.17007.18022': 21, '4.14.17613.18039': 22, '4.11.15063.1155': 23, '4.10.14393.1794': 24, '4.8.207.0': 25, '4.8.10240.17946': 26, '4.12.17007.18011': 27, '4.8.10240.17354': 28, '4.10.14393.1066': 29, '4.13.17134.191': 30, '4.8.10240.17113': 31, '4.9.10586.873': 32, '4.10.14393.1198': 33, '4.9.10586.839': 34, '4.8.204.0': 35, '4.9.10586.962': 36, '4.9.10586.672': 37, '4.10.14393.1613': 38, '4.9.10586.1045': 39, '4.13.17134.112': 40, '4.12.17007.17123': 41, '4.9.10586.916': 42, '4.10.14393.2273': 43, '4.8.10240.17319': 44, '4.9.10586.965': 45, '4.8.10240.17184': 46, '4.8.10240.17889': 47, '4.10.14393.953': 48, '4.8.10240.17861': 49, '4.8.10240.17071': 50, '4.8.10240.17202': 51, '4.18.1807.18072': 52, '4.13.17134.319': 53, '4.10.205.0': 54, '4.6.305.0': 55, '4.4.304.0': 56, '4.14.17613.18038': 57, '4.8.10240.17609': 58, '4.7.205.0': 59, '4.18.1806.20021': 60, '4.17.17677.1000': 61, '4.8.10240.17533': 62, '4.5.218.0': 63, '4.10.14393.2248': 64, '4.8.10240.17146': 65, '4.8.10240.17394': 66, '4.8.10240.17797': 67, '4.10.14393.1532': 68, '4.8.10240.17918': 69, '4.10.14393.2457': 70, '4.16.17656.18051': 71, '4.8.10240.17770': 72, '4.9.10586.456': 73, '4.18.1806.20015': 74, '4.15.17661.1001': 75, '4.15.17643.1000': 76, '4.18.1807.20063': 77, '4.5.216.0': 78, '4.17.17686.1003': 79, '4.10.14393.726': 80, '4.8.10240.17446': 81, '4.9.10586.1177': 82, '4.17.17672.1000': 83, '4.11.15063.994': 84, '4.15.17666.1000': 85, '4.17.17682.1000': 86, '4.15.17650.1001': 87, '4.4.306.0': 88, '4.17.17685.20082': 89, '4.18.1806.20033': 90, '4.13.17634.1000': 91, '4.12.17007.18021': 92, '4.15.17655.1000': 93, '4.12.17007.17121': 94, '4.13.17627.1000': 95, '4.18.1807.18070': 96, '4.13.17639.1000': 97, '4.9.219.0': 98, '4.8.203.0': 99, '4.11.15063.1154': 100, '4.8.10240.16425': 101, '4.17.17686.1004': 102, '4.5.212.0': 103, '4.13.17134.226': 104, '4.7.209.0': 105, '4.13.17618.1000': 106, '4.12.16299.0': 107, '4.8.10240.17943': 108, '4.13.17604.1000': 109, '4.18.1810.5': 110, '4.13.17134.320': 111, '4.18.1810.20021': 112, '4.8.10240.18036': 113, '4.18.1809.20006': 114, '4.18.1810.20017': 115, '4.18.1810.20029': 116, '4.10.14393.2608': 117, '4.6.302.0': 118, '4.10.207.0': 119, '4.18.1810.20037': 120, '4.13.17623.1002': 121, '4.13.17133.1': 122, '4.9.10586.461': 123})
    train['AvSigVersion'] = train['AvSigVersion'].map({'1.273.1735.0': 0, '1.263.48.0': 1, '1.273.1341.0': 2, '1.273.1527.0': 3, '1.273.1379.0': 4, '1.273.1094.0': 5, '1.273.845.0': 6, '1.273.1393.0': 7, '1.275.988.0': 8, '1.273.973.0': 9, '1.273.869.0': 10, '1.273.1826.0': 11, '1.273.1372.0': 12, '1.275.895.0': 13, '1.273.925.0': 14, '1.275.130.0': 15, '1.275.300.0': 16, '1.273.466.0': 17, '1.277.48.0': 18, '1.273.1795.0': 19, '1.273.781.0': 20, '1.273.356.0': 21, '1.275.488.0': 22, '1.275.173.0': 23, '1.275.879.0': 24, '1.275.91.0': 25, '1.273.1376.0': 26, '1.277.25.0': 27, '1.275.26.0': 28, '1.273.1561.0': 29, '1.275.1025.0': 30, '1.237.607.0': 31, '1.275.511.0': 32, '1.275.482.0': 33, '1.273.1622.0': 34, '1.275.249.0': 35, '1.273.1273.0': 36, '1.267.1242.0': 37, '1.271.795.0': 38, '1.273.894.0': 39, '1.273.1124.0': 40, '1.273.851.0': 41, '1.273.1034.0': 42, '1.251.42.0': 43, '1.263.469.0': 44, '1.273.371.0': 45, '1.275.1414.0': 46, '1.275.112.0': 47, '1.271.613.0': 48, '1.275.445.0': 49, '1.245.492.0': 50, '1.275.795.0': 51, '1.275.1231.0': 52, '1.257.902.0': 53, '1.275.288.0': 54, '1.273.912.0': 55, '1.275.30.0': 56, '1.275.98.0': 57, '1.269.1420.0': 58, '1.235.1997.0': 59, '1.259.693.0': 60, '1.275.350.0': 61, '1.273.1620.0': 62, '1.275.1244.0': 63, '1.275.941.0': 64, '1.275.330.0': 65, '1.275.237.0': 66, '1.275.1140.0': 67, '1.275.1621.0': 68, '1.275.342.0': 69, '1.273.443.0': 70, '1.273.810.0': 71, '1.273.1073.0': 72, '1.275.1550.0': 73, '1.275.1765.0': 74, '1.273.1565.0': 75, '1.275.1176.0': 76, '1.273.1631.0': 77, '1.273.1721.0': 78, '1.273.486.0': 79, '1.275.1682.0': 80, '1.273.337.0': 81, '1.273.1416.0': 82, '1.273.1613.0': 83, '1.273.509.0': 84, '1.273.1420.0': 85, '1.275.727.0': 86, '1.273.1571.0': 87, '1.275.1534.0': 88, '1.275.1108.0': 89, '1.269.520.0': 90, '1.275.1209.0': 91, '1.273.1135.0': 92, '1.263.908.0': 93, '1.259.112.0': 94, '1.273.1400.0': 95, '1.273.1276.0': 96, '1.275.901.0': 97, '1.273.1232.0': 98, '1.253.971.0': 99, '1.275.1555.0': 100, '1.275.314.0': 101, '1.275.1397.0': 102, '1.273.597.0': 103, '1.275.256.0': 104, '1.275.1606.0': 105, '1.253.725.0': 106, '1.275.1433.0': 107, '1.267.711.0': 108, '1.269.1570.0': 109, '1.275.1325.0': 110, '1.273.1401.0': 111, '1.233.2039.0': 112, '1.267.15.0': 113, '1.275.241.0': 114, '1.273.330.0': 115, '1.273.802.0': 116, '1.275.561.0': 117, '1.275.1759.0': 118, '1.275.263.0': 119, '1.275.613.0': 120, '1.273.505.0': 121, '1.273.1471.0': 122, '1.275.1435.0': 123, '1.273.717.0': 124, '1.275.790.0': 125, '1.275.1503.0': 126, '1.275.1311.0': 127, '1.273.1668.0': 128, '1.275.909.0': 129, '1.275.1343.0': 130, '1.273.1558.0': 131, '1.273.858.0': 132, '1.273.774.0': 133, '1.275.665.0': 134, '1.275.54.0': 135, '1.237.0.0': 136, '1.275.365.0': 137, '1.273.1454.0': 138, '1.275.1525.0': 139, '1.273.529.0': 140, '1.275.378.0': 141, '1.273.933.0': 142, '1.271.1003.0': 143, '1.275.410.0': 144, '1.273.1749.0': 145, '1.273.1523.0': 146, '1.273.1610.0': 147, '1.275.757.0': 148, '1.267.359.0': 149, '1.275.1293.0': 150, '1.273.1304.0': 151, '1.273.1413.0': 152, '1.273.1574.0': 153, '1.275.461.0': 154, '1.241.1278.0': 155, '1.275.1744.0': 156, '1.275.699.0': 157, '1.275.49.0': 158, '1.273.1677.0': 159, '1.275.563.0': 160, '1.273.1608.0': 161, '1.273.1494.0': 162, '1.263.1879.0': 163, '1.269.345.0': 164, '1.273.1165.0': 165, '1.275.948.0': 166, '1.275.1380.0': 167, '1.275.1783.0': 168, '1.275.1058.0': 169, '1.275.500.0': 170, '1.267.1884.0': 171, '1.275.710.0': 172, '1.273.1167.0': 173, '1.275.199.0': 174, '1.271.135.0': 175, '1.275.569.0': 176, '1.273.1054.0': 177, '1.275.866.0': 178, '1.273.989.0': 179, '1.259.1663.0': 180, '1.275.11.0': 181, '1.275.1562.0': 182, '1.275.431.0': 183, '1.267.891.0': 184, '1.271.617.0': 185, '1.273.1788.0': 186, '1.275.491.0': 187, '1.275.138.0': 188, '1.271.676.0': 189, '1.277.49.0': 190, '1.273.658.0': 191, '1.273.848.0': 192, '1.273.625.0': 193, '1.271.501.0': 194, '1.273.1660.0': 195, '1.233.643.0': 196, '1.275.1011.0': 197, '1.267.1076.0': 198, '1.273.679.0': 199, '1.273.1051.0': 200, '1.275.1545.0': 201, '1.275.470.0': 202, '1.275.1694.0': 203, '1.275.1107.0': 204, '1.227.1509.0': 205, '1.275.1603.0': 206, '1.275.363.0': 207, '1.273.1516.0': 208, '1.273.665.0': 209, '1.273.293.0': 210, '1.275.499.0': 211, '1.273.536.0': 212, '1.275.1669.0': 213, '1.275.1683.0': 214, '1.275.779.0': 215, '1.275.1105.0': 216, '1.273.1629.0': 217, '1.275.913.0': 218, '1.271.720.0': 219, '1.273.1714.0': 220, '1.275.782.0': 221, '1.271.260.0': 222, '1.273.1440.0': 223, '1.233.2317.0': 224, '1.275.843.0': 225, '1.273.1460.0': 226, '1.275.1798.0': 227, '1.275.1429.0': 228, '1.273.461.0': 229, '1.261.1275.0': 230, '1.269.149.0': 231, '1.273.410.0': 232, '1.273.1699.0': 233, '1.273.1505.0': 234, '1.273.493.0': 235, '1.273.920.0': 236, '1.273.1125.0': 237, '1.275.1546.0': 238, '1.273.1282.0': 239, '1.273.1498.0': 240, '1.275.1332.0': 241, '1.273.488.0': 242, '1.273.430.0': 243, '1.273.975.0': 244, '1.273.1644.0': 245, '1.273.1664.0': 246, '1.273.1635.0': 247, '1.263.571.0': 248, '1.269.1255.0': 249, '1.265.642.0': 250, '1.275.570.0': 251, '1.273.1305.0': 252, '1.275.995.0': 253, '1.273.881.0': 254, '1.277.67.0': 255, '1.275.1378.0': 256, '1.263.448.0': 257, '1.273.484.0': 258, '1.273.1234.0': 259, '1.275.1120.0': 260, '1.275.1658.0': 261, '1.275.155.0': 262, '1.275.702.0': 263, '1.275.1134.0': 264, '1.275.1047.0': 265, '1.273.366.0': 266, '1.275.434.0': 267, '1.273.551.0': 268, '1.277.4.0': 269, '1.273.950.0': 270, '1.275.644.0': 271, '1.273.1598.0': 272, '1.273.491.0': 273, '1.273.425.0': 274, '1.253.85.0': 275, '1.273.1410.0': 276, '1.275.568.0': 277, '1.273.1179.0': 278, '1.275.1632.0': 279, '1.273.1140.0': 280, '1.275.18.0': 281, '1.275.223.0': 282, '1.275.1069.0': 283, '1.273.520.0': 284, '1.273.1127.0': 285, '1.271.360.0': 286, '1.273.514.0': 287, '1.273.738.0': 288, '1.275.1509.0': 289, '1.269.141.0': 290, '1.275.144.0': 291, '1.273.806.0': 292, '1.271.669.0': 293, '1.261.1606.0': 294, '1.273.1329.0': 295, '1.273.289.0': 296, '1.271.528.0': 297, '1.275.1086.0': 298, '1.273.266.0': 299, '1.275.188.0': 300, '1.275.1668.0': 301, '1.269.1733.0': 302, '1.269.768.0': 303, '1.275.190.0': 304, '1.275.1774.0': 305, '1.275.627.0': 306, '1.275.1362.0': 307, '1.265.948.0': 308, '1.273.1005.0': 309, '1.275.1458.0': 310, '1.273.212.0': 311, '1.273.1405.0': 312, '1.275.970.0': 313, '1.275.174.0': 314, '1.273.1490.0': 315, '1.273.512.0': 316, '1.275.1442.0': 317, '1.271.722.0': 318, '1.275.672.0': 319, '1.273.601.0': 320, '1.271.1102.0': 321, '1.273.922.0': 322, '1.275.938.0': 323, '1.273.1504.0': 324, '1.267.924.0': 325, '1.273.534.0': 326, '1.273.929.0': 327, '1.271.491.0': 328, '1.273.545.0': 329, '1.275.508.0': 330, '1.273.1208.0': 331, '1.273.241.0': 332, '1.271.778.0': 333, '1.275.158.0': 334, '1.275.1283.0': 335, '1.233.4036.0': 336, '1.273.506.0': 337, '1.273.1311.0': 338, '1.273.1809.0': 339, '1.273.1328.0': 340, '1.273.675.0': 341, '1.269.595.0': 342, '1.273.511.0': 343, '1.275.1062.0': 344, '1.275.1791.0': 345, '1.275.87.0': 346, '1.275.178.0': 347, '1.273.1585.0': 348, '1.249.202.0': 349, '1.275.1341.0': 350, '1.275.934.0': 351, '1.275.1761.0': 352, '1.275.760.0': 353, '1.273.483.0': 354, '1.275.810.0': 355, '1.273.1667.0': 356, '1.275.571.0': 357, '1.273.589.0': 358, '1.275.957.0': 359, '1.275.1662.0': 360, '1.267.1645.0': 361, '1.271.1001.0': 362, '1.273.1478.0': 363, '1.275.609.0': 364, '1.273.857.0': 365, '1.273.1704.0': 366, '1.275.557.0': 367, '1.275.1695.0': 368, '1.275.483.0': 369, '1.259.1638.0': 370, '1.275.1349.0': 371, '1.275.1198.0': 372, '1.275.850.0': 373, '1.271.120.0': 374, '1.275.1484.0': 375, '1.267.386.0': 376, '1.275.388.0': 377, '1.275.463.0': 378, '1.275.368.0': 379, '1.267.965.0': 380, '1.273.438.0': 381, '1.273.1509.0': 382, '1.275.348.0': 383, '1.275.59.0': 384, '1.257.835.0': 385, '1.275.1141.0': 386, '1.273.587.0': 387, '1.273.1702.0': 388, '1.275.1090.0': 389, '1.275.642.0': 390, '1.275.1609.0': 391, '1.275.1613.0': 392, '1.273.428.0': 393, '1.273.612.0': 394, '1.271.1027.0': 395, '1.275.1600.0': 396, '1.267.267.0': 397, '1.273.836.0': 398, '1.275.893.0': 399, '1.275.828.0': 400, '1.273.1056.0': 401, '1.261.744.0': 402, '1.273.1307.0': 403, '1.275.1643.0': 404, '1.273.510.0': 405, '1.275.969.0': 406, '1.271.1085.0': 407, '1.273.1362.0': 408, '1.273.541.0': 409, '1.275.327.0': 410, '1.269.1304.0': 411, '1.275.167.0': 412, '1.273.768.0': 413, '1.257.1434.0': 414, '1.273.654.0': 415, '1.233.3745.0': 416, '1.275.540.0': 417, '1.273.706.0': 418, '1.273.1626.0': 419, '1.273.1259.0': 420, '1.275.1080.0': 421, '1.239.1069.0': 422, '1.275.981.0': 423, '1.275.595.0': 424, '1.275.319.0': 425, '1.271.849.0': 426, '1.273.413.0': 427, '1.273.1188.0': 428, '1.263.321.0': 429, '1.275.687.0': 430, '1.275.390.0': 431, '1.233.3338.0': 432, '1.267.1681.0': 433, '1.273.334.0': 434, '1.263.828.0': 435, '1.277.43.0': 436, '1.275.150.0': 437, '1.275.84.0': 438, '1.275.81.0': 439, '1.275.598.0': 440, '1.273.861.0': 441, '1.275.974.0': 442, '1.275.611.0': 443, '1.275.1115.0': 444, '1.269.1427.0': 445, '1.275.321.0': 446, '1.275.1075.0': 447, '1.273.1260.0': 448, '1.275.667.0': 449, '1.273.1144.0': 450, '1.271.861.0': 451, '1.267.1562.0': 452, '1.273.1468.0': 453, '1.269.1961.0': 454, '1.273.1860.0': 455, '1.235.3368.0': 456, '1.263.559.0': 457, '1.275.504.0': 458, '1.275.413.0': 459, '1.251.546.0': 460, '1.275.414.0': 461, '1.275.576.0': 462, '1.275.1537.0': 463, '1.275.497.0': 464, '1.273.1666.0': 465, '1.275.684.0': 466, '1.263.445.0': 467, '1.275.1570.0': 468, '1.273.141.0': 469, '1.275.876.0': 470, '1.275.1527.0': 471, '1.269.1391.0': 472, '1.275.400.0': 473, '1.267.1524.0': 474, '1.273.280.0': 475, '1.273.1729.0': 476, '1.273.1718.0': 477, '1.273.1723.0': 478, '1.275.921.0': 479, '1.275.663.0': 480, '1.275.604.0': 481, '1.275.640.0': 482, '1.275.429.0': 483, '1.275.1417.0': 484, '1.269.1574.0': 485, '1.267.1013.0': 486, '1.275.1373.0': 487, '1.273.516.0': 488, '1.275.722.0': 489, '1.273.315.0': 490, '1.275.1276.0': 491, '1.275.1001.0': 492, '1.273.1465.0': 493, '1.275.1312.0': 494, '1.275.23.0': 495, '1.273.1519.0': 496, '1.271.815.0': 497, '1.259.1390.0': 498, '1.273.915.0': 499, '1.273.1223.0': 500, '1.273.1482.0': 501, '1.267.960.0': 502, '1.275.1106.0': 503, '1.273.778.0': 504, '1.267.675.0': 505, '1.275.864.0': 506, '1.275.453.0': 507, '1.247.459.0': 508, '1.271.78.0': 509, '1.275.1101.0': 510, '1.273.1244.0': 511, '1.275.641.0': 512, '1.271.1065.0': 513, '1.273.1616.0': 514, '1.275.686.0': 515, '1.273.1823.0': 516, '1.275.498.0': 517, '1.273.1707.0': 518, '1.275.617.0': 519, '1.271.75.0': 520, '1.275.1187.0': 521, '1.267.1375.0': 522, '1.275.852.0': 523, '1.275.682.0': 524, '1.273.1568.0': 525, '1.271.473.0': 526, '1.273.1691.0': 527, '1.275.1242.0': 528, '1.275.884.0': 529, '1.275.68.0': 530, '1.273.696.0': 531, '1.275.477.0': 532, '1.273.1784.0': 533, '1.275.1616.0': 534, '1.275.1103.0': 535, '1.275.1519.0': 536, '1.275.633.0': 537, '1.275.590.0': 538, '1.263.1029.0': 539, '1.265.270.0': 540, '1.273.312.0': 541, '1.271.105.0': 542, '1.273.1112.0': 543, '1.275.1104.0': 544, '1.273.1601.0': 545, '1.275.771.0': 546, '1.275.1478.0': 547, '1.275.555.0': 548, '1.273.1346.0': 549, '1.275.706.0': 550, '1.273.1870.0': 551, '1.275.1573.0': 552, '1.275.148.0': 553, '1.239.768.0': 554, '1.271.751.0': 555, '1.275.1487.0': 556, '1.275.1635.0': 557, '1.275.202.0': 558, '1.269.1786.0': 559, '1.273.750.0': 560, '1.259.537.0': 561, '1.269.400.0': 562, '1.273.1652.0': 563, '1.275.582.0': 564, '1.269.1192.0': 565, '1.273.1108.0': 566, '1.273.198.0': 567, '1.275.481.0': 568, '1.237.1445.0': 569, '1.275.292.0': 570, '1.273.1104.0': 571, '1.273.160.0': 572, '1.275.486.0': 573, '1.273.1548.0': 574, '1.275.352.0': 575, '1.265.164.0': 576, '1.273.518.0': 577, '1.275.794.0': 578, '1.259.124.0': 579, '1.273.76.0': 580, '1.273.1623.0': 581, '1.265.813.0': 582, '1.273.418.0': 583, '1.267.1569.0': 584, '1.275.448.0': 585, '1.273.526.0': 586, '1.275.1734.0': 587, '1.273.865.0': 588, '1.271.322.0': 589, '1.271.682.0': 590, '1.275.323.0': 591, '1.275.1565.0': 592, '1.273.513.0': 593, '1.247.865.0': 594, '1.275.674.0': 595, '1.275.485.0': 596, '1.273.1732.0': 597, '1.251.115.0': 598, '1.273.557.0': 599, '1.271.913.0': 600, '1.269.1663.0': 601, '1.273.703.0': 602, '1.275.94.0': 603, '1.273.798.0': 604, '1.233.119.0': 605, '1.271.236.0': 606, '1.275.1779.0': 607, '1.275.608.0': 608, '1.275.1067.0': 609, '1.275.715.0': 610, '1.275.495.0': 611, '1.273.689.0': 612, '1.275.929.0': 613, '1.275.132.0': 614, '1.263.859.0': 615, '1.275.281.0': 616, '1.275.209.0': 617, '1.271.1136.0': 618, '1.275.1727.0': 619, '1.273.559.0': 620, '1.275.373.0': 621, '1.271.1198.0': 622, '1.269.764.0': 623, '1.275.278.0': 624, '1.275.836.0': 625, '1.273.440.0': 626, '1.275.1320.0': 627, '1.273.1301.0': 628, '1.273.571.0': 629, '1.275.450.0': 630, '1.271.1068.0': 631, '1.275.359.0': 632, '1.275.792.0': 633, '1.269.408.0': 634, '1.275.423.0': 635, '1.275.101.0': 636, '1.273.609.0': 637, '1.275.357.0': 638, '1.275.41.0': 639, '1.269.1708.0': 640, '1.275.758.0': 641, '1.275.1796.0': 642, '1.273.986.0': 643, '1.273.157.0': 644, '1.273.995.0': 645, '1.269.1699.0': 646, '1.273.633.0': 647, '1.269.1042.0': 648, '1.273.1697.0': 649, '1.271.1178.0': 650, '1.273.1486.0': 651, '1.275.1691.0': 652, '1.275.151.0': 653, '1.237.286.0': 654, '1.265.880.0': 655, '1.273.1004.0': 656, '1.273.866.0': 657, '1.273.1270.0': 658, '1.273.1618.0': 659, '1.271.114.0': 660, '1.273.313.0': 661, '1.275.197.0': 662, '1.275.184.0': 663, '1.271.417.0': 664, '1.271.49.0': 665, '1.273.637.0': 666, '1.275.580.0': 667, '1.273.1863.0': 668, '1.261.1543.0': 669, '1.275.260.0': 670, '1.275.932.0': 671, '1.273.1633.0': 672, '1.275.586.0': 673, '1.265.524.0': 674, '1.271.738.0': 675, '1.271.1119.0': 676, '1.261.404.0': 677, '1.273.642.0': 678, '1.271.190.0': 679, '1.275.226.0': 680, '1.275.1448.0': 681, '1.263.752.0': 682, '1.261.1710.0': 683, '1.275.1732.0': 684, '1.247.520.0': 685, '1.275.935.0': 686, '1.267.537.0': 687, '1.275.180.0': 688, '1.273.20.0': 689, '1.271.693.0': 690, '1.275.1628.0': 691, '1.275.398.0': 692, '1.277.15.0': 693, '1.271.878.0': 694, '1.275.907.0': 695, '1.273.1650.0': 696, '1.271.71.0': 697, '1.275.545.0': 698, '1.269.1646.0': 699, '1.273.138.0': 700, '1.273.1131.0': 701, '1.275.1278.0': 702, '1.275.997.0': 703, '1.275.171.0': 704, '1.259.1155.0': 705, '1.275.566.0': 706, '1.267.1212.0': 707, '1.275.1498.0': 708, '1.275.307.0': 709, '1.273.195.0': 710, '1.269.809.0': 711, '1.275.376.0': 712, '1.265.692.0': 713, '1.257.366.0': 714, '1.271.826.0': 715, '1.273.692.0': 716, '1.273.584.0': 717, '1.263.1824.0': 718, '1.253.229.0': 719, '1.265.786.0': 720, '1.273.757.0': 721, '1.273.1501.0': 722, '1.275.718.0': 723, '1.271.485.0': 724, '1.275.46.0': 725, '1.275.535.0': 726, '1.275.952.0': 727, '1.275.1719.0': 728, '1.259.921.0': 729, '1.275.193.0': 730, '1.275.1431.0': 731, '1.275.1118.0': 732, '1.273.761.0': 733, '1.269.1479.0': 734, '1.275.883.0': 735, '1.265.389.0': 736, '1.259.272.0': 737, '1.275.284.0': 738, '1.271.536.0': 739, '1.275.72.0': 740, '1.267.1350.0': 741, '1.275.1753.0': 742, '1.275.455.0': 743, '1.275.559.0': 744, '1.273.167.0': 745, '1.235.428.0': 746, '1.269.1000.0': 747, '1.271.1050.0': 748, '1.261.880.0': 749, '1.275.65.0': 750, '1.273.208.0': 751, '1.273.1710.0': 752, '1.275.1384.0': 753, '1.263.132.0': 754, '1.275.975.0': 755, '1.273.1552.0': 756, '1.273.595.0': 757, '1.275.339.0': 758, '1.265.610.0': 759, '1.259.37.0': 760, '1.277.20.0': 761, '1.271.153.0': 762, '1.275.40.0': 763, '1.273.713.0': 764, '1.275.1567.0': 765, '1.271.1182.0': 766, '1.275.819.0': 767, '1.273.1592.0': 768, '1.273.1780.0': 769, '1.269.1188.0': 770, '1.271.294.0': 771, '1.269.1807.0': 772, '1.275.1540.0': 773, '1.259.1256.0': 774, '1.271.886.0': 775, '1.275.161.0': 776, '1.271.1016.0': 777, '1.275.1338.0': 778, '1.275.33.0': 779, '1.271.516.0': 780, '1.261.1683.0': 781, '1.273.1369.0': 782, '1.273.408.0': 783, '1.271.543.0': 784, '1.259.1000.0': 785, '1.275.474.0': 786, '1.275.1382.0': 787, '1.269.1889.0': 788, '1.273.1156.0': 789, '1.273.1545.0': 790, '1.267.1832.0': 791, '1.273.1293.0': 792, '1.275.1762.0': 793, '1.275.631.0': 794, '1.275.697.0': 795, '1.273.501.0': 796, '1.275.355.0': 797, '1.275.1578.0': 798, '1.275.360.0': 799, '1.251.1167.0': 800, '1.273.1338.0': 801, '1.269.211.0': 802, '1.275.1327.0': 803, '1.275.8.0': 804, '1.275.993.0': 805, '1.275.441.0': 806, '1.273.868.0': 807, '1.273.1605.0': 808, '1.267.1719.0': 809, '1.275.1500.0': 810, '1.267.1715.0': 811, '1.271.996.0': 812, '1.273.765.0': 813, '1.271.1129.0': 814, '1.235.3184.0': 815, '1.273.1327.0': 816, '1.273.165.0': 817, '1.271.679.0': 818, '1.273.489.0': 819, '1.273.1354.0': 820, '1.261.996.0': 821, '1.275.615.0': 822, '1.273.1734.0': 823, '1.261.947.0': 824, '1.271.1042.0': 825, '1.261.1614.0': 826, '1.273.1397.0': 827, '1.275.784.0': 828, '1.275.1329.0': 829, '1.277.13.0': 830, '1.273.459.0': 831, '1.273.630.0': 832, '1.273.1136.0': 833, '1.275.36.0': 834, '1.273.1512.0': 835, '1.271.598.0': 836, '1.257.809.0': 837, '1.275.1748.0': 838, '1.277.64.0': 839, '1.273.1360.0': 840, '1.237.932.0': 841, '1.267.611.0': 842, '1.275.622.0': 843, '1.269.465.0': 844, '1.275.1425.0': 845, '1.267.1804.0': 846, '1.275.1346.0': 847, '1.275.108.0': 848, '1.273.638.0': 849, '1.275.1234.0': 850, '1.221.14.0': 851, '1.233.2491.0': 852, '1.273.709.0': 853, '1.253.1055.0': 854, '1.269.1066.0': 855, '1.245.912.0': 856, '1.275.1639.0': 857, '1.275.1013.0': 858, '1.275.230.0': 859, '1.267.1629.0': 860, '1.261.957.0': 861, '1.263.830.0': 862, '1.275.911.0': 863, '1.275.1686.0': 864, '1.275.245.0': 865, '1.263.325.0': 866, '1.275.805.0': 867, '1.269.1145.0': 868, '1.273.1679.0': 869, '1.269.1134.0': 870, '1.249.707.0': 871, '1.273.1520.0': 872, '1.263.154.0': 873, '1.275.599.0': 874, '1.273.927.0': 875, '1.273.456.0': 876, '1.263.121.0': 877, '1.275.769.0': 878, '1.273.1475.0': 879, '1.275.1129.0': 880, '1.271.728.0': 881, '1.269.1356.0': 882, '1.275.1286.0': 883, '1.271.764.0': 884, '1.271.325.0': 885, '1.273.1656.0': 886, '1.261.1561.0': 887, '1.271.643.0': 888, '1.261.1428.0': 889, '1.231.195.0': 890, '1.267.1800.0': 891, '1.275.767.0': 892, '1.265.36.0': 893, '1.273.1791.0': 894, '1.275.1712.0': 895, '1.275.1771.0': 896, '1.269.519.0': 897, '1.273.422.0': 898, '1.271.798.0': 899, '1.275.542.0': 900, '1.273.326.0': 901, '1.275.943.0': 902, '1.273.499.0': 903, '1.273.1688.0': 904, '1.267.1669.0': 905, '1.275.380.0': 906, '1.267.1502.0': 907, '1.275.636.0': 908, '1.275.310.0': 909, '1.273.94.0': 910, '1.275.847.0': 911, '1.239.688.0': 912, '1.273.841.0': 913, '1.259.809.0': 914, '1.273.1176.0': 915, '1.269.396.0': 916, '1.275.835.0': 917, '1.257.575.0': 918, '1.271.654.0': 919, '1.273.128.0': 920, '1.275.1598.0': 921, '1.275.849.0': 922, '1.275.788.0': 923, '1.275.826.0': 924, '1.273.434.0': 925, '1.275.821.0': 926, '1.263.124.0': 927, '1.261.806.0': 928, '1.273.581.0': 929, '1.273.1685.0': 930, '1.267.1477.0': 931, '1.271.957.0': 932, '1.275.713.0': 933, '1.271.657.0': 934, '1.275.1421.0': 935, '1.271.771.0': 936, '1.273.1642.0': 937, '1.273.523.0': 938, '1.275.543.0': 939, '1.257.660.0': 940, '1.273.163.0': 941, '1.275.1776.0': 942, '1.275.1552.0': 943, '1.259.1371.0': 944, '1.259.1667.0': 945, '1.235.632.0': 946, '1.273.309.0': 947, '1.273.286.0': 948, '1.271.334.0': 949, '1.273.568.0': 950, '1.273.1589.0': 951, '1.273.314.0': 952, '1.273.1693.0': 953, '1.273.452.0': 954, '1.251.363.0': 955, '1.275.578.0': 956, '1.277.51.0': 957, '1.275.370.0': 958, '1.259.387.0': 959, '1.273.538.0': 960, '1.269.1095.0': 961, '1.275.443.0': 962, '1.275.1507.0': 963, '1.275.1659.0': 964, '1.271.824.0': 965, '1.271.897.0': 966, '1.273.1555.0': 967, '1.275.891.0': 968, '1.261.1503.0': 969, '1.263.1989.0': 970, '1.273.453.0': 971, '1.275.903.0': 972, '1.277.41.0': 973, '1.275.900.0': 974, '1.261.1025.0': 975, '1.275.164.0': 976, '1.259.1041.0': 977, '1.275.219.0': 978, '1.271.1100.0': 979, '1.267.1702.0': 980, '1.225.2575.0': 981, '1.275.149.0': 982, '1.239.990.0': 983, '1.273.1324.0': 984, '1.273.735.0': 985, '1.271.111.0': 986, '1.269.28.0': 987, '1.273.154.0': 988, '1.269.1109.0': 989, '1.265.444.0': 990, '1.269.846.0': 991, '1.275.438.0': 992, '1.271.711.0': 993, '1.273.770.0': 994, '1.273.55.0': 995, '1.267.1004.0': 996, '1.267.1692.0': 997, '1.275.1741.0': 998, '1.271.1173.0': 999, '1.257.478.0': 1000, '1.267.1751.0': 1001, '1.275.105.0': 1002, '1.275.1656.0': 1003, '1.263.944.0': 1004, '1.269.1220.0': 1005, '1.273.591.0': 1006, '1.263.1843.0': 1007, '1.271.1140.0': 1008, '1.275.573.0': 1009, '1.273.685.0': 1010, '1.267.1739.0': 1011, '1.263.1133.0': 1012, '1.275.316.0': 1013, '1.261.1325.0': 1014, '1.261.1319.0': 1015, '1.273.61.0': 1016, '1.273.1812.0': 1017, '1.275.1709.0': 1018, '1.275.898.0': 1019, '1.267.632.0': 1020, '1.273.1634.0': 1021, '1.273.421.0': 1022, '1.269.1216.0': 1023, '1.269.1483.0': 1024, '1.275.1718.0': 1025, '1.273.129.0': 1026, '1.275.623.0': 1027, '1.253.96.0': 1028, '1.273.1655.0': 1029, '1.275.1236.0': 1030, '1.259.1328.0': 1031, '1.271.701.0': 1032, '1.273.879.0': 1033, '1.273.578.0': 1034, '1.277.46.0': 1035, '1.275.723.0': 1036, '1.263.1955.0': 1037, '1.275.182.0': 1038, '1.255.226.0': 1039, '1.275.1739.0': 1040, '1.273.1326.0': 1041, '1.275.940.0': 1042, '1.275.166.0': 1043, '1.275.584.0': 1044, '1.275.1593.0': 1045, '1.273.1296.0': 1046, '1.269.1690.0': 1047, '1.269.224.0': 1048, '1.271.213.0': 1049, '1.245.1013.0': 1050, '1.275.704.0': 1051, '1.275.198.0': 1052, '1.267.1740.0': 1053, '1.273.522.0': 1054, '1.263.598.0': 1055, '1.275.804.0': 1056, '1.259.1496.0': 1057, '1.271.239.0': 1058, '1.269.412.0': 1059, '1.271.550.0': 1060, '1.275.602.0': 1061, '1.271.864.0': 1062, '1.271.127.0': 1063, '1.275.725.0': 1064, '1.243.700.0': 1065, '1.269.1410.0': 1066, '1.271.1156.0': 1067, '1.233.1332.0': 1068, '1.275.420.0': 1069, '1.275.1794.0': 1070, '1.271.488.0': 1071, '1.271.341.0': 1072, '1.273.1595.0': 1073, '1.265.947.0': 1074, '1.269.69.0': 1075, '1.275.1582.0': 1076, '1.273.262.0': 1077, '1.275.1454.0': 1078, '1.265.485.0': 1079, '1.273.1031.0': 1080, '1.253.1046.0': 1081, '1.231.1796.0': 1082, '1.277.18.0': 1083, '1.271.730.0': 1084, '1.271.876.0': 1085, '1.275.1733.0': 1086, '1.245.82.0': 1087, '1.267.773.0': 1088, '1.269.1050.0': 1089, '1.261.1042.0': 1090, '1.251.936.0': 1091, '1.273.1267.0': 1092, '1.235.714.0': 1093, '1.275.968.0': 1094, '1.263.1580.0': 1095, '1.261.70.0': 1096, '1.275.1756.0': 1097, '1.275.1396.0': 1098, '1.275.1642.0': 1099, '1.233.3645.0': 1100, '1.275.213.0': 1101, '1.273.1299.0': 1102, '1.273.1638.0': 1103, '1.265.157.0': 1104, '1.263.190.0': 1105, '1.263.971.0': 1106, '1.273.1253.0': 1107, '1.269.1577.0': 1108, '1.267.943.0': 1109, '1.261.1277.0': 1110, '1.251.334.0': 1111, '1.259.512.0': 1112, '1.273.259.0': 1113, '1.261.46.0': 1114, '1.275.383.0': 1115, '1.263.787.0': 1116, '1.269.625.0': 1117, '1.271.757.0': 1118, '1.263.328.0': 1119, '1.251.733.0': 1120, '1.269.1212.0': 1121, '1.269.427.0': 1122, '1.259.1718.0': 1123, '1.271.825.0': 1124, '1.275.1654.0': 1125, '1.249.197.0': 1126, '1.263.490.0': 1127, '1.269.1338.0': 1128, '1.275.971.0': 1129, '1.273.125.0': 1130, '1.271.328.0': 1131, '1.267.1680.0': 1132, '1.269.518.0': 1133, '1.265.111.0': 1134, '1.269.989.0': 1135, '1.273.70.0': 1136, '1.275.1423.0': 1137, '1.277.23.0': 1138, '1.277.58.0': 1139, '1.271.670.0': 1140, '1.273.174.0': 1141, '1.275.466.0': 1142, '1.273.1356.0': 1143, '1.273.605.0': 1144, '1.259.16.0': 1145, '1.275.624.0': 1146, '1.275.1003.0': 1147, '1.273.1388.0': 1148, '1.273.808.0': 1149, '1.273.1241.0': 1150, '1.271.1121.0': 1151, '1.263.890.0': 1152, '1.273.1257.0': 1153, '1.275.479.0': 1154, '1.263.1930.0': 1155, '1.267.850.0': 1156, '1.265.768.0': 1157, '1.275.1665.0': 1158, '1.269.1229.0': 1159, '1.275.396.0': 1160, '1.273.132.0': 1161, '1.269.48.0': 1162, '1.271.885.0': 1163, '1.245.591.0': 1164, '1.269.1747.0': 1165, '1.239.861.0': 1166, '1.247.221.0': 1167, '1.271.89.0': 1168, '1.237.1329.0': 1169, '1.269.818.0': 1170, '1.267.910.0': 1171, '1.259.805.0': 1172, '1.275.437.0': 1173, '1.273.205.0': 1174, '1.249.99.0': 1175, '1.273.1657.0': 1176, '1.271.193.0': 1177, '1.275.798.0': 1178, '1.273.283.0': 1179, '1.239.1273.0': 1180, '1.275.1645.0': 1181, '1.265.737.0': 1182, '1.273.100.0': 1183, '1.269.888.0': 1184, '1.267.159.0': 1185, '1.247.1082.0': 1186, '1.251.716.0': 1187, '1.245.577.0': 1188, '1.257.1075.0': 1189, '1.251.637.0': 1190, '1.275.552.0': 1191, '1.271.971.0': 1192, '1.275.635.0': 1193, '1.261.1314.0': 1194, '1.273.166.0': 1195, '1.265.517.0': 1196, '1.275.490.0': 1197, '1.273.290.0': 1198, '1.249.991.0': 1199, '1.257.1096.0': 1200, '1.275.1730.0': 1201, '1.273.837.0': 1202, '1.253.1053.0': 1203, '1.275.1318.0': 1204, '1.275.5.0': 1205, '1.273.1264.0': 1206, '1.273.844.0': 1207, '1.273.1607.0': 1208, '1.261.139.0': 1209, '1.275.1390.0': 1210, '1.273.1682.0': 1211, '1.271.181.0': 1212, '1.267.1677.0': 1213, '1.271.1023.0': 1214, '1.251.891.0': 1215, '1.267.1870.0': 1216, '1.273.740.0': 1217, '1.275.1685.0': 1218, '1.275.777.0': 1219, '1.233.1137.0': 1220, '1.273.150.0': 1221, '1.271.525.0': 1222, '1.271.198.0': 1223, '1.271.1074.0': 1224, '1.233.4285.0': 1225, '1.263.305.0': 1226, '1.251.1003.0': 1227, '1.275.720.0': 1228, '1.269.261.0': 1229, '1.275.877.0': 1230, '1.267.1238.0': 1231, '1.269.461.0': 1232, '1.267.1164.0': 1233, '1.269.146.0': 1234, '1.271.184.0': 1235, '1.245.526.0': 1236, '1.243.98.0': 1237, '1.257.1417.0': 1238, '1.269.1880.0': 1239, '1.263.346.0': 1240, '1.261.77.0': 1241, '1.275.1769.0': 1242, '1.241.1395.0': 1243, '1.275.493.0': 1244, '1.273.256.0': 1245, '1.263.737.0': 1246, '1.263.1652.0': 1247, '1.273.416.0': 1248, '1.275.638.0': 1249, '1.245.868.0': 1250, '1.269.1904.0': 1251, '1.263.544.0': 1252, '1.269.1190.0': 1253, '1.271.1117.0': 1254, '1.269.1919.0': 1255, '1.271.358.0': 1256, '1.239.1546.0': 1257, '1.271.939.0': 1258, '1.275.845.0': 1259, '1.271.1124.0': 1260, '1.275.781.0': 1261, '1.265.74.0': 1262, '1.259.1634.0': 1263, '1.263.743.0': 1264, '1.239.567.0': 1265, '1.275.1113.0': 1266, '1.273.1162.0': 1267, '1.269.521.0': 1268, '1.269.669.0': 1269, '1.271.668.0': 1270, '1.259.1741.0': 1271, '1.275.823.0': 1272, '1.245.154.0': 1273, '1.271.159.0': 1274, '1.265.589.0': 1275, '1.233.2114.0': 1276, '1.263.455.0': 1277, '1.269.973.0': 1278, '1.273.477.0': 1279, '1.275.950.0': 1280, '1.225.191.0': 1281, '1.267.1468.0': 1282, '1.251.840.0': 1283, '1.267.682.0': 1284, '1.247.93.0': 1285, '1.235.182.0': 1286, '1.265.549.0': 1287, '1.275.550.0': 1288, '1.261.185.0': 1289, '1.269.512.0': 1290, '1.273.1261.0': 1291, '1.265.51.0': 1292, '1.267.1328.0': 1293, '1.271.555.0': 1294, '1.275.75.0': 1295, '1.267.1435.0': 1296, '1.271.1031.0': 1297, '1.271.608.0': 1298, '1.251.1343.0': 1299, '1.261.1541.0': 1300, '1.249.1320.0': 1301, '1.239.1040.0': 1302, '1.269.1752.0': 1303, '1.257.642.0': 1304, '1.273.1228.0': 1305, '1.273.43.0': 1306, '1.269.793.0': 1307, '1.269.584.0': 1308, '1.275.881.0': 1309, '1.273.1726.0': 1310, '1.275.1051.0': 1311, '1.259.546.0': 1312, '1.267.1727.0': 1313, '1.271.677.0': 1314, '1.267.478.0': 1315, '1.265.83.0': 1316, '1.269.944.0': 1317, '1.267.98.0': 1318, '1.261.191.0': 1319, '1.269.950.0': 1320, '1.271.832.0': 1321, '1.275.146.0': 1322, '1.271.345.0': 1323, '1.259.574.0': 1324, '1.259.986.0': 1325, '1.247.450.0': 1326, '1.265.143.0': 1327, '1.275.79.0': 1328, '1.263.1654.0': 1329, '1.257.615.0': 1330, '1.269.565.0': 1331, '1.271.572.0': 1332, '1.265.384.0': 1333, '1.263.1938.0': 1334, '1.261.1721.0': 1335, '1.273.152.0': 1336, '1.249.984.0': 1337, '1.257.1317.0': 1338, '1.263.428.0': 1339, '1.263.730.0': 1340, '1.267.1260.0': 1341, '1.273.164.0': 1342, '1.275.1411.0': 1343, '1.273.187.0': 1344, '1.269.641.0': 1345, '1.237.323.0': 1346, '1.275.1450.0': 1347, '1.275.1754.0': 1348, '1.231.1493.0': 1349, '1.267.526.0': 1350, '1.267.571.0': 1351, '1.259.1443.0': 1352, '1.273.177.0': 1353, '1.261.1230.0': 1354, '1.263.1951.0': 1355, '1.271.874.0': 1356, '1.267.1197.0': 1357, '1.263.205.0': 1358, '1.271.987.0': 1359, '1.253.629.0': 1360, '1.225.1590.0': 1361, '1.269.551.0': 1362, '1.269.1224.0': 1363, '1.259.590.0': 1364, '1.273.1587.0': 1365, '1.271.366.0': 1366, '1.273.1240.0': 1367, '1.267.1557.0': 1368, '1.233.977.0': 1369, '1.267.615.0': 1370, '1.261.885.0': 1371, '1.257.841.0': 1372, '1.265.108.0': 1373, '1.267.1672.0': 1374, '1.267.1272.0': 1375, '1.257.738.0': 1376, '1.265.318.0': 1377, '1.267.208.0': 1378, '1.271.363.0': 1379, '1.257.373.0': 1380, '1.271.900.0': 1381, '1.251.119.0': 1382, '1.275.547.0': 1383, '1.267.889.0': 1384, '1.275.1650.0': 1385, '1.263.160.0': 1386, '1.257.6.0': 1387, '1.267.1157.0': 1388, '1.275.583.0': 1389, '1.263.1728.0': 1390, '1.269.723.0': 1391, '1.267.550.0': 1392, '1.233.744.0': 1393, '1.275.194.0': 1394, '1.233.672.0': 1395, '1.251.359.0': 1396, '1.271.935.0': 1397, '1.251.1225.0': 1398, '1.269.1925.0': 1399, '1.269.1071.0': 1400, '1.275.177.0': 1401, '1.267.1284.0': 1402, '1.265.695.0': 1403, '1.237.196.0': 1404, '1.271.307.0': 1405, '1.257.991.0': 1406, '1.251.477.0': 1407, '1.269.1157.0': 1408, '1.273.1159.0': 1409, '1.275.832.0': 1410, '1.267.864.0': 1411, '1.265.197.0': 1412, '1.271.326.0': 1413, '1.273.188.0': 1414, '1.261.1510.0': 1415, '1.269.1231.0': 1416, '1.263.171.0': 1417, '1.267.1688.0': 1418, '1.257.957.0': 1419, '1.261.370.0': 1420, '1.263.1863.0': 1421, '1.277.2.0': 1422, '1.269.775.0': 1423, '1.269.1834.0': 1424, '1.267.78.0': 1425, '1.273.202.0': 1426, '1.263.1166.0': 1427, '1.275.588.0': 1428, '1.275.606.0': 1429, '1.271.359.0': 1430, '1.271.253.0': 1431, '1.255.161.0': 1432, '1.275.1589.0': 1433, '1.237.399.0': 1434, '1.263.1878.0': 1435, '1.267.784.0': 1436, '1.271.990.0': 1437, '1.269.277.0': 1438, '1.269.1239.0': 1439, '1.265.515.0': 1440, '1.271.804.0': 1441, '1.269.474.0': 1442, '1.251.1398.0': 1443, '1.237.238.0': 1444, '1.249.262.0': 1445, '1.271.891.0': 1446, '1.259.953.0': 1447, '1.261.585.0': 1448, '1.271.1166.0': 1449, '1.273.122.0': 1450, '1.273.179.0': 1451, '1.275.472.0': 1452, '1.259.1345.0': 1453, '1.271.678.0': 1454, '1.275.1486.0': 1455, '1.259.926.0': 1456, '1.227.2034.0': 1457, '1.233.2306.0': 1458, '1.269.202.0': 1459, '1.257.1116.0': 1460, '1.263.1841.0': 1461, '1.271.660.0': 1462, '1.271.704.0': 1463, '1.271.55.0': 1464, '1.267.503.0': 1465, '1.233.4167.0': 1466, '1.271.859.0': 1467, '1.267.1255.0': 1468, '1.269.178.0': 1469, '1.273.32.0': 1470, '1.269.1300.0': 1471, '1.237.951.0': 1472, '1.275.861.0': 1473, '1.229.1054.0': 1474, '1.245.730.0': 1475, '1.261.523.0': 1476, '1.263.1709.0': 1477, '1.269.272.0': 1478, '1.267.1133.0': 1479, '1.253.140.0': 1480, '1.265.311.0': 1481, '1.263.85.0': 1482, '1.267.1516.0': 1483, '1.263.1766.0': 1484, '1.269.940.0': 1485, '1.261.1138.0': 1486, '1.271.663.0': 1487, '1.267.1333.0': 1488, '1.267.807.0': 1489, '1.271.984.0': 1490, '1.247.385.0': 1491, '1.269.1385.0': 1492, '1.269.708.0': 1493, '1.275.459.0': 1494, '1.271.903.0': 1495, '1.225.1459.0': 1496, '1.271.736.0': 1497, '1.239.377.0': 1498, '1.253.269.0': 1499, '1.273.46.0': 1500, '1.261.104.0': 1501, '1.263.112.0': 1502, '1.267.154.0': 1503, '1.269.1974.0': 1504, '1.269.1776.0': 1505, '1.255.113.0': 1506, '1.269.692.0': 1507, '1.271.586.0': 1508, '1.269.967.0': 1509, '1.253.904.0': 1510, '1.269.411.0': 1511, '1.269.423.0': 1512, '1.269.1301.0': 1513, '1.269.761.0': 1514, '1.269.544.0': 1515, '1.257.830.0': 1516, '1.233.216.0': 1517, '1.275.538.0': 1518, '1.263.1006.0': 1519, '1.271.22.0': 1520, '1.275.972.0': 1521, '1.273.1336.0': 1522, '1.269.1851.0': 1523, '1.261.838.0': 1524, '1.243.10.0': 1525, '1.257.124.0': 1526, '1.269.830.0': 1527, '1.265.333.0': 1528, '1.257.436.0': 1529, '1.269.488.0': 1530, '1.271.460.0': 1531, '1.263.585.0': 1532, '1.261.1700.0': 1533, '1.273.754.0': 1534, '1.259.707.0': 1535, '1.233.984.0': 1536, '1.261.65.0': 1537, '1.269.51.0': 1538, '1.257.1003.0': 1539, '1.261.1577.0': 1540, '1.271.799.0': 1541, '1.271.1097.0': 1542, '1.237.77.0': 1543, '1.267.1207.0': 1544, '1.267.1778.0': 1545, '1.271.388.0': 1546, '1.251.581.0': 1547, '1.273.130.0': 1548, '1.269.909.0': 1549, '1.257.100.0': 1550, '1.261.354.0': 1551, '1.261.876.0': 1552, '1.235.3285.0': 1553, '1.271.456.0': 1554, '1.265.194.0': 1555, '1.245.232.0': 1556, '1.261.72.0': 1557, '1.267.860.0': 1558, '1.263.278.0': 1559, '1.267.813.0': 1560, '1.261.1633.0': 1561, '1.271.601.0': 1562, '1.257.1318.0': 1563, '1.269.1061.0': 1564, '1.237.219.0': 1565, '1.269.1492.0': 1566, '1.269.448.0': 1567, '1.237.1021.0': 1568, '1.245.857.0': 1569, '1.247.650.0': 1570, '1.251.1412.0': 1571, '1.233.285.0': 1572, '1.267.777.0': 1573, '1.265.527.0': 1574, '1.269.515.0': 1575, '1.273.144.0': 1576, '1.271.623.0': 1577, '1.261.566.0': 1578, '1.253.736.0': 1579, '1.271.232.0': 1580, '1.267.1161.0': 1581, '1.259.734.0': 1582, '1.271.498.0': 1583, '1.269.1773.0': 1584, '1.269.380.0': 1585, '1.249.790.0': 1586, '1.273.37.0': 1587, '1.241.1226.0': 1588, '1.259.1709.0': 1589, '1.269.965.0': 1590, '1.267.1142.0': 1591, '1.267.306.0': 1592, '1.271.108.0': 1593, '1.251.771.0': 1594, '1.243.883.0': 1595, '1.263.537.0': 1596, '1.239.1208.0': 1597, '1.257.1160.0': 1598, '1.259.640.0': 1599, '1.271.967.0': 1600, '1.267.1263.0': 1601, '1.261.666.0': 1602, '1.259.517.0': 1603, '1.273.1254.0': 1604, '1.261.1355.0': 1605, '1.273.463.0': 1606, '1.271.124.0': 1607, '1.235.2544.0': 1608, '1.269.1538.0': 1609, '1.271.769.0': 1610, '1.257.713.0': 1611, '1.257.451.0': 1612, '1.269.1363.0': 1613, '1.265.759.0': 1614, '1.249.918.0': 1615, '1.271.548.0': 1616, '1.271.1054.0': 1617, '1.269.578.0': 1618, '1.229.1992.0': 1619, '1.271.884.0': 1620, '1.267.1896.0': 1621, '1.269.1882.0': 1622, '1.267.1101.0': 1623, '1.269.320.0': 1624, '1.265.269.0': 1625, '1.257.534.0': 1626, '1.275.1717.0': 1627, '1.269.791.0': 1628, '1.257.429.0': 1629, '1.257.800.0': 1630, '1.255.77.0': 1631, '1.259.416.0': 1632, '1.253.783.0': 1633, '1.269.559.0': 1634, '1.269.1505.0': 1635, '1.267.932.0': 1636, '1.267.1627.0': 1637, '1.267.11.0': 1638, '1.271.963.0': 1639, '1.273.131.0': 1640, '1.259.94.0': 1641, '1.275.215.0': 1642, '1.263.1918.0': 1643, '1.269.314.0': 1644, '1.261.1006.0': 1645, '1.271.1104.0': 1646, '1.229.1456.0': 1647, '1.263.1118.0': 1648, '1.271.545.0': 1649, '1.245.1110.0': 1650, '1.263.119.0': 1651, '1.267.151.0': 1652, '1.265.251.0': 1653, '1.231.2045.0': 1654, '1.235.604.0': 1655, '1.271.348.0': 1656, '1.275.998.0': 1657, '1.267.538.0': 1658, '1.261.505.0': 1659, '1.267.525.0': 1660, '1.265.434.0': 1661, '1.263.563.0': 1662, '1.261.441.0': 1663, '1.259.1088.0': 1664, '1.271.1063.0': 1665, '1.261.1718.0': 1666, '1.269.264.0': 1667, '1.269.1374.0': 1668, '1.253.345.0': 1669, '1.273.91.0': 1670, '1.275.928.0': 1671, '1.267.245.0': 1672, '1.267.1139.0': 1673, '1.261.1322.0': 1674, '1.269.986.0': 1675, '1.253.1085.0': 1676, '1.267.1345.0': 1677, '1.269.206.0': 1678, '1.263.137.0': 1679, '1.251.38.0': 1680, '1.269.268.0': 1681, '1.265.543.0': 1682, '1.261.1310.0': 1683, '1.267.79.0': 1684, '1.267.739.0': 1685, '1.269.619.0': 1686, '1.227.347.0': 1687, '1.271.1133.0': 1688, '1.261.761.0': 1689, '1.271.638.0': 1690, '1.241.551.0': 1691, '1.269.558.0': 1692, '1.267.1513.0': 1693, '1.253.751.0': 1694, '1.273.482.0': 1695, '1.265.264.0': 1696, '1.251.991.0': 1697, '1.261.1278.0': 1698, '1.261.1547.0': 1699, '1.259.1546.0': 1700, '1.259.1623.0': 1701, '1.267.1497.0': 1702, '1.267.1203.0': 1703, '1.261.1555.0': 1704, '1.249.827.0': 1705, '1.263.1645.0': 1706, '1.263.1837.0': 1707, '1.263.643.0': 1708, '1.267.1301.0': 1709, '1.269.1474.0': 1710, '1.233.3570.0': 1711, '1.267.713.0': 1712, '1.271.420.0': 1713, '1.269.1235.0': 1714, '1.271.116.0': 1715, '1.257.930.0': 1716, '1.257.1349.0': 1717, '1.269.1953.0': 1718, '1.267.1293.0': 1719, '1.269.548.0': 1720, '1.263.876.0': 1721, '1.261.1339.0': 1722, '1.269.153.0': 1723, '1.263.144.0': 1724, '1.271.143.0': 1725, '1.267.535.0': 1726, '1.247.763.0': 1727, '1.271.245.0': 1728, '1.265.707.0': 1729, '1.265.680.0': 1730, '1.271.210.0': 1731, '1.263.1737.0': 1732, '1.263.954.0': 1733, '1.253.475.0': 1734, '1.265.560.0': 1735, '1.249.847.0': 1736, '1.273.110.0': 1737, '1.271.40.0': 1738, '1.233.1429.0': 1739, '1.271.324.0': 1740, '1.259.1298.0': 1741, '1.249.979.0': 1742, '1.261.1082.0': 1743, '1.263.1059.0': 1744, '1.267.86.0': 1745, '1.265.547.0': 1746, '1.267.380.0': 1747, '1.271.582.0': 1748, '1.241.869.0': 1749, '1.269.1501.0': 1750, '1.247.376.0': 1751, '1.263.267.0': 1752, '1.267.809.0': 1753, '1.275.977.0': 1754, '1.253.450.0': 1755, '1.261.297.0': 1756, '1.273.126.0': 1757, '1.265.349.0': 1758, '1.257.826.0': 1759, '1.271.511.0': 1760, '1.271.314.0': 1761, '1.229.1218.0': 1762, '1.251.1294.0': 1763, '1.263.968.0': 1764, '1.265.378.0': 1765, '1.273.467.0': 1766, '1.265.809.0': 1767, '1.267.1310.0': 1768, '1.269.1107.0': 1769, '1.231.1362.0': 1770, '1.273.113.0': 1771, '1.233.3781.0': 1772, '1.269.1263.0': 1773, '1.269.1293.0': 1774, '1.259.1629.0': 1775, '1.271.845.0': 1776, '1.263.454.0': 1777, '1.271.463.0': 1778, '1.269.1131.0': 1779, '1.269.236.0': 1780, '1.267.90.0': 1781, '1.271.1015.0': 1782, '1.259.1750.0': 1783, '1.253.138.0': 1784, '1.271.946.0': 1785, '1.233.1601.0': 1786, '1.273.107.0': 1787, '1.263.733.0': 1788, '1.229.727.0': 1789, '1.251.1002.0': 1790, '1.267.1769.0': 1791, '1.269.517.0': 1792, '1.271.101.0': 1793, '1.259.731.0': 1794, '1.247.645.0': 1795, '1.249.505.0': 1796, '1.257.132.0': 1797, '1.229.1981.0': 1798, '1.249.883.0': 1799, '1.271.403.0': 1800, '1.231.2154.0': 1801, '1.273.251.0': 1802, '1.249.1237.0': 1803, '1.265.826.0': 1804, '1.267.790.0': 1805, '1.249.95.0': 1806, '1.269.898.0': 1807, '1.269.1161.0': 1808, '1.273.182.0': 1809, '1.267.848.0': 1810, '1.243.894.0': 1811, '1.233.1185.0': 1812, '1.261.1202.0': 1813, '1.271.787.0': 1814, '1.257.873.0': 1815, '1.271.146.0': 1816, '1.251.1045.0': 1817, '1.267.521.0': 1818, '1.261.1661.0': 1819, '1.269.1557.0': 1820, '1.267.1158.0': 1821, '1.253.834.0': 1822, '1.259.1551.0': 1823, '1.263.567.0': 1824, '1.251.409.0': 1825, '1.271.1010.0': 1826, '1.267.446.0': 1827, '1.265.202.0': 1828, '1.271.1028.0': 1829, '1.267.1450.0': 1830, '1.269.982.0': 1831, '1.261.1558.0': 1832, '1.263.380.0': 1833, '1.259.1386.0': 1834, '1.257.45.0': 1835, '1.257.1169.0': 1836, '1.253.130.0': 1837, '1.269.1354.0': 1838, '1.271.452.0': 1839, '1.253.680.0': 1840, '1.261.1293.0': 1841, '1.261.158.0': 1842, '1.267.762.0': 1843, '1.225.1913.0': 1844, '1.257.1092.0': 1845, '1.269.1283.0': 1846, '1.269.407.0': 1847, '1.263.450.0': 1848, '1.225.3703.0': 1849, '1.233.1099.0': 1850, '1.267.1684.0': 1851, '1.269.1251.0': 1852, '1.271.981.0': 1853, '1.255.2.0': 1854, '1.263.800.0': 1855, '1.271.1114.0': 1856, '1.251.808.0': 1857, '1.267.886.0': 1858, '1.251.28.0': 1859, '1.259.1279.0': 1860, '1.235.3236.0': 1861, '1.263.403.0': 1862, '1.247.472.0': 1863, '1.261.1302.0': 1864, '1.261.1563.0': 1865, '1.269.1243.0': 1866, '1.251.927.0': 1867, '1.265.726.0': 1868, '1.271.1012.0': 1869, '1.235.1991.0': 1870, '1.249.1244.0': 1871, '1.269.435.0': 1872, '1.267.1879.0': 1873, '1.267.232.0': 1874, '1.267.686.0': 1875, '1.257.1076.0': 1876, '1.263.1873.0': 1877, '1.263.647.0': 1878, '1.261.430.0': 1879, '1.233.4235.0': 1880, '1.269.1329.0': 1881, '1.269.730.0': 1882, '1.271.870.0': 1883, '1.251.1466.0': 1884, '1.275.992.0': 1885, '1.259.1049.0': 1886, '1.271.355.0': 1887, '1.255.203.0': 1888, '1.275.468.0': 1889, '1.269.13.0': 1890, '1.261.39.0': 1891, '1.269.1091.0': 1892, '1.273.191.0': 1893, '1.265.91.0': 1894, '1.263.900.0': 1895, '1.271.92.0': 1896, '1.269.1596.0': 1897, '1.255.107.0': 1898, '1.271.907.0': 1899, '1.267.65.0': 1900, '1.251.1339.0': 1901, '1.271.767.0': 1902, '1.251.1101.0': 1903, '1.271.648.0': 1904, '1.247.406.0': 1905, '1.229.68.0': 1906, '1.273.135.0': 1907, '1.267.1459.0': 1908, '1.235.2715.0': 1909, '1.269.600.0': 1910, '1.267.1239.0': 1911, '1.267.883.0': 1912, '1.263.1805.0': 1913, '1.241.1031.0': 1914, '1.271.652.0': 1915, '1.259.742.0': 1916, '1.225.2203.0': 1917, '1.269.569.0': 1918, '1.269.672.0': 1919, '1.269.588.0': 1920, '1.269.756.0': 1921, '1.267.1764.0': 1922, '1.271.19.0': 1923, '1.267.1343.0': 1924, '1.247.985.0': 1925, '1.225.4071.0': 1926, '1.269.483.0': 1927, '1.271.513.0': 1928, '1.225.1932.0': 1929, '1.269.142.0': 1930, '1.263.870.0': 1931, '1.267.873.0': 1932, '1.265.215.0': 1933, '1.243.910.0': 1934, '1.245.617.0': 1935, '1.263.242.0': 1936, '1.243.550.0': 1937, '1.263.332.0': 1938, '1.271.928.0': 1939, '1.269.840.0': 1940, '1.263.1883.0': 1941, '1.271.753.0': 1942, '1.269.1325.0': 1943, '1.257.24.0': 1944, '1.267.1906.0': 1945, '1.267.1522.0': 1946, '1.233.1206.0': 1947, '1.259.308.0': 1948, '1.261.285.0': 1949, '1.259.641.0': 1950, '1.273.127.0': 1951, '1.267.463.0': 1952, '1.241.953.0': 1953, '1.235.357.0': 1954, '1.239.303.0': 1955, '1.275.752.0': 1956, '1.267.1090.0': 1957, '1.267.1355.0': 1958, '1.249.751.0': 1959, '1.269.814.0': 1960, '1.235.2586.0': 1961, '1.263.721.0': 1962, '1.269.337.0': 1963, '1.271.51.0': 1964, '1.267.921.0': 1965, '1.269.877.0': 1966, '1.259.1596.0': 1967, '1.273.478.0': 1968, '1.229.823.0': 1969, '1.269.1650.0': 1970, '1.269.509.0': 1971, '1.267.696.0': 1972, '1.247.617.0': 1973, '1.275.1480.0': 1974, '1.277.62.0': 1975, '1.263.460.0': 1976, '1.267.639.0': 1977, '1.271.931.0': 1978, '1.271.632.0': 1979, '1.271.665.0': 1980, '1.257.1383.0': 1981, '1.241.491.0': 1982, '1.271.1060.0': 1983, '1.259.1806.0': 1984, '1.267.561.0': 1985, '1.267.76.0': 1986, '1.247.662.0': 1987, '1.255.118.0': 1988, '1.237.203.0': 1989, '1.259.428.0': 1990, '1.267.1119.0': 1991, '1.267.1420.0': 1992, '1.245.172.0': 1993, '1.269.181.0': 1994, '1.229.400.0': 1995, '1.245.1173.0': 1996, '1.263.656.0': 1997, '1.261.1109.0': 1998, '1.275.919.0': 1999, '1.259.848.0': 2000, '1.271.666.0': 2001, '1.271.650.0': 2002, '1.267.138.0': 2003, '1.259.539.0': 2004, '1.271.960.0': 2005, '1.261.1186.0': 2006, '1.263.125.0': 2007, '1.261.1261.0': 2008, '1.245.1172.0': 2009, '1.263.509.0': 2010, '1.269.999.0': 2011, '1.255.289.0': 2012, '1.251.736.0': 2013, '1.259.1223.0': 2014, '1.229.732.0': 2015, '1.233.3617.0': 2016, '1.263.2034.0': 2017, '1.271.655.0': 2018, '1.257.1202.0': 2019, '1.249.608.0': 2020, '1.259.1.0': 2021, '1.257.1306.0': 2022, '1.257.15.0': 2023, '1.245.775.0': 2024, '1.261.1375.0': 2025, '1.269.1694.0': 2026, '1.269.916.0': 2027, '1.247.931.0': 2028, '1.229.1843.0': 2029, '1.247.368.0': 2030, '1.271.1002.0': 2031, '1.263.672.0': 2032, '1.269.1344.0': 2033, '1.263.1895.0': 2034, '1.259.1703.0': 2035, '1.261.44.0': 2036, '1.253.1036.0': 2037, '1.271.1162.0': 2038, '1.247.1015.0': 2039, '1.253.822.0': 2040, '1.267.1462.0': 2041, '1.259.1263.0': 2042, '1.249.799.0': 2043, '1.269.1497.0': 2044, '1.263.420.0': 2045, '1.269.679.0': 2046, '1.259.1293.0': 2047, '1.269.725.0': 2048, '1.257.381.0': 2049, '1.267.1689.0': 2050, '1.273.66.0': 2051, '1.269.1526.0': 2052, '1.261.1143.0': 2053, '1.265.3.0': 2054, '1.269.1087.0': 2055, '1.261.1372.0': 2056, '1.261.118.0': 2057, '1.267.481.0': 2058, '1.269.747.0': 2059, '1.263.582.0': 2060, '1.259.721.0': 2061, '1.259.1232.0': 2062, '1.261.1398.0': 2063, '1.263.1044.0': 2064, '1.259.1565.0': 2065, '1.253.1052.0': 2066, '1.247.347.0': 2067, '1.253.574.0': 2068, '1.271.183.0': 2069, '1.249.281.0': 2070, '1.269.1914.0': 2071, '1.263.393.0': 2072, '1.269.148.0': 2073, '1.269.1055.0': 2074, '1.269.860.0': 2075, '1.253.1077.0': 2076, '1.267.1250.0': 2077, '1.271.1047.0': 2078, '1.259.1555.0': 2079, '1.261.1497.0': 2080, '1.259.1769.0': 2081, '1.269.606.0': 2082, '1.271.992.0': 2083, '1.249.167.0': 2084, '1.265.647.0': 2085, '1.257.959.0': 2086, '1.261.1578.0': 2087, '1.245.945.0': 2088, '1.263.1842.0': 2089, '1.247.291.0': 2090, '1.255.250.0': 2091, '1.263.1021.0': 2092, '1.251.463.0': 2093, '1.267.913.0': 2094, '1.271.1008.0': 2095, '1.247.1040.0': 2096, '1.269.1369.0': 2097, '1.265.682.0': 2098, '1.235.336.0': 2099, '1.227.1489.0': 2100, '1.253.758.0': 2101, '1.261.679.0': 2102, '1.245.795.0': 2103, '1.263.532.0': 2104, '1.257.1037.0': 2105, '1.257.1312.0': 2106, '1.271.379.0': 2107, '1.261.1507.0': 2108, '1.263.1934.0': 2109, '1.253.532.0': 2110, '1.259.1300.0': 2111, '1.269.924.0': 2112, '1.233.3850.0': 2113, '1.259.527.0': 2114, '1.257.201.0': 2115, '1.267.1067.0': 2116, '1.267.607.0': 2117, '1.231.1471.0': 2118, '1.263.156.0': 2119, '1.231.1152.0': 2120, '1.261.1431.0': 2121, '1.263.116.0': 2122, '1.267.1314.0': 2123, '1.273.105.0': 2124, '1.263.1135.0': 2125, '1.261.1069.0': 2126, '1.267.643.0': 2127, '1.247.823.0': 2128, '1.253.571.0': 2129, '1.269.680.0': 2130, '1.265.699.0': 2131, '1.249.1361.0': 2132, '1.269.1742.0': 2133, '1.265.385.0': 2134, '1.269.638.0': 2135, '1.265.621.0': 2136, '1.263.635.0': 2137, '1.269.740.0': 2138, '1.261.1536.0': 2139, '1.233.3477.0': 2140, '1.237.1702.0': 2141, '1.247.1024.0': 2142, '1.275.931.0': 2143, '1.259.209.0': 2144, '1.267.555.0': 2145, '1.253.777.0': 2146, '1.261.1622.0': 2147, '1.269.555.0': 2148, '1.251.223.0': 2149, '1.267.168.0': 2150, '1.267.565.0': 2151, '1.273.254.0': 2152, '1.243.578.0': 2153, '1.235.1852.0': 2154, '1.271.219.0': 2155, '1.237.787.0': 2156, '1.263.1130.0': 2157, '1.269.650.0': 2158, '1.269.958.0': 2159, '1.267.1889.0': 2160, '1.247.1022.0': 2161, '1.267.1736.0': 2162, '1.271.204.0': 2163, '1.249.615.0': 2164, '1.271.186.0': 2165, '1.271.721.0': 2166, '1.273.17.0': 2167, '1.257.395.0': 2168, '1.249.986.0': 2169, '1.251.1135.0': 2170, '1.269.1842.0': 2171, '1.263.867.0': 2172, '1.247.251.0': 2173, '1.263.80.0': 2174, '1.227.2704.0': 2175, '1.267.1442.0': 2176, '1.267.1508.0': 2177, '1.265.937.0': 2178, '1.265.722.0': 2179, '1.273.170.0': 2180, '1.259.1604.0': 2181, '1.263.1176.0': 2182, '1.263.1018.0': 2183, '1.235.3069.0': 2184, '1.259.698.0': 2185, '1.259.1614.0': 2186, '1.231.1052.0': 2187, '1.259.298.0': 2188, '1.269.1846.0': 2189, '1.267.834.0': 2190, '1.265.873.0': 2191, '1.273.142.0': 2192, '1.261.293.0': 2193, '1.225.1988.0': 2194, '1.271.672.0': 2195, '1.267.539.0': 2196, '1.253.790.0': 2197, '1.253.538.0': 2198, '1.251.344.0': 2199, '1.263.113.0': 2200, '1.265.303.0': 2201, '1.271.240.0': 2202, '1.269.58.0': 2203, '1.229.210.0': 2204, '1.227.2846.0': 2205, '1.271.333.0': 2206, '1.257.1264.0': 2207, '1.271.733.0': 2208, '1.269.1118.0': 2209, '1.251.392.0': 2210, '1.237.378.0': 2211, '1.261.1320.0': 2212, '1.251.1046.0': 2213, '1.263.1926.0': 2214, '1.267.1901.0': 2215, '1.265.211.0': 2216, '1.275.754.0': 2217, '1.259.733.0': 2218, '1.271.999.0': 2219, '1.225.164.0': 2220, '1.233.3662.0': 2221, '1.257.1252.0': 2222, '1.239.956.0': 2223, '1.235.195.0': 2224, '1.237.1302.0': 2225, '1.269.901.0': 2226, '1.261.1088.0': 2227, '1.261.1716.0': 2228, '1.269.239.0': 2229, '1.235.495.0': 2230, '1.229.1212.0': 2231, '1.271.327.0': 2232, '1.263.751.0': 2233, '1.263.212.0': 2234, '1.271.495.0': 2235, '1.269.1068.0': 2236, '1.251.731.0': 2237, '1.269.471.0': 2238, '1.257.313.0': 2239, '1.267.1414.0': 2240, '1.241.819.0': 2241, '1.235.2078.0': 2242, '1.251.1360.0': 2243, '1.267.341.0': 2244, '1.259.1583.0': 2245, '1.267.768.0': 2246, '1.233.154.0': 2247, '1.255.307.0': 2248, '1.259.1547.0': 2249, '1.269.1376.0': 2250, '1.237.1049.0': 2251, '1.267.1437.0': 2252, '1.261.558.0': 2253, '1.237.1578.0': 2254, '1.269.405.0': 2255, '1.269.927.0': 2256, '1.263.440.0': 2257, '1.253.176.0': 2258, '1.267.765.0': 2259, '1.267.868.0': 2260, '1.259.451.0': 2261, '1.245.536.0': 2262, '1.267.1724.0': 2263, '1.241.600.0': 2264, '1.267.840.0': 2265, '1.269.140.0': 2266, '1.265.519.0': 2267, '1.267.1431.0': 2268, '1.249.1260.0': 2269, '1.255.74.0': 2270, '1.237.665.0': 2271, '1.273.114.0': 2272, '1.243.583.0': 2273, '1.249.348.0': 2274, '1.263.67.0': 2275, '1.255.252.0': 2276, '1.269.1907.0': 2277, '1.269.1489.0': 2278, '1.235.260.0': 2279, '1.269.484.0': 2280, '1.259.1220.0': 2281, '1.249.118.0': 2282, '1.229.431.0': 2283, '1.271.658.0': 2284, '1.243.3.0': 2285, '1.237.1567.0': 2286, '1.267.1071.0': 2287, '1.269.385.0': 2288, '1.231.456.0': 2289, '1.243.866.0': 2290, '1.261.1524.0': 2291, '1.233.3801.0': 2292, '1.241.1019.0': 2293, '1.251.318.0': 2294, '1.263.482.0': 2295, '1.249.380.0': 2296, '1.259.1600.0': 2297, '1.269.1075.0': 2298, '1.271.918.0': 2299, '1.261.1345.0': 2300, '1.271.1111.0': 2301, '1.269.1053.0': 2302, '1.267.1268.0': 2303, '1.239.1350.0': 2304, '1.249.370.0': 2305, '1.259.1235.0': 2306, '1.267.1728.0': 2307, '1.267.346.0': 2308, '1.239.1256.0': 2309, '1.269.301.0': 2310, '1.259.1613.0': 2311, '1.271.332.0': 2312, '1.251.268.0': 2313, '1.267.133.0': 2314, '1.265.917.0': 2315, '1.253.1093.0': 2316, '1.227.2357.0': 2317, '1.269.439.0': 2318, '1.257.417.0': 2319, '1.267.880.0': 2320, '1.249.561.0': 2321, '1.233.1343.0': 2322, '1.263.712.0': 2323, '1.273.123.0': 2324, '1.267.196.0': 2325, '1.233.1009.0': 2326, '1.259.578.0': 2327, '1.263.707.0': 2328, '1.241.262.0': 2329, '1.261.252.0': 2330, '1.269.873.0': 2331, '1.245.700.0': 2332, '1.225.361.0': 2333, '1.251.471.0': 2334, '1.269.1857.0': 2335, '1.231.585.0': 2336, '1.261.1163.0': 2337, '1.271.605.0': 2338, '1.267.1009.0': 2339, '1.259.125.0': 2340, '1.251.507.0': 2341, '1.261.728.0': 2342, '1.271.47.0': 2343, '1.223.1327.0': 2344, '1.269.1130.0': 2345, '1.273.149.0': 2346, '1.265.155.0': 2347, '1.271.228.0': 2348, '1.275.985.0': 2349, '1.271.44.0': 2350, '1.269.1387.0': 2351, '1.249.947.0': 2352, '1.263.1813.0': 2353, '1.257.920.0': 2354, '1.233.1899.0': 2355, '1.259.1780.0': 2356, '1.235.2880.0': 2357, '1.269.1423.0': 2358, '1.233.1990.0': 2359, '1.269.665.0': 2360, '1.271.808.0': 2361, '1.259.1488.0': 2362, '1.271.811.0': 2363, '1.251.917.0': 2364, '1.239.1616.0': 2365, '1.227.1563.0': 2366, '1.259.49.0': 2367, '1.261.244.0': 2368, '1.269.341.0': 2369, '1.253.151.0': 2370, '1.263.1832.0': 2371, '1.269.491.0': 2372, '1.269.634.0': 2373, '1.247.544.0': 2374, '1.265.277.0': 2375, '1.225.3235.0': 2376, '1.251.831.0': 2377, '1.261.1406.0': 2378, '1.259.464.0': 2379, '1.269.1469.0': 2380, '1.257.243.0': 2381, '1.253.445.0': 2382, '1.251.922.0': 2383, '1.251.1016.0': 2384, '1.249.690.0': 2385, '1.257.1290.0': 2386, '1.263.1649.0': 2387, '1.269.1099.0': 2388, '1.259.722.0': 2389, '1.247.88.0': 2390, '1.263.374.0': 2391, '1.265.658.0': 2392, '1.227.2293.0': 2393, '1.257.1469.0': 2394, '1.261.1613.0': 2395, '1.267.373.0': 2396, '1.249.1227.0': 2397, '1.267.1472.0': 2398, '1.233.161.0': 2399, '1.271.454.0': 2400, '1.259.1291.0': 2401, '1.239.214.0': 2402, '1.259.1153.0': 2403, '1.261.92.0': 2404, '1.271.916.0': 2405, '1.251.452.0': 2406, '1.267.991.0': 2407, '1.235.1069.0': 2408, '1.263.1650.0': 2409, '1.259.144.0': 2410, '1.257.334.0': 2411, '1.251.1063.0': 2412, '1.253.352.0': 2413, '1.261.1192.0': 2414, '1.259.1075.0': 2415, '1.261.1581.0': 2416, '1.267.1494.0': 2417, '1.235.3124.0': 2418, '1.269.1531.0': 2419, '1.261.1019.0': 2420, '1.263.996.0': 2421, '1.271.821.0': 2422, '1.261.868.0': 2423, '1.263.100.0': 2424, '1.265.228.0': 2425, '1.251.192.0': 2426, '1.257.58.0': 2427, '1.245.771.0': 2428, '1.257.36.0': 2429, '1.269.1866.0': 2430, '1.271.470.0': 2431, '1.231.1844.0': 2432, '1.273.116.0': 2433, '1.235.1047.0': 2434, '1.225.2809.0': 2435, '1.249.550.0': 2436, '1.245.348.0': 2437, '1.273.39.0': 2438, '1.247.129.0': 2439, '1.245.877.0': 2440, '1.247.527.0': 2441, '1.265.632.0': 2442, '1.271.246.0': 2443, '1.269.23.0': 2444, '1.271.310.0': 2445, '1.257.121.0': 2446, '1.263.1946.0': 2447, '1.261.336.0': 2448, '1.233.610.0': 2449, '1.259.114.0': 2450, '1.261.351.0': 2451, '1.233.3214.0': 2452, '1.251.420.0': 2453, '1.251.1265.0': 2454, '1.259.1746.0': 2455, '1.261.676.0': 2456, '1.225.2944.0': 2457, '1.261.1393.0': 2458, '1.271.925.0': 2459, '1.259.1117.0': 2460, '1.261.1353.0': 2461, '1.235.270.0': 2462, '1.241.35.0': 2463, '1.233.1783.0': 2464, '1.251.939.0': 2465, '1.269.945.0': 2466, '1.259.1187.0': 2467, '1.265.859.0': 2468, '1.263.1596.0': 2469, '1.253.865.0': 2470, '1.255.163.0': 2471, '1.259.115.0': 2472, '1.235.3116.0': 2473, '1.265.412.0': 2474, '1.267.600.0': 2475, '1.269.564.0': 2476, '1.251.1177.0': 2477, '1.261.1402.0': 2478, '1.271.790.0': 2479, '1.269.1522.0': 2480, '1.269.913.0': 2481, '1.269.1415.0': 2482, '1.247.537.0': 2483, '1.267.188.0': 2484, '1.263.83.0': 2485, '1.269.689.0': 2486, '1.261.237.0': 2487, '1.233.3766.0': 2488, '1.265.929.0': 2489, '1.267.69.0': 2490, '1.233.3790.0': 2491, '1.269.52.0': 2492, '1.247.556.0': 2493, '1.271.914.0': 2494, '1.263.999.0': 2495, '1.225.2906.0': 2496, '1.271.37.0': 2497, '1.251.505.0': 2498, '1.267.771.0': 2499, '1.271.659.0': 2500, '1.257.774.0': 2501, '1.261.1605.0': 2502, '1.255.245.0': 2503, '1.259.1585.0': 2504, '1.269.143.0': 2505, '1.249.990.0': 2506, '1.261.747.0': 2507, '1.249.688.0': 2508, '1.247.287.0': 2509, '1.271.370.0': 2510, '1.231.1163.0': 2511, '1.239.1016.0': 2512, '1.249.367.0': 2513, '1.261.1053.0': 2514, '1.251.513.0': 2515, '1.263.162.0': 2516, '1.271.235.0': 2517, '1.257.945.0': 2518, '1.259.1018.0': 2519, '1.223.2310.0': 2520, '1.259.1455.0': 2521, '1.263.1962.0': 2522, '1.271.680.0': 2523, '1.241.1357.0': 2524, '1.255.143.0': 2525, '1.267.143.0': 2526, '1.235.2968.0': 2527, '1.243.438.0': 2528, '1.271.155.0': 2529, '1.249.1403.0': 2530, '1.251.740.0': 2531, '1.251.1021.0': 2532, '1.257.1244.0': 2533, '1.233.3457.0': 2534, '1.223.2685.0': 2535, '1.233.2671.0': 2536, '1.253.972.0': 2537, '1.253.742.0': 2538, '1.257.488.0': 2539, '1.263.1846.0': 2540, '1.259.241.0': 2541, '1.271.362.0': 2542, '1.235.1929.0': 2543, '1.239.313.0': 2544, '1.261.1066.0': 2545, '1.265.804.0': 2546, '1.265.862.0': 2547, '1.269.841.0': 2548, '1.267.658.0': 2549, '1.227.2410.0': 2550, '1.263.146.0': 2551, '1.271.1039.0': 2552, '1.267.93.0': 2553, '1.233.3331.0': 2554, '1.235.247.0': 2555, '1.259.704.0': 2556, '1.261.1583.0': 2557, '1.261.1303.0': 2558, '1.267.256.0': 2559, '1.229.449.0': 2560, '1.237.1082.0': 2561, '1.271.532.0': 2562, '1.253.798.0': 2563, '1.253.1015.0': 2564, '1.267.1454.0': 2565, '1.227.638.0': 2566, '1.227.2071.0': 2567, '1.263.808.0': 2568, '1.269.1467.0': 2569, '1.267.420.0': 2570, '1.271.667.0': 2571, '1.239.1605.0': 2572, '1.249.116.0': 2573, '1.233.3693.0': 2574, '1.267.1298.0': 2575, '1.257.257.0': 2576, '1.249.1170.0': 2577, '1.259.1098.0': 2578, '1.259.310.0': 2579, '1.261.367.0': 2580, '1.259.1761.0': 2581, '1.243.870.0': 2582, '1.263.1607.0': 2583, '1.233.3066.0': 2584, '1.233.3312.0': 2585, '1.237.510.0': 2586, '1.233.2085.0': 2587, '1.239.197.0': 2588, '1.237.998.0': 2589, '1.247.742.0': 2590, '1.269.1730.0': 2591, '1.247.969.0': 2592, '1.267.1269.0': 2593, '1.261.855.0': 2594, '1.249.805.0': 2595, '1.235.244.0': 2596, '1.271.323.0': 2597, '1.271.1044.0': 2598, '1.263.176.0': 2599, '1.235.229.0': 2600, '1.265.924.0': 2601, '1.271.442.0': 2602, '1.269.311.0': 2603, '1.261.589.0': 2604, '1.269.303.0': 2605, '1.259.1664.0': 2606, '1.269.480.0': 2607, '1.251.1131.0': 2608, '1.235.2849.0': 2609, '1.265.660.0': 2610, '1.233.4175.0': 2611, '1.231.1008.0': 2612, '1.269.694.0': 2613, '1.271.661.0': 2614, '1.239.104.0': 2615, '1.265.352.0': 2616, '1.235.3399.0': 2617, '1.251.1500.0': 2618, '1.263.506.0': 2619, '1.231.149.0': 2620, '1.265.419.0': 2621, '1.269.597.0': 2622, '1.267.1780.0': 2623, '1.267.471.0': 2624, '1.261.1386.0': 2625, '1.251.848.0': 2626, '1.263.14.0': 2627, '1.267.876.0': 2628, '1.243.861.0': 2629, '1.231.2114.0': 2630, '1.269.197.0': 2631, '1.245.459.0': 2632, '1.269.1197.0': 2633, '1.239.205.0': 2634, '1.269.678.0': 2635, '1.249.237.0': 2636, '1.251.742.0': 2637, '1.267.775.0': 2638, '1.263.814.0': 2639, '1.263.384.0': 2640, '1.271.708.0': 2641, '1.239.713.0': 2642, '1.229.1557.0': 2643, '1.267.780.0': 2644, '1.273.146.0': 2645, '1.241.120.0': 2646, '1.263.1092.0': 2647, '1.263.435.0': 2648, '1.267.323.0': 2649, '1.253.1030.0': 2650, '1.267.906.0': 2651, '1.265.416.0': 2652, '1.269.652.0': 2653, '1.233.370.0': 2654, '1.259.501.0': 2655, '1.245.148.0': 2656, '1.251.1120.0': 2657, '1.267.1699.0': 2658, '1.247.899.0': 2659, '1.261.610.0': 2660, '1.265.5.0': 2661, '1.261.725.0': 2662, '1.265.86.0': 2663, '1.251.271.0': 2664, '1.253.846.0': 2665, '1.273.120.0': 2666, '1.253.784.0': 2667, '1.253.34.0': 2668, '1.271.514.0': 2669, '1.265.933.0': 2670, '1.253.462.0': 2671, '1.261.1595.0': 2672, '1.267.523.0': 2673, '1.261.416.0': 2674, '1.257.74.0': 2675, '1.235.872.0': 2676, '1.227.2184.0': 2677, '1.233.875.0': 2678, '1.225.3982.0': 2679, '1.239.1156.0': 2680, '1.253.936.0': 2681, '1.253.360.0': 2682, '1.267.1210.0': 2683, '1.265.182.0': 2684, '1.259.1340.0': 2685, '1.267.548.0': 2686, '1.261.485.0': 2687, '1.243.52.0': 2688, '1.261.51.0': 2689, '1.265.307.0': 2690, '1.275.1065.0': 2691, '1.251.1038.0': 2692, '1.227.1718.0': 2693, '1.267.1874.0': 2694, '1.251.943.0': 2695, '1.239.603.0': 2696, '1.263.1128.0': 2697, '1.259.766.0': 2698, '1.247.170.0': 2699, '1.275.863.0': 2700, '1.271.201.0': 2701, '1.269.1296.0': 2702, '1.239.5.0': 2703, '1.269.893.0': 2704, '1.253.275.0': 2705, '1.269.1873.0': 2706, '1.267.224.0': 2707, '1.263.562.0': 2708, '1.265.704.0': 2709, '1.257.749.0': 2710, '1.225.2249.0': 2711, '1.267.831.0': 2712, '1.253.876.0': 2713, '1.265.213.0': 2714, '1.265.684.0': 2715, '1.261.1388.0': 2716, '1.233.3122.0': 2717, '1.245.435.0': 2718, '1.261.125.0': 2719, '1.225.2465.0': 2720, '1.229.1431.0': 2721, '1.243.474.0': 2722, '1.247.614.0': 2723, '1.239.176.0': 2724, '1.245.1157.0': 2725, '1.271.41.0': 2726, '1.237.645.0': 2727, '1.263.1564.0': 2728, '1.259.1753.0': 2729, '1.257.565.0': 2730, '1.267.1000.0': 2731, '1.263.940.0': 2732, '1.261.820.0': 2733, '1.269.1802.0': 2734, '1.265.522.0': 2735, '1.261.554.0': 2736, '1.271.656.0': 2737, '1.255.358.0': 2738, '1.233.2597.0': 2739, '1.267.467.0': 2740, '1.253.773.0': 2741, '1.231.1834.0': 2742, '1.257.1020.0': 2743, '1.229.259.0': 2744, '1.267.564.0': 2745, '1.223.1873.0': 2746, '1.269.123.0': 2747, '1.233.3224.0': 2748, '1.235.252.0': 2749, '1.269.137.0': 2750, '1.263.231.0': 2751, '1.227.2248.0': 2752, '1.269.771.0': 2753, '1.269.611.0': 2754, '1.233.2312.0': 2755, '1.255.340.0': 2756, '1.269.287.0': 2757, '1.251.117.0': 2758, '1.259.1594.0': 2759, '1.271.504.0': 2760, '1.263.59.0': 2761, '1.249.215.0': 2762, '1.231.184.0': 2763, '1.241.890.0': 2764, '1.259.570.0': 2765, '1.235.505.0': 2766, '1.259.1394.0': 2767, '1.249.450.0': 2768, '1.269.698.0': 2769, '1.263.568.0': 2770, '1.227.613.0': 2771, '1.243.724.0': 2772, '1.243.780.0': 2773, '1.237.1259.0': 2774, '1.259.194.0': 2775, '1.261.1689.0': 2776, '1.251.1027.0': 2777, '1.261.1620.0': 2778, '1.261.1300.0': 2779, '1.271.17.0': 2780, '1.263.239.0': 2781, '1.271.894.0': 2782, '1.263.1122.0': 2783, '1.251.702.0': 2784, '1.247.977.0': 2785, '1.245.1118.0': 2786, '1.269.192.0': 2787, '1.269.1121.0': 2788, '1.247.178.0': 2789, '1.251.901.0': 2790, '1.269.1113.0': 2791, '1.263.667.0': 2792, '1.259.1799.0': 2793, '1.271.953.0': 2794, '1.235.2094.0': 2795, '1.265.729.0': 2796, '1.235.417.0': 2797, '1.267.532.0': 2798, '1.251.859.0': 2799, '1.271.800.0': 2800, '1.233.4096.0': 2801, '1.237.119.0': 2802, '1.267.1078.0': 2803, '1.265.945.0': 2804, '1.271.664.0': 2805, '1.263.895.0': 2806, '1.271.949.0': 2807, '1.251.958.0': 2808, '1.263.740.0': 2809, '1.235.3315.0': 2810, '1.253.723.0': 2811, '1.233.2971.0': 2812, '1.267.497.0': 2813, '1.257.356.0': 2814, '1.261.1612.0': 2815, '1.225.1484.0': 2816, '1.249.1159.0': 2817, '1.229.111.0': 2818, '1.235.3012.0': 2819, '1.261.1260.0': 2820, '1.249.1315.0': 2821, '1.233.4226.0': 2822, '1.229.1609.0': 2823, '1.223.2495.0': 2824, '1.231.353.0': 2825, '1.271.519.0': 2826, '1.229.518.0': 2827, '1.247.339.0': 2828, '1.257.1260.0': 2829, '1.249.1283.0': 2830, '1.263.1966.0': 2831, '1.267.108.0': 2832, '1.269.738.0': 2833, '1.251.127.0': 2834, '1.267.1474.0': 2835, '1.269.516.0': 2836, '1.269.310.0': 2837, '1.247.43.0': 2838, '1.245.398.0': 2839, '1.249.1262.0': 2840, '1.249.1272.0': 2841, '1.259.934.0': 2842, '1.261.607.0': 2843, '1.257.917.0': 2844, '1.225.1338.0': 2845, '1.251.379.0': 2846, '1.249.474.0': 2847, '1.243.539.0': 2848, '1.249.269.0': 2849, '1.261.1676.0': 2850, '1.263.484.0': 2851, '1.251.232.0': 2852, '1.249.528.0': 2853, '1.269.934.0': 2854, '1.249.476.0': 2855, '1.257.359.0': 2856, '1.269.5.0': 2857, '1.253.11.0': 2858, '1.263.1113.0': 2859, '1.263.272.0': 2860, '1.269.456.0': 2861, '1.251.1161.0': 2862, '1.259.850.0': 2863, '1.249.994.0': 2864, '1.265.103.0': 2865, '1.249.675.0': 2866, '1.235.3091.0': 2867, '1.261.127.0': 2868, '1.233.3881.0': 2869, '1.261.739.0': 2870, '1.225.2230.0': 2871, '1.257.862.0': 2872, '1.265.258.0': 2873, '1.233.2285.0': 2874, '1.259.1342.0': 2875, '1.251.713.0': 2876, '1.267.872.0': 2877, '1.271.921.0': 2878, '1.259.583.0': 2879, '1.237.276.0': 2880, '1.259.941.0': 2881, '1.249.764.0': 2882, '1.275.924.0': 2883, '1.251.885.0': 2884, '1.257.742.0': 2885, '1.269.42.0': 2886, '1.253.109.0': 2887, '1.253.973.0': 2888, '1.267.628.0': 2889, '1.269.185.0': 2890, '1.263.134.0': 2891, '1.245.379.0': 2892, '1.259.1590.0': 2893, '1.249.712.0': 2894, '1.259.384.0': 2895, '1.267.1288.0': 2896, '1.253.527.0': 2897, '1.267.1055.0': 2898, '1.251.1299.0': 2899, '1.273.52.0': 2900, '1.259.682.0': 2901, '1.249.130.0': 2902, '1.269.1126.0': 2903, '1.249.774.0': 2904, '1.233.395.0': 2905, '1.259.1324.0': 2906, '1.253.654.0': 2907, '1.267.708.0': 2908, '1.267.939.0': 2909, '1.243.384.0': 2910, '1.249.1124.0': 2911, '1.267.1795.0': 2912, '1.267.1712.0': 2913, '1.261.682.0': 2914, '1.239.411.0': 2915, '1.225.1660.0': 2916, '1.249.733.0': 2917, '1.267.898.0': 2918, '1.233.3481.0': 2919, '1.233.1408.0': 2920, '1.259.1584.0': 2921, '1.235.2528.0': 2922, '1.263.223.0': 2923, '1.265.248.0': 2924, '1.259.1051.0': 2925, '1.275.933.0': 2926, '1.271.351.0': 2927, '1.269.55.0': 2928, '1.273.58.0': 2929, '1.249.28.0': 2930, '1.259.906.0': 2931, '1.267.1295.0': 2932, '1.251.959.0': 2933, '1.259.1383.0': 2934, '1.251.1376.0': 2935, '1.263.411.0': 2936, '1.259.1310.0': 2937, '1.259.904.0': 2938, '1.249.1164.0': 2939, '1.271.760.0': 2940, '1.269.1292.0': 2941, '1.265.104.0': 2942, '1.251.1270.0': 2943, '1.259.257.0': 2944, '1.263.63.0': 2945, '1.265.701.0': 2946, '1.241.366.0': 2947, '1.223.2112.0': 2948, '1.259.247.0': 2949, '1.271.224.0': 2950, '1.253.717.0': 2951, '1.237.1247.0': 2952, '1.249.976.0': 2953, '1.269.1103.0': 2954, '1.259.1268.0': 2955, '1.251.727.0': 2956, '1.257.500.0': 2957, '1.261.232.0': 2958, '1.249.1204.0': 2959, '1.233.1176.0': 2960, '1.269.629.0': 2961, '1.265.245.0': 2962, '1.237.290.0': 2963, '1.261.724.0': 2964, '1.269.307.0': 2965, '1.245.210.0': 2966, '1.251.447.0': 2967, '1.269.696.0': 2968, '1.259.1748.0': 2969, '1.271.252.0': 2970, '1.251.131.0': 2971, '1.257.543.0': 2972, '1.241.176.0': 2973, '1.223.2483.0': 2974, '1.245.1225.0': 2975, '1.233.3738.0': 2976, '1.257.195.0': 2977, '1.239.920.0': 2978, '1.233.4066.0': 2979, '1.269.1032.0': 2980, '1.263.1851.0': 2981, '1.249.1169.0': 2982, '1.243.389.0': 2983, '1.269.1036.0': 2984, '1.263.465.0': 2985, '1.255.236.0': 2986, '1.245.41.0': 2987, '1.257.1457.0': 2988, '1.265.150.0': 2989, '1.243.858.0': 2990, '1.231.1110.0': 2991, '1.269.930.0': 2992, '1.267.955.0': 2993, '1.267.337.0': 2994, '1.237.961.0': 2995, '1.259.434.0': 2996, '1.231.848.0': 2997, '1.263.739.0': 2998, '1.251.427.0': 2999, '1.229.14.0': 3000, '1.267.1283.0': 3001, '1.271.243.0': 3002, '1.265.921.0': 3003, '1.241.590.0': 3004, '1.267.1520.0': 3005, '1.259.1378.0': 3006, '1.267.746.0': 3007, '1.239.1340.0': 3008, '1.263.51.0': 3009, '1.269.1360.0': 3010, '1.259.1507.0': 3011, '1.273.73.0': 3012, '1.267.522.0': 3013, '1.259.701.0': 3014, '1.259.1254.0': 3015, '1.247.955.0': 3016, '1.233.1654.0': 3017, '1.245.140.0': 3018, '1.267.533.0': 3019, '1.259.703.0': 3020, '1.261.1134.0': 3021, '1.253.820.0': 3022, '1.253.193.0': 3023, '1.265.146.0': 3024, '1.269.145.0': 3025, '1.267.1237.0': 3026, '1.253.923.0': 3027, '1.261.1280.0': 3028, '1.271.1081.0': 3029, '1.261.1216.0': 3030, '1.269.495.0': 3031, '1.261.1173.0': 3032, '1.267.425.0': 3033, '1.259.875.0': 3034, '1.263.936.0': 3035, '1.227.2404.0': 3036, '1.269.580.0': 3037, '1.267.1134.0': 3038, '1.267.443.0': 3039, '1.253.143.0': 3040, '1.239.37.0': 3041, '1.267.804.0': 3042, '1.235.408.0': 3043, '1.261.564.0': 3044, '1.267.122.0': 3045, '1.267.1427.0': 3046, '1.267.1130.0': 3047, '1.269.684.0': 3048, '1.271.614.0': 3049, '1.271.629.0': 3050, '1.245.977.0': 3051, '1.257.283.0': 3052, '1.261.748.0': 3053, '1.261.1392.0': 3054, '1.269.1280.0': 3055, '1.261.276.0': 3056, '1.251.649.0': 3057, '1.261.1708.0': 3058, '1.261.61.0': 3059, '1.235.904.0': 3060, '1.265.225.0': 3061, '1.263.776.0': 3062, '1.235.2287.0': 3063, '1.269.1861.0': 3064, '1.251.1361.0': 3065, '1.241.1167.0': 3066, '1.267.1641.0': 3067, '1.247.216.0': 3068, '1.235.2012.0': 3069, '1.263.1052.0': 3070, '1.269.1203.0': 3071, '1.267.529.0': 3072, '1.261.633.0': 3073, '1.267.908.0': 3074, '1.227.1575.0': 3075, '1.271.540.0': 3076, '1.231.1310.0': 3077, '1.263.434.0': 3078, '1.259.772.0': 3079, '1.247.82.0': 3080, '1.229.1960.0': 3081, '1.245.774.0': 3082, '1.241.1073.0': 3083, '1.265.441.0': 3084, '1.263.696.0': 3085, '1.267.531.0': 3086, '1.229.709.0': 3087, '1.263.1691.0': 3088, '1.233.1158.0': 3089, '1.267.486.0': 3090, '1.269.1407.0': 3091, '1.257.0.0': 3092, '1.253.1033.0': 3093, '1.271.131.0': 3094, '1.267.150.0': 3095, '1.263.133.0': 3096, '1.245.777.0': 3097, '1.259.47.0': 3098, '1.271.216.0': 3099, '1.271.312.0': 3100, '1.259.1609.0': 3101, '1.267.1324.0': 3102, '1.263.464.0': 3103, '1.265.645.0': 3104, '1.265.69.0': 3105, '1.229.252.0': 3106, '1.267.262.0': 3107, '1.225.1261.0': 3108, '1.267.942.0': 3109, '1.239.1124.0': 3110, '1.247.783.0': 3111, '1.243.185.0': 3112, '1.245.98.0': 3113, '1.263.1550.0': 3114, '1.245.137.0': 3115, '1.247.642.0': 3116, '1.241.847.0': 3117, '1.263.457.0': 3118, '1.263.1075.0': 3119, '1.253.624.0': 3120, '1.269.1040.0': 3121, '1.245.627.0': 3122, '1.269.1797.0': 3123, '1.269.937.0': 3124, '1.261.1502.0': 3125, '1.247.785.0': 3126, '1.235.1805.0': 3127, '1.267.823.0': 3128, '1.263.270.0': 3129, '1.245.434.0': 3130, '1.257.229.0': 3131, '1.273.497.0': 3132, '1.231.782.0': 3133, '1.245.265.0': 3134, '1.267.105.0': 3135, '1.267.473.0': 3136, '1.257.886.0': 3137, '1.261.357.0': 3138, '1.239.1512.0': 3139, '1.229.1965.0': 3140, '1.269.392.0': 3141, '1.249.1077.0': 3142, '1.247.491.0': 3143, '1.249.1332.0': 3144, '1.251.1057.0': 3145, '1.271.621.0': 3146, '1.227.1731.0': 3147, '1.237.1284.0': 3148, '1.233.1594.0': 3149, '1.267.812.0': 3150, '1.259.987.0': 3151, '1.257.967.0': 3152, '1.271.699.0': 3153, '1.229.1907.0': 3154, '1.259.849.0': 3155, '1.229.1155.0': 3156, '1.261.151.0': 3157, '1.229.426.0': 3158, '1.247.904.0': 3159, '1.263.142.0': 3160, '1.245.550.0': 3161, '1.267.1045.0': 3162, '1.261.637.0': 3163, '1.263.1913.0': 3164, '1.263.659.0': 3165, '1.271.645.0': 3166, '1.261.814.0': 3167, '1.233.849.0': 3168, '1.227.586.0': 3169, '1.265.625.0': 3170, '1.263.1102.0': 3171, '1.249.342.0': 3172, '1.233.3284.0': 3173, '1.233.3562.0': 3174, '1.257.1438.0': 3175, '1.263.227.0': 3176, '1.267.878.0': 3177, '1.251.981.0': 3178, '1.233.2458.0': 3179, '1.235.1217.0': 3180, '1.235.209.0': 3181, '1.225.2604.0': 3182, '1.265.370.0': 3183, '1.235.735.0': 3184, '1.237.531.0': 3185, '1.227.1587.0': 3186, '1.227.2324.0': 3187, '1.241.574.0': 3188, '1.233.2780.0': 3189, '1.259.78.0': 3190, '1.259.269.0': 3191, '1.263.1055.0': 3192, '1.251.519.0': 3193, '1.263.1885.0': 3194, '1.231.438.0': 3195, '1.259.778.0': 3196, '1.251.1078.0': 3197, '1.233.3319.0': 3198, '1.243.1021.0': 3199, '1.253.851.0': 3200, '1.227.848.0': 3201, '1.261.707.0': 3202, '1.239.898.0': 3203, '1.227.254.0': 3204, '1.263.433.0': 3205, '1.253.102.0': 3206, '1.239.1315.0': 3207, '1.253.144.0': 3208, '1.263.496.0': 3209, '1.269.477.0': 3210, '1.233.748.0': 3211, '1.225.412.0': 3212, '1.269.615.0': 3213, '1.267.1697.0': 3214, '1.259.106.0': 3215, '1.235.618.0': 3216, '1.237.1116.0': 3217, '1.265.241.0': 3218, '1.227.931.0': 3219, '1.261.1097.0': 3220, '1.259.958.0': 3221, '1.251.170.0': 3222, '1.255.160.0': 3223, '1.259.1744.0': 3224, '1.247.655.0': 3225, '1.259.462.0': 3226, '1.259.497.0': 3227, '1.235.1053.0': 3228, '1.257.995.0': 3229, '1.257.441.0': 3230, '1.259.964.0': 3231, '1.263.1981.0': 3232, '1.251.140.0': 3233, '1.259.379.0': 3234, '1.269.1828.0': 3235, '1.261.331.0': 3236, '1.251.759.0': 3237, '1.269.431.0': 3238, '1.237.1160.0': 3239, '1.235.2087.0': 3240, '1.261.1371.0': 3241, '1.237.536.0': 3242, '1.271.575.0': 3243, '1.253.400.0': 3244, '1.269.882.0': 3245, '1.267.238.0': 3246, '1.223.2949.0': 3247, '1.263.1893.0': 3248, '1.241.1008.0': 3249, '1.245.212.0': 3250, '1.271.909.0': 3251, '1.235.1383.0': 3252, '1.263.1163.0': 3253, '1.253.153.0': 3254, '1.247.341.0': 3255, '1.267.1085.0': 3256, '1.231.1992.0': 3257, '1.239.745.0': 3258, '1.239.1430.0': 3259, '1.223.1814.0': 3260, '1.251.739.0': 3261, '1.235.2543.0': 3262, '1.247.480.0': 3263, '1.263.824.0': 3264, '1.259.120.0': 3265, '1.263.536.0': 3266, '1.267.1249.0': 3267, '1.257.114.0': 3268, '1.259.736.0': 3269, '1.267.671.0': 3270, '1.263.446.0': 3271, '1.239.365.0': 3272, '1.235.2447.0': 3273, '1.261.415.0': 3274, '1.265.290.0': 3275, '1.237.144.0': 3276, '1.257.639.0': 3277, '1.259.1301.0': 3278, '1.263.688.0': 3279, '1.239.1412.0': 3280, '1.261.1394.0': 3281, '1.251.597.0': 3282, '1.225.255.0': 3283, '1.239.1521.0': 3284, '1.237.1086.0': 3285, '1.235.2697.0': 3286, '1.261.794.0': 3287, '1.241.183.0': 3288, '1.237.721.0': 3289, '1.263.1011.0': 3290, '1.237.1560.0': 3291, '1.227.1841.0': 3292, '1.251.1409.0': 3293, '1.223.1964.0': 3294, '1.231.258.0': 3295, '1.259.1610.0': 3296, '1.267.221.0': 3297, '1.253.626.0': 3298, '1.263.1535.0': 3299, '1.227.1306.0': 3300, '1.253.804.0': 3301, '1.229.1628.0': 3302, '1.225.2535.0': 3303, '1.237.1595.0': 3304, '1.247.929.0': 3305, '1.231.1958.0': 3306, '1.261.1041.0': 3307, '1.247.847.0': 3308, '1.259.1072.0': 3309, '1.251.704.0': 3310, '1.267.993.0': 3311, '1.265.803.0': 3312, '1.253.557.0': 3313, '1.263.1584.0': 3314, '1.269.744.0': 3315, '1.267.328.0': 3316, '1.239.151.0': 3317, '1.261.1717.0': 3318, '1.237.1165.0': 3319, '1.251.632.0': 3320, '1.263.580.0': 3321, '1.229.1588.0': 3322, '1.261.1070.0': 3323, '1.257.1086.0': 3324, '1.259.798.0': 3325, '1.267.139.0': 3326, '1.247.173.0': 3327, '1.245.436.0': 3328, '1.267.490.0': 3329, '1.259.1659.0': 3330, '1.247.979.0': 3331, '1.261.1049.0': 3332, '1.265.799.0': 3333, '1.257.527.0': 3334, '1.245.313.0': 3335, '1.251.516.0': 3336, '1.249.324.0': 3337, '1.259.156.0': 3338, '1.255.335.0': 3339, '1.223.2397.0': 3340, '1.243.272.0': 3341, '1.225.3827.0': 3342, '1.243.266.0': 3343, '1.231.1272.0': 3344, '1.239.434.0': 3345, '1.245.126.0': 3346, '1.235.805.0': 3347, '1.247.413.0': 3348, '1.239.1596.0': 3349, '1.261.1498.0': 3350, '1.269.929.0': 3351, '1.267.494.0': 3352, '1.263.429.0': 3353, '1.265.913.0': 3354, '1.239.752.0': 3355, '1.245.91.0': 3356, '1.267.1748.0': 3357, '1.259.591.0': 3358, '1.261.654.0': 3359, '1.225.3529.0': 3360, '1.251.1312.0': 3361, '1.231.261.0': 3362, '1.257.387.0': 3363, '1.223.2192.0': 3364, '1.265.714.0': 3365, '1.261.431.0': 3366, '1.257.1109.0': 3367, '1.265.666.0': 3368, '1.247.966.0': 3369, '1.229.698.0': 3370, '1.233.1198.0': 3371, '1.255.219.0': 3372, '1.237.1657.0': 3373, '1.259.348.0': 3374, '1.267.479.0': 3375, '1.257.1191.0': 3376, '1.245.237.0': 3377, '1.257.951.0': 3378, '1.239.488.0': 3379, '1.251.1033.0': 3380, '1.239.792.0': 3381, '1.239.1119.0': 3382, '1.237.1636.0': 3383, '1.247.485.0': 3384, '1.259.985.0': 3385, '1.263.123.0': 3386, '1.241.745.0': 3387, '1.265.186.0': 3388, '1.239.173.0': 3389, '1.251.1356.0': 3390, '1.261.1272.0': 3391, '1.233.1190.0': 3392, '1.229.1836.0': 3393, '1.225.595.0': 3394, '1.265.676.0': 3395, '1.229.863.0': 3396, '1.245.882.0': 3397, '1.235.1132.0': 3398, '1.249.187.0': 3399, '1.257.1217.0': 3400, '1.251.1400.0': 3401, '1.269.1335.0': 3402, '1.259.764.0': 3403, '1.267.56.0': 3404, '1.259.1005.0': 3405, '1.225.2185.0': 3406, '1.247.482.0': 3407, '1.237.488.0': 3408, '1.239.450.0': 3409, '1.269.499.0': 3410, '1.263.1145.0': 3411, '1.261.1533.0': 3412, '1.251.1007.0': 3413, '1.267.547.0': 3414, '1.263.94.0': 3415, '1.241.257.0': 3416, '1.261.1326.0': 3417, '1.257.1313.0': 3418, '1.237.1357.0': 3419, '1.263.1040.0': 3420, '1.259.478.0': 3421, '1.233.833.0': 3422, '1.251.475.0': 3423, '1.267.475.0': 3424, '1.247.131.0': 3425, '1.263.201.0': 3426, '1.267.437.0': 3427, '1.259.1358.0': 3428, '1.239.252.0': 3429, '1.239.785.0': 3430, '1.265.380.0': 3431, '1.251.403.0': 3432, '1.237.112.0': 3433, '1.249.1318.0': 3434, '1.265.126.0': 3435, '1.251.705.0': 3436, '1.263.681.0': 3437, '1.251.978.0': 3438, '1.261.734.0': 3439, '1.247.937.0': 3440, '1.251.1336.0': 3441, '1.271.85.0': 3442, '1.237.52.0': 3443, '1.251.290.0': 3444, '1.229.1912.0': 3445, '1.261.1060.0': 3446, '1.233.2008.0': 3447, '1.261.627.0': 3448, '1.229.1851.0': 3449, '1.243.479.0': 3450, '1.247.301.0': 3451, '1.267.1290.0': 3452, '1.259.1633.0': 3453, '1.271.774.0': 3454, '1.237.1198.0': 3455, '1.243.242.0': 3456, '1.241.972.0': 3457, '1.249.955.0': 3458, '1.267.119.0': 3459, '1.227.537.0': 3460, '1.257.364.0': 3461, '1.199.1615.0': 3462, '1.259.995.0': 3463, '1.259.631.0': 3464, '1.233.4172.0': 3465, '1.247.932.0': 3466, '1.257.1429.0': 3467, '1.235.1600.0': 3468, '1.263.692.0': 3469, '1.259.151.0': 3470, '1.243.282.0': 3471, '1.249.1294.0': 3472, '1.263.1922.0': 3473, '1.225.2978.0': 3474, '1.263.854.0': 3475, '1.237.1785.0': 3476, '1.259.284.0': 3477, '1.245.449.0': 3478, '1.241.1423.0': 3479, '1.263.1142.0': 3480, '1.267.543.0': 3481, '1.257.1230.0': 3482, '1.225.3320.0': 3483, '1.267.1229.0': 3484, '1.227.2037.0': 3485, '1.267.552.0': 3486, '1.263.1070.0': 3487, '1.253.442.0': 3488, '1.261.1650.0': 3489, '1.265.214.0': 3490, '1.233.3684.0': 3491, '1.259.1356.0': 3492, '1.229.2065.0': 3493, '1.261.1586.0': 3494, '1.251.973.0': 3495, '1.267.527.0': 3496, '1.251.878.0': 3497, '1.253.1096.0': 3498, '1.239.185.0': 3499, '1.261.452.0': 3500, '1.257.536.0': 3501, '1.261.1603.0': 3502, '1.259.1402.0': 3503, '1.235.2059.0': 3504, '1.243.1039.0': 3505, '1.241.333.0': 3506, '1.245.494.0': 3507, '1.235.1504.0': 3508, '1.249.101.0': 3509, '1.269.1089.0': 3510, '1.249.219.0': 3511, '1.269.878.0': 3512, '1.225.3316.0': 3513, '1.263.1856.0': 3514, '1.267.1256.0': 3515, '1.229.1110.0': 3516, '1.233.3726.0': 3517, '1.267.836.0': 3518, '1.263.3.0': 3519, '1.229.1669.0': 3520, '1.241.1283.0': 3521, '1.261.1572.0': 3522, '1.225.3519.0': 3523, '1.235.1367.0': 3524, '1.263.120.0': 3525, '1.249.762.0': 3526, '1.257.1300.0': 3527, '1.229.888.0': 3528, '1.233.3739.0': 3529, '1.239.762.0': 3530, '1.273.35.0': 3531, '1.271.449.0': 3532, '1.269.17.0': 3533, '1.251.1146.0': 3534, '1.265.27.0': 3535, '1.259.1042.0': 3536, '1.253.761.0': 3537, '1.261.1656.0': 3538, '1.257.1323.0': 3539, '1.267.472.0': 3540, '1.269.994.0': 3541, '1.249.109.0': 3542, '1.235.3152.0': 3543, '1.229.372.0': 3544, '1.267.1735.0': 3545, '1.237.928.0': 3546, '1.261.632.0': 3547, '1.259.509.0': 3548, '1.265.326.0': 3549, '1.237.173.0': 3550, '1.261.1644.0': 3551, '1.259.1096.0': 3552, '1.245.489.0': 3553, '1.261.511.0': 3554, '1.261.1693.0': 3555, '1.261.1531.0': 3556, '1.269.911.0': 3557, '1.267.249.0': 3558, '1.257.938.0': 3559, '1.227.1864.0': 3560, '1.263.387.0': 3561, '1.241.162.0': 3562, '1.249.430.0': 3563, '1.249.911.0': 3564, '1.227.1478.0': 3565, '1.259.507.0': 3566, '1.251.187.0': 3567, '1.269.1248.0': 3568, '1.225.387.0': 3569, '1.259.1710.0': 3570, '1.259.1637.0': 3571, '1.247.635.0': 3572, '1.243.731.0': 3573, '1.253.281.0': 3574, '1.267.857.0': 3575, '1.263.1632.0': 3576, '1.259.610.0': 3577, '1.245.1208.0': 3578, '1.263.1587.0': 3579, '1.231.1329.0': 3580, '1.263.985.0': 3581, '1.257.70.0': 3582, '1.257.1149.0': 3583, '1.265.904.0': 3584, '1.227.359.0': 3585, '1.245.335.0': 3586, '1.251.1.0': 3587, '1.233.86.0': 3588, '1.225.1311.0': 3589, '1.245.451.0': 3590, '1.259.612.0': 3591, '1.263.1915.0': 3592, '1.255.214.0': 3593, '1.257.1311.0': 3594, '1.257.579.0': 3595, '1.233.1001.0': 3596, '1.235.1652.0': 3597, '1.233.1211.0': 3598, '1.253.285.0': 3599, '1.235.1892.0': 3600, '1.247.269.0': 3601, '1.239.1447.0': 3602, '1.233.3189.0': 3603, '1.245.941.0': 3604, '1.223.2135.0': 3605, '1.267.1264.0': 3606, '1.241.686.0': 3607, '1.213.475.0': 3608, '1.253.142.0': 3609, '1.229.1866.0': 3610, '1.259.774.0': 3611, '1.251.699.0': 3612, '1.267.332.0': 3613, '1.243.868.0': 3614, '1.231.396.0': 3615, '1.263.1897.0': 3616, '1.237.899.0': 3617, '1.263.1869.0': 3618, '1.251.1397.0': 3619, '1.231.2071.0': 3620, '1.255.273.0': 3621, '1.243.859.0': 3622, '1.263.1182.0': 3623, '1.263.398.0': 3624, '1.263.187.0': 3625, '1.253.350.0': 3626, '1.243.825.0': 3627, '1.267.623.0': 3628, '1.263.1819.0': 3629, '1.233.3874.0': 3630, '1.269.1049.0': 3631, '1.235.957.0': 3632, '1.263.111.0': 3633, '1.267.476.0': 3634, '1.261.466.0': 3635, '1.251.545.0': 3636, '1.267.1773.0': 3637, '1.263.1563.0': 3638, '1.251.272.0': 3639, '1.251.995.0': 3640, '1.259.1184.0': 3641, '1.261.204.0': 3642, '1.271.320.0': 3643, '1.243.768.0': 3644, '1.267.879.0': 3645, '1.247.1010.0': 3646, '1.251.813.0': 3647, '1.261.1598.0': 3648, '1.249.685.0': 3649, '1.261.678.0': 3650, '1.267.1675.0': 3651, '1.265.32.0': 3652, '1.255.354.0': 3653, '1.263.447.0': 3654, '1.265.702.0': 3655, '1.259.107.0': 3656, '1.247.654.0': 3657, '1.255.39.0': 3658, '1.253.6.0': 3659, '1.251.1540.0': 3660, '1.259.795.0': 3661, '1.271.1035.0': 3662, '1.251.568.0': 3663, '1.257.1241.0': 3664, '1.261.385.0': 3665, '1.227.1655.0': 3666, '1.249.142.0': 3667, '1.233.1907.0': 3668, '1.233.3315.0': 3669, '1.259.447.0': 3670, '1.263.1594.0': 3671, '1.265.80.0': 3672, '1.261.45.0': 3673, '1.267.1424.0': 3674, '1.241.1337.0': 3675, '1.245.65.0': 3676, '1.259.713.0': 3677, '1.237.1231.0': 3678, '1.245.549.0': 3679, '1.253.325.0': 3680, '1.267.1733.0': 3681, '1.267.131.0': 3682, '1.261.470.0': 3683, '1.231.385.0': 3684, '1.239.1103.0': 3685, '1.259.725.0': 3686, '1.239.797.0': 3687, '1.245.1078.0': 3688, '1.259.435.0': 3689, '1.271.139.0': 3690, '1.263.316.0': 3691, '1.239.1488.0': 3692, '1.253.171.0': 3693, '1.263.423.0': 3694, '1.261.563.0': 3695, '1.261.228.0': 3696, '1.253.19.0': 3697, '1.251.434.0': 3698, '1.235.2398.0': 3699, '1.235.1560.0': 3700, '1.233.3197.0': 3701, '1.265.299.0': 3702, '1.229.353.0': 3703, '1.233.233.0': 3704, '1.259.360.0': 3705, '1.235.1519.0': 3706, '1.231.1671.0': 3707, '1.231.1619.0': 3708, '1.273.63.0': 3709, '1.261.1580.0': 3710, '1.261.1400.0': 3711, '1.237.262.0': 3712, '1.241.1380.0': 3713, '1.245.215.0': 3714, '1.223.2407.0': 3715, '1.259.886.0': 3716, '1.251.678.0': 3717, '1.229.607.0': 3718, '1.249.543.0': 3719, '1.259.858.0': 3720, '1.261.13.0': 3721, '1.263.1173.0': 3722, '1.237.871.0': 3723, '1.257.1463.0': 3724, '1.229.654.0': 3725, '1.251.429.0': 3726, '1.259.1368.0': 3727, '1.271.1006.0': 3728, '1.259.1292.0': 3729, '1.269.1043.0': 3730, '1.259.1550.0': 3731, '1.249.113.0': 3732, '1.225.2872.0': 3733, '1.251.555.0': 3734, '1.245.33.0': 3735, '1.251.432.0': 3736, '1.229.481.0': 3737, '1.225.3885.0': 3738, '1.247.692.0': 3739, '1.257.942.0': 3740, '1.263.702.0': 3741, '1.263.167.0': 3742, '1.229.1207.0': 3743, '1.245.889.0': 3744, '1.263.1089.0': 3745, '1.229.1001.0': 3746, '1.225.1289.0': 3747, '1.239.1217.0': 3748, '1.251.1212.0': 3749, '1.257.1398.0': 3750, '1.255.210.0': 3751, '1.247.127.0': 3752, '1.261.1065.0': 3753, '1.263.574.0': 3754, '1.245.568.0': 3755, '1.263.530.0': 3756, '1.255.346.0': 3757, '1.269.297.0': 3758, '1.257.530.0': 3759, '1.233.2202.0': 3760, '1.231.1294.0': 3761, '1.249.1404.0': 3762, '1.239.1507.0': 3763, '1.225.2843.0': 3764, '1.233.126.0': 3765, '1.227.803.0': 3766, '1.227.2655.0': 3767, '1.259.365.0': 3768, '1.235.2358.0': 3769, '1.261.347.0': 3770, '1.223.2074.0': 3771, '1.267.60.0': 3772, '1.227.2069.0': 3773, '1.263.1082.0': 3774, '1.259.1524.0': 3775, '1.261.692.0': 3776, '1.251.1036.0': 3777, '1.241.172.0': 3778, '1.233.1355.0': 3779, '1.229.1821.0': 3780, '1.245.802.0': 3781, '1.255.359.0': 3782, '1.225.3072.0': 3783, '1.267.252.0': 3784, '1.265.172.0': 3785, '1.253.403.0': 3786, '1.263.978.0': 3787, '1.259.1375.0': 3788, '1.235.510.0': 3789, '1.259.1553.0': 3790, '1.261.1015.0': 3791, '1.223.1749.0': 3792, '1.251.116.0': 3793, '1.261.520.0': 3794, '1.231.239.0': 3795, '1.259.1516.0': 3796, '1.269.865.0': 3797, '1.261.1701.0': 3798, '1.263.1850.0': 3799, '1.259.1009.0': 3800, '1.257.1365.0': 3801, '1.239.92.0': 3802, '1.233.1808.0': 3803, '1.231.1283.0': 3804, '1.261.1282.0': 3805, '1.253.1069.0': 3806, '1.249.90.0': 3807, '1.235.3301.0': 3808, '1.249.30.0': 3809, '1.233.3809.0': 3810, '1.237.1041.0': 3811, '1.261.1575.0': 3812, '1.253.446.0': 3813, '1.257.788.0': 3814, '1.249.734.0': 3815, '1.225.2298.0': 3816, '1.247.25.0': 3817, '1.229.1736.0': 3818, '1.257.539.0': 3819, '1.237.1618.0': 3820, '1.225.347.0': 3821, '1.265.329.0': 3822, '1.235.3207.0': 3823, '1.261.1512.0': 3824, '1.225.433.0': 3825, '1.265.53.0': 3826, '1.263.381.0': 3827, '1.271.81.0': 3828, '1.257.521.0': 3829, '1.263.414.0': 3830, '1.257.586.0': 3831, '1.253.674.0': 3832, '1.263.662.0': 3833, '1.259.891.0': 3834, '1.253.1048.0': 3835, '1.235.963.0': 3836, '1.233.3748.0': 3837, '1.263.82.0': 3838, '1.249.715.0': 3839, '1.257.555.0': 3840, '1.249.1279.0': 3841, '1.259.218.0': 3842, '1.251.1331.0': 3843, '1.267.459.0': 3844, '1.251.1542.0': 3845, '1.253.231.0': 3846, '1.243.286.0': 3847, '1.259.1276.0': 3848, '1.231.1033.0': 3849, '1.235.1873.0': 3850, '1.257.1040.0': 3851, '1.253.564.0': 3852, '1.231.1088.0': 3853, '1.259.56.0': 3854, '1.253.667.0': 3855, '1.237.1157.0': 3856, '1.261.1223.0': 3857, '1.267.129.0': 3858, '1.263.1146.0': 3859, '1.259.116.0': 3860, '1.233.1718.0': 3861, '1.247.814.0': 3862, '1.259.691.0': 3863, '1.259.221.0': 3864, '1.249.11.0': 3865, '1.263.1116.0': 3866, '1.249.727.0': 3867, '1.249.1289.0': 3868, '1.239.1426.0': 3869, '1.261.76.0': 3870, '1.243.847.0': 3871, '1.265.275.0': 3872, '1.259.644.0': 3873, '1.227.1263.0': 3874, '1.241.4.0': 3875, '1.261.727.0': 3876, '1.229.1349.0': 3877, '1.263.1067.0': 3878, '1.265.252.0': 3879, '1.253.418.0': 3880, '1.233.1822.0': 3881, '1.233.4320.0': 3882, '1.267.1198.0': 3883, '1.251.889.0': 3884, '1.235.2629.0': 3885, '1.253.746.0': 3886, '1.263.438.0': 3887, '1.259.142.0': 3888, '1.229.1791.0': 3889, '1.265.866.0': 3890, '1.249.682.0': 3891, '1.249.191.0': 3892, '1.247.295.0': 3893, '1.235.329.0': 3894, '1.243.260.0': 3895, '1.253.771.0': 3896, '1.263.150.0': 3897, '1.265.89.0': 3898, '1.257.383.0': 3899, '1.257.961.0': 3900, '1.257.76.0': 3901, '1.257.976.0': 3902, '1.241.812.0': 3903, '1.245.359.0': 3904, '1.251.969.0': 3905, '1.257.702.0': 3906, '1.257.953.0': 3907, '1.251.173.0': 3908, '1.239.460.0': 3909, '1.249.1399.0': 3910, '1.243.402.0': 3911, '1.257.127.0': 3912, '1.225.1391.0': 3913, '1.249.1405.0': 3914, '1.269.186.0': 3915, '1.269.659.0': 3916, '1.263.1888.0': 3917, '1.249.1180.0': 3918, '1.229.1453.0': 3919, '1.257.564.0': 3920, '1.265.206.0': 3921, '1.249.603.0': 3922, '1.265.691.0': 3923, '1.223.2098.0': 3924, '1.261.753.0': 3925, '1.227.1988.0': 3926, '1.265.408.0': 3927, '1.245.923.0': 3928, '1.257.1288.0': 3929, '1.253.618.0': 3930, '1.237.1324.0': 3931, '1.263.1687.0': 3932, '1.261.301.0': 3933, '1.271.978.0': 3934, '1.263.148.0': 3935, '1.245.1009.0': 3936, '1.251.435.0': 3937, '1.245.339.0': 3938, '1.265.793.0': 3939, '1.255.232.0': 3940, '1.245.772.0': 3941, '1.227.167.0': 3942, '1.261.827.0': 3943, '1.251.1014.0': 3944, '1.257.1433.0': 3945, '1.253.449.0': 3946, '1.251.947.0': 3947, '1.261.634.0': 3948, '1.245.365.0': 3949, '1.253.94.0': 3950, '1.267.916.0': 3951, '1.237.1169.0': 3952, '1.257.1327.0': 3953, '1.243.361.0': 3954, '1.263.1100.0': 3955, '1.249.878.0': 3956, '1.229.35.0': 3957, '1.239.1191.0': 3958, '1.263.1764.0': 3959, '1.245.96.0': 3960, '1.271.11.0': 3961, '1.261.417.0': 3962, '1.265.513.0': 3963, '1.229.1461.0': 3964, '1.261.449.0': 3965, '1.263.1159.0': 3966, '1.263.1111.0': 3967, '1.259.894.0': 3968, '1.249.336.0': 3969, '1.261.218.0': 3970, '1.261.433.0': 3971, '1.245.131.0': 3972, '1.263.442.0': 3973, '1.271.866.0': 3974, '1.251.211.0': 3975, '1.267.544.0': 3976, '1.263.716.0': 3977, '1.225.1295.0': 3978, '1.253.1043.0': 3979, '1.253.779.0': 3980, '1.243.600.0': 3981, '1.235.1858.0': 3982, '1.261.1496.0': 3983, '1.229.232.0': 3984, '1.257.1175.0': 3985, '1.227.2472.0': 3986, '1.251.70.0': 3987, '1.227.372.0': 3988, '1.231.917.0': 3989, '1.259.35.0': 3990, '1.261.629.0': 3991, '1.229.1916.0': 3992, '1.269.1459.0': 3993, '1.259.1146.0': 3994, '1.239.178.0': 3995, '1.251.711.0': 3996, '1.259.505.0': 3997, '1.267.235.0': 3998, '1.267.536.0': 3999, '1.267.820.0': 4000, '1.233.1830.0': 4001, '1.251.1219.0': 4002, '1.245.849.0': 4003, '1.263.804.0': 4004, '1.241.407.0': 4005, '1.267.854.0': 4006, '1.235.996.0': 4007, '1.233.3363.0': 4008, '1.237.1006.0': 4009, '1.263.1706.0': 4010, '1.223.2807.0': 4011, '1.253.929.0': 4012, '1.251.1070.0': 4013, '1.251.1484.0': 4014, '1.251.1011.0': 4015, '1.259.686.0': 4016, '1.229.1834.0': 4017, '1.259.844.0': 4018, '1.261.1527.0': 4019, '1.225.2876.0': 4020, '1.251.1386.0': 4021, '1.269.451.0': 4022, '1.259.975.0': 4023, '1.249.466.0': 4024, '1.265.292.0': 4025, '1.257.1460.0': 4026, '1.263.1641.0': 4027, '1.233.1393.0': 4028, '1.241.41.0': 4029, '1.229.1190.0': 4030, '1.259.757.0': 4031, '1.247.408.0': 4032, '1.253.879.0': 4033, '1.239.1678.0': 4034, '1.263.215.0': 4035, '1.261.1515.0': 4036, '1.257.1282.0': 4037, '1.239.1089.0': 4038, '1.263.426.0': 4039, '1.251.663.0': 4040, '1.259.87.0': 4041, '1.247.442.0': 4042, '1.261.34.0': 4043, '1.233.4218.0': 4044, '1.267.541.0': 4045, '1.249.953.0': 4046, '1.251.584.0': 4047, '1.239.1381.0': 4048, '1.263.1540.0': 4049, '1.251.236.0': 4050, '1.257.1142.0': 4051, '1.257.848.0': 4052, '1.263.819.0': 4053, '1.231.80.0': 4054, '1.225.1607.0': 4055, '1.243.856.0': 4056, '1.257.200.0': 4057, '1.249.898.0': 4058, '1.261.697.0': 4059, '1.231.1810.0': 4060, '1.225.62.0': 4061, '1.267.826.0': 4062, '1.245.162.0': 4063, '1.233.1637.0': 4064, '1.231.2020.0': 4065, '1.231.615.0': 4066, '1.245.256.0': 4067, '1.227.2817.0': 4068, '1.261.1552.0': 4069, '1.263.905.0': 4070, '1.231.255.0': 4071, '1.263.1635.0': 4072, '1.243.493.0': 4073, '1.231.129.0': 4074, '1.241.383.0': 4075, '1.259.136.0': 4076, '1.255.102.0': 4077, '1.249.999.0': 4078, '1.251.1022.0': 4079, '1.263.934.0': 4080, '1.235.754.0': 4081, '1.245.822.0': 4082, '1.253.170.0': 4083, '1.261.1632.0': 4084, '1.259.989.0': 4085, '1.263.1985.0': 4086, '1.259.381.0': 4087, '1.249.1023.0': 4088, '1.243.350.0': 4089, '1.261.1609.0': 4090, '1.255.48.0': 4091, '1.261.833.0': 4092, '1.243.638.0': 4093, '1.265.255.0': 4094, '1.249.228.0': 4095, '1.269.133.0': 4096, '1.265.869.0': 4097, '1.251.863.0': 4098, '1.237.742.0': 4099, '1.267.413.0': 4100, '1.261.599.0': 4101, '1.261.1592.0': 4102, '1.253.621.0': 4103, '1.267.126.0': 4104, '1.239.558.0': 4105, '1.263.437.0': 4106, '1.253.30.0': 4107, '1.251.112.0': 4108, '1.225.674.0': 4109, '1.241.286.0': 4110, '1.239.1201.0': 4111, '1.225.2207.0': 4112, '1.237.759.0': 4113, '1.249.1068.0': 4114, '1.263.1853.0': 4115, '1.229.937.0': 4116, '1.237.565.0': 4117, '1.259.111.0': 4118, '1.261.551.0': 4119, '1.251.122.0': 4120, '1.235.1288.0': 4121, '1.263.534.0': 4122, '1.247.96.0': 4123, '1.269.869.0': 4124, '1.263.543.0': 4125, '1.245.327.0': 4126, '1.225.3649.0': 4127, '1.243.706.0': 4128, '1.259.304.0': 4129, '1.269.856.0': 4130, '1.245.683.0': 4131, '1.263.792.0': 4132, '1.249.462.0': 4133, '1.259.1447.0': 4134, '1.235.1017.0': 4135, '1.253.969.0': 4136, '1.259.563.0': 4137, '1.241.853.0': 4138, '1.225.246.0': 4139, '1.247.663.0': 4140, '1.269.936.0': 4141, '1.229.1440.0': 4142, '1.263.1048.0': 4143, '1.261.1646.0': 4144, '1.261.1338.0': 4145, '1.267.498.0': 4146, '1.249.1199.0': 4147, '1.249.821.0': 4148, '1.251.459.0': 4149, '1.243.110.0': 4150, '1.261.310.0': 4151, '1.257.247.0': 4152, '1.255.95.0': 4153, '1.251.20.0': 4154, '0.0.0.0': 4155, '1.263.453.0': 4156, '1.259.475.0': 4157, '1.235.1063.0': 4158, '1.231.2078.0': 4159, '1.267.1244.0': 4160, '1.263.1014.0': 4161, '1.249.967.0': 4162, '1.245.2.0': 4163, '1.257.637.0': 4164, '1.235.3392.0': 4165, '1.233.3444.0': 4166, '1.241.1402.0': 4167, '1.253.341.0': 4168, '1.263.984.0': 4169, '1.269.751.0': 4170, '1.249.13.0': 4171, '1.247.882.0': 4172, '1.257.1079.0': 4173, '1.263.1974.0': 4174, '1.251.1006.0': 4175, '1.259.668.0': 4176, '1.247.505.0': 4177, '1.261.1369.0': 4178, '1.267.1445.0': 4179, '1.257.890.0': 4180, '1.239.1611.0': 4181, '1.237.726.0': 4182, '1.233.1663.0': 4183, '1.225.4044.0': 4184, '1.261.1660.0': 4185, '1.263.1007.0': 4186, '1.261.1626.0': 4187, '1.253.770.0': 4188, '1.259.630.0': 4189, '1.239.586.0': 4190, '1.261.1268.0': 4191, '1.265.49.0': 4192, '1.249.810.0': 4193, '1.231.1318.0': 4194, '1.263.502.0': 4195, '1.251.428.0': 4196, '1.261.217.0': 4197, '1.271.377.0': 4198, '1.241.1221.0': 4199, '1.231.1286.0': 4200, '1.225.2997.0': 4201, '1.241.1079.0': 4202, '1.261.683.0': 4203, '1.263.1638.0': 4204, '1.261.221.0': 4205, '1.237.1760.0': 4206, '1.247.407.0': 4207, '1.261.1636.0': 4208, '1.237.1827.0': 4209, '1.233.802.0': 4210, '1.261.424.0': 4211, '1.257.851.0': 4212, '1.261.1321.0': 4213, '1.257.877.0': 4214, '1.235.174.0': 4215, '1.261.344.0': 4216, '1.225.3281.0': 4217, '1.257.998.0': 4218, '1.261.328.0': 4219, '1.241.251.0': 4220, '1.241.1252.0': 4221, '1.257.233.0': 4222, '1.259.1588.0': 4223, '1.231.502.0': 4224, '1.257.1328.0': 4225, '1.251.1031.0': 4226, '1.247.448.0': 4227, '1.243.454.0': 4228, '1.251.836.0': 4229, '1.241.1307.0': 4230, '1.237.1801.0': 4231, '1.223.2178.0': 4232, '1.253.673.0': 4233, '1.229.1548.0': 4234, '1.235.2855.0': 4235, '1.263.425.0': 4236, '1.227.2300.0': 4237, '1.259.206.0': 4238, '1.233.428.0': 4239, '1.247.501.0': 4240, '1.259.690.0': 4241, '1.259.137.0': 4242, '1.257.1344.0': 4243, '1.251.762.0': 4244, '1.237.58.0': 4245, '1.243.972.0': 4246, '1.261.1312.0': 4247, '1.227.310.0': 4248, '1.237.185.0': 4249, '1.259.951.0': 4250, '1.241.413.0': 4251, '1.229.740.0': 4252, '1.259.452.0': 4253, '1.241.1216.0': 4254, '1.251.1053.0': 4255, '1.269.444.0': 4256, '1.227.1929.0': 4257, '1.247.359.0': 4258, '1.251.1018.0': 4259, '1.261.1383.0': 4260, '1.251.423.0': 4261, '1.263.1684.0': 4262, '1.249.1002.0': 4263, '1.261.1642.0': 4264, '1.233.3222.0': 4265, '1.263.782.0': 4266, '1.231.1771.0': 4267, '1.251.1028.0': 4268, '1.247.389.0': 4269, '1.235.2115.0': 4270, '1.247.928.0': 4271, '1.249.993.0': 4272, '1.247.917.0': 4273, '1.251.351.0': 4274, '1.235.2163.0': 4275, '1.257.806.0': 4276, '1.235.3101.0': 4277, '1.263.1829.0': 4278, '1.263.218.0': 4279, '1.263.152.0': 4280, '1.261.1380.0': 4281, '1.259.113.0': 4282, '1.257.459.0': 4283, '1.255.53.0': 4284, '1.251.609.0': 4285, '1.251.377.0': 4286, '1.245.431.0': 4287, '1.255.312.0': 4288, '1.257.1284.0': 4289, '1.259.984.0': 4290, '1.249.278.0': 4291, '1.245.71.0': 4292, '1.257.1070.0': 4293, '1.253.444.0': 4294, '1.245.902.0': 4295, '1.261.1621.0': 4296, '1.259.224.0': 4297, '1.263.117.0': 4298, '1.259.626.0': 4299, '1.251.754.0': 4300, '1.243.237.0': 4301, '1.247.640.0': 4302, '1.227.2544.0': 4303, '1.225.2713.0': 4304, '1.251.32.0': 4305, '1.269.1056.0': 4306, '1.249.418.0': 4307, '1.269.24.0': 4308, '1.257.1090.0': 4309, '1.235.638.0': 4310, '1.251.1065.0': 4311, '1.263.1970.0': 4312, '1.245.768.0': 4313, '1.261.1366.0': 4314, '1.231.948.0': 4315, '1.255.60.0': 4316, '1.249.980.0': 4317, '1.235.3222.0': 4318, '1.267.935.0': 4319, '1.253.721.0': 4320, '1.267.423.0': 4321, '1.263.652.0': 4322, '1.247.890.0': 4323, '1.263.1105.0': 4324, '1.233.2872.0': 4325, '1.235.80.0': 4326, '1.237.47.0': 4327, '1.251.574.0': 4328, '1.261.700.0': 4329, '1.241.665.0': 4330, '1.239.230.0': 4331, '1.249.1352.0': 4332, '1.249.1194.0': 4333, '1.229.287.0': 4334, '1.235.1535.0': 4335, '1.263.173.0': 4336, '1.245.865.0': 4337, '1.249.590.0': 4338, '1.225.2048.0': 4339, '1.261.756.0': 4340, '1.251.679.0': 4341, '1.257.599.0': 4342, '1.247.604.0': 4343, '1.247.440.0': 4344, '1.257.992.0': 4345, '1.259.1409.0': 4346, '1.263.87.0': 4347, '1.267.1305.0': 4348, '1.225.2043.0': 4349, '1.259.857.0': 4350, '1.251.1029.0': 4351, '1.263.52.0': 4352, '1.261.1641.0': 4353, '1.259.879.0': 4354, '1.243.414.0': 4355, '1.241.965.0': 4356, '1.237.914.0': 4357, '1.247.401.0': 4358, '1.237.484.0': 4359, '1.247.399.0': 4360, '1.247.633.0': 4361, '1.243.1034.0': 4362, '1.249.131.0': 4363, '1.249.126.0': 4364, '1.225.426.0': 4365, '1.239.517.0': 4366, '1.251.1303.0': 4367, '1.229.46.0': 4368, '1.251.536.0': 4369, '1.251.708.0': 4370, '1.251.252.0': 4371, '1.227.2754.0': 4372, '1.243.517.0': 4373, '1.263.229.0': 4374, '1.229.471.0': 4375, '1.237.191.0': 4376, '1.225.4025.0': 4377, '1.241.1291.0': 4378, '1.251.1142.0': 4379, '1.261.381.0': 4380, '1.241.709.0': 4381, '1.263.922.0': 4382, '1.247.545.0': 4383, '1.253.775.0': 4384, '1.251.1261.0': 4385, '1.235.2003.0': 4386, '1.251.256.0': 4387, '1.261.321.0': 4388, '1.263.913.0': 4389, '1.261.738.0': 4390, '1.259.674.0': 4391, '1.267.742.0': 4392, '1.231.566.0': 4393, '1.233.3218.0': 4394, '1.259.147.0': 4395, '1.239.849.0': 4396, '1.267.994.0': 4397, '1.257.1036.0': 4398, '1.229.441.0': 4399, '1.263.1801.0': 4400, '1.247.451.0': 4401, '1.233.3735.0': 4402, '1.265.282.0': 4403, '1.259.923.0': 4404, '1.235.2224.0': 4405, '1.243.39.0': 4406, '1.233.1752.0': 4407, '1.233.1576.0': 4408, '1.241.521.0': 4409, '1.227.2343.0': 4410, '1.249.706.0': 4411, '1.233.2411.0': 4412, '1.229.1177.0': 4413, '1.243.862.0': 4414, '1.249.1388.0': 4415, '1.235.2293.0': 4416, '1.265.351.0': 4417, '1.261.1074.0': 4418, '1.249.1333.0': 4419, '1.233.884.0': 4420, '1.225.2072.0': 4421, '1.235.2105.0': 4422, '1.247.119.0': 4423, '1.237.851.0': 4424, '1.265.675.0': 4425, '1.235.2109.0': 4426, '1.261.437.0': 4427, '1.251.530.0': 4428, '1.261.1500.0': 4429, '1.241.1146.0': 4430, '1.241.1323.0': 4431, '1.259.1270.0': 4432, '1.261.374.0': 4433, '1.259.477.0': 4434, '1.245.200.0': 4435, '1.261.791.0': 4436, '1.245.432.0': 4437, '1.245.530.0': 4438, '1.225.2971.0': 4439, '1.245.221.0': 4440, '1.237.640.0': 4441, '1.249.745.0': 4442, '1.225.3604.0': 4443, '1.233.3590.0': 4444, '1.225.2841.0': 4445, '1.229.781.0': 4446, '1.239.892.0': 4447, '1.259.994.0': 4448, '1.259.621.0': 4449, '1.239.867.0': 4450, '1.247.666.0': 4451, '1.223.2847.0': 4452, '1.227.2934.0': 4453, '1.267.540.0': 4454, '1.249.174.0': 4455, '1.237.1347.0': 4456, '1.239.938.0': 4457, '1.249.872.0': 4458, '1.237.1149.0': 4459, '1.223.1374.0': 4460, '1.259.201.0': 4461, '1.233.3898.0': 4462, '1.269.920.0': 4463, '1.257.225.0': 4464, '1.259.1217.0': 4465, '1.235.1349.0': 4466, '1.259.1141.0': 4467, '1.253.1045.0': 4468, '1.253.660.0': 4469, '1.249.334.0': 4470, '1.237.994.0': 4471, '1.263.385.0': 4472, '1.261.1131.0': 4473, '1.229.2009.0': 4474, '1.251.666.0': 4475, '1.259.1267.0': 4476, '1.259.415.0': 4477, '1.263.421.0': 4478, '1.247.665.0': 4479, '1.245.203.0': 4480, '1.267.545.0': 4481, '1.233.3461.0': 4482, '1.251.613.0': 4483, '1.259.956.0': 4484, '1.253.396.0': 4485, '1.239.1325.0': 4486, '1.245.931.0': 4487, '1.249.958.0': 4488, '1.257.856.0': 4489, '1.269.387.0': 4490, '1.259.785.0': 4491, '1.245.390.0': 4492, '1.261.758.0': 4493, '1.235.1750.0': 4494, '1.257.1416.0': 4495, '1.263.864.0': 4496, '1.259.708.0': 4497, '1.259.1540.0': 4498, '1.231.247.0': 4499, '1.251.168.0': 4500, '1.231.1103.0': 4501, '1.225.3510.0': 4502, '1.233.1912.0': 4503, '1.263.42.0': 4504, '1.263.492.0': 4505, '1.267.1372.0': 4506, '1.259.228.0': 4507, '1.233.1505.0': 4508, '1.229.596.0': 4509, '1.249.726.0': 4510, '1.245.250.0': 4511, '1.265.102.0': 4512, '1.247.948.0': 4513, '1.237.540.0': 4514, '1.233.649.0': 4515, '1.247.770.0': 4516, '1.229.1769.0': 4517, '1.225.4092.0': 4518, '1.251.491.0': 4519, '1.275.1078.0': 4520, '1.233.3706.0': 4521, '1.259.1451.0': 4522, '1.231.953.0': 4523, '1.241.412.0': 4524, '1.241.989.0': 4525, '1.259.1001.0': 4526, '1.257.759.0': 4527, '1.267.815.0': 4528, '1.229.1251.0': 4529, '1.253.461.0': 4530, '1.261.508.0': 4531, '1.263.130.0': 4532, '1.241.1127.0': 4533, '1.247.384.0': 4534, '1.261.274.0': 4535, '1.265.437.0': 4536, '1.231.123.0': 4537, '1.225.598.0': 4538, '1.227.2617.0': 4539, '1.235.369.0': 4540, '1.257.549.0': 4541, '1.267.191.0': 4542, '1.239.999.0': 4543, '1.227.2777.0': 4544, '1.243.653.0': 4545, '1.251.135.0': 4546, '1.233.1316.0': 4547, '1.259.1265.0': 4548, '1.235.695.0': 4549, '1.261.521.0': 4550, '1.253.928.0': 4551, '1.257.1332.0': 4552, '1.233.177.0': 4553, '1.227.674.0': 4554, '1.233.4265.0': 4555, '1.229.681.0': 4556, '1.259.717.0': 4557, '1.257.1389.0': 4558, '1.253.1005.0': 4559, '1.249.735.0': 4560, '1.261.749.0': 4561, '1.253.552.0': 4562, '1.255.280.0': 4563, '1.247.811.0': 4564, '1.261.1047.0': 4565, '1.245.1012.0': 4566, '1.229.850.0': 4567, '1.257.401.0': 4568, '1.257.1001.0': 4569, '1.227.2352.0': 4570, '1.241.229.0': 4571, '1.239.202.0': 4572, '1.247.793.0': 4573, '1.247.935.0': 4574, '1.249.1321.0': 4575, '1.231.2187.0': 4576, '1.261.287.0': 4577, '1.267.610.0': 4578, '1.259.601.0': 4579, '1.235.1035.0': 4580, '1.239.1388.0': 4581, '1.261.436.0': 4582, '1.239.239.0': 4583, '1.261.648.0': 4584, '1.229.382.0': 4585, '1.225.3028.0': 4586, '1.251.670.0': 4587, '1.243.948.0': 4588, '1.247.76.0': 4589, '1.257.1340.0': 4590, '1.237.947.0': 4591, '1.253.1089.0': 4592, '1.259.354.0': 4593, '1.247.213.0': 4594, '1.253.135.0': 4595, '1.247.351.0': 4596, '1.251.692.0': 4597, '1.263.419.0': 4598, '1.225.584.0': 4599, '1.259.123.0': 4600, '1.247.418.0': 4601, '1.253.103.0': 4602, '1.259.1595.0': 4603, '1.237.866.0': 4604, '1.233.543.0': 4605, '1.229.1870.0': 4606, '1.259.60.0': 4607, '1.231.1116.0': 4608, '1.253.785.0': 4609, '1.225.4102.0': 4610, '1.251.95.0': 4611, '1.259.1414.0': 4612, '1.229.545.0': 4613, '1.247.657.0': 4614, '1.269.506.0': 4615, '1.231.449.0': 4616, '1.249.729.0': 4617, '1.227.1516.0': 4618, '1.263.443.0': 4619, '1.263.91.0': 4620, '1.225.4157.0': 4621, '1.259.1440.0': 4622, '1.263.431.0': 4623, '1.257.1094.0': 4624, '1.259.1311.0': 4625, '1.249.595.0': 4626, '1.239.1079.0': 4627, '1.239.701.0': 4628, '1.225.2254.0': 4629, '1.225.3819.0': 4630, '1.259.557.0': 4631, '1.233.2001.0': 4632, '1.251.1009.0': 4633, '1.233.136.0': 4634, '1.265.673.0': 4635, '1.225.2875.0': 4636, '1.231.864.0': 4637, '1.245.546.0': 4638, '1.249.949.0': 4639, '1.241.377.0': 4640, '1.253.455.0': 4641, '1.229.615.0': 4642, '1.243.475.0': 4643, '1.243.695.0': 4644, '1.233.4161.0': 4645, '1.237.969.0': 4646, '1.239.1134.0': 4647, '1.247.396.0': 4648, '1.233.2828.0': 4649, '1.223.1766.0': 4650, '1.247.639.0': 4651, '1.249.654.0': 4652, '1.267.435.0': 4653, '1.225.1983.0': 4654, '1.261.502.0': 4655, '1.261.29.0': 4656, '1.257.1281.0': 4657, '1.247.379.0': 4658, '1.227.1796.0': 4659, '1.223.2897.0': 4660, '1.231.44.0': 4661, '1.245.517.0': 4662, '1.235.345.0': 4663, '1.239.1321.0': 4664, '1.261.364.0': 4665, '1.241.612.0': 4666, '1.259.203.0': 4667, '1.263.122.0': 4668, '1.237.819.0': 4669, '1.263.1631.0': 4670, '1.231.2087.0': 4671, '1.259.580.0': 4672, '1.259.446.0': 4673, '1.225.2152.0': 4674, '1.265.733.0': 4675, '1.231.842.0': 4676, '1.233.3667.0': 4677, '1.257.99.0': 4678, '1.225.2276.0': 4679, '1.233.2404.0': 4680, '1.239.923.0': 4681, '1.257.1278.0': 4682, '1.261.1079.0': 4683, '1.251.876.0': 4684, '1.235.866.0': 4685, '1.259.696.0': 4686, '1.225.825.0': 4687, '1.261.1509.0': 4688, '1.247.160.0': 4689, '1.253.693.0': 4690, '1.257.1374.0': 4691, '1.245.401.0': 4692, '1.245.259.0': 4693, '1.267.704.0': 4694, '1.245.17.0': 4695, '1.261.313.0': 4696, '1.259.1259.0': 4697, '1.237.1214.0': 4698, '1.237.464.0': 4699, '1.263.494.0': 4700, '1.251.430.0': 4701, '1.227.1584.0': 4702, '1.259.1605.0': 4703, '1.249.987.0': 4704, '1.239.1374.0': 4705, '1.259.851.0': 4706, '1.263.958.0': 4707, '1.241.1209.0': 4708, '1.233.3357.0': 4709, '1.251.499.0': 4710, '1.247.641.0': 4711, '1.249.807.0': 4712, '1.259.667.0': 4713, '1.233.627.0': 4714, '1.225.1999.0': 4715, '1.243.133.0': 4716, '1.251.688.0': 4717, '1.257.38.0': 4718, '1.225.1574.0': 4719, '1.265.295.0': 4720, '1.227.2561.0': 4721, '1.237.1585.0': 4722, '1.261.1056.0': 4723, '1.231.818.0': 4724, '1.261.781.0': 4725, '1.231.1660.0': 4726, '1.227.1465.0': 4727, '1.243.345.0': 4728, '1.261.25.0': 4729, '1.249.732.0': 4730, '1.235.545.0': 4731, '1.247.592.0': 4732, '1.263.131.0': 4733, '1.233.1746.0': 4734, '1.233.4114.0': 4735, '1.251.643.0': 4736, '1.251.1371.0': 4737, '1.263.1062.0': 4738, '1.227.2939.0': 4739, '1.225.1889.0': 4740, '1.247.798.0': 4741, '1.263.487.0': 4742, '1.243.846.0': 4743, '1.229.1161.0': 4744, '1.257.903.0': 4745, '1.229.626.0': 4746, '1.265.717.0': 4747, '1.261.807.0': 4748, '1.249.91.0': 4749, '1.231.226.0': 4750, '1.229.1562.0': 4751, '1.243.610.0': 4752, '1.261.1653.0': 4753, '1.237.1102.0': 4754, '1.263.299.0': 4755, '1.261.1647.0': 4756, '1.265.430.0': 4757, '1.227.1115.0': 4758, '1.249.909.0': 4759, '1.225.1442.0': 4760, '1.261.499.0': 4761, '1.251.1375.0': 4762, '1.233.3051.0': 4763, '1.231.631.0': 4764, '1.263.1126.0': 4765, '1.231.740.0': 4766, '1.261.757.0': 4767, '1.259.443.0': 4768, '1.225.3129.0': 4769, '1.233.2270.0': 4770, '1.257.733.0': 4771, '1.247.999.0': 4772, '1.227.2720.0': 4773, '1.253.906.0': 4774, '1.225.1943.0': 4775, '1.237.1438.0': 4776, '1.261.746.0': 4777, '1.257.78.0': 4778, '1.251.219.0': 4779, '1.225.1401.0': 4780, '1.223.1538.0': 4781, '1.259.254.0': 4782, '1.259.565.0': 4783, '1.253.579.0': 4784, '1.263.136.0': 4785, '1.251.464.0': 4786, '1.241.211.0': 4787, '1.265.240.0': 4788, '1.233.1951.0': 4789, '1.257.1060.0': 4790, '1.229.355.0': 4791, '1.265.315.0': 4792, '1.251.240.0': 4793, '1.239.1401.0': 4794, '1.239.930.0': 4795, '1.261.844.0': 4796, '1.249.477.0': 4797, '1.257.1404.0': 4798, '1.249.1074.0': 4799, '1.247.647.0': 4800, '1.263.135.0': 4801, '1.245.40.0': 4802, '1.247.421.0': 4803, '1.233.3162.0': 4804, '1.261.1037.0': 4805, '1.263.1903.0': 4806, '1.223.1640.0': 4807, '1.237.981.0': 4808, '1.233.1357.0': 4809, '1.237.1852.0': 4810, '1.243.477.0': 4811, '1.263.390.0': 4812, '1.259.1314.0': 4813, '1.241.628.0': 4814, '1.247.667.0': 4815, '1.257.442.0': 4816, '1.225.2482.0': 4817, '1.235.1075.0': 4818, '1.225.1511.0': 4819, '1.233.1934.0': 4820, '1.247.735.0': 4821, '1.235.597.0': 4822, '1.263.183.0': 4823, '1.253.1029.0': 4824, '1.261.395.0': 4825, '1.263.409.0': 4826, '1.265.99.0': 4827, '1.267.1226.0': 4828, '1.227.1980.0': 4829, '1.261.353.0': 4830, '1.239.285.0': 4831, '1.253.916.0': 4832, '1.245.183.0': 4833, '1.239.1544.0': 4834, '1.243.804.0': 4835, '1.233.3178.0': 4836, '1.267.1051.0': 4837, '1.229.1225.0': 4838, '1.259.1454.0': 4839, '1.253.815.0': 4840, '1.247.439.0': 4841, '1.231.1456.0': 4842, '1.257.787.0': 4843, '1.255.147.0': 4844, '1.235.3215.0': 4845, '1.267.204.0': 4846, '1.225.1529.0': 4847, '1.261.1374.0': 4848, '1.267.303.0': 4849, '1.249.20.0': 4850, '1.243.943.0': 4851, '1.227.823.0': 4852, '1.245.305.0': 4853, '1.257.822.0': 4854, '1.249.136.0': 4855, '1.257.906.0': 4856, '1.263.78.0': 4857, '1.259.662.0': 4858, '1.225.4034.0': 4859, '1.225.905.0': 4860, '1.245.1010.0': 4861, '1.259.1273.0': 4862, '1.263.1833.0': 4863, '1.249.1189.0': 4864, '1.233.948.0': 4865, '1.261.1610.0': 4866, '1.235.557.0': 4867, '1.239.76.0': 4868, '1.261.1695.0': 4869, '1.239.1394.0': 4870, '1.237.1681.0': 4871, '1.255.29.0': 4872, '1.259.358.0': 4873, '1.231.1821.0': 4874, '1.245.908.0': 4875, '1.249.257.0': 4876, '1.263.526.0': 4877, '1.247.989.0': 4878, '1.257.979.0': 4879, '1.233.2434.0': 4880, '1.243.459.0': 4881, '1.259.445.0': 4882, '1.241.434.0': 4883, '1.233.3169.0': 4884, '1.253.669.0': 4885, '1.265.190.0': 4886, '1.201.1538.0': 4887, '1.259.1431.0': 4888, '1.231.280.0': 4889, '1.229.1596.0': 4890, '1.233.501.0': 4891, '1.265.366.0': 4892, '1.261.388.0': 4893, '1.255.324.0': 4894, '1.239.0.0': 4895, '1.259.1569.0': 4896, '1.257.705.0': 4897, '1.231.1128.0': 4898, '1.257.533.0': 4899, '1.263.1139.0': 4900, '1.245.1082.0': 4901, '1.225.1020.0': 4902, '1.249.941.0': 4903, '1.261.651.0': 4904, '1.257.700.0': 4905, '1.253.772.0': 4906, '1.261.1596.0': 4907, '1.233.495.0': 4908, '1.229.694.0': 4909, '1.249.842.0': 4910, '1.237.1789.0': 4911, '1.251.1308.0': 4912, '1.247.566.0': 4913, '1.233.4211.0': 4914, '1.243.1007.0': 4915, '1.263.235.0': 4916, '1.251.746.0': 4917, '1.233.2717.0': 4918, '1.233.294.0': 4919, '1.229.759.0': 4920, '1.245.447.0': 4921, '1.257.763.0': 4922, '1.239.1252.0': 4923, '1.233.2789.0': 4924, '1.263.467.0': 4925, '1.245.949.0': 4926, '1.241.857.0': 4927, '1.235.1940.0': 4928, '1.257.399.0': 4929, '1.231.1908.0': 4930, '1.259.1626.0': 4931, '1.257.1412.0': 4932, '1.253.374.0': 4933, '1.225.1465.0': 4934, '1.263.313.0': 4935, '1.263.632.0': 4936, '1.243.670.0': 4937, '1.261.1350.0': 4938, '1.267.551.0': 4939, '1.261.1478.0': 4940, '1.227.38.0': 4941, '1.229.1582.0': 4942, '1.267.636.0': 4943, '1.229.603.0': 4944, '1.231.1206.0': 4945, '1.247.29.0': 4946, '1.241.0.0': 4947, '1.231.1513.0': 4948, '1.249.47.0': 4949, '1.249.316.0': 4950, '1.265.285.0': 4951, '1.229.1493.0': 4952, '1.257.84.0': 4953, '1.269.734.0': 4954, '1.259.442.0': 4955, '1.233.657.0': 4956, '1.249.443.0': 4957, '1.231.901.0': 4958, '1.247.860.0': 4959, '1.227.8.0': 4960, '1.259.1067.0': 4961, '1.235.163.0': 4962, '1.231.407.0': 4963, '1.253.24.0': 4964, '1.169.55.0': 4965, '1.261.1481.0': 4966, '1.253.225.0': 4967, '1.245.124.0': 4968, '1.233.938.0': 4969, '1.273.1804.0': 4970, '1.235.1432.0': 4971, '1.241.460.0': 4972, '1.225.2288.0': 4973, '1.241.775.0': 4974, '1.261.1714.0': 4975, '1.251.628.0': 4976, '1.251.336.0': 4977, '1.257.1307.0': 4978, '1.245.198.0': 4979, '1.267.692.0': 4980, '1.261.1597.0': 4981, '1.245.1007.0': 4982, '1.259.752.0': 4983, '1.263.319.0': 4984, '1.229.1293.0': 4985, '1.229.1733.0': 4986, '1.259.1578.0': 4987, '1.251.279.0': 4988, '1.261.306.0': 4989, '1.235.470.0': 4990, '1.231.1806.0': 4991, '1.237.449.0': 4992, '1.249.1060.0': 4993, '1.247.1038.0': 4994, '1.265.374.0': 4995, '1.227.29.0': 4996, '1.263.1892.0': 4997, '1.231.814.0': 4998, '1.257.1190.0': 4999, '1.225.1328.0': 5000, '1.249.1285.0': 5001, '1.231.1740.0': 5002, '1.261.18.0': 5003, '1.237.1188.0': 5004, '1.263.456.0': 5005, '1.245.230.0': 5006, '1.259.235.0': 5007, '1.253.451.0': 5008, '1.233.489.0': 5009, '1.243.522.0': 5010, '1.259.676.0': 5011, '1.225.4056.0': 5012, '1.249.681.0': 5013, '1.227.2599.0': 5014, '1.239.874.0': 5015, '1.249.557.0': 5016, '1.231.1715.0': 5017, '1.225.2354.0': 5018, '1.245.1003.0': 5019, '1.245.1167.0': 5020, '1.261.674.0': 5021, '1.235.331.0': 5022, '1.247.380.0': 5023, '1.259.121.0': 5024, '1.231.537.0': 5025, '1.259.675.0': 5026, '1.263.1110.0': 5027, '1.235.1542.0': 5028, '1.233.841.0': 5029, '1.261.1404.0': 5030, '1.235.1948.0': 5031, '1.249.1052.0': 5032, '1.225.469.0': 5033, '1.245.968.0': 5034, '1.245.670.0': 5035, '1.241.995.0': 5036, '1.223.2675.0': 5037, '1.251.852.0': 5038, '1.241.654.0': 5039, '1.225.1745.0': 5040, '1.223.1723.0': 5041, '1.247.434.0': 5042, '1.247.619.0': 5043, '1.263.261.0': 5044, '1.263.129.0': 5045, '1.225.3181.0': 5046, '1.249.1265.0': 5047, '1.249.1313.0': 5048, '1.233.582.0': 5049, '1.233.2733.0': 5050, '1.249.1266.0': 5051, '1.249.894.0': 5052, '1.261.560.0': 5053, '1.241.483.0': 5054, '1.263.577.0': 5055, '1.239.1672.0': 5056, '1.245.323.0': 5057, '1.223.2229.0': 5058, '1.239.1559.0': 5059, '1.227.2583.0': 5060, '1.231.746.0': 5061, '1.253.712.0': 5062, '1.259.1085.0': 5063, '1.247.322.0': 5064, '1.257.379.0': 5065, '1.265.654.0': 5066, '1.245.924.0': 5067, '1.251.518.0': 5068, '1.231.1002.0': 5069, '1.249.480.0': 5070, '1.241.234.0': 5071, '1.261.1378.0': 5072, '1.253.1024.0': 5073, '1.237.479.0': 5074, '1.231.1074.0': 5075, '1.253.80.0': 5076, '1.259.192.0': 5077, '1.243.996.0': 5078, '1.249.1057.0': 5079, '1.241.394.0': 5080, '1.263.71.0': 5081, '1.257.1293.0': 5082, '1.261.1602.0': 5083, '1.247.733.0': 5084, '1.243.219.0': 5085, '1.229.1361.0': 5086, '1.239.592.0': 5087, '1.259.172.0': 5088, '1.235.2080.0': 5089, '1.237.890.0': 5090, '1.251.424.0': 5091, '1.249.17.0': 5092, '1.235.3322.0': 5093, '1.239.823.0': 5094, '1.253.132.0': 5095, '1.225.1250.0': 5096, '1.251.1289.0': 5097, '1.263.504.0': 5098, '1.255.221.0': 5099, '1.233.2933.0': 5100, '1.261.1518.0': 5101, '1.247.388.0': 5102, '1.231.117.0': 5103, '1.261.645.0': 5104, '1.247.203.0': 5105, '1.233.3396.0': 5106, '1.247.74.0': 5107, '1.237.260.0': 5108, '1.265.877.0': 5109, '1.261.693.0': 5110, '1.235.3406.0': 5111, '1.259.1579.0': 5112, '1.235.3027.0': 5113, '1.257.1007.0': 5114, '1.241.477.0': 5115, '1.225.1345.0': 5116, '1.247.494.0': 5117, '1.257.1022.0': 5118, '1.225.2306.0': 5119, '1.259.748.0': 5120, '1.245.443.0': 5121, '1.251.1024.0': 5122, '1.231.1262.0': 5123, '1.241.839.0': 5124, '1.257.1187.0': 5125, '1.239.912.0': 5126, '1.245.842.0': 5127, '1.261.341.0': 5128, '1.253.356.0': 5129, '1.241.1373.0': 5130, '1.227.2289.0': 5131, '1.229.1783.0': 5132, '1.237.1819.0': 5133, '1.225.1925.0': 5134, '1.249.1181.0': 5135, '1.227.2259.0': 5136, '1.229.876.0': 5137, '1.225.2382.0': 5138, '1.257.1360.0': 5139, '1.233.3743.0': 5140, '1.233.1380.0': 5141, '1.249.887.0': 5142, '1.253.324.0': 5143, '1.225.1467.0': 5144, '1.245.138.0': 5145, '1.235.2283.0': 5146, '1.231.1020.0': 5147, '1.243.92.0': 5148, '1.257.363.0': 5149, '1.253.1013.0': 5150, '1.231.1236.0': 5151, '1.267.678.0': 5152, '1.253.522.0': 5153, '1.261.665.0': 5154, '1.263.401.0': 5155, '1.253.544.0': 5156, '1.253.458.0': 5157, '1.235.308.0': 5158, '1.233.3351.0': 5159, '1.259.1591.0': 5160, '1.229.63.0': 5161, '1.235.1715.0': 5162, '1.245.88.0': 5163, '1.253.894.0': 5164, '1.229.465.0': 5165, '1.247.909.0': 5166, '1.237.42.0': 5167, '1.253.139.0': 5168, '1.233.3747.0': 5169, '1.247.990.0': 5170, '1.263.498.0': 5171, '1.227.489.0': 5172, '1.237.1415.0': 5173, '1.249.691.0': 5174, '1.233.303.0': 5175, '1.251.549.0': 5176, '1.247.765.0': 5177, '1.229.1042.0': 5178, '1.225.512.0': 5179, '1.231.621.0': 5180, '1.251.374.0': 5181, '1.259.414.0': 5182, '1.259.1701.0': 5183, '1.225.1845.0': 5184, '1.231.1248.0': 5185, '1.233.3209.0': 5186, '1.259.663.0': 5187, '1.257.471.0': 5188, '1.249.93.0': 5189, '1.225.3391.0': 5190, '1.263.638.0': 5191, '1.251.1461.0': 5192, '1.249.357.0': 5193, '1.237.1553.0': 5194, '1.251.1044.0': 5195, '1.245.211.0': 5196, '1.249.483.0': 5197, '1.229.385.0': 5198, '1.245.1091.0': 5199, '1.247.620.0': 5200, '1.233.603.0': 5201, '1.259.741.0': 5202, '1.227.82.0': 5203, '1.229.1258.0': 5204, '1.263.303.0': 5205, '1.249.989.0': 5206, '1.251.1536.0': 5207, '1.259.965.0': 5208, '1.241.660.0': 5209, '1.257.1446.0': 5210, '1.225.4151.0': 5211, '1.261.1167.0': 5212, '1.227.2140.0': 5213, '1.259.1736.0': 5214, '1.231.870.0': 5215, '1.249.1270.0': 5216, '1.261.1126.0': 5217, '1.229.1407.0': 5218, '1.259.1700.0': 5219, '1.247.959.0': 5220, '1.231.1999.0': 5221, '1.251.703.0': 5222, '1.261.239.0': 5223, '1.223.2221.0': 5224, '1.263.439.0': 5225, '1.239.1500.0': 5226, '1.261.229.0': 5227, '1.253.155.0': 5228, '1.251.566.0': 5229, '1.239.64.0': 5230, '1.257.996.0': 5231, '1.229.2054.0': 5232, '1.253.600.0': 5233, '1.265.289.0': 5234, '1.247.38.0': 5235, '1.233.768.0': 5236, '1.259.882.0': 5237, '1.245.225.0': 5238, '1.259.1006.0': 5239, '1.253.793.0': 5240, '1.261.542.0': 5241, '1.253.379.0': 5242, '1.239.1442.0': 5243, '1.245.890.0': 5244, '1.263.1037.0': 5245, '1.251.455.0': 5246, '1.251.113.0': 5247, '1.257.390.0': 5248, '1.247.473.0': 5249, '1.267.242.0': 5250, '1.261.548.0': 5251, '1.235.809.0': 5252, '1.245.1011.0': 5253, '1.227.1764.0': 5254, '1.235.2246.0': 5255, '1.251.706.0': 5256, '1.241.200.0': 5257, '1.229.417.0': 5258, '1.231.942.0': 5259, '1.259.1351.0': 5260, '1.247.624.0': 5261, '1.253.128.0': 5262, '1.263.118.0': 5263, '1.233.3779.0': 5264, '1.249.112.0': 5265, '1.247.404.0': 5266, '1.245.1169.0': 5267, '1.229.856.0': 5268, '1.235.707.0': 5269, '1.267.700.0': 5270, '1.231.227.0': 5271, '1.227.2335.0': 5272, '1.249.649.0': 5273, '1.223.1370.0': 5274, '1.249.855.0': 5275, '1.241.1158.0': 5276, '1.231.39.0': 5277, '1.261.333.0': 5278, '1.251.906.0': 5279, '1.257.1451.0': 5280, '1.235.566.0': 5281, '1.249.938.0': 5282, '1.237.434.0': 5283, '1.237.208.0': 5284, '1.261.832.0': 5285, '1.217.1286.0': 5286, '1.231.230.0': 5287, '1.253.767.0': 5288, '1.245.193.0': 5289, '1.237.227.0': 5290, '1.255.209.0': 5291, '1.233.870.0': 5292, '1.245.122.0': 5293, '1.239.1035.0': 5294, '1.261.427.0': 5295, '1.259.534.0': 5296, '1.249.163.0': 5297, '1.259.185.0': 5298, '1.259.666.0': 5299, '1.241.624.0': 5300, '1.227.1711.0': 5301, '1.249.139.0': 5302, '1.241.466.0': 5303, '1.235.256.0': 5304, '1.235.2272.0': 5305, '1.259.595.0': 5306, '1.231.1552.0': 5307, '1.251.719.0': 5308, '1.253.794.0': 5309, '1.259.1714.0': 5310, '1.231.1158.0': 5311, '1.227.2124.0': 5312, '1.227.2570.0': 5313, '1.229.794.0': 5314, '1.233.1984.0': 5315, '1.251.674.0': 5316, '1.263.1800.0': 5317, '1.235.3328.0': 5318, '1.249.584.0': 5319, '1.233.1323.0': 5320, '1.261.434.0': 5321, '1.229.362.0': 5322, '1.239.371.0': 5323, '1.233.3774.0': 5324, '1.241.72.0': 5325, '1.245.959.0': 5326, '1.243.729.0': 5327, '1.239.1469.0': 5328, '1.239.1384.0': 5329, '1.267.620.0': 5330, '1.233.1679.0': 5331, '1.263.75.0': 5332, '1.259.1266.0': 5333, '1.233.3328.0': 5334, '1.261.689.0': 5335, '1.243.173.0': 5336, '1.253.149.0': 5337, '1.233.2865.0': 5338, '1.259.104.0': 5339, '1.225.2568.0': 5340, '1.237.30.0': 5341, '1.259.654.0': 5342, '1.233.535.0': 5343, '1.255.88.0': 5344, '1.263.92.0': 5345, '1.245.18.0': 5346, '1.251.538.0': 5347, '1.253.788.0': 5348, '1.257.1065.0': 5349, '1.233.2628.0': 5350, '1.225.793.0': 5351, '1.259.931.0': 5352, '1.231.550.0': 5353, '1.229.1535.0': 5354, '1.247.90.0': 5355, '1.237.1334.0': 5356, '1.245.871.0': 5357, '1.219.1716.0': 5358, '1.243.890.0': 5359, '1.223.2580.0': 5360, '1.253.412.0': 5361, '1.261.590.0': 5362, '1.253.841.0': 5363, '1.251.1352.0': 5364, '1.235.2232.0': 5365, '1.239.903.0': 5366, '1.257.940.0': 5367, '1.263.726.0': 5368, '1.241.1300.0': 5369, '1.235.1851.0': 5370, '1.249.103.0': 5371, '1.245.144.0': 5372, '1.249.252.0': 5373, '1.263.500.0': 5374, '1.257.1419.0': 5375, '1.233.1119.0': 5376, '1.231.192.0': 5377, '1.225.4173.0': 5378, '1.259.559.0': 5379, '1.239.1653.0': 5380, '1.255.216.0': 5381, '1.259.554.0': 5382, '1.227.2059.0': 5383, '1.241.877.0': 5384, '1.261.1652.0': 5385, '1.225.837.0': 5386, '1.253.134.0': 5387, '1.227.2105.0': 5388, '1.235.1980.0': 5389, '1.231.1298.0': 5390, '1.241.472.0': 5391, '1.235.39.0': 5392, '1.253.931.0': 5393, '1.247.390.0': 5394, '1.245.1087.0': 5395, '1.229.1617.0': 5396, '1.225.1221.0': 5397, '1.235.1453.0': 5398, '1.263.44.0': 5399, '1.233.3540.0': 5400, '1.239.1485.0': 5401, '1.261.1600.0': 5402, '1.263.209.0': 5403, '1.259.647.0': 5404, '1.265.656.0': 5405, '1.247.607.0': 5406, '1.231.881.0': 5407, '1.241.296.0': 5408, '1.267.528.0': 5409, '1.223.2118.0': 5410, '1.245.1102.0': 5411, '1.231.1139.0': 5412, '1.267.441.0': 5413, '1.251.331.0': 5414, '1.259.188.0': 5415, '1.225.531.0': 5416, '1.229.899.0': 5417, '1.223.2048.0': 5418, '1.249.1286.0': 5419, '1.253.148.0': 5420, '1.229.1141.0': 5421, '1.251.258.0': 5422, '1.241.63.0': 5423, '1.225.2176.0': 5424, '1.241.217.0': 5425, '1.229.197.0': 5426, '1.259.709.0': 5427, '1.263.458.0': 5428, '1.225.3933.0': 5429, '1.225.1773.0': 5430, '1.263.65.0': 5431, '1.233.3058.0': 5432, '1.247.554.0': 5433, '1.233.1442.0': 5434, '1.251.712.0': 5435, '1.205.1338.0': 5436, '1.227.656.0': 5437, '1.231.22.0': 5438, '1.253.782.0': 5439, '1.231.1278.0': 5440, '1.237.1280.0': 5441, '1.239.494.0': 5442, '1.243.549.0': 5443, '1.263.677.0': 5444, '1.257.1466.0': 5445, '1.253.791.0': 5446, '1.235.860.0': 5447, '1.259.1513.0': 5448, '1.247.704.0': 5449, '1.231.1613.0': 5450, '1.243.913.0': 5451, '1.261.317.0': 5452, '1.257.1153.0': 5453, '1.253.69.0': 5454, '1.247.864.0': 5455, '1.243.316.0': 5456, '1.233.1284.0': 5457, '1.245.452.0': 5458, '1.247.514.0': 5459, '1.235.1758.0': 5460, '1.245.209.0': 5461, '1.235.1566.0': 5462, '1.261.473.0': 5463, '1.233.775.0': 5464, '1.251.1275.0': 5465, '1.247.456.0': 5466, '1.257.519.0': 5467, '1.257.1248.0': 5468, '1.237.360.0': 5469, '1.263.795.0': 5470, '1.249.670.0': 5471, '1.233.1838.0': 5472, '1.251.975.0': 5473, '1.247.739.0': 5474, '1.229.163.0': 5475, '1.251.282.0': 5476, '1.253.890.0': 5477, '1.243.140.0': 5478, '1.231.1801.0': 5479, '1.249.1154.0': 5480, '1.241.1172.0': 5481, '1.225.3974.0': 5482, '1.259.1405.0': 5483, '1.227.949.0': 5484, '1.239.431.0': 5485, '1.225.1206.0': 5486, '1.249.1282.0': 5487, '1.253.569.0': 5488, '1.243.544.0': 5489, '1.253.443.0': 5490, '1.259.369.0': 5491, '1.249.992.0': 5492, '1.247.1068.0': 5493, '1.235.362.0': 5494, '1.257.396.0': 5495, '1.231.1012.0': 5496, '1.263.233.0': 5497, '1.225.1447.0': 5498, '1.233.3370.0': 5499, '1.227.2328.0': 5500, '1.243.1015.0': 5501, '1.227.101.0': 5502, '1.245.177.0': 5503, '1.225.1784.0': 5504, '1.257.1287.0': 5505, '1.225.3090.0': 5506, '1.243.573.0': 5507, '1.241.1330.0': 5508, '1.231.327.0': 5509, '1.239.242.0': 5510, '1.259.970.0': 5511, '1.263.686.0': 5512, '1.233.2056.0': 5513, '1.243.880.0': 5514, '1.257.1051.0': 5515, '1.263.46.0': 5516, '1.237.1766.0': 5517, '1.263.170.0': 5518, '1.261.565.0': 5519, '1.233.2210.0': 5520, '1.263.987.0': 5521, '1.241.312.0': 5522, '1.239.1663.0': 5523, '1.249.155.0': 5524, '1.263.1590.0': 5525, '1.247.944.0': 5526, '1.243.207.0': 5527, '1.227.1023.0': 5528, '1.237.691.0': 5529, '1.263.436.0': 5530, '1.263.58.0': 5531, '1.263.1003.0': 5532, '1.227.810.0': 5533, '1.245.399.0': 5534, '1.225.86.0': 5535, '1.233.1732.0': 5536, '1.225.2667.0': 5537, '1.261.544.0': 5538, '1.253.160.0': 5539, '1.235.3220.0': 5540, '1.253.676.0': 5541, '1.245.510.0': 5542, '1.247.867.0': 5543, '1.227.302.0': 5544, '1.233.200.0': 5545, '1.249.983.0': 5546, '1.255.155.0': 5547, '1.227.512.0': 5548, '1.263.1096.0': 5549, '1.233.821.0': 5550, '1.231.1097.0': 5551, '1.223.2464.0': 5552, '1.243.291.0': 5553, '1.235.2373.0': 5554, '1.231.959.0': 5555, '1.259.1043.0': 5556, '1.245.895.0': 5557, '1.249.817.0': 5558, '1.235.578.0': 5559, '1.247.56.0': 5560, '1.225.4051.0': 5561, '1.247.951.0': 5562, '1.263.165.0': 5563, '1.229.786.0': 5564, '1.225.2097.0': 5565, '1.249.1291.0': 5566, '1.227.1628.0': 5567, '1.249.1267.0': 5568, '1.235.1570.0': 5569, '1.233.2659.0': 5570, '1.257.1301.0': 5571, '1.225.583.0': 5572, '1.223.1414.0': 5573, '1.259.437.0': 5574, '1.233.955.0': 5575, '1.259.1607.0': 5576, '1.257.948.0': 5577, '1.257.609.0': 5578, '1.249.169.0': 5579, '1.225.3579.0': 5580, '1.253.703.0': 5581, '1.235.571.0': 5582, '1.253.31.0': 5583, '1.257.560.0': 5584, '1.231.797.0': 5585, '1.259.948.0': 5586, '1.245.880.0': 5587, '1.235.397.0': 5588, '1.225.3422.0': 5589, '1.251.128.0': 5590, '1.237.347.0': 5591, '1.241.166.0': 5592, '1.241.361.0': 5593, '1.245.954.0': 5594, '1.257.606.0': 5595, '1.263.427.0': 5596, '1.259.141.0': 5597, '1.233.3763.0': 5598, '1.225.2784.0': 5599, '1.229.1262.0': 5600, '1.223.2667.0': 5601, '1.257.638.0': 5602, '1.239.1151.0': 5603, '1.241.1318.0': 5604, '1.249.738.0': 5605, '1.237.809.0': 5606, '1.227.1639.0': 5607, '1.225.1548.0': 5608, '1.251.395.0': 5609, '1.233.3556.0': 5610, '1.233.863.0': 5611, '1.239.1005.0': 5612, '1.257.937.0': 5613, '1.249.962.0': 5614, '1.249.1026.0': 5615, '1.237.815.0': 5616, '1.259.178.0': 5617, '1.229.1875.0': 5618, '1.225.1238.0': 5619, '1.233.3698.0': 5620, '1.259.788.0': 5621, '1.249.233.0': 5622, '1.233.897.0': 5623, '1.261.183.0': 5624, '1.233.3746.0': 5625, '1.263.1547.0': 5626, '1.233.2253.0': 5627, '1.245.716.0': 5628, '1.257.190.0': 5629, '1.243.605.0': 5630, '1.249.870.0': 5631, '1.249.985.0': 5632, '1.249.1048.0': 5633, '1.257.754.0': 5634, '1.261.349.0': 5635, '1.225.1756.0': 5636, '1.225.2689.0': 5637, '1.247.84.0': 5638, '1.259.715.0': 5639, '1.259.1581.0': 5640, '1.233.3382.0': 5641, '1.229.192.0': 5642, '1.225.235.0': 5643, '1.257.1228.0': 5644, '1.245.454.0': 5645, '1.241.1274.0': 5646, '1.235.1663.0': 5647, '1.229.1926.0': 5648, '1.259.973.0': 5649, '1.235.133.0': 5650, '1.249.724.0': 5651, '1.257.993.0': 5652, '1.239.422.0': 5653, '1.249.575.0': 5654, '1.263.197.0': 5655, '1.239.778.0': 5656, '1.237.1342.0': 5657, '1.225.2433.0': 5658, '1.263.1155.0': 5659, '1.245.444.0': 5660, '1.241.401.0': 5661, '1.247.78.0': 5662, '1.225.3341.0': 5663, '1.229.18.0': 5664, '1.237.1313.0': 5665, '1.259.1363.0': 5666, '1.233.2396.0': 5667, '1.239.1667.0': 5668, '1.245.962.0': 5669, '1.229.788.0': 5670, '1.249.499.0': 5671, '1.225.2377.0': 5672, '1.243.407.0': 5673, '1.233.3261.0': 5674, '1.261.1615.0': 5675, '1.233.765.0': 5676, '1.229.1483.0': 5677, '1.245.426.0': 5678, '1.241.643.0': 5679, '1.235.538.0': 5680, '1.251.391.0': 5681, '1.249.1293.0': 5682, '1.225.2313.0': 5683, '1.237.841.0': 5684, '1.261.377.0': 5685, '1.225.2280.0': 5686, '1.267.73.0': 5687, '1.261.1582.0': 5688, '1.237.965.0': 5689, '1.241.831.0': 5690, '1.259.196.0': 5691, '1.231.743.0': 5692, '1.225.3814.0': 5693, '1.243.593.0': 5694, '1.249.836.0': 5695, '1.241.983.0': 5696, '1.259.1095.0': 5697, '1.227.1294.0': 5698, '1.227.2150.0': 5699, '1.243.850.0': 5700, '1.223.2880.0': 5701, '1.253.472.0': 5702, '1.233.2941.0': 5703, '1.225.2804.0': 5704, '1.237.739.0': 5705, '1.243.855.0': 5706, '1.231.1459.0': 5707, '1.235.1005.0': 5708, '1.243.774.0': 5709, '1.227.2118.0': 5710, '1.225.2171.0': 5711, '1.251.722.0': 5712, '1.247.63.0': 5713, '1.229.645.0': 5714, '1.231.859.0': 5715, '1.229.1398.0': 5716, '1.249.742.0': 5717, '1.257.1369.0': 5718, '1.247.643.0': 5719, '1.231.752.0': 5720, '1.223.2747.0': 5721, '1.239.506.0': 5722, '1.241.1383.0': 5723, '1.229.1788.0': 5724, '1.223.1384.0': 5725, '1.261.1390.0': 5726, '1.235.3155.0': 5727, '1.233.1940.0': 5728, '1.231.201.0': 5729, '1.247.949.0': 5730, '1.241.1354.0': 5731, '1.237.1293.0': 5732, '1.229.1357.0': 5733, '1.225.2592.0': 5734, '1.225.3595.0': 5735, '1.249.813.0': 5736, '1.247.906.0': 5737, '1.231.75.0': 5738, '1.249.493.0': 5739, '1.235.1013.0': 5740, '1.231.2006.0': 5741, '1.233.3486.0': 5742, '1.259.902.0': 5743, '1.259.901.0': 5744, '1.223.1847.0': 5745, '1.233.3658.0': 5746, '1.235.833.0': 5747, '1.259.712.0': 5748, '1.235.1446.0': 5749, '1.231.1245.0': 5750, '1.225.2237.0': 5751, '1.257.1140.0': 5752, '1.239.188.0': 5753, '1.257.854.0': 5754, '1.245.442.0': 5755, '1.261.241.0': 5756, '1.259.480.0': 5757, '1.259.109.0': 5758, '1.247.387.0': 5759, '1.251.911.0': 5760, '1.249.754.0': 5761, '1.237.305.0': 5762, '1.233.3671.0': 5763, '1.267.1347.0': 5764, '1.225.1299.0': 5765, '1.235.591.0': 5766, '1.261.291.0': 5767, '1.231.821.0': 5768, '1.223.2337.0': 5769, '1.229.988.0': 5770, '1.251.486.0': 5771, '1.257.1029.0': 5772, '1.227.2633.0': 5773, '1.263.1078.0': 5774, '1.227.2129.0': 5775, '1.241.564.0': 5776, '1.259.227.0': 5777, '1.249.757.0': 5778, '1.253.808.0': 5779, '1.263.691.0': 5780, '1.237.11.0': 5781, '1.233.1424.0': 5782, '1.257.950.0': 5783, '1.243.927.0': 5784, '1.243.334.0': 5785, '1.225.1531.0': 5786, '1.233.51.0': 5787, '1.241.720.0': 5788, '1.259.1756.0': 5789, '1.235.1919.0': 5790, '1.247.274.0': 5791, '1.231.1786.0': 5792, '1.263.1979.0': 5793, '1.227.2736.0': 5794, '1.249.1013.0': 5795, '1.223.2844.0': 5796, '1.235.1404.0': 5797, '1.231.1641.0': 5798, '1.225.1739.0': 5799, '1.253.126.0': 5800, '1.261.701.0': 5801, '1.239.171.0': 5802, '1.249.1309.0': 5803, '1.267.619.0': 5804, '1.247.353.0': 5805, '1.241.317.0': 5806, '1.225.3407.0': 5807, '1.251.375.0': 5808, '1.257.956.0': 5809, '1.225.1811.0': 5810, '1.251.1492.0': 5811, '1.249.536.0': 5812, '1.249.580.0': 5813, '1.259.493.0': 5814, '1.231.50.0': 5815, '1.241.1346.0': 5816, '1.235.953.0': 5817, '1.229.1571.0': 5818, '1.219.1436.0': 5819, '1.257.250.0': 5820, '1.233.2025.0': 5821, '1.235.2155.0': 5822, '1.227.652.0': 5823, '1.227.2271.0': 5824, '1.263.331.0': 5825, '1.213.3814.0': 5826, '1.241.1385.0': 5827, '1.223.2819.0': 5828, '1.247.393.0': 5829, '1.259.375.0': 5830, '1.235.2033.0': 5831, '1.257.1143.0': 5832, '1.247.789.0': 5833, '1.249.547.0': 5834, '1.233.1964.0': 5835, '1.233.112.0': 5836, '1.259.1533.0': 5837, '1.227.59.0': 5838, '1.251.36.0': 5839, '1.259.1274.0': 5840, '1.225.3041.0': 5841, '1.247.513.0': 5842, '1.231.1570.0': 5843, '1.231.178.0': 5844, '1.235.1822.0': 5845, '1.225.2266.0': 5846, '1.247.23.0': 5847, '1.229.969.0': 5848, '1.233.1027.0': 5849, '1.257.1181.0': 5850, '1.261.361.0': 5851, '1.253.218.0': 5852, '1.233.892.0': 5853, '1.229.228.0': 5854, '1.249.665.0': 5855, '1.259.302.0': 5856, '1.259.1554.0': 5857, '1.249.1028.0': 5858, '1.263.81.0': 5859, '1.239.1186.0': 5860, '1.237.755.0': 5861, '1.247.79.0': 5862, '1.229.819.0': 5863, '1.235.2277.0': 5864, '1.251.1138.0': 5865, '1.227.2154.0': 5866, '1.233.2078.0': 5867, '1.259.1573.0': 5868, '1.259.181.0': 5869, '1.241.674.0': 5870, '1.247.855.0': 5871, '1.235.863.0': 5872, '1.253.857.0': 5873, '1.207.2950.0': 5874, '1.223.2664.0': 5875, '1.229.1257.0': 5876, '1.259.1320.0': 5877, '1.235.1460.0': 5878, '1.261.1494.0': 5879, '1.229.2036.0': 5880, '1.247.94.0': 5881, '1.231.1146.0': 5882, '1.253.776.0': 5883, '1.259.344.0': 5884, '1.245.307.0': 5885, '1.239.548.0': 5886, '1.225.199.0': 5887, '1.233.2572.0': 5888, '1.239.1524.0': 5889, '1.263.1150.0': 5890, '1.261.596.0': 5891, '1.231.1347.0': 5892, '1.223.2826.0': 5893, '1.253.664.0': 5894, '1.229.539.0': 5895, '1.227.408.0': 5896, '1.233.1374.0': 5897, '1.225.1994.0': 5898, '1.227.441.0': 5899, '1.257.1107.0': 5900, '1.235.3173.0': 5901, '1.261.1426.0': 5902, '1.229.1639.0': 5903, '1.245.1206.0': 5904, '1.235.2549.0': 5905, '1.257.1005.0': 5906, '1.239.1011.0': 5907, '1.233.4207.0': 5908, '1.251.1490.0': 5909, '1.225.35.0': 5910, '1.241.945.0': 5911, '1.251.1073.0': 5912, '1.257.793.0': 5913, '1.231.1238.0': 5914, '1.227.1138.0': 5915, '1.233.3818.0': 5916, '1.229.2003.0': 5917, '1.251.406.0': 5918, '1.265.200.0': 5919, '1.263.265.0': 5920, '1.237.905.0': 5921, '1.235.1354.0': 5922, '1.223.2587.0': 5923, '1.235.3277.0': 5924, '1.259.1319.0': 5925, '1.235.1812.0': 5926, '1.227.2682.0': 5927, '1.225.145.0': 5928, '1.249.736.0': 5929, '1.237.924.0': 5930, '1.261.429.0': 5931, '1.231.903.0': 5932, '1.233.245.0': 5933, '1.225.978.0': 5934, '1.261.324.0': 5935, '1.223.2023.0': 5936, '1.247.923.0': 5937, '1.227.1364.0': 5938, '1.233.1111.0': 5939, '1.235.3290.0': 5940, '1.245.133.0': 5941, '1.247.1004.0': 5942, '1.237.1001.0': 5943, '1.233.1946.0': 5944, '1.243.296.0': 5945, '1.249.658.0': 5946, '1.233.443.0': 5947, '1.243.842.0': 5948, '1.249.1172.0': 5949, '1.243.229.0': 5950, '1.257.704.0': 5951, '1.231.216.0': 5952, '1.225.129.0': 5953, '1.245.410.0': 5954, '1.227.1784.0': 5955, '1.233.526.0': 5956, '1.251.437.0': 5957, '1.261.752.0': 5958, '1.217.797.0': 5959, '1.243.338.0': 5960, '1.237.524.0': 5961, '1.237.1605.0': 5962, '1.259.588.0': 5963, '1.251.1076.0': 5964, '1.267.688.0': 5965, '1.231.189.0': 5966, '1.249.641.0': 5967, '1.229.1319.0': 5968, '1.263.1153.0': 5969, '1.253.1041.0': 5970, '1.249.696.0': 5971, '1.253.425.0': 5972, '1.245.438.0': 5973, '1.259.597.0': 5974, '1.251.590.0': 5975, '1.233.2109.0': 5976, '1.237.1098.0': 5977, '1.227.1419.0': 5978, '1.241.680.0': 5979, '1.227.1885.0': 5980, '1.233.2919.0': 5981, '1.263.1032.0': 5982, '1.261.1376.0': 5983, '1.233.332.0': 5984, '1.227.2628.0': 5985, '1.227.589.0': 5986, '1.261.483.0': 5987, '1.259.429.0': 5988, '1.251.626.0': 5989, '1.225.3215.0': 5990, '1.247.698.0': 5991, '1.249.902.0': 5992, '1.225.1644.0': 5993, '1.237.986.0': 5994, '1.233.165.0': 5995, '1.261.848.0': 5996, '1.237.1219.0': 5997, '1.225.1873.0': 5998, '1.235.2250.0': 5999, '1.227.721.0': 6000, '1.239.1376.0': 6001, '1.241.768.0': 6002, '1.251.764.0': 6003, '1.251.103.0': 6004, '1.225.3909.0': 6005, '1.261.1573.0': 6006, '1.225.2829.0': 6007, '1.243.979.0': 6008, '1.231.1926.0': 6009, '1.241.541.0': 6010, '1.263.565.0': 6011, '1.235.2038.0': 6012, '1.261.788.0': 6013, '1.259.1775.0': 6014, '1.223.850.0': 6015, '1.229.2048.0': 6016, '1.251.182.0': 6017, '1.247.66.0': 6018, '1.229.892.0': 6019, '1.233.2636.0': 6020, '1.237.1319.0': 6021, '1.233.1768.0': 6022, '1.233.922.0': 6023, '1.227.2672.0': 6024, '1.249.1168.0': 6025, '1.251.1151.0': 6026, '1.233.3594.0': 6027, '1.257.1139.0': 6028, '1.257.412.0': 6029, '1.249.971.0': 6030, '1.239.1096.0': 6031, '1.231.1774.0': 6032, '1.235.1961.0': 6033, '1.227.206.0': 6034, '1.245.173.0': 6035, '1.251.99.0': 6036, '1.249.831.0': 6037, '1.227.2534.0': 6038, '1.229.1953.0': 6039, '1.233.1970.0': 6040, '1.255.313.0': 6041, '1.233.237.0': 6042, '1.235.70.0': 6043, '1.243.463.0': 6044, '1.259.639.0': 6045, '1.225.2005.0': 6046, '1.245.674.0': 6047, '1.241.1122.0': 6048, '1.239.1181.0': 6049, '1.223.2758.0': 6050, '1.261.1607.0': 6051, '1.235.1985.0': 6052, '1.259.108.0': 6053, '1.229.703.0': 6054, '1.239.169.0': 6055, '1.251.179.0': 6056, '1.235.461.0': 6057, '1.245.766.0': 6058, '1.239.949.0': 6059, '1.231.1506.0': 6060, '1.239.145.0': 6061, '1.237.1601.0': 6062, '1.225.3854.0': 6063, '1.249.1392.0': 6064, '1.231.1629.0': 6065, '1.241.151.0': 6066, '1.241.454.0': 6067, '1.257.913.0': 6068, '1.227.322.0': 6069, '1.257.583.0': 6070, '1.231.66.0': 6071, '1.251.659.0': 6072, '1.231.206.0': 6073, '1.237.1338.0': 6074, '1.257.802.0': 6075, '1.259.938.0': 6076, '1.263.0.0': 6077, '1.237.128.0': 6078, '1.235.1058.0': 6079, '1.227.756.0': 6080, '1.243.919.0': 6081, '1.233.3230.0': 6082, '1.237.102.0': 6083, '1.251.1363.0': 6084, '1.229.581.0': 6085, '1.261.1370.0': 6086, '1.229.1175.0': 6087, '1.255.195.0': 6088, '1.251.495.0': 6089, '1.225.381.0': 6090, '1.245.856.0': 6091, '1.233.719.0': 6092, '1.223.2346.0': 6093, '1.229.1893.0': 6094, '1.249.211.0': 6095, '1.231.1000.0': 6096, '1.225.462.0': 6097, '1.247.894.0': 6098, '1.237.354.0': 6099, '1.237.178.0': 6100, '1.231.300.0': 6101, '1.257.1399.0': 6102, '1.253.838.0': 6103, '1.229.800.0': 6104, '1.223.1709.0': 6105, '1.233.61.0': 6106, '1.249.779.0': 6107, '1.235.216.0': 6108, '1.239.1534.0': 6109, '1.233.1524.0': 6110, '1.253.125.0': 6111, '1.247.516.0': 6112, '1.227.2284.0': 6113, '1.241.323.0': 6114, '1.225.1232.0': 6115, '1.231.688.0': 6116, '1.237.1381.0': 6117, '1.249.723.0': 6118, '1.249.944.0': 6119, '1.225.1100.0': 6120, '1.247.744.0': 6121, '1.237.654.0': 6122, '1.251.248.0': 6123, '1.241.1408.0': 6124, '1.243.792.0': 6125, '1.243.69.0': 6126, '1.257.1247.0': 6127, '1.251.1478.0': 6128, '1.235.2144.0': 6129, '1.251.844.0': 6130, '1.249.1001.0': 6131, '1.239.194.0': 6132, '1.247.933.0': 6133, '1.235.316.0': 6134, '1.263.2.0': 6135, '1.247.627.0': 6136, '1.247.72.0': 6137, '1.237.558.0': 6138, '1.243.631.0': 6139, '1.247.175.0': 6140, '1.247.490.0': 6141, '1.241.849.0': 6142, '1.223.2544.0': 6143, '1.261.22.0': 6144, '1.233.4189.0': 6145, '1.233.1128.0': 6146, '1.247.316.0': 6147, '1.245.565.0': 6148, '1.227.502.0': 6149, '1.235.343.0': 6150, '1.253.610.0': 6151, '1.237.243.0': 6152, '1.225.3429.0': 6153, '1.245.1111.0': 6154, '1.227.2835.0': 6155, '1.251.110.0': 6156, '1.245.1059.0': 6157, '1.227.2915.0': 6158, '1.231.2139.0': 6159, '1.251.1222.0': 6160, '1.231.103.0': 6161, '1.251.618.0': 6162, '1.233.3524.0': 6163, '1.225.1417.0': 6164, '1.227.991.0': 6165, '1.229.658.0': 6166, '1.227.1155.0': 6167, '1.245.433.0': 6168, '1.259.670.0': 6169, '1.237.976.0': 6170, '1.223.2796.0': 6171, '1.261.463.0': 6172, '1.241.1239.0': 6173, '1.259.758.0': 6174, '1.263.417.0': 6175, '1.233.665.0': 6176, '1.223.1669.0': 6177, '1.231.977.0': 6178, '1.241.117.0': 6179, '1.229.1802.0': 6180, '1.229.749.0': 6181, '1.253.547.0': 6182, '1.251.1048.0': 6183, '1.241.147.0': 6184, '1.231.1933.0': 6185, '1.261.1160.0': 6186, '1.223.2713.0': 6187, '1.263.1109.0': 6188, '1.225.668.0': 6189, '1.261.1490.0': 6190, '1.231.2083.0': 6191, '1.249.501.0': 6192, '1.223.1145.0': 6193, '1.239.1477.0': 6194, '1.251.622.0': 6195, '1.229.1970.0': 6196, '1.233.2092.0': 6197, '1.233.1886.0': 6198, '1.241.1268.0': 6199, '1.251.1202.0': 6200, '1.245.1008.0': 6201, '1.223.2446.0': 6202, '1.243.80.0': 6203, '1.233.278.0': 6204, '1.225.1517.0': 6205, '1.251.709.0': 6206, '1.233.4270.0': 6207, '1.225.312.0': 6208, '1.225.366.0': 6209, '1.235.1362.0': 6210, '1.249.158.0': 6211, '1.229.979.0': 6212, '1.231.151.0': 6213, '1.225.1650.0': 6214, '1.225.3918.0': 6215, '1.237.1055.0': 6216, '1.259.1538.0': 6217, '1.227.363.0': 6218, '1.233.2217.0': 6219, '1.233.4104.0': 6220, '1.251.1364.0': 6221, '1.239.1146.0': 6222, '1.227.2144.0': 6223, '1.231.749.0': 6224, '1.261.1389.0': 6225, '1.223.1944.0': 6226, '1.261.695.0': 6227, '1.223.721.0': 6228, '1.225.135.0': 6229, '1.259.293.0': 6230, '1.245.214.0': 6231, '1.233.2963.0': 6232, '1.235.1418.0': 6233, '1.235.420.0': 6234, '1.257.1291.0': 6235, '1.249.1034.0': 6236, '1.239.841.0': 6237, '1.253.357.0': 6238, '1.235.1318.0': 6239, '1.245.314.0': 6240, '1.245.419.0': 6241, '1.251.738.0': 6242, '1.243.121.0': 6243, '1.259.1419.0': 6244, '1.259.237.0': 6245, '1.233.2651.0': 6246, '1.231.1951.0': 6247, '1.253.174.0': 6248, '1.245.612.0': 6249, '1.257.1215.0': 6250, '1.235.1335.0': 6251, '1.233.1403.0': 6252, '1.233.4165.0': 6253, '1.227.1385.0': 6254, '1.237.87.0': 6255, '1.235.3087.0': 6256, '1.243.378.0': 6257, '1.243.145.0': 6258, '1.237.139.0': 6259, '1.249.1005.0': 6260, '1.233.786.0': 6261, '1.231.1343.0': 6262, '1.247.87.0': 6263, '1.263.141.0': 6264, '1.231.1186.0': 6265, '1.227.471.0': 6266, '1.233.4157.0': 6267, '1.231.443.0': 6268, '1.231.243.0': 6269, '1.231.2027.0': 6270, '1.237.729.0': 6271, '1.235.988.0': 6272, '1.225.2061.0': 6273, '1.235.842.0': 6274, '1.249.1031.0': 6275, '1.235.1279.0': 6276, '1.231.809.0': 6277, '1.231.1226.0': 6278, '1.249.760.0': 6279, '1.231.1895.0': 6280, '1.247.660.0': 6281, '1.243.470.0': 6282, '1.233.1263.0': 6283, '1.253.607.0': 6284, '1.243.876.0': 6285, '1.239.174.0': 6286, '1.229.1938.0': 6287, '1.259.363.0': 6288, '1.237.772.0': 6289, '1.227.328.0': 6290, '1.229.1528.0': 6291, '1.247.394.0': 6292, '1.229.460.0': 6293, '1.225.3419.0': 6294, '1.225.2402.0': 6295, '1.225.3344.0': 6296, '1.223.2235.0': 6297, '1.227.854.0': 6298, '1.247.947.0': 6299, '1.259.1399.0': 6300, '1.259.635.0': 6301, '1.257.932.0': 6302, '1.259.1412.0': 6303, '1.233.4080.0': 6304, '1.251.602.0': 6305, '1.233.3794.0': 6306, '1.263.452.0': 6307, '1.231.788.0': 6308, '1.229.902.0': 6309, '1.237.1073.0': 6310, '1.241.557.0': 6311, '1.247.644.0': 6312, '1.251.1158.0': 6313, '1.233.1417.0': 6314, '1.243.921.0': 6315, '1.225.2471.0': 6316, '1.261.1589.0': 6317, '1.235.3353.0': 6318, '1.241.1175.0': 6319, '1.247.279.0': 6320, '1.235.1780.0': 6321, '1.235.3056.0': 6322, '1.233.3599.0': 6323, '1.247.553.0': 6324, '1.233.3623.0': 6325, '1.231.12.0': 6326, '1.235.742.0': 6327, '1.241.1414.0': 6328, '1.249.224.0': 6329, '1.253.464.0': 6330, '1.257.611.0': 6331, '1.233.2098.0': 6332, '1.239.573.0': 6333, '1.251.338.0': 6334, '1.231.736.0': 6335, '1.241.787.0': 6336, '1.261.413.0': 6337, '1.227.2498.0': 6338, '1.259.1531.0': 6339, '1.233.770.0': 6340, '1.245.60.0': 6341, '1.233.3534.0': 6342, '1.257.12.0': 6343, '1.225.2826.0': 6344, '1.231.111.0': 6345, '1.245.146.0': 6346, '1.249.1177.0': 6347, '1.227.2760.0': 6348, '1.225.4126.0': 6349, '1.257.572.0': 6350, '1.229.1829.0': 6351, '1.227.1079.0': 6352, '1.229.96.0': 6353, '1.235.1886.0': 6354, '1.243.983.0': 6355, '1.227.1846.0': 6356, '1.225.2091.0': 6357, '1.237.1572.0': 6358, '1.233.2197.0': 6359, '1.259.1589.0': 6360, '1.263.741.0': 6361, '1.223.2787.0': 6362, '1.253.166.0': 6363, '1.235.238.0': 6364, '1.257.947.0': 6365, '1.225.3846.0': 6366, '1.235.1637.0': 6367, '1.235.2212.0': 6368, '1.229.387.0': 6369, '1.231.545.0': 6370, '1.257.984.0': 6371, '1.249.490.0': 6372, '1.257.255.0': 6373, '1.235.2301.0': 6374, '1.247.851.0': 6375, '1.225.2529.0': 6376, '1.247.207.0': 6377, '1.225.1760.0': 6378, '1.225.1693.0': 6379, '1.237.635.0': 6380, '1.257.1303.0': 6381, '1.223.1838.0': 6382, '1.245.394.0': 6383, '1.229.1579.0': 6384, '1.233.969.0': 6385, '1.235.1518.0': 6386, '1.245.457.0': 6387, '1.229.1372.0': 6388, '1.247.532.0': 6389, '1.253.603.0': 6390, '1.225.3124.0': 6391, '1.223.564.0': 6392, '1.261.288.0': 6393, '1.231.2061.0': 6394, '1.261.737.0': 6395, '1.233.1854.0': 6396, '1.225.3863.0': 6397, '1.257.634.0': 6398, '1.223.1828.0': 6399, '1.251.387.0': 6400, '1.231.2173.0': 6401, '1.251.126.0': 6402, '1.241.196.0': 6403, '1.225.1834.0': 6404, '1.233.195.0': 6405, '1.225.1565.0': 6406, '1.225.3381.0': 6407, '1.233.1973.0': 6408, '1.239.221.0': 6409, '1.233.3907.0': 6410, '1.233.3375.0': 6411, '1.251.570.0': 6412, '1.223.174.0': 6413, '1.257.406.0': 6414, '1.241.509.0': 6415, '1.247.95.0': 6416, '1.237.1832.0': 6417, '1.249.456.0': 6418, '1.249.1225.0': 6419, '1.251.696.0': 6420, '1.251.441.0': 6421, '1.167.387.0': 6422, '1.257.540.0': 6423, '1.233.2426.0': 6424, '1.243.302.0': 6425, '1.255.303.0': 6426, '1.227.2179.0': 6427, '1.223.2699.0': 6428, '1.225.1006.0': 6429, '1.241.841.0': 6430, '1.227.1672.0': 6431, '1.259.1308.0': 6432, '1.235.168.0': 6433, '1.225.4118.0': 6434, '1.225.591.0': 6435, '1.249.907.0': 6436, '1.231.1915.0': 6437, '1.223.1880.0': 6438, '1.259.761.0': 6439, '1.253.699.0': 6440, '1.225.1177.0': 6441, '1.251.1223.0': 6442, '1.231.908.0': 6443, '1.231.1265.0': 6444, '1.233.65.0': 6445, '1.233.3654.0': 6446, '1.227.2810.0': 6447, '1.241.568.0': 6448, '1.231.531.0': 6449, '1.223.1733.0': 6450, '1.227.1635.0': 6451, '1.235.2257.0': 6452, '1.235.854.0': 6453, '1.225.989.0': 6454, '1.247.1079.0': 6455, '1.235.688.0': 6456, '1.223.1437.0': 6457, '1.237.881.0': 6458, '1.227.1597.0': 6459, '1.241.1027.0': 6460, '1.257.909.0': 6461, '1.259.13.0': 6462, '1.237.660.0': 6463, '1.231.792.0': 6464, '1.227.2559.0': 6465, '1.225.2385.0': 6466, '1.225.2293.0': 6467, '1.225.3019.0': 6468, '1.257.603.0': 6469, '1.237.182.0': 6470, '1.239.139.0': 6471, '1.247.777.0': 6472, '1.239.1454.0': 6473, '1.237.1135.0': 6474, '1.231.87.0': 6475, '1.227.840.0': 6476, '1.247.510.0': 6477, '1.225.4001.0': 6478, '1.245.272.0': 6479, '1.251.1164.0': 6480, '1.231.372.0': 6481, '1.233.3687.0': 6482, '1.237.1844.0': 6483, '1.251.829.0': 6484, '1.235.119.0': 6485, '1.231.1595.0': 6486, '1.235.1642.0': 6487, '1.229.649.0': 6488, '1.247.64.0': 6489, '1.235.375.0': 6490, '1.259.1365.0': 6491, '1.259.652.0': 6492, '1.261.704.0': 6493, '1.247.188.0': 6494, '1.227.2725.0': 6495, '1.223.1196.0': 6496, '1.223.1314.0': 6497, '1.231.503.0': 6498, '1.233.21.0': 6499, '1.227.2529.0': 6500, '1.253.499.0': 6501, '1.225.1868.0': 6502, '1.243.354.0': 6503, '1.247.246.0': 6504, '1.241.703.0': 6505, '1.247.840.0': 6506, '1.245.1107.0': 6507, '1.247.345.0': 6508, '1.227.1096.0': 6509, '1.227.270.0': 6510, '1.259.699.0': 6511, '1.225.2458.0': 6512, '1.263.992.0': 6513, '1.233.1519.0': 6514, '1.235.1416.0': 6515, '1.257.818.0': 6516, '1.241.1328.0': 6517, '1.233.3586.0': 6518, '1.225.3631.0': 6519, '1.259.103.0': 6520, '1.225.2649.0': 6521, '1.237.1206.0': 6522, '1.249.1292.0': 6523, '1.213.5817.0': 6524, '1.245.188.0': 6525, '1.225.3305.0': 6526, '1.239.359.0': 6527, '1.247.957.0': 6528, '1.241.1312.0': 6529, '1.223.2886.0': 6530, '1.239.165.0': 6531, '1.235.1170.0': 6532, '1.229.690.0': 6533, '1.229.1771.0': 6534, '1.247.454.0': 6535, '1.231.1709.0': 6536, '1.229.916.0': 6537, '1.225.618.0': 6538, '1.257.1055.0': 6539, '1.245.300.0': 6540, '1.225.4167.0': 6541, '1.229.436.0': 6542, '1.229.106.0': 6543, '1.241.976.0': 6544, '1.225.1083.0': 6545, '1.229.1647.0': 6546, '1.231.1122.0': 6547, '1.241.780.0': 6548, '1.251.620.0': 6549, '1.225.4114.0': 6550, '1.225.3084.0': 6551, '1.263.389.0': 6552, '1.235.2338.0': 6553, '1.247.197.0': 6554, '1.229.764.0': 6555, '1.223.2871.0': 6556, '1.247.283.0': 6557, '1.245.1051.0': 6558, '1.227.2051.0': 6559, '1.243.990.0': 6560, '1.259.1384.0': 6561, '1.245.416.0': 6562, '1.239.1408.0': 6563, '1.241.547.0': 6564, '1.257.653.0': 6565, '1.229.668.0': 6566, '1.249.107.0': 6567, '1.235.991.0': 6568, '1.241.1110.0': 6569, '1.251.1334.0': 6570, '1.231.1025.0': 6571, '1.245.767.0': 6572, '1.233.3772.0': 6573, '1.235.729.0': 6574, '1.249.151.0': 6575, '1.223.1758.0': 6576, '1.261.270.0': 6577, '1.225.2540.0': 6578, '1.233.3613.0': 6579, '1.237.823.0': 6580, '1.225.2214.0': 6581, '1.225.1751.0': 6582, '1.225.3963.0': 6583, '1.245.839.0': 6584, '1.241.737.0': 6585, '1.231.682.0': 6586, '1.229.181.0': 6587, '1.225.3401.0': 6588, '1.235.1773.0': 6589, '1.251.1156.0': 6590, '1.233.325.0': 6591, '1.231.1281.0': 6592, '1.241.1244.0': 6593, '1.247.991.0': 6594, '1.231.1882.0': 6595, '1.253.414.0': 6596, '1.251.698.0': 6597, '1.237.829.0': 6598, '1.253.63.0': 6599, '1.235.1538.0': 6600, '1.233.3279.0': 6601, '1.225.1489.0': 6602, '1.263.925.0': 6603, '1.227.2253.0': 6604, '1.233.1547.0': 6605, '1.235.108.0': 6606, '1.249.1269.0': 6607, '1.225.565.0': 6608, '1.251.1373.0': 6609, '1.245.36.0': 6610, '1.231.1587.0': 6611, '1.237.393.0': 6612, '1.259.1233.0': 6613, '1.225.3568.0': 6614, '1.241.290.0': 6615, '1.235.1905.0': 6616, '1.227.1625.0': 6617, '1.233.2589.0': 6618, '1.227.1203.0': 6619, '1.235.1911.0': 6620, '1.227.458.0': 6621, '1.225.3393.0': 6622, '1.225.928.0': 6623, '1.245.369.0': 6624, '1.253.766.0': 6625, '1.251.107.0': 6626, '1.225.3914.0': 6627, '1.233.1438.0': 6628, '1.225.4067.0': 6629, '1.237.15.0': 6630, '1.251.1405.0': 6631, '1.253.470.0': 6632, '1.251.481.0': 6633, '1.251.707.0': 6634, '1.249.1238.0': 6635, '1.247.623.0': 6636, '1.235.1742.0': 6637, '1.237.1209.0': 6638, '1.225.1244.0': 6639, '1.225.1578.0': 6640, '1.239.1462.0': 6641, '1.225.498.0': 6642, '1.227.2640.0': 6643, '1.229.686.0': 6644, '1.223.1159.0': 6645, '1.259.170.0': 6646, '1.225.3561.0': 6647, '1.237.127.0': 6648, '1.259.791.0': 6649, '1.225.3447.0': 6650, '1.233.598.0': 6651, '1.247.956.0': 6652, '1.225.1333.0': 6653, '1.233.4010.0': 6654, '1.235.350.0': 6655, '1.229.172.0': 6656, '1.243.839.0': 6657, '1.247.1076.0': 6658, '1.239.1397.0': 6659, '1.223.2061.0': 6660, '1.249.787.0': 6661, '1.253.469.0': 6662, '1.231.832.0': 6663, '1.233.1996.0': 6664, '1.229.955.0': 6665, '1.249.25.0': 6666, '1.225.2806.0': 6667, '1.243.534.0': 6668, '1.245.792.0': 6669, '1.245.204.0': 6670, '1.247.69.0': 6671, '1.247.398.0': 6672, '1.237.1228.0': 6673, '1.233.3217.0': 6674, '1.229.1637.0': 6675, '1.259.692.0': 6676, '1.229.1136.0': 6677, '1.253.163.0': 6678, '1.249.271.0': 6679, '1.227.275.0': 6680, '1.249.146.0': 6681, '1.241.1257.0': 6682, '1.247.405.0': 6683, '1.227.477.0': 6684, '1.243.872.0': 6685, '1.229.589.0': 6686, '1.253.427.0': 6687, '1.259.960.0': 6688, '1.233.71.0': 6689, '1.245.384.0': 6690, '1.249.1229.0': 6691, '1.247.1007.0': 6692, '1.239.1128.0': 6693, '1.253.74.0': 6694, '1.249.1016.0': 6695, '1.237.546.0': 6696, '1.251.185.0': 6697, '1.233.29.0': 6698, '1.253.358.0': 6699, '1.223.1720.0': 6700, '1.241.800.0': 6701, '1.229.120.0': 6702, '1.251.1326.0': 6703, '1.229.804.0': 6704, '1.229.1047.0': 6705, '1.237.1425.0': 6706, '1.225.3424.0': 6707, '1.233.2750.0': 6708, '1.253.466.0': 6709, '1.193.418.0': 6710, '1.251.421.0': 6711, '1.251.298.0': 6712, '1.241.648.0': 6713, '1.241.274.0': 6714, '1.235.1660.0': 6715, '1.231.199.0': 6716, '1.235.2103.0': 6717, '1.249.22.0': 6718, '1.245.621.0': 6719, '1.253.488.0': 6720, '1.229.666.0': 6721, '1.237.1201.0': 6722, '1.259.724.0': 6723, '1.235.447.0': 6724, '1.235.415.0': 6725, '1.225.1969.0': 6726, '1.231.964.0': 6727, '1.231.1361.0': 6728, '1.235.2151.0': 6729, '1.237.782.0': 6730, '1.233.781.0': 6731, '1.229.299.0': 6732, '1.231.1901.0': 6733, '1.237.459.0': 6734, '1.245.930.0': 6735, '1.225.1119.0': 6736, '1.237.876.0': 6737, '1.251.205.0': 6738, '1.243.711.0': 6739, '1.239.1378.0': 6740, '1.233.4250.0': 6741, '1.245.679.0': 6742, '1.233.1090.0': 6743, '1.231.27.0': 6744, '1.227.2644.0': 6745, '1.229.1943.0': 6746, '1.241.279.0': 6747, '1.231.766.0': 6748, '1.239.1542.0': 6749, '1.225.205.0': 6750, '1.231.1061.0': 6751, '1.237.749.0': 6752, '1.233.3154.0': 6753, '1.237.470.0': 6754, '1.235.3159.0': 6755, '1.249.951.0': 6756, '1.241.1392.0': 6757, '1.237.1407.0': 6758, '1.227.298.0': 6759, '1.225.2359.0': 6760, '1.241.792.0': 6761, '1.233.2244.0': 6762, '1.225.3368.0': 6763, '1.231.1566.0': 6764, '1.249.1233.0': 6765, '1.233.3421.0': 6766, '1.249.1076.0': 6767, '1.227.334.0': 6768, '1.251.802.0': 6769, '1.249.711.0': 6770, '1.239.1590.0': 6771, '1.223.2717.0': 6772, '1.231.1241.0': 6773, '1.239.309.0': 6774, '1.229.674.0': 6775, '1.257.885.0': 6776, '1.247.1034.0': 6777, '1.229.1182.0': 6778, '1.235.2344.0': 6779, '1.225.1824.0': 6780, '1.231.346.0': 6781, '1.225.3258.0': 6782, '1.251.123.0': 6783, '1.247.722.0': 6784, '1.245.448.0': 6785, '1.241.940.0': 6786, '1.225.3897.0': 6787, '1.231.556.0': 6788, '1.247.334.0': 6789, '1.227.2238.0': 6790, '1.233.2927.0': 6791, '1.233.1724.0': 6792, '1.239.201.0': 6793, '1.229.609.0': 6794, '1.235.1343.0': 6795, '1.227.746.0': 6796, '1.255.110.0': 6797, '1.243.529.0': 6798, '1.225.160.0': 6799, '1.245.76.0': 6800, '1.233.4244.0': 6801, '1.253.789.0': 6802, '1.235.1648.0': 6803, '1.233.3413.0': 6804, '1.243.196.0': 6805, '1.233.2473.0': 6806, '1.235.2027.0': 6807, '1.225.2556.0': 6808, '1.227.2306.0': 6809, '1.227.2901.0': 6810, '1.247.876.0': 6811, '1.261.754.0': 6812, '1.235.450.0': 6813, '1.259.779.0': 6814, '1.227.2112.0': 6815, '1.243.303.0': 6816, '1.233.201.0': 6817, '1.259.1543.0': 6818, '1.229.1786.0': 6819, '1.223.2015.0': 6820, '1.227.2830.0': 6821, '1.233.14.0': 6822, '1.233.4256.0': 6823, '1.223.2641.0': 6824, '1.245.515.0': 6825, '1.227.2491.0': 6826, '1.233.1257.0': 6827, '1.239.1599.0': 6828, '1.231.801.0': 6829, '1.233.1349.0': 6830, '1.235.100.0': 6831, '1.251.1026.0': 6832, '1.233.388.0': 6833, '1.249.364.0': 6834, '1.223.2255.0': 6835, '1.233.3703.0': 6836, '1.241.351.0': 6837, '1.233.2236.0': 6838, '1.225.3842.0': 6839, '1.221.334.0': 6840, '1.223.1978.0': 6841, '1.241.698.0': 6842, '1.241.190.0': 6843, '1.247.1058.0': 6844, '1.233.963.0': 6845, '1.259.776.0': 6846, '1.231.525.0': 6847, '1.229.318.0': 6848, '1.227.2825.0': 6849, '1.245.456.0': 6850, '1.253.454.0': 6851, '1.223.2816.0': 6852, '1.233.1294.0': 6853, '1.227.278.0': 6854, '1.247.658.0': 6855, '1.251.1495.0': 6856, '1.235.720.0': 6857, '1.225.24.0': 6858, '1.235.2149.0': 6859, '1.223.2468.0': 6860, '1.233.858.0': 6861, '1.245.116.0': 6862, '1.233.1981.0': 6863, '1.223.1046.0': 6864, '1.249.997.0': 6865, '1.253.474.0': 6866, '1.249.264.0': 6867, '1.249.794.0': 6868, '1.231.1232.0': 6869, '1.227.89.0': 6870, '1.229.1880.0': 6871, '1.223.2317.0': 6872, '1.229.838.0': 6873, '1.239.291.0': 6874, '1.231.757.0': 6875, '1.229.1445.0': 6876, '1.247.886.0': 6877, '1.223.2302.0': 6878, '1.235.1438.0': 6879, '1.245.971.0': 6880, '1.247.85.0': 6881, '1.233.3951.0': 6882, '1.241.1418.0': 6883, '1.229.1193.0': 6884, '1.235.749.0': 6885, '1.233.1921.0': 6886, '1.249.1036.0': 6887, '1.233.3430.0': 6888, '1.245.375.0': 6889, '1.211.2659.0': 6890, '1.225.1722.0': 6891, '1.229.737.0': 6892, '1.233.2851.0': 6893, '1.233.1958.0': 6894, '1.231.1722.0': 6895, '1.251.34.0': 6896, '1.245.504.0': 6897, '1.225.3482.0': 6898, '1.261.733.0': 6899, '1.249.532.0': 6900, '1.243.317.0': 6901, '1.227.202.0': 6902, '1.239.1529.0': 6903, '1.233.4182.0': 6904, '1.227.466.0': 6905, '1.225.2852.0': 6906, '1.235.2106.0': 6907, '1.251.1039.0': 6908, '1.251.1043.0': 6909, '1.261.479.0': 6910, '1.239.774.0': 6911, '1.261.459.0': 6912, '1.227.2262.0': 6913, '1.227.1662.0': 6914, '1.247.718.0': 6915, '1.241.883.0': 6916, '1.235.1380.0': 6917, '1.235.1578.0': 6918, '1.249.1268.0': 6919, '1.229.706.0': 6920, '1.257.710.0': 6921, '1.229.1606.0': 6922, '1.235.1725.0': 6923, '1.227.2618.0': 6924, '1.249.771.0': 6925, '1.233.435.0': 6926, '1.245.667.0': 6927, '1.245.180.0': 6928, '1.241.716.0': 6929, '1.253.79.0': 6930, '1.239.1313.0': 6931, '1.249.86.0': 6932, '1.229.1827.0': 6933, '1.249.783.0': 6934, '1.241.206.0': 6935, '1.227.2742.0': 6936, '1.223.2352.0': 6937, '1.233.3669.0': 6938, '1.225.290.0': 6939, '1.237.188.0': 6940, '1.235.473.0': 6941, '1.235.1598.0': 6942, '1.235.489.0': 6943, '1.259.1577.0': 6944, '1.257.515.0': 6945, '1.231.560.0': 6946, '1.225.2775.0': 6947, '1.225.3323.0': 6948, '1.229.1269.0': 6949, '1.227.2312.0': 6950, '1.225.3807.0': 6951, '1.239.1065.0': 6952, '1.239.1335.0': 6953, '1.225.2984.0': 6954, '1.235.984.0': 6955, '1.227.2732.0': 6956, '1.227.2676.0': 6957, '1.233.3468.0': 6958, '1.235.126.0': 6959, '1.257.1394.0': 6960, '1.233.1307.0': 6961, '1.233.3388.0': 6962, '1.235.224.0': 6963, '1.241.744.0': 6964, '1.251.654.0': 6965, '1.233.1569.0': 6966, '1.239.391.0': 6967, '1.245.423.0': 6968, '1.251.710.0': 6969, '1.251.751.0': 6970, '1.229.508.0': 6971, '1.235.1974.0': 6972, '1.261.510.0': 6973, '1.233.412.0': 6974, '1.247.930.0': 6975, '1.225.3363.0': 6976, '1.235.1209.0': 6977, '1.251.137.0': 6978, '1.233.4279.0': 6979, '1.237.551.0': 6980, '1.237.602.0': 6981, '1.225.3840.0': 6982, '1.237.838.0': 6983, '1.241.825.0': 6984, '1.233.1497.0': 6985, '1.229.2076.0': 6986, '1.231.1172.0': 6987, '1.223.1660.0': 6988, '1.225.1106.0': 6989, '1.237.474.0': 6990, '1.227.943.0': 6991, '1.257.982.0': 6992, '1.233.2579.0': 6993, '1.225.1767.0': 6994, '1.243.396.0': 6995, '1.225.1778.0': 6996, '1.229.790.0': 6997, '1.231.1134.0': 6998, '1.235.202.0': 6999, '1.225.3417.0': 7000, '1.237.1092.0': 7001, '1.241.806.0': 7002, '1.225.323.0': 7003, '1.227.2489.0': 7004, '1.243.203.0': 7005, '1.227.54.0': 7006, '1.235.1575.0': 7007, '1.233.2072.0': 7008, '1.229.52.0': 7009, '1.231.1545.0': 7010, '1.231.820.0': 7011, '1.243.864.0': 7012, '1.231.1886.0': 7013, '1.233.3890.0': 7014, '1.233.2813.0': 7015, '1.237.371.0': 7016, '1.231.433.0': 7017, '1.235.53.0': 7018, '1.253.649.0': 7019, '1.235.302.0': 7020, '1.247.696.0': 7021, '1.227.1102.0': 7022, '1.257.1054.0': 7023, '1.247.920.0': 7024, '1.241.619.0': 7025, '1.225.1683.0': 7026, '1.259.967.0': 7027, '1.225.394.0': 7028, '1.225.2243.0': 7029, '1.227.2159.0': 7030, '1.227.1179.0': 7031, '1.251.202.0': 7032, '1.249.600.0': 7033, '1.233.516.0': 7034, '1.227.1236.0': 7035, '1.225.3541.0': 7036, '1.209.2591.0': 7037, '1.233.3952.0': 7038, '1.245.899.0': 7039, '1.225.3867.0': 7040, '1.227.971.0': 7041, '1.257.1113.0': 7042, '1.237.1138.0': 7043, '1.231.1067.0': 7044, '1.233.4073.0': 7045, '1.231.2177.0': 7046, '1.227.2163.0': 7047, '1.225.3198.0': 7048, '1.223.1683.0': 7049, '1.227.289.0': 7050, '1.231.2040.0': 7051, '1.235.1510.0': 7052, '1.225.2067.0': 7053, '1.227.248.0': 7054, '1.247.913.0': 7055, '1.233.2482.0': 7056, '1.263.194.0': 7057, '1.227.2602.0': 7058, '1.225.222.0': 7059, '1.227.2391.0': 7060, '1.231.1191.0': 7061, '1.241.594.0': 7062, '1.227.1029.0': 7063, '1.249.1357.0': 7064, '1.237.1144.0': 7065, '1.225.1063.0': 7066, '1.227.549.0': 7067, '1.235.149.0': 7068, '1.227.295.0': 7069, '1.263.296.0': 7070, '1.225.301.0': 7071, '1.225.2926.0': 7072, '1.227.1007.0': 7073, '1.235.424.0': 7074, '1.203.1253.0': 7075, '1.235.2219.0': 7076, '1.225.3387.0': 7077, '1.259.122.0': 7078, '1.225.264.0': 7079, '1.229.1814.0': 7080, '1.225.2018.0': 7081, '1.239.1570.0': 7082, '1.227.2120.0': 7083, '1.229.1382.0': 7084, '1.233.4020.0': 7085, '1.239.131.0': 7086, '1.237.629.0': 7087, '1.235.625.0': 7088, '1.227.1619.0': 7089, '1.249.1022.0': 7090, '1.245.220.0': 7091, '1.237.1110.0': 7092, '1.227.296.0': 7093, '1.257.1337.0': 7094, '1.227.1315.0': 7095, '1.231.924.0': 7096, '1.231.1.0': 7097, '1.233.2902.0': 7098, '1.247.878.0': 7099, '1.225.3262.0': 7100, '1.203.1017.0': 7101, '1.199.3431.0': 7102, '1.233.4240.0': 7103, '1.227.1435.0': 7104, '1.233.1893.0': 7105, '1.261.281.0': 7106, '1.247.75.0': 7107, '1.229.633.0': 7108, '1.259.1436.0': 7109, '1.231.780.0': 7110, '1.225.370.0': 7111, '1.229.2060.0': 7112, '1.237.32.0': 7113, '1.239.71.0': 7114, '1.225.2523.0': 7115, '1.235.2142.0': 7116, '1.249.1009.0': 7117, '1.223.2736.0': 7118, '1.241.1099.0': 7119, '1.231.1754.0': 7120, '1.225.3032.0': 7121, '1.233.1301.0': 7122, '1.227.661.0': 7123, '1.233.3641.0': 7124, '1.247.28.0': 7125, '1.225.320.0': 7126, '1.223.948.0': 7127, '1.223.1636.0': 7128, '1.227.1067.0': 7129, '1.235.2007.0': 7130, '1.235.141.0': 7131, '1.247.400.0': 7132, '1.223.788.0': 7133, '1.231.182.0': 7134, '1.227.2415.0': 7135, '1.225.715.0': 7136, '1.223.2689.0': 7137, '1.251.408.0': 7138, '1.231.1793.0': 7139, '1.225.970.0': 7140, '1.241.340.0': 7141, '1.223.1737.0': 7142, '1.225.3222.0': 7143, '1.235.549.0': 7144, '1.233.402.0': 7145, '1.251.1094.0': 7146, '1.231.55.0': 7147, '1.247.535.0': 7148, '1.233.451.0': 7149, '1.223.2360.0': 7150, '1.231.419.0': 7151, '1.237.710.0': 7152, '1.227.594.0': 7153, '1.249.540.0': 7154, '1.235.391.0': 7155, '1.233.809.0': 7156, '1.235.1766.0': 7157, '1.227.1648.0': 7158, '1.243.626.0': 7159, '1.225.2758.0': 7160, '1.233.184.0': 7161, '1.253.912.0': 7162, '1.229.190.0': 7163, '1.245.885.0': 7164, '1.231.137.0': 7165, '1.225.1735.0': 7166, '1.237.1838.0': 7167, '1.233.3784.0': 7168, '1.263.918.0': 7169, '1.225.1583.0': 7170, '1.223.2905.0': 7171, '1.225.1711.0': 7172, '1.241.1117.0': 7173, '1.229.1652.0': 7174, '1.225.1499.0': 7175, '1.239.31.0': 7176, '1.245.55.0': 7177, '1.231.31.0': 7178, '1.229.1859.0': 7179, '1.261.418.0': 7180, '1.223.2766.0': 7181, '1.231.1251.0': 7182, '1.247.409.0': 7183, '1.231.992.0': 7184, '1.239.443.0': 7185, '1.223.2643.0': 7186, '1.225.1429.0': 7187, '1.231.94.0': 7188, '1.235.1499.0': 7189, '1.227.1503.0': 7190, '1.233.3276.0': 7191, '1.259.755.0': 7192, '1.249.713.0': 7193, '1.225.101.0': 7194, '1.227.354.0': 7195, '1.247.83.0': 7196, '1.249.662.0': 7197, '1.225.2780.0': 7198, '1.245.1075.0': 7199, '1.259.460.0': 7200, '1.233.457.0': 7201, '1.225.2181.0': 7202, '1.245.534.0': 7203, '1.237.5.0': 7204, '1.237.72.0': 7205, '1.235.585.0': 7206, '1.231.1919.0': 7207, '1.225.1478.0': 7208, '1.233.3603.0': 7209, '1.239.405.0': 7210, '1.257.1200.0': 7211, '1.235.1968.0': 7212, '1.231.806.0': 7213, '1.237.956.0': 7214, '1.223.1151.0': 7215, '1.225.1133.0': 7216, '1.229.361.0': 7217, '1.249.644.0': 7218, '1.245.227.0': 7219, '1.239.544.0': 7220, '1.249.988.0': 7221, '1.241.1204.0': 7222, '1.231.1690.0': 7223, '1.227.1341.0': 7224, '1.233.3112.0': 7225, '1.239.854.0': 7226, '1.235.497.0': 7227, '1.227.1780.0': 7228, '1.231.249.0': 7229, '1.233.993.0': 7230, '1.235.1801.0': 7231, '1.227.1353.0': 7232, '1.235.1592.0': 7233, '1.251.819.0': 7234, '1.225.3535.0': 7235, '1.223.2903.0': 7236, '1.223.1406.0': 7237, '1.223.985.0': 7238, '1.225.2119.0': 7239, '1.223.2940.0': 7240, '1.243.0.0': 7241, '1.227.1822.0': 7242, '1.233.1372.0': 7243, '1.253.508.0': 7244, '1.249.1000.0': 7245, '1.225.3013.0': 7246, '1.233.4026.0': 7247, '1.253.550.0': 7248, '1.227.1700.0': 7249, '1.225.3411.0': 7250, '1.233.168.0': 7251, '1.249.1246.0': 7252, '1.223.2382.0': 7253, '1.233.3203.0': 7254, '1.233.172.0': 7255, '1.229.1481.0': 7256, '1.235.2016.0': 7257, '1.223.1936.0': 7258, '1.225.3457.0': 7259, '1.261.225.0': 7260, '1.225.2800.0': 7261, '1.225.3005.0': 7262, '1.227.2210.0': 7263, '1.249.111.0': 7264, '1.247.447.0': 7265, '1.233.2019.0': 7266, '1.233.714.0': 7267, '1.239.677.0': 7268, '1.235.113.0': 7269, '1.233.2845.0': 7270, '1.225.2817.0': 7271, '1.245.342.0': 7272, '1.225.610.0': 7273, '1.231.288.0': 7274, '1.235.611.0': 7275, '1.235.3296.0': 7276, '1.225.3066.0': 7277, '1.227.620.0': 7278, '1.223.2126.0': 7279, '1.225.3911.0': 7280, '1.253.482.0': 7281, '1.223.2875.0': 7282, '1.233.4232.0': 7283, '1.235.45.0': 7284, '1.237.1420.0': 7285, '1.243.647.0': 7286, '1.233.2374.0': 7287, '1.263.930.0': 7288, '1.233.795.0': 7289, '1.223.649.0': 7290, '1.229.867.0': 7291, '1.225.2273.0': 7292, '1.225.3436.0': 7293, '1.229.676.0': 7294, '1.249.122.0': 7295, '1.249.100.0': 7296, '1.227.1922.0': 7297, '1.239.1246.0': 7298, '1.245.445.0': 7299, '1.225.2815.0': 7300, '1.241.1157.0': 7301, '1.225.1829.0': 7302, '1.231.1874.0': 7303, '1.239.1649.0': 7304, '1.245.302.0': 7305, '1.235.31.0': 7306, '1.257.795.0': 7307, '1.239.1531.0': 7308, '1.229.1186.0': 7309, '1.225.2192.0': 7310, '1.237.343.0': 7311, '1.229.722.0': 7312, '1.235.1028.0': 7313, '1.229.141.0': 7314, '1.225.3505.0': 7315, '1.243.74.0': 7316, '1.223.1281.0': 7317, '1.227.1577.0': 7318, '1.229.808.0': 7319, '1.233.2771.0': 7320, '1.229.2021.0': 7321, '1.231.1938.0': 7322, '1.245.722.0': 7323, '1.249.96.0': 7324, '1.231.1178.0': 7325, '1.223.2332.0': 7326, '1.253.158.0': 7327, '1.237.1353.0': 7328, '1.233.1967.0': 7329, '1.227.179.0': 7330, '1.229.833.0': 7331, '1.225.2867.0': 7332, '1.231.855.0': 7333, '1.225.2825.0': 7334, '1.241.1104.0': 7335, '1.229.1203.0': 7336, '1.227.1161.0': 7337, '1.233.1619.0': 7338, '1.241.188.0': 7339, '1.225.831.0': 7340, '1.247.397.0': 7341, '1.227.2613.0': 7342, '1.233.595.0': 7343, '1.233.3935.0': 7344, '1.231.1580.0': 7345, '1.233.3409.0': 7346, '1.225.2290.0': 7347, '1.231.179.0': 7348, '1.237.1106.0': 7349, '1.233.181.0': 7350, '1.225.3210.0': 7351, '1.225.3309.0': 7352, '1.249.82.0': 7353, '1.235.1615.0': 7354, '1.231.1486.0': 7355, '1.233.3200.0': 7356, '1.225.997.0': 7357, '1.227.1835.0': 7358, '1.225.351.0': 7359, '1.227.686.0': 7360, '1.227.2229.0': 7361, '1.229.1623.0': 7362, '1.239.129.0': 7363, '1.225.1069.0': 7364, '1.231.541.0': 7365, '1.235.533.0': 7366, '1.225.2493.0': 7367, '1.231.1212.0': 7368, '1.233.2894.0': 7369, '1.251.129.0': 7370, '1.225.1852.0': 7371, '1.225.244.0': 7372, '1.231.1871.0': 7373, '1.239.1445.0': 7374, '1.227.1151.0': 7375, '1.227.2513.0': 7376, '1.227.2319.0': 7377, '1.237.240.0': 7378, '1.227.1550.0': 7379, '1.233.56.0': 7380, '1.231.1677.0': 7381, '1.227.2558.0': 7382, '1.233.350.0': 7383, '1.237.91.0': 7384, '1.229.1854.0': 7385, '1.237.200.0': 7386, '1.231.1831.0': 7387, '1.223.679.0': 7388, '1.249.701.0': 7389, '1.225.3638.0': 7390, '1.225.1665.0': 7391, '1.223.2804.0': 7392, '1.229.377.0': 7393, '1.227.634.0': 7394, '1.243.658.0': 7395, '1.225.2332.0': 7396, '1.225.3501.0': 7397, '1.245.128.0': 7398, '1.251.468.0': 7399, '1.233.3576.0': 7400, '1.227.266.0': 7401, '1.251.354.0': 7402, '1.231.828.0': 7403, '1.233.2797.0': 7404, '1.243.167.0': 7405, '1.229.1518.0': 7406, '1.235.828.0': 7407, '1.239.581.0': 7408, '1.225.2225.0': 7409, '1.227.1378.0': 7410, '1.223.2635.0': 7411, '1.233.4088.0': 7412, '1.235.187.0': 7413, '1.225.2113.0': 7414, '1.225.1436.0': 7415, '1.249.871.0': 7416, '1.259.1353.0': 7417, '1.231.514.0': 7418, '1.243.937.0': 7419, '1.223.2319.0': 7420, '1.225.3880.0': 7421, '1.225.2133.0': 7422, '1.231.304.0': 7423, '1.223.2489.0': 7424, '1.243.268.0': 7425, '1.227.385.0': 7426, '1.233.1759.0': 7427, '1.245.848.0': 7428, '1.235.1735.0': 7429, '1.235.1530.0': 7430, '1.225.2125.0': 7431, '1.239.300.0': 7432, '1.231.1865.0': 7433, '1.225.4121.0': 7434, '1.225.2740.0': 7435, '1.223.2694.0': 7436, '1.227.349.0': 7437, '1.225.78.0': 7438, '1.225.2798.0': 7439, '1.249.725.0': 7440, '1.241.667.0': 7441, '1.235.2516.0': 7442, '1.229.1509.0': 7443, '1.235.1552.0': 7444, '1.227.2219.0': 7445, '1.225.1728.0': 7446, '1.241.1002.0': 7447, '1.221.510.0': 7448, '1.223.2935.0': 7449, '1.247.298.0': 7450, '1.237.266.0': 7451, '1.247.728.0': 7452, '1.225.3620.0': 7453, '1.229.771.0': 7454, '1.237.1305.0': 7455, '1.233.2064.0': 7456, '1.225.1863.0': 7457, '1.245.1162.0': 7458, '1.235.1727.0': 7459, '1.229.138.0': 7460, '1.227.2350.0': 7461, '1.233.194.0': 7462, '1.231.224.0': 7463, '1.235.2096.0': 7464, '1.225.897.0': 7465, '1.201.2151.0': 7466, '1.239.122.0': 7467, '1.235.2126.0': 7468, '1.225.2707.0': 7469, '1.215.2092.0': 7470, '1.225.629.0': 7471, '1.229.881.0': 7472, '1.225.1900.0': 7473, '1.227.23.0': 7474, '1.237.1674.0': 7475, '1.261.421.0': 7476, '1.225.3804.0': 7477, '1.231.1944.0': 7478, '1.225.3858.0': 7479, '1.231.271.0': 7480, '1.227.482.0': 7481, '1.233.814.0': 7482, '1.249.699.0': 7483, '1.225.2477.0': 7484, '1.235.2025.0': 7485, '1.247.395.0': 7486, '1.223.1671.0': 7487, '1.231.734.0': 7488, '1.235.395.0': 7489, '1.251.347.0': 7490, '1.231.794.0': 7491, '1.229.202.0': 7492, '1.233.3272.0': 7493, '1.223.2708.0': 7494, '1.245.344.0': 7495, '1.239.1582.0': 7496, '1.231.519.0': 7497, '1.237.619.0': 7498, '1.237.175.0': 7499, '1.227.2770.0': 7500, '1.225.2146.0': 7501, '1.241.1334.0': 7502, '1.237.1298.0': 7503, '1.223.1824.0': 7504, '1.235.2265.0': 7505, '1.247.1070.0': 7506, '1.227.1228.0': 7507, '1.225.1627.0': 7508, '1.227.1695.0': 7509, '1.231.320.0': 7510, '1.237.1078.0': 7511, '1.215.422.0': 7512, '1.247.715.0': 7513, '1.201.224.0': 7514, '1.227.123.0': 7515, '1.235.2271.0': 7516, '1.247.392.0': 7517, '1.225.2842.0': 7518, '1.225.1213.0': 7519, '1.225.2857.0': 7520, '1.223.2724.0': 7521, '1.239.533.0': 7522, '1.235.2131.0': 7523, '1.217.1832.0': 7524, '1.231.802.0': 7525, '1.239.1058.0': 7526, '1.227.2764.0': 7527, '1.223.1443.0': 7528, '1.227.1393.0': 7529, '1.231.887.0': 7530, '1.227.2398.0': 7531, '1.253.705.0': 7532, '1.223.124.0': 7533, '1.227.2666.0': 7534, '1.229.1947.0': 7535, '1.243.998.0': 7536, '1.225.1014.0': 7537, '1.225.183.0': 7538, '1.223.839.0': 7539, '1.227.432.0': 7540, '1.231.1167.0': 7541, '1.235.1374.0': 7542, '1.227.2841.0': 7543, '1.227.274.0': 7544, '1.235.1794.0': 7545, '1.227.785.0': 7546, '1.231.1182.0': 7547, '1.225.2396.0': 7548, '1.241.1263.0': 7549, '1.221.390.0': 7550, '1.223.1716.0': 7551, '1.223.2055.0': 7552, '1.233.3427.0': 7553, '1.229.2027.0': 7554, '1.225.3887.0': 7555, '1.227.2046.0': 7556, '1.225.727.0': 7557, '1.259.42.0': 7558, '1.227.108.0': 7559, '1.225.2827.0': 7560, '1.225.1421.0': 7561, '1.223.1424.0': 7562, '1.259.100.0': 7563, '1.237.499.0': 7564, '1.225.2661.0': 7565, '1.233.1369.0': 7566, '1.259.723.0': 7567, '1.225.1671.0': 7568, '1.235.562.0': 7569, '1.251.1399.0': 7570, '1.223.2623.0': 7571, '1.261.391.0': 7572, '1.225.241.0': 7573, '1.223.2163.0': 7574, '1.247.436.0': 7575, '1.195.3092.0': 7576, '1.223.2928.0': 7577, '1.241.1366.0': 7578, '1.237.1613.0': 7579, '1.229.844.0': 7580, '1.241.186.0': 7581, '1.227.325.0': 7582, '1.231.274.0': 7583, '1.223.1793.0': 7584, '1.229.346.0': 7585, '1.223.1665.0': 7586, '1.233.1739.0': 7587, '1.231.1825.0': 7588, '1.229.122.0': 7589, '1.247.68.0': 7590, '1.231.1520.0': 7591, '1.155.266.0': 7592, '1.231.234.0': 7593, '1.231.311.0': 7594, '1.233.2860.0': 7595, '1.225.1149.0': 7596, '1.237.374.0': 7597, '1.225.258.0': 7598, '1.245.1147.0': 7599, '1.231.896.0': 7600, '1.225.606.0': 7601, '1.223.1180.0': 7602, '1.225.356.0': 7603, '1.223.1010.0': 7604, '1.231.219.0': 7605, '1.225.345.0': 7606, '1.225.3165.0': 7607, '1.225.2678.0': 7608, '1.223.2704.0': 7609, '1.233.887.0': 7610, '1.227.893.0': 7611, '1.229.453.0': 7612, '1.223.15.0': 7613, '1.239.887.0': 7614, '1.227.2066.0': 7615, '1.225.3467.0': 7616, '1.197.52.0': 7617, '1.225.2937.0': 7618, '1.235.383.0': 7619, '1.235.65.0': 7620, '1.223.1597.0': 7621, '1.231.1082.0': 7622, '1.225.2640.0': 7623, '1.225.2513.0': 7624, '1.243.308.0': 7625, '1.251.130.0': 7626, '1.231.1221.0': 7627, '1.223.2752.0': 7628, '1.231.1500.0': 7629, '1.229.1550.0': 7630, '1.243.851.0': 7631, '1.227.1330.0': 7632, '1.237.1432.0': 7633, '1.227.700.0': 7634, '1.225.1248.0': 7635, '1.235.1512.0': 7636, '1.225.950.0': 7637, '1.229.776.0': 7638, '1.223.1648.0': 7639, '1.229.207.0': 7640, '1.227.2578.0': 7641, '1.231.1324.0': 7642, '1.233.2263.0': 7643, '1.229.475.0': 7644, '1.225.1317.0': 7645, '1.235.2352.0': 7646, '1.237.123.0': 7647, '1.237.885.0': 7648, '1.227.2462.0': 7649, '1.225.3300.0': 7650, '1.223.2440.0': 7651, '1.237.686.0': 7652, '1.235.2095.0': 7653, '1.261.643.0': 7654, '1.243.871.0': 7655, '1.227.2637.0': 7656, '1.241.1271.0': 7657, '1.229.1031.0': 7658, '1.229.1034.0': 7659, '1.249.488.0': 7660, '1.231.2065.0': 7661, '1.233.2014.0': 7662, '1.227.2715.0': 7663, '1.233.3649.0': 7664, '1.227.711.0': 7665, '1.223.1259.0': 7666, '1.223.2743.0': 7667, '1.237.733.0': 7668, '1.239.196.0': 7669, '1.223.667.0': 7670, '1.229.1027.0': 7671, '1.229.1806.0': 7672, '1.233.3266.0': 7673, '1.247.584.0': 7674, '1.229.870.0': 7675, '1.231.893.0': 7676, '1.207.1891.0': 7677, '1.223.1755.0': 7678, '1.225.334.0': 7679, '1.243.1020.0': 7680, '1.231.627.0': 7681, '1.249.88.0': 7682, '1.233.2880.0': 7683, '1.225.3478.0': 7684, '1.223.2198.0': 7685, '1.229.1997.0': 7686, '1.245.29.0': 7687, '1.247.927.0': 7688, '1.225.1092.0': 7689, '1.239.600.0': 7690, '1.241.506.0': 7691, '1.229.1633.0': 7692, '1.245.608.0': 7693, '1.231.1357.0': 7694, '1.239.1536.0': 7695, '1.225.4084.0': 7696, '1.227.2687.0': 7697, '1.227.2620.0': 7698, '1.231.175.0': 7699, '1.243.436.0': 7700, '1.235.500.0': 7701, '1.225.3440.0': 7702, '1.229.1570.0': 7703, '1.233.46.0': 7704, '1.231.2044.0': 7705, '1.227.2280.0': 7706, '1.237.1089.0': 7707, '1.225.1817.0': 7708, '1.225.1954.0': 7709, '1.223.2791.0': 7710, '1.239.539.0': 7711, '1.227.1818.0': 7712, '1.243.797.0': 7713, '1.231.930.0': 7714, '1.233.652.0': 7715, '1.237.555.0': 7716, '1.227.726.0': 7717, '1.229.1475.0': 7718, '1.239.530.0': 7719, '1.259.91.0': 7720, '1.225.2302.0': 7721, '1.227.394.0': 7722, '1.231.7.0': 7723, '1.225.3824.0': 7724, '1.229.340.0': 7725, '1.225.3486.0': 7726, '1.225.3378.0': 7727, '1.233.38.0': 7728, '1.225.1883.0': 7729, '1.241.637.0': 7730, '1.231.1268.0': 7731, '1.241.157.0': 7732, '1.225.3354.0': 7733, '1.225.1655.0': 7734, '1.235.456.0': 7735, '1.231.2033.0': 7736, '1.233.2756.0': 7737, '1.233.131.0': 7738, '1.237.429.0': 7739, '1.225.1305.0': 7740, '1.231.1684.0': 7741, '1.243.157.0': 7742, '1.239.217.0': 7743, '1.239.1053.0': 7744, '1.223.489.0': 7745, '1.223.2839.0': 7746, '1.229.1393.0': 7747, '1.227.1470.0': 7748, '1.225.3203.0': 7749, '1.229.1797.0': 7750, '1.237.193.0': 7751, '1.225.1965.0': 7752, '1.229.530.0': 7753, '1.225.2864.0': 7754, '1.223.2506.0': 7755, '1.231.61.0': 7756, '1.221.457.0': 7757, '1.225.2958.0': 7758, '1.225.659.0': 7759, '1.225.3446.0': 7760, '1.231.1841.0': 7761, '1.227.150.0': 7762, '1.225.4087.0': 7763, '1.231.1750.0': 7764, '1.227.1413.0': 7765, '1.227.1605.0': 7766, '1.225.2859.0': 7767, '1.223.1185.0': 7768, '1.227.2297.0': 7769, '1.243.926.0': 7770, '1.233.1929.0': 7771, '1.247.157.0': 7772, '1.229.126.0': 7773, '1.225.268.0': 7774, '1.231.779.0': 7775, '1.233.1627.0': 7776, '1.231.107.0': 7777, '1.237.134.0': 7778, '1.241.345.0': 7779, '1.225.2763.0': 7780, '1.225.4098.0': 7781, '1.235.2145.0': 7782, '1.247.941.0': 7783, '1.227.551.0': 7784, '1.227.763.0': 7785, '1.225.1840.0': 7786, '1.225.2196.0': 7787, '1.251.1401.0': 7788, '1.223.992.0': 7789, '1.235.2138.0': 7790, '1.227.2926.0': 7791, '1.225.2311.0': 7792, '1.229.1594.0': 7793, '1.247.356.0': 7794, '1.225.630.0': 7795, '1.233.3227.0': 7796, '1.245.112.0': 7797, '1.253.524.0': 7798, '1.229.313.0': 7799, '1.225.2107.0': 7800, '1.227.313.0': 7801, '1.223.804.0': 7802, '1.225.3929.0': 7803, '1.231.1737.0': 7804, '1.225.3599.0': 7805, '1.233.3403.0': 7806, '1.239.512.0': 7807, '1.233.4158.0': 7808, '1.225.3514.0': 7809, '1.225.81.0': 7810, '1.227.2574.0': 7811, '1.233.3817.0': 7812, '1.227.2099.0': 7813, '1.237.281.0': 7814, '1.225.315.0': 7815, '1.237.1401.0': 7816, '1.227.2710.0': 7817, '1.237.212.0': 7818, '1.225.154.0': 7819, '1.223.2029.0': 7820, '1.223.1849.0': 7821, '1.227.752.0': 7822, '1.231.2134.0': 7823, '1.225.2327.0': 7824, '1.253.503.0': 7825, '1.235.476.0': 7826, '1.229.949.0': 7827, '1.231.761.0': 7828, '1.225.3850.0': 7829, '1.229.168.0': 7830, '1.229.404.0': 7831, '1.225.2391.0': 7832, '1.225.1959.0': 7833, '1.225.2902.0': 7834, '1.225.3926.0': 7835, '1.225.2336.0': 7836, '1.225.2348.0': 7837, '1.225.919.0': 7838, '1.263.989.0': 7839, '1.227.2662.0': 7840, '1.223.2894.0': 7841, '1.225.2406.0': 7842, '1.231.1766.0': 7843, '1.227.316.0': 7844, '1.223.2169.0': 7845, '1.233.2105.0': 7846, '1.261.584.0': 7847, '1.229.160.0': 7848, '1.223.2784.0': 7849, '1.237.22.0': 7850, '1.231.1352.0': 7851, '1.169.1625.0': 7852, '1.231.1222.0': 7853, '1.225.570.0': 7854, '1.229.2014.0': 7855, '1.233.155.0': 7856, '1.227.2649.0': 7857, '1.225.3573.0': 7858, '1.233.2955.0': 7859, '1.213.5578.0': 7860, '1.223.1809.0': 7861, '1.225.2339.0': 7862, '1.231.36.0': 7863, '1.237.413.0': 7864, '1.233.2562.0': 7865, '1.229.304.0': 7866, '1.227.2420.0': 7867, '1.223.505.0': 7868, '1.235.1000.0': 7869, '1.227.1558.0': 7870, '1.229.513.0': 7871, '1.209.3251.0': 7872, '1.245.728.0': 7873, '1.225.3275.0': 7874, '1.223.844.0': 7875, '1.227.1196.0': 7876, '1.231.1306.0': 7877, '1.229.1662.0': 7878, '1.231.223.0': 7879, '1.233.1777.0': 7880, '1.237.894.0': 7881, '1.225.2666.0': 7882, '1.229.829.0': 7883, '1.227.1391.0': 7884, '1.225.1506.0': 7885, '1.229.1831.0': 7886, '1.225.2752.0': 7887, '1.245.1143.0': 7888, '1.225.985.0': 7889, '1.227.293.0': 7890, '1.233.2644.0': 7891, '1.233.2820.0': 7892, '1.235.2054.0': 7893, '1.233.356.0': 7894, '1.223.1137.0': 7895, '1.229.1080.0': 7896, '1.227.307.0': 7897, '1.225.2684.0': 7898, '1.223.2207.0': 7899, '1.235.3121.0': 7900, '1.225.237.0': 7901, '1.233.587.0': 7902, '1.235.1708.0': 7903, '1.227.779.0': 7904, '1.247.936.0': 7905, '1.227.2094.0': 7906, '1.231.1698.0': 7907, '1.231.922.0': 7908, '1.237.409.0': 7909, '1.233.3644.0': 7910, '1.229.982.0': 7911, '1.235.677.0': 7912, '1.225.736.0': 7913, '1.227.2694.0': 7914, '1.223.2531.0': 7915, '1.231.1216.0': 7916, '1.235.3050.0': 7917, '1.223.2184.0': 7918, '1.227.2552.0': 7919, '1.229.860.0': 7920, '1.229.947.0': 7921, '1.231.2160.0': 7922, '1.229.1281.0': 7923, '1.225.3489.0': 7924, '1.231.172.0': 7925, '1.225.110.0': 7926, '1.225.2258.0': 7927, '1.229.1848.0': 7928, '1.225.1522.0': 7929, '1.211.1981.0': 7930, '1.225.272.0': 7931, '1.235.2960.0': 7932, '1.247.381.0': 7933, '1.227.229.0': 7934, '1.223.2853.0': 7935, '1.223.1752.0': 7936, '1.225.279.0': 7937, '1.225.4061.0': 7938, '1.237.352.0': 7939, '1.233.914.0': 7940, '1.233.460.0': 7941, '1.229.1147.0': 7942, '1.229.637.0': 7943, '1.227.2242.0': 7944, '1.225.288.0': 7945, '1.225.2794.0': 7946, '1.225.398.0': 7947, '1.223.1365.0': 7948, '1.223.2471.0': 7949, '1.233.1287.0': 7950, '1.235.156.0': 7951, '1.223.2619.0': 7952, '1.235.960.0': 7953, '1.229.294.0': 7954, '1.205.1689.0': 7955, '1.245.1054.0': 7956, '1.231.293.0': 7957, '1.213.5318.0': 7958, '1.235.3177.0': 7959, '1.233.1845.0': 7960, '1.229.1296.0': 7961, '1.235.3340.0': 7962, '1.223.1800.0': 7963, '1.231.1258.0': 7964, '1.229.1301.0': 7965, '1.233.2888.0': 7966, '1.229.151.0': 7967, '1.223.2155.0': 7968, '1.239.1022.0': 7969, '1.243.717.0': 7970, '1.223.2923.0': 7971, '1.227.1493.0': 7972, '1.245.152.0': 7973, '1.233.4151.0': 7974, '1.239.126.0': 7975, '1.225.749.0': 7976, '1.223.2916.0': 7977, '1.225.527.0': 7978, '1.227.2092.0': 7979, '1.231.837.0': 7980, '1.227.2466.0': 7981, '1.227.2806.0': 7982, '1.233.1253.0': 7983, '1.231.1705.0': 7984, '1.231.2183.0': 7985, '1.225.517.0': 7986, '1.231.890.0': 7987, '1.225.4039.0': 7988, '1.249.181.0': 7989, '1.225.2695.0': 7990, '1.231.2147.0': 7991, '1.229.247.0': 7992, '1.225.1155.0': 7993, '1.225.2260.0': 7994, '1.225.2916.0': 7995, '1.227.2538.0': 7996, '1.229.115.0': 7997, '1.227.2791.0': 7998, '1.223.800.0': 7999, '1.233.80.0': 8000, '1.237.832.0': 8001, '1.225.2954.0': 8002, '1.227.2920.0': 8003, '1.225.2242.0': 8004, '1.225.3642.0': 8005, '1.233.342.0': 8006, '1.225.3525.0': 8007, '1.237.504.0': 8008, '1.225.67.0': 8009, '1.229.912.0': 8010, '1.235.484.0': 8011, '1.213.5655.0': 8012, '1.225.3334.0': 8013, '1.227.2909.0': 8014, '1.227.1469.0': 8015, '1.223.523.0': 8016, '1.233.3942.0': 8017, '1.213.790.0': 8018, '1.225.3835.0': 8019, '1.229.394.0': 8020, '1.245.1097.0': 8021, '1.227.2542.0': 8022, '1.233.202.0': 8023, '1.227.2624.0': 8024, '1.223.2738.0': 8025, '1.225.3848.0': 8026, '1.215.3020.0': 8027, '1.225.298.0': 8028, '1.227.2548.0': 8029, '1.235.1547.0': 8030, '1.229.1888.0': 8031, '1.213.3329.0': 8032, '1.225.1718.0': 8033, '1.223.2835.0': 8034, '1.233.3764.0': 8035, '1.239.1642.0': 8036, '1.223.1292.0': 8037, '1.229.327.0': 8038, '1.227.288.0': 8039, '1.225.624.0': 8040, '1.195.3225.0': 8041, '1.227.1001.0': 8042, '1.237.650.0': 8043, '1.223.810.0': 8044, '1.235.1007.0': 8045, '1.235.1328.0': 8046, '1.223.1743.0': 8047, '1.227.14.0': 8048, '1.203.961.0': 8049, '1.225.94.0': 8050, '1.233.480.0': 8051, '1.235.2160.0': 8052, '1.221.699.0': 8053, '1.225.1937.0': 8054, '1.225.3350.0': 8055, '1.223.1533.0': 8056, '1.227.95.0': 8057, '1.225.3893.0': 8058, '1.241.1342.0': 8059, '1.225.3187.0': 8060, '1.229.1195.0': 8061, '1.225.1047.0': 8062, '1.233.474.0': 8063, '1.229.617.0': 8064, '1.227.1836.0': 8065, '1.223.739.0': 8066, '1.227.623.0': 8067, '1.225.1322.0': 8068, '1.231.723.0': 8069, '1.225.3059.0': 8070, '1.231.643.0': 8071, '1.207.853.0': 8072, '1.233.4205.0': 8073, '1.237.1547.0': 8074, '1.223.699.0': 8075, '1.229.962.0': 8076, '1.227.2225.0': 8077, '1.213.1546.0': 8078, '1.225.3556.0': 8079, '1.213.3699.0': 8080, '1.219.115.0': 8081, '1.223.1412.0': 8082, '1.225.2054.0': 8083, '1.227.2508.0': 8084, '1.229.2032.0': 8085, '1.223.2006.0': 8086, '1.225.1600.0': 8087, '1.225.1908.0': 8088, '1.235.680.0': 8089, '1.203.928.0': 8090, '1.223.1020.0': 8091, '1.225.574.0': 8092, '1.223.2388.0': 8093, '1.247.631.0': 8094, '1.225.250.0': 8095, '1.225.1226.0': 8096, '1.223.1712.0': 8097, '1.227.680.0': 8098, '1.227.2802.0': 8099, '1.227.113.0': 8100, '1.225.73.0': 8101, '1.225.283.0': 8102, '1.257.30.0': 8103, '1.225.1634.0': 8104, '1.231.413.0': 8105, '1.233.3926.0': 8106, '1.193.1928.0': 8107, '1.225.2453.0': 8108, '1.225.1169.0': 8109, '1.243.665.0': 8110, '1.231.1271.0': 8111, '1.253.516.0': 8112, '1.223.1193.0': 8113, '1.229.535.0': 8114, '1.227.738.0': 8115, '1.225.2992.0': 8116, '1.229.136.0': 8117, '1.223.959.0': 8118, '1.231.2168.0': 8119, '1.223.1264.0': 8120, '1.247.714.0': 8121, '1.219.254.0': 8122, '1.225.2505.0': 8123, '1.227.542.0': 8124, '1.233.2911.0': 8125, '1.231.391.0': 8126, '1.225.418.0': 8127, '1.229.2042.0': 8128, '1.231.873.0': 8129, '1.231.1816.0': 8130, '1.233.1386.0': 8131, '1.239.987.0': 8132, '1.225.1896.0': 8133, '1.225.766.0': 8134, '1.229.663.0': 8135, '1.225.423.0': 8136, '1.231.777.0': 8137, '1.229.59.0': 8138, '1.229.754.0': 8139, '1.225.2223.0': 8140, '1.225.2811.0': 8141, '1.229.641.0': 8142, '1.225.338.0': 8143, '1.225.1127.0': 8144, '1.229.413.0': 8145, '1.227.370.0': 8146, '1.213.5826.0': 8147, '1.225.456.0': 8148, '1.225.2847.0': 8149, '1.195.109.0': 8150, '1.233.3701.0': 8151, '1.225.1163.0': 8152, '1.225.913.0': 8153, '1.223.1112.0': 8154, '1.227.2333.0': 8155, '1.223.2610.0': 8156, '1.225.4135.0': 8157, '1.237.419.0': 8158, '1.225.3545.0': 8159, '1.227.1359.0': 8160, '1.223.2550.0': 8161, '1.205.2185.0': 8162, '1.233.1360.0': 8163, '1.227.283.0': 8164, '1.227.2608.0': 8165, '1.203.125.0': 8166, '1.225.2251.0': 8167, '1.231.97.0': 8168, '1.227.2146.0': 8169, '1.227.193.0': 8170, '1.223.1062.0': 8171, '1.227.1495.0': 8172, '1.225.1688.0': 8173, '1.225.883.0': 8174, '1.227.304.0': 8175, '1.225.1454.0': 8176, '1.225.140.0': 8177, '1.199.2708.0': 8178, '1.225.1638.0': 8179, '1.225.3050.0': 8180, '1.229.1287.0': 8181, '1.223.2661.0': 8182, '1.197.1237.0': 8183, '1.225.48.0': 8184, '1.227.2233.0': 8185, '1.225.2818.0': 8186, '1.223.1109.0': 8187, '1.227.0.0': 8188, '1.237.108.0': 8189, '1.225.2268.0': 8190, '1.233.2948.0': 8191, '1.223.1344.0': 8192, '1.225.2233.0': 8193, '1.223.2775.0': 8194, '1.223.2223.0': 8195, '1.227.404.0': 8196, '1.223.2527.0': 8197, '1.223.1470.0': 8198, '1.225.2131.0': 8199, '1.225.2343.0': 8200, '1.223.1983.0': 8201, '1.223.2730.0': 8202, '1.229.9.0': 8203, '1.231.936.0': 8204, '1.227.921.0': 8205, '1.225.2323.0': 8206, '1.225.3626.0': 8207, '1.235.2018.0': 8208, '1.229.146.0': 8209, '1.225.2931.0': 8210, '1.227.173.0': 8211, '1.227.959.0': 8212, '1.223.934.0': 8213, '1.225.1611.0': 8214, '1.223.623.0': 8215, '1.227.376.0': 8216, '1.225.2438.0': 8217, '1.257.465.0': 8218, '1.225.2856.0': 8219, '1.231.69.0': 8220, '1.225.227.0': 8221, '1.235.23.0': 8222, '1.225.2822.0': 8223, '1.225.3462.0': 8224, '1.223.186.0': 8225, '1.223.208.0': 8226, '1.227.1253.0': 8227, '1.227.120.0': 8228, '1.225.4145.0': 8229, '1.223.2553.0': 8230, '1.223.1440.0': 8231, '1.225.1947.0': 8232, '1.225.3550.0': 8233, '1.223.1473.0': 8234, '1.227.2503.0': 8235, '1.227.1484.0': 8236, '1.245.1152.0': 8237, '1.229.384.0': 8238, '1.219.420.0': 8239, '1.223.54.0': 8240, '1.237.495.0': 8241, '1.233.637.0': 8242, '1.219.1130.0': 8243, '1.215.2252.0': 8244, '1.225.306.0': 8245, '1.233.1564.0': 8246, '1.231.1047.0': 8247, '1.213.451.0': 8248, '1.253.495.0': 8249, '1.223.2858.0': 8250, '1.229.422.0': 8251, '1.219.1310.0': 8252, '1.223.1427.0': 8253, '1.233.3804.0': 8254, '1.225.3871.0': 8255, '1.213.2626.0': 8256, '1.213.6352.0': 8257, '1.229.1387.0': 8258, '1.223.1991.0': 8259, '1.199.3370.0': 8260, '1.225.1028.0': 8261, '1.231.1256.0': 8262, '1.231.343.0': 8263, '1.227.1514.0': 8264, '1.231.1890.0': 8265, '1.225.3077.0': 8266, '1.227.985.0': 8267, '1.199.2088.0': 8268, '1.235.3019.0': 8269, '1.225.2500.0': 8270, '1.237.1373.0': 8271, '1.227.76.0': 8272, '1.225.2589.0': 8273, '1.221.439.0': 8274, '1.225.559.0': 8275, '1.239.837.0': 8276, '1.223.798.0': 8277, '1.221.606.0': 8278, '1.227.380.0': 8279, '1.231.1291.0': 8280, '1.213.2121.0': 8281, '1.197.2843.0': 8282, '1.223.2313.0': 8283, '1.223.2249.0': 8284, '1.229.91.0': 8285, '1.221.299.0': 8286, '1.225.4128.0': 8287, '1.233.3205.0': 8288, '1.231.423.0': 8289, '1.223.2519.0': 8290, '1.223.1351.0': 8291, '1.203.1944.0': 8292, '1.223.145.0': 8293, '1.223.2538.0': 8294, '1.223.1093.0': 8295, '1.229.976.0': 8296, '1.239.135.0': 8297, '1.203.933.0': 8298, '1.209.466.0': 8299, '1.225.2786.0': 8300, '1.213.2558.0': 8301, '1.223.2912.0': 8302, '1.225.275.0': 8303, '1.219.58.0': 8304, '1.221.745.0': 8305, '1.223.2658.0': 8306, '1.217.1446.0': 8307, '1.223.927.0': 8308, '1.223.2764.0': 8309, '1.223.2393.0': 8310, '1.219.1500.0': 8311, '1.223.1465.0': 8312, '1.245.1138.0': 8313, '1.209.1587.0': 8314, '1.215.104.0': 8315, '1.201.395.0': 8316, '1.225.1678.0': 8317, '1.231.426.0': 8318, '1.233.148.0': 8319, '1.219.1707.0': 8320, '1.223.1927.0': 8321, '1.217.1180.0': 8322, '1.223.1506.0': 8323, '1.223.2614.0': 8324, '1.225.2655.0': 8325, '1.223.1429.0': 8326, '1.223.755.0': 8327, '1.225.233.0': 8328, '1.207.1058.0': 8329, '1.225.1857.0': 8330, '1.223.2670.0': 8331, '1.225.1042.0': 8332, '1.225.405.0': 8333, '1.213.1070.0': 8334, '1.233.3353.0': 8335, '1.213.4923.0': 8336, '1.231.18.0': 8337, '1.223.2460.0': 8338, '1.223.1318.0': 8339, '1.223.2451.0': 8340, '1.223.2722.0': 8341, '1.223.1049.0': 8342, '1.235.1704.0': 8343, '1.225.3564.0': 8344, '1.225.55.0': 8345, '1.197.2142.0': 8346, '1.225.216.0': 8347, '1.227.820.0': 8348, '1.231.770.0': 8349, '1.223.1276.0': 8350, '1.227.964.0': 8351, '1.223.1056.0': 8352, '1.195.2226.0': 8353, '1.227.706.0': 8354, '1.223.692.0': 8355, '1.213.3536.0': 8356, '1.227.285.0': 8357, '1.241.111.0': 8358, '1.223.1997.0': 8359, '1.195.1107.0': 8360, '1.223.1360.0': 8361, '1.207.1367.0': 8362, '1.223.830.0': 8363, '1.229.333.0': 8364, '1.225.3359.0': 8365, '1.227.773.0': 8366, '1.227.326.0': 8367, '1.207.405.0': 8368, '1.189.189.0': 8369, '1.223.1492.0': 8370, '1.223.127.0': 8371, '1.225.3270.0': 8372, '1.233.539.0': 8373, '1.229.186.0': 8374, '1.187.1121.0': 8375, '1.223.2864.0': 8376, '1.223.791.0': 8377, '1.217.477.0': 8378, '1.2&#x17;3.1144.0': 8379, '1.225.0.0': 8380, '1.229.222.0': 8381, '1.219.2167.0': 8382, '1.235.3345.0': 8383, '1.215.1327.0': 8384, '1.223.1266.0': 8385, '1.225.41.0': 8386, '1.207.1186.0': 8387, '1.229.338.0': 8388, '1.223.370.0': 8389, '1.233.707.0': 8390, '1.225.3193.0': 8391, '1.223.2606.0': 8392, '1.225.294.0': 8393, '1.223.196.0': 8394, '1.211.1734.0': 8395, '1.249.83.0': 8396, '1.247.708.0': 8397, '1.231.1518.0': 8398, '1.225.4108.0': 8399, '1.217.1034.0': 8400, '1.223.1342.0': 8401, '1.213.5059.0': 8402, '1.215.2506.0': 8403, '1.217.599.0': 8404, '1.205.2121.0': 8405, '1.207.467.0': 8406, '1.227.291.0': 8407, '1.213.7751.0': 8408, '1.231.1301.0': 8409, '1.223.1101.0': 8410, '1.217.613.0': 8411, '1.227.2595.0': 8412, '1.213.7527.0': 8413, '1.223.1400.0': 8414, '1.215.1582.0': 8415, '1.227.2097.0': 8416, '1.193.185.0': 8417, '1.223.2454.0': 8418, '1.213.1796.0': 8419, '1.219.485.0': 8420, '1.223.751.0': 8421, '1.223.2646.0': 8422, '1.225.2284.0': 8423, '1.217.1366.0': 8424, '1.223.2781.0': 8425, '1.223.1433.0': 8426, '1.261.180.0': 8427, '1.223.2684.0': 8428, '1.227.2042.0': 8429, '1.207.3631.0': 8430, '1.223.2240.0': 8431, '1.223.1002.0': 8432, '1.227.2038.0': 8433, '1.225.1921.0': 8434, '1.225.1617.0': 8435, '1.221.714.0': 8436, '1.225.3877.0': 8437, '1.223.834.0': 8438, '1.223.713.0': 8439, '1.227.1555.0': 8440, '1.225.2850.0': 8441, '1.213.3131.0': 8442, '1.217.1755.0': 8443, '1.213.2997.0': 8444, '1.225.3021.0': 8445, '1.223.1127.0': 8446, '1.207.1618.0': 8447, '1.223.1376.0': 8448, '1.223.2711.0': 8449, '1.197.2441.0': 8450, '1.239.1540.0': 8451, '1.223.1419.0': 8452, '1.223.1338.0': 8453, '1.225.2858.0': 8454, '1.205.391.0': 8455, '1.215.1710.0': 8456, '1.245.13.0': 8457, '1.223.742.0': 8458, '1.223.1459.0': 8459, '1.207.2253.0': 8460, '1.207.3625.0': 8461, '1.237.316.0': 8462, '1.215.2461.0': 8463, '1.219.1822.0': 8464, '1.223.260.0': 8465, '1.223.392.0': 8466, '1.221.397.0': 8467, '1.219.1973.0': 8468, '1.223.1333.0': 8469, '1.213.6196.0': 8470, '1.223.1485.0': 8471, '1.207.2504.0': 8472, '1.223.1499.0': 8473, '1.215.2563.0': 8474, '1.199.1324.0': 8475, '1.225.1596.0': 8476, '1.203.59.0': 8477, '1.223.715.0': 8478, '1.217.141.0': 8479, '1.213.1063.0': 8480, '1.213.4702.0': 8481, '1.223.675.0': 8482, '1.275.1027.0': 8483, '1.225.5.0': 8484, '1.223.957.0': 8485, '1.219.2355.0': 8486, '1.217.390.0': 8487, '1.237.98.0': 8488, '1.197.1076.0': 8489, '1.205.995.0': 8490, '1.223.1089.0': 8491, '1.225.1623.0': 8492, '1.229.1721.0': 8493, '1.223.784.0': 8494, '1.221.570.0': 8495, '1.223.683.0': 8496, '1.223.1121.0': 8497, '1.225.1878.0': 8498, '1.213.4918.0': 8499, '1.177.229.0': 8500, '1.223.646.0': 8501, '1.223.331.0': 8502, '1.217.223.0': 8503, '1.221.508.0': 8504, '1.223.968.0': 8505, '1.227.222.0': 8506, '1.223.251.0': 8507, '1.199.1981.0': 8508, '1.217.1571.0': 8509, '1.219.190.0': 8510, '1.207.1349.0': 8511, '1.223.2891.0': 8512, '1.219.1665.0': 8513, '1.217.824.0': 8514, '1.225.1055.0': 8515, '1.223.540.0': 8516, '1.213.6234.0': 8517, '1.203.2139.0': 8518, '1.213.6253.0': 8519, '1.223.2632.0': 8520, '1.209.3189.0': 8521, '1.209.99.0': 8522, '1.213.6023.0': 8523, '1.223.1357.0': 8524, '1.223.555.0': 8525, '1.213.6073.0': 8526, '1.217.500.0': 8527, '1.213.1918.0': 8528, '1.203.1966.0': 8529, '1.223.1075.0': 8530, '1.281.501.0': 8531, '1.279.301.0': 8532, '1.277.230.0': 8533, '1.281.664.0': 8534, '1.279.236.0': 8535, '1.277.724.0': 8536, '1.279.1373.0': 8537, '1.277.1185.0': 8538, '1.281.675.0': 8539, '1.279.922.0': 8540, '1.281.638.0': 8541, '1.281.95.0': 8542, '1.281.685.0': 8543, '1.281.708.0': 8544, '1.281.482.0': 8545, '1.279.292.0': 8546, '1.279.684.0': 8547, '1.279.739.0': 8548, '1.277.861.0': 8549, '1.277.706.0': 8550, '1.277.862.0': 8551, '1.279.336.0': 8552, '1.277.1042.0': 8553, '1.279.367.0': 8554, '1.281.670.0': 8555, '1.277.248.0': 8556, '1.277.881.0': 8557, '1.277.936.0': 8558, '1.277.989.0': 8559, '1.277.811.0': 8560, '1.277.832.0': 8561, '1.277.641.0': 8562, '1.277.950.0': 8563, '1.277.749.0': 8564, '1.281.211.0': 8565, '1.277.1049.0': 8566, '1.279.955.0': 8567, '1.279.1442.0': 8568, '1.279.1282.0': 8569, '1.279.59.0': 8570, '1.279.158.0': 8571, '1.279.1245.0': 8572, '1.277.176.0': 8573, '1.281.695.0': 8574, '1.277.1037.0': 8575, '1.279.806.0': 8576, '1.277.911.0': 8577, '1.281.241.0': 8578, '1.279.8.0': 8579, '1.277.1047.0': 8580, '1.281.293.0': 8581, '1.277.921.0': 8582, '1.279.212.0': 8583, '1.281.618.0': 8584, '1.279.1324.0': 8585, '1.277.1071.0': 8586, '1.277.1044.0': 8587, '1.281.613.0': 8588, '1.277.1212.0': 8589, '1.277.445.0': 8590, '1.279.768.0': 8591, '1.281.782.0': 8592, '1.281.788.0': 8593, '1.277.871.0': 8594, '1.277.701.0': 8595, '1.277.1054.0': 8596, '1.277.1089.0': 8597, '1.277.515.0': 8598, '1.277.1156.0': 8599, '1.277.575.0': 8600, '1.281.657.0': 8601, '1.277.1010.0': 8602, '1.281.113.0': 8603, '1.277.1102.0': 8604, '1.279.295.0': 8605, '1.279.1713.0': 8606, '1.277.400.0': 8607, '1.277.587.0': 8608, '1.281.718.0': 8609, '1.281.594.0': 8610, '1.279.1675.0': 8611, '1.279.390.0': 8612, '1.279.102.0': 8613, '1.281.627.0': 8614, '1.279.1361.0': 8615, '1.279.452.0': 8616, '1.277.1035.0': 8617, '1.277.938.0': 8618, '1.279.361.0': 8619, '1.279.244.0': 8620, '1.279.1569.0': 8621, '1.277.1188.0': 8622, '1.279.208.0': 8623, '1.279.1522.0': 8624, '1.279.32.0': 8625, '1.279.351.0': 8626, '1.279.788.0': 8627, '1.277.1053.0': 8628, '1.279.383.0': 8629, '1.277.1162.0': 8630, '1.277.160.0': 8631, '1.281.777.0': 8632, '1.279.275.0': 8633, '1.277.96.0': 8634, '1.277.853.0': 8635, '1.281.634.0': 8636, '1.277.118.0': 8637, '1.281.574.0': 8638, '1.281.756.0': 8639, '1.281.382.0': 8640, '1.277.378.0': 8641, '1.277.761.0': 8642, '1.279.1171.0': 8643, '1.281.700.0': 8644, '1.277.1150.0': 8645, '1.277.1243.0': 8646, '1.277.1220.0': 8647, '1.281.411.0': 8648, '1.279.1546.0': 8649, '1.277.740.0': 8650, '1.279.247.0': 8651, '1.279.1380.0': 8652, '1.279.1104.0': 8653, '1.279.737.0': 8654, '1.277.195.0': 8655, '1.277.264.0': 8656, '1.277.747.0': 8657, '1.281.739.0': 8658, '1.277.631.0': 8659, '1.277.682.0': 8660, '1.277.1124.0': 8661, '1.279.99.0': 8662, '1.281.465.0': 8663, '1.277.666.0': 8664, '1.281.723.0': 8665, '1.277.558.0': 8666, '1.277.235.0': 8667, '1.277.225.0': 8668, '1.277.385.0': 8669, '1.277.322.0': 8670, '1.277.562.0': 8671, '1.279.413.0': 8672, '1.279.1230.0': 8673, '1.281.645.0': 8674, '1.279.192.0': 8675, '1.279.67.0': 8676, '1.279.150.0': 8677, '1.281.806.0': 8678, '1.277.702.0': 8679, '1.277.279.0': 8680, '1.279.663.0': 8681, '1.281.565.0': 8682, '1.277.858.0': 8683, '1.277.648.0': 8684, '1.277.624.0': 8685, '1.281.812.0': 8686, '1.279.175.0': 8687, '1.279.1633.0': 8688, '1.281.536.0': 8689, '1.281.785.0': 8690, '1.279.1407.0': 8691, '1.277.657.0': 8692, '1.281.732.0': 8693, '1.281.479.0': 8694, '1.277.1050.0': 8695, '1.277.145.0': 8696, '1.277.1132.0': 8697, '1.279.387.0': 8698, '1.277.645.0': 8699, '1.281.473.0': 8700, '1.277.334.0': 8701, '1.277.716.0': 8702, '1.277.893.0': 8703, '1.277.1025.0': 8704, '1.277.798.0': 8705, '1.279.1128.0': 8706, '1.277.164.0': 8707, '1.277.283.0': 8708, '1.277.806.0': 8709, '1.281.453.0': 8710, '1.281.646.0': 8711, '1.279.14.0': 8712, '1.277.305.0': 8713, '1.279.1719.0': 8714, '1.281.59.0': 8715, '1.277.1083.0': 8716, '1.277.284.0': 8717, '1.277.442.0': 8718, '1.281.758.0': 8719, '1.277.407.0': 8720, '1.279.416.0': 8721, '1.279.489.0': 8722, '1.277.662.0': 8723, '1.277.1121.0': 8724, '1.277.714.0': 8725, '1.277.289.0': 8726, '1.279.753.0': 8727, '1.277.157.0': 8728, '1.281.405.0': 8729, '1.277.121.0': 8730, '1.277.550.0': 8731, '1.279.230.0': 8732, '1.279.1649.0': 8733, '1.281.736.0': 8734, '1.279.314.0': 8735, '1.277.471.0': 8736, '1.281.538.0': 8737, '1.277.775.0': 8738, '1.277.736.0': 8739, '1.277.1174.0': 8740, '1.277.850.0': 8741, '1.281.400.0': 8742, '1.277.397.0': 8743, '1.277.711.0': 8744, '1.277.581.0': 8745, '1.281.774.0': 8746, '1.281.178.0': 8747, '1.277.506.0': 8748, '1.277.313.0': 8749, '1.277.704.0': 8750, '1.277.688.0': 8751, '1.279.902.0': 8752, '1.281.462.0': 8753, '1.277.679.0': 8754, '1.277.1126.0': 8755, '1.281.157.0': 8756, '1.281.160.0': 8757, '1.281.42.0': 8758, '1.281.510.0': 8759, '1.277.1164.0': 8760, '1.279.555.0': 8761, '1.277.986.0': 8762, '1.277.717.0': 8763, '1.279.298.0': 8764, '1.277.745.0': 8765, '1.277.204.0': 8766, '1.279.785.0': 8767, '1.279.682.0': 8768, '1.277.612.0': 8769, '1.279.803.0': 8770, '1.281.487.0': 8771, '1.277.227.0': 8772, '1.281.239.0': 8773, '1.277.328.0': 8774, '1.277.683.0': 8775, '1.281.261.0': 8776, '1.281.126.0': 8777, '1.279.166.0': 8778, '1.277.350.0': 8779, '1.281.50.0': 8780, '1.277.933.0': 8781, '1.281.90.0': 8782, '1.281.721.0': 8783, '1.281.801.0': 8784, '1.281.815.0': 8785, '1.279.87.0': 8786, '1.277.690.0': 8787, '1.277.439.0': 8788, '1.277.329.0': 8789, '1.279.607.0': 8790, '1.279.140.0': 8791, '1.279.136.0': 8792, '1.277.336.0': 8793, '1.279.857.0': 8794, '1.279.434.0': 8795, '1.279.1581.0': 8796, '1.277.1064.0': 8797, '1.281.750.0': 8798, '1.277.271.0': 8799, '1.277.730.0': 8800, '1.281.711.0': 8801, '1.277.457.0': 8802, '1.277.1234.0': 8803, '1.279.422.0': 8804, '1.277.705.0': 8805, '1.279.419.0': 8806, '1.277.380.0': 8807, '1.279.526.0': 8808, '1.279.468.0': 8809, '1.279.22.0': 8810, '1.277.809.0': 8811, '1.277.274.0': 8812, '1.281.170.0': 8813, '1.281.541.0': 8814, '1.281.444.0': 8815, '1.277.413.0': 8816, '1.277.375.0': 8817, '1.277.785.0': 8818, '1.277.997.0': 8819, '1.281.596.0': 8820, '1.277.979.0': 8821, '1.279.182.0': 8822, '1.279.459.0': 8823, '1.277.485.0': 8824, '1.279.425.0': 8825, '1.279.226.0': 8826, '1.279.795.0': 8827, '1.279.1221.0': 8828, '1.277.674.0': 8829, '1.277.1041.0': 8830, '1.277.568.0': 8831, '1.279.1340.0': 8832, '1.277.352.0': 8833, '1.277.1240.0': 8834, '1.279.885.0': 8835, '1.281.589.0': 8836, '1.279.131.0': 8837, '1.281.598.0': 8838, '1.279.523.0': 8839, '1.277.602.0': 8840, '1.279.1239.0': 8841, '1.279.218.0': 8842, '1.281.308.0': 8843, '1.281.729.0': 8844, '1.279.894.0': 8845, '1.281.101.0': 8846, '1.277.294.0': 8847, '1.281.610.0': 8848, '1.277.992.0': 8849, '1.279.1164.0': 8850, '1.277.395.0': 8851, '1.279.630.0': 8852, '1.281.181.0': 8853, '1.279.1681.0': 8854, '1.281.715.0': 8855, '1.279.441.0': 8856, '1.277.795.0': 8857, '1.277.1217.0': 8858, '1.277.868.0': 8859, '1.277.318.0': 8860, '1.279.987.0': 8861, '1.277.564.0': 8862, '1.277.692.0': 8863, '1.279.169.0': 8864, '1.279.1663.0': 8865, '1.277.1152.0': 8866, '1.279.410.0': 8867, '1.277.583.0': 8868, '1.279.161.0': 8869, '1.277.590.0': 8870, '1.279.1141.0': 8871, '1.277.366.0': 8872, '1.281.767.0': 8873, '1.277.973.0': 8874, '1.277.726.0': 8875, '1.277.303.0': 8876, '1.279.891.0': 8877, '1.277.1032.0': 8878, '1.277.151.0': 8879, '1.277.906.0': 8880, '1.279.227.0': 8881, '1.279.1219.0': 8882, '1.279.888.0': 8883, '1.277.387.0': 8884, '1.281.118.0': 8885, '1.279.604.0': 8886, '1.277.1036.0': 8887, '1.279.153.0': 8888, '1.281.521.0': 8889, '1.279.1014.0': 8890, '1.281.748.0': 8891, '1.279.1233.0': 8892, '1.279.96.0': 8893, '1.277.769.0': 8894, '1.279.995.0': 8895, '1.279.693.0': 8896, '1.279.1518.0': 8897, '1.279.1696.0': 8898, '1.277.560.0': 8899, '1.279.286.0': 8900, '1.277.947.0': 8901, '1.277.459.0': 8902, '1.279.991.0': 8903, '1.279.1430.0': 8904, '1.281.622.0': 8905, '1.279.1358.0': 8906, '1.277.918.0': 8907, '1.279.1503.0': 8908, '1.279.1266.0': 8909, '1.281.651.0': 8910, '1.279.764.0': 8911, '1.279.269.0': 8912, '1.277.1046.0': 8913, '1.277.790.0': 8914, '1.279.165.0': 8915, '1.277.907.0': 8916, '1.277.592.0': 8917, '1.279.289.0': 8918, '1.277.800.0': 8919, '1.277.677.0': 8920, '1.279.375.0': 8921, '1.277.156.0': 8922, '1.279.800.0': 8923, '1.277.865.0': 8924, '1.277.1062.0': 8925, '1.279.1294.0': 8926, '1.279.1178.0': 8927, '1.279.1150.0': 8928, '1.279.1693.0': 8929, '1.277.766.0': 8930, '1.279.1027.0': 8931, '1.277.354.0': 8932, '1.277.434.0': 8933, '1.279.756.0': 8934, '1.279.502.0': 8935, '1.277.654.0': 8936, '1.281.692.0': 8937, '1.279.850.0': 8938, '1.279.619.0': 8939, '1.277.1030.0': 8940, '1.281.662.0': 8941, '1.281.459.0': 8942, '1.279.1314.0': 8943, '1.279.1203.0': 8944, '1.279.1635.0': 8945, '1.279.493.0': 8946, '1.277.1228.0': 8947, '1.277.142.0': 8948, '1.279.728.0': 8949, '1.279.699.0': 8950, '1.279.325.0': 8951, '1.277.684.0': 8952, '1.279.364.0': 8953, '1.277.222.0': 8954, '1.279.259.0': 8955, '1.279.1272.0': 8956, '1.281.61.0': 8957, '1.277.695.0': 8958, '1.279.373.0': 8959, '1.279.676.0': 8960, '1.281.253.0': 8961, '1.277.676.0': 8962, '1.279.1652.0': 8963, '1.277.678.0': 8964, '1.279.1260.0': 8965, '1.279.1701.0': 8966, '1.279.1528.0': 8967, '1.277.1168.0': 8968, '1.277.197.0': 8969, '1.279.330.0': 8970, '1.279.1024.0': 8971, '1.279.720.0': 8972, '1.279.333.0': 8973, '1.277.324.0': 8974, '1.277.148.0': 8975, '1.279.90.0': 8976, '1.281.354.0': 8977, '1.277.822.0': 8978, '1.281.431.0': 8979, '1.277.803.0': 8980, '1.277.87.0': 8981, '1.279.771.0': 8982, '1.281.570.0': 8983, '1.277.703.0': 8984, '1.281.637.0': 8985, '1.279.847.0': 8986, '1.281.422.0': 8987, '1.277.447.0': 8988, '1.281.72.0': 8989, '1.279.1660.0': 8990, '1.279.1572.0': 8991, '1.279.1626.0': 8992, '1.279.74.0': 8993, '1.277.621.0': 8994, '1.279.1101.0': 8995, '1.279.499.0': 8996, '1.277.153.0': 8997, '1.277.732.0': 8998, '1.279.747.0': 8999, '1.281.765.0': 9000, '1.277.125.0': 9001, '1.279.1540.0': 9002, '1.279.190.0': 9003, '1.277.930.0': 9004, '1.279.1537.0': 9005, '1.279.836.0': 9006, '1.277.424.0': 9007, '1.281.408.0': 9008, '1.281.36.0': 9009, '1.281.790.0': 9010, '1.279.173.0': 9011, '1.277.464.0': 9012, '1.281.760.0': 9013, '1.281.607.0': 9014, '1.277.1191.0': 9015, '1.281.684.0': 9016, '1.279.1593.0': 9017, '1.277.1209.0': 9018, '1.277.330.0': 9019, '1.277.686.0': 9020, '1.279.465.0': 9021, '1.277.735.0': 9022, '1.281.661.0': 9023, '1.277.348.0': 9024, '1.277.368.0': 9025, '1.279.1620.0': 9026, '1.277.1205.0': 9027, '1.281.677.0': 9028, '1.277.707.0': 9029, '1.279.478.0': 9030, '1.279.1082.0': 9031, '1.281.476.0': 9032, '1.277.643.0': 9033, '1.277.421.0': 9034, '1.281.244.0': 9035, '1.279.345.0': 9036, '1.279.702.0': 9037, '1.277.830.0': 9038, '1.277.150.0': 9039, '1.281.507.0': 9040, '1.279.256.0': 9041, '1.279.147.0': 9042, '1.279.1307.0': 9043, '1.277.824.0': 9044, '1.279.1337.0': 9045, '1.279.1549.0': 9046, '1.277.141.0': 9047, '1.279.188.0': 9048, '1.279.1291.0': 9049, '1.279.873.0': 9050, '1.277.316.0': 9051, '1.279.257.0': 9052, '1.279.694.0': 9053, '1.277.663.0': 9054, '1.277.219.0': 9055, '1.277.331.0': 9056, '1.277.975.0': 9057, '1.279.1168.0': 9058, '1.279.10.0': 9059, '1.277.981.0': 9060, '1.279.1063.0': 9061, '1.277.75.0': 9062, '1.277.708.0': 9063, '1.277.681.0': 9064, '1.279.1352.0': 9065, '1.277.89.0': 9066, '1.279.431.0': 9067, '1.281.451.0': 9068, '1.279.1644.0': 9069, '1.277.793.0': 9070, '1.277.267.0': 9071, '1.279.572.0': 9072, '1.281.67.0': 9073, '1.281.111.0': 9074, '1.277.856.0': 9075, '1.279.1022.0': 9076, '1.277.1200.0': 9077, '1.279.644.0': 9078, '1.281.414.0': 9079, '1.277.389.0': 9080, '1.277.467.0': 9081, '1.277.1237.0': 9082, '1.277.977.0': 9083, '1.277.1007.0': 9084, '1.279.1216.0': 9085, '1.279.1288.0': 9086, '1.277.84.0': 9087, '1.277.734.0': 9088, '1.281.821.0': 9089, '1.277.259.0': 9090, '1.281.363.0': 9091, '1.281.545.0': 9092, '1.279.1313.0': 9093, '1.279.69.0': 9094, '1.277.721.0': 9095, '1.279.1134.0': 9096, '1.279.504.0': 9097, '1.279.353.0': 9098, '1.277.1158.0': 9099, '1.279.178.0': 9100, '1.279.1684.0': 9101, '1.279.62.0': 9102, '1.277.880.0': 9103, '1.277.727.0': 9104, '1.279.616.0': 9105, '1.277.455.0': 9106, '1.281.297.0': 9107, '1.279.758.0': 9108, '1.279.766.0': 9109, '1.281.335.0': 9110, '1.277.1203.0': 9111, '1.279.471.0': 9112, '1.279.233.0': 9113, '1.279.638.0': 9114, '1.281.187.0': 9115, '1.281.45.0': 9116, '1.279.1001.0': 9117, '1.279.1560.0': 9118, '1.279.1658.0': 9119, '1.277.1005.0': 9120, '1.277.595.0': 9121, '1.277.968.0': 9122, '1.279.1328.0': 9123, '1.277.1058.0': 9124, '1.279.163.0': 9125, '1.281.587.0': 9126, '1.277.552.0': 9127, '1.277.319.0': 9128, '1.281.380.0': 9129, '1.281.569.0': 9130, '1.281.73.0': 9131, '1.277.291.0': 9132, '1.277.402.0': 9133, '1.279.1506.0': 9134, '1.279.359.0': 9135, '1.277.546.0': 9136, '1.279.519.0': 9137, '1.277.508.0': 9138, '1.281.64.0': 9139, '1.279.842.0': 9140, '1.277.452.0': 9141, '1.277.169.0': 9142, '1.279.632.0': 9143, '1.279.1515.0': 9144, '1.281.416.0': 9145, '1.279.679.0': 9146, '1.281.25.0': 9147, '1.277.694.0': 9148, '1.277.689.0': 9149, '1.279.1707.0': 9150, '1.277.362.0': 9151, '1.277.659.0': 9152, '1.277.715.0': 9153, '1.277.240.0': 9154, '1.277.269.0': 9155, '1.279.216.0': 9156, '1.281.796.0': 9157, '1.277.410.0': 9158, '1.277.166.0': 9159, '1.277.723.0': 9160, '1.279.1543.0': 9161, '1.279.1555.0': 9162, '1.277.344.0': 9163, '1.279.998.0': 9164, '1.277.1086.0': 9165, '1.279.1698.0': 9166, '1.277.605.0': 9167, '1.279.93.0': 9168, '1.277.246.0': 9169, '1.279.1180.0': 9170, '1.279.1242.0': 9171, '1.277.237.0': 9172, '1.279.1525.0': 9173, '1.279.1121.0': 9174, '1.279.1687.0': 9175, '1.277.994.0': 9176, '1.277.346.0': 9177, '1.279.1093.0': 9178, '1.281.544.0': 9179, '1.281.803.0': 9180, '1.281.604.0': 9181, '1.277.469.0': 9182, '1.277.728.0': 9183, '1.279.1367.0': 9184, '1.277.500.0': 9185, '1.279.339.0': 9186, '1.281.467.0': 9187, '1.279.1144.0': 9188, '1.279.516.0': 9189, '1.279.988.0': 9190, '1.277.356.0': 9191, '1.277.432.0': 9192, '1.279.171.0': 9193, '1.277.364.0': 9194, '1.277.697.0': 9195, '1.279.1439.0': 9196, '1.277.257.0': 9197, '1.279.437.0': 9198, '1.277.848.0': 9199, '1.277.978.0': 9200, '1.279.1210.0': 9201, '1.279.1147.0': 9202, '1.281.192.0': 9203, '1.279.1710.0': 9204, '1.277.360.0': 9205, '1.281.47.0': 9206, '1.279.635.0': 9207, '1.279.348.0': 9208, '1.279.1611.0': 9209, '1.277.1119.0': 9210, '1.279.1433.0': 9211, '1.279.697.0': 9212, '1.279.655.0': 9213, '1.279.1095.0': 9214, '1.277.437.0': 9215, '1.279.1617.0': 9216, '1.281.349.0': 9217, '1.277.69.0': 9218, '1.277.680.0': 9219, '1.277.693.0': 9220, '1.279.156.0': 9221, '1.281.19.0': 9222, '1.279.19.0': 9223, '1.279.1258.0': 9224, '1.277.511.0': 9225, '1.279.1059.0': 9226, '1.281.76.0': 9227, '1.279.1318.0': 9228, '1.279.641.0': 9229, '1.279.1269.0': 9230, '1.279.1030.0': 9231, '1.279.1017.0': 9232, '1.277.277.0': 9233, '1.281.34.0': 9234, '1.277.571.0': 9235, '1.277.685.0': 9236, '1.279.1669.0': 9237, '1.277.513.0': 9238, '1.277.904.0': 9239, '1.281.39.0': 9240, '1.277.722.0': 9241, '1.281.53.0': 9242, '1.281.402.0': 9243, '1.277.725.0': 9244, '1.279.1201.0': 9245, '1.279.1054.0': 9246, '1.279.1301.0': 9247, '1.277.382.0': 9248, '1.279.1486.0': 9249, '1.277.548.0': 9250, '1.279.1489.0': 9251, '1.277.1197.0': 9252, '1.277.341.0': 9253, '1.279.1298.0': 9254, '1.279.1186.0': 9255, '1.277.282.0': 9256, '1.281.496.0': 9257, '1.279.1331.0': 9258, '1.277.700.0': 9259, '1.277.719.0': 9260, '1.279.1079.0': 9261, '1.281.87.0': 9262, '1.279.328.0': 9263, '1.277.233.0': 9264, '1.279.1509.0': 9265, '1.277.417.0': 9266, '1.279.1076.0': 9267, '1.279.1716.0': 9268, '1.279.1126.0': 9269, '1.279.1046.0': 9270, '1.277.243.0': 9271, '1.279.1115.0': 9272, '1.279.1421.0': 9273, '1.277.171.0': 9274, '1.279.84.0': 9275, '1.279.870.0': 9276, '1.277.405.0': 9277, '1.279.761.0': 9278, '1.281.98.0': 9279, '1.279.1587.0': 9280, '1.279.1183.0': 9281, '1.277.598.0': 9282, '1.281.484.0': 9283, '1.279.656.0': 9284, '1.281.470.0': 9285, '1.279.64.0': 9286, '1.279.1424.0': 9287, '1.279.879.0': 9288, '1.279.1448.0': 9289, '1.279.508.0': 9290, '1.279.322.0': 9291, '1.279.1057.0': 9292, '1.277.729.0': 9293, '1.279.688.0': 9294, '1.277.503.0': 9295, '1.279.1007.0': 9296, '1.277.983.0': 9297, '1.277.1098.0': 9298, '1.281.360.0': 9299, '1.279.356.0': 9300, '1.279.78.0': 9301, '1.277.339.0': 9302, '1.279.1480.0': 9303, '1.277.314.0': 9304, '1.279.1036.0': 9305, '1.279.1236.0': 9306, '1.281.202.0': 9307, '1.281.351.0': 9308, '1.279.566.0': 9309, '1.277.449.0': 9310, '1.277.1055.0': 9311, '1.281.498.0': 9312, '1.279.622.0': 9313, '1.277.619.0': 9314, '1.277.392.0': 9315, '1.277.138.0': 9316, '1.279.691.0': 9317, '1.279.1346.0': 9318, '1.279.1004.0': 9319, '1.279.81.0': 9320, '1.279.319.0': 9321, '1.279.1673.0': 9322, '1.281.304.0': 9323, '1.279.864.0': 9324, '1.279.1436.0': 9325, '1.279.1049.0': 9326, '1.279.1552.0': 9327, '1.277.1059.0': 9328, '1.279.1575.0': 9329, '1.279.342.0': 9330, '1.279.253.0': 9331, '1.279.1019.0': 9332, '1.279.1641.0': 9333, '1.279.210.0': 9334, '1.279.860.0': 9335, '1.277.555.0': 9336, '1.279.617.0': 9337, '1.279.731.0': 9338, '1.277.174.0': 9339, '1.281.346.0': 9340, '1.279.1160.0': 9341, '1.279.798.0': 9342, '1.277.357.0': 9343, '1.279.514.0': 9344, '1.281.82.0': 9345, '1.279.462.0': 9346, '1.277.311.0': 9347, '1.279.672.0': 9348, '1.277.95.0': 9349, '1.279.1483.0': 9350, '1.277.599.0': 9351, '1.277.72.0': 9352, '1.277.787.0': 9353, '1.279.185.0': 9354, '1.279.625.0': 9355, '1.277.430.0': 9356, '1.277.301.0': 9357, '1.279.1595.0': 9358, '1.279.1033.0': 9359, '1.279.1498.0': 9360, '1.279.711.0': 9361, '1.279.1501.0': 9362, '1.277.359.0': 9363, '1.281.687.0': 9364, '1.279.1623.0': 9365, '1.277.82.0': 9366, '1.279.1309.0': 9367, '1.279.782.0': 9368, '1.281.439.0': 9369, '1.279.1349.0': 9370, '1.279.652.0': 9371, '1.281.208.0': 9372, '1.279.1512.0': 9373, '1.277.1079.0': 9374, '1.279.1198.0': 9375, '1.279.984.0': 9376, '1.279.282.0': 9377, '1.279.474.0': 9378, '1.279.1364.0': 9379, '1.281.189.0': 9380, '1.279.1495.0': 9381, '1.279.660.0': 9382, '1.281.419.0': 9383, '1.279.370.0': 9384, '1.281.194.0': 9385, '1.279.1355.0': 9386, '1.279.1464.0': 9387, '1.279.1690.0': 9388, '1.279.670.0': 9389, '1.279.900.0': 9390, '1.281.27.0': 9391, '1.281.330.0': 9392, '1.279.627.0': 9393, '1.281.429.0': 9394, '1.279.649.0': 9395, '1.281.306.0': 9396, '1.279.272.0': 9397, '1.279.542.0': 9398, '1.279.1614.0': 9399, '1.279.316.0': 9400, '1.279.1578.0': 9401, '1.277.321.0': 9402, '1.279.725.0': 9403, '1.279.552.0': 9404, '1.277.573.0': 9405, '1.279.1599.0': 9406, '1.281.205.0': 9407, '1.277.664.0': 9408, '1.279.569.0': 9409, '1.279.1666.0': 9410, '1.279.1404.0': 9411, '1.279.1090.0': 9412, '1.279.1343.0': 9413, '1.279.1317.0': 9414, '1.279.867.0': 9415, '1.279.876.0': 9416, '1.279.511.0': 9417, '1.279.549.0': 9418, '1.279.1130.0': 9419, '1.279.1334.0': 9420, '1.281.78.0': 9421, '1.279.1678.0': 9422, '1.279.1605.0': 9423, '1.279.539.0': 9424, '1.281.327.0': 9425, '1.279.705.0': 9426, '1.279.674.0': 9427, '1.277.262.0': 9428, '1.279.734.0': 9429, '1.279.646.0': 9430, '1.281.518.0': 9431, '1.279.1533.0': 9432, '1.279.1043.0': 9433, '1.281.70.0': 9434, '1.279.1117.0': 9435, '1.279.1558.0': 9436, '1.279.1638.0': 9437, '1.281.343.0': 9438, '1.279.601.0': 9439, '1.279.722.0': 9440, '1.281.707.0': 9441, '1.281.85.0': 9442, '1.281.673.0': 9443, '1.279.1608.0': 9444, '1.215.780.0': 9445, '1.281.726.0': 9446, '1.279.1304.0': 9447, '1.279.532.0': 9448, '1.279.708.0': 9449, '1.279.714.0': 9450, '1.279.536.0': 9451, '1.279.1254.0': 9452, '1.279.1052.0': 9453, '1.279.717.0': 9454, '1.281.456.0': 9455, '1.279.658.0': 9456, '1.281.818.0': 9457, '1.281.332.0': 9458, '1.281.625.0': 9459, '1.223.2035.0': 9460, '1.279.1478.0': 9461, '1.211.1747.0': 9462, '1.221.665.0': 9463, '1.213.3963.0': 9464, '1.209.730.0': 9465, '1.279.1383.0': 9466, '1.209.2697.0': 9467, '1.223.735.0': 9468, '1.221.302.0': 9469, '1.213.3199.0': 9470, '1.211.916.0': 9471, '1.209.2509.0': 9472, '1.187.322.0': 9473, '1.213.3429.0': 9474, '1.213.2223.0': 9475, '1.215.2826.0': 9476, '1.225.2922.0': 9477, '1.223.2044.0': 9478, '1.209.3245.0': 9479, '1.223.580.0': 9480, '1.207.283.0': 9481, '1.201.857.0': 9482, '1.217.1277.0': 9483, '1.223.2405.0': 9484, '1.213.6523.0': 9485, '1.213.608.0': 9486, '1.219.1126.0': 9487, '1.223.588.0': 9488, '1.219.1090.0': 9489, '1.209.2137.0': 9490, '1.217.1220.0': 9491, '1.219.2351.0': 9492, '1.211.2264.0': 9493, '1.215.1641.0': 9494, '1.215.254.0': 9495, '1.219.662.0': 9496, '1.217.1699.0': 9497, '1.211.1197.0': 9498, '1.217.941.0': 9499, '1.203.1628.0': 9500, '1.201.1744.0': 9501, '1.205.797.0': 9502, '1.217.2218.0': 9503, '1.213.1665.0': 9504, '1.219.2085.0': 9505, '1.213.307.0': 9506, '1.219.2160.0': 9507, '1.215.1053.0': 9508, '1.215.1115.0': 9509, '1.221.629.0': 9510, '1.213.5308.0': 9511, '1.213.5341.0': 9512, '1.213.3168.0': 9513, '1.213.6613.0': 9514, '1.217.204.0': 9515, '1.207.3087.0': 9516, '1.209.2169.0': 9517, '1.219.2327.0': 9518, '1.213.5692.0': 9519, '1.215.3077.0': 9520, '1.201.779.0': 9521, '1.225.2768.0': 9522, '1.215.1037.0': 9523, '1.223.3.0': 9524, '1.223.903.0': 9525, '1.219.1284.0': 9526, '1.211.2433.0': 9527, '1.215.291.0': 9528, '1.223.2104.0': 9529, '1.209.408.0': 9530, '1.223.730.0': 9531, '1.213.6952.0': 9532, '1.213.2695.0': 9533, '1.211.41.0': 9534, '1.191.897.0': 9535, '1.217.399.0': 9536, '1.217.2090.0': 9537, '1.213.6986.0': 9538, '1.203.808.0': 9539, '1.217.1293.0': 9540, '1.213.5004.0': 9541, '1.203.2776.0': 9542, '1.223.2637.0': 9543, '1.213.5352.0': 9544, '1.223.515.0': 9545, '1.205.102.0': 9546, '1.193.3388.0': 9547, '1.215.1603.0': 9548, '1.211.2201.0': 9549, '1.207.243.0': 9550, '1.213.7291.0': 9551, '1.221.795.0': 9552, '1.213.3521.0': 9553, '1.203.667.0': 9554, '1.193.3377.0': 9555, '1.203.2155.0': 9556, '1.223.1476.0': 9557, '1.215.911.0': 9558, '1.223.1083.0': 9559, '1.215.389.0': 9560, '1.217.490.0': 9561, '1.217.915.0': 9562, '1.213.5806.0': 9563, '1.199.1515.0': 9564, '1.211.1689.0': 9565, '1.211.111.0': 9566, '1.225.32.0': 9567, '1.205.1354.0': 9568, '1.213.7066.0': 9569, '1.213.6725.0': 9570, '1.201.725.0': 9571, '1.213.2859.0': 9572, '1.219.771.0': 9573, '1.213.1735.0': 9574, '1.217.747.0': 9575, '1.223.1028.0': 9576, '1.219.1365.0': 9577, '1.217.2259.0': 9578, '1.223.908.0': 9579, '1.219.1074.0': 9580, '1.213.6205.0': 9581, '1.169.2478.0': 9582, '1.213.2684.0': 9583, '1.213.1725.0': 9584, '1.209.781.0': 9585, '1.203.1062.0': 9586, '1.203.1837.0': 9587, '1.221.817.0': 9588, '1.217.2157.0': 9589, '1.223.455.0': 9590, '1.211.2006.0': 9591, '1.213.6583.0': 9592, '1.213.5329.0': 9593, '1.219.2198.0': 9594, '1.213.3458.0': 9595, '1.207.1687.0': 9596, '1.207.1955.0': 9597, '1.211.176.0': 9598, '1.217.1503.0': 9599, '1.205.203.0': 9600, '1.219.1026.0': 9601, '1.223.943.0': 9602, '1.223.1545.0': 9603, '1.223.710.0': 9604, '1.223.2687.0': 9605, '1.221.491.0': 9606, '1.205.2452.0': 9607, '1.207.2059.0': 9608, '1.199.2323.0': 9609, '1.213.134.0': 9610, '1.207.3196.0': 9611, '1.223.695.0': 9612, '1.211.2598.0': 9613, '1.215.1002.0': 9614, '1.243.64.0': 9615, '1.213.5087.0': 9616, '1.213.1010.0': 9617, '1.223.133.0': 9618, '1.213.2188.0': 9619, '1.223.2678.0': 9620, '1.223.2655.0': 9621, '1.207.1049.0': 9622})
    train['Platform'] = train['Platform'].map({'windows10': 0, 'windows7': 1, 'windows8': 2, 'windows2016': 3})
    train['Processor'] = train['Processor'].map({'x64': 0, 'arm64': 1, 'x86': 2})
    train['OsVer'] = train['OsVer'].map({'10.0.0.0': 0, '6.1.1.0': 1, '6.3.0.0': 2, '10.0.1.44': 3, '6.1.0.0': 4, '6.3.1.0': 5, '6.1.3.0': 6, '10.0.3.0': 7, '10.0.1.0': 8, '10.0.32.72': 9, '10.0.2.0': 10, '10.0.5.18': 11, '10.0.0.1': 12, '10.0.21.0': 13, '6.3.3.0': 14, '10.0.4.0': 15, '10.0.80.0': 16, '10.0.7.0': 17, '10.0.5.0': 18, '6.3.4.0': 19, '10.0.32.0': 20, '10.0.3.80': 21, '10.0.2.86': 22, '10.0.19.80': 23, '10.0.16.36': 24, '6.1.0.112': 25, '10.0.5.117': 26, '10.0.48.0': 27, '6.3.16.0': 28, '10.0.16.0': 29, '10.0.1.144': 30, '10.0.7.101': 31, '6.1.7.0': 32, '6.1.0.128': 33, '10.0.0.96': 34, '10.0.0.112': 35, '10.0.0.22': 36, '10.0.153.153': 37, '10.0.8.0': 38, '10.0.23.0': 39, '6.3.0.16': 40, '10.0.0.3': 41, '6.3.5.0': 42, '10.0.6.0': 43, '6.1.2.0': 44, '6.3.80.0': 45, '6.1.6.0': 46, '6.1.4.0': 47, '6.3.32.72': 48, '6.1.16.36': 49, '10.0.4.80': 50, '6.3.1.144': 51, '6.3.0.2': 52, '6.3.0.117': 53, '10.0.0.2': 54, '6.3.0.112': 55, '10.0.0.80': 56, '6.3.7.0': 57, '6.3.2.0': 58, '10.0.2.80': 59, '6.1.80.0': 60, '10.0.3.232': 61, '10.0.0.250': 62, '10.0.7.80': 63, '6.3.153.153': 64, '10.0.64.150': 65, '10.0.1.244': 66, '10.0.72.0': 67, '10.0.26.128': 68})
    train['OsPlatformSubRelease'] = train['OsPlatformSubRelease'].map({'rs4': 0, 'rs1': 1, 'rs3': 2, 'windows7': 3, 'windows8.1': 4, 'th1': 5, 'rs2': 6, 'th2': 7, 'prers5': 8})
    train['OsBuildLab'] = train['OsBuildLab'].map({'17134.1.amd64fre.rs4_release.180410-1804': 0, '14393.0.amd64fre.rs1_release.160715-1616': 1, '16299.431.amd64fre.rs3_release_svc_escrow.180502-1908': 2, '17134.1.arm64fre.rs4_release.180410-1804': 3, '7601.18409.amd64fre.win7sp1_gdr.140303-2144': 4, '16299.15.amd64fre.rs3_release.170928-1534': 5, '9600.19067.x86fre.winblue_ltsb_escrow.180619-2033': 6, '10240.17914.amd64fre.th1.180627-1911': 7, '17134.1.x86fre.rs4_release.180410-1804': 8, '16299.15.x86fre.rs3_release.170928-1534': 9, '14393.576.x86fre.rs1_release_inmarket.161208-2252': 10, '15063.0.amd64fre.rs2_release.170317-1834': 11, '9600.19067.amd64fre.winblue_ltsb_escrow.180619-2033': 12, '10586.545.amd64fre.th2_release.160802-1857': 13, '9600.19125.amd64fre.winblue_ltsb.180812-0703': 14, '14393.1593.amd64fre.rs1_release.170731-1934': 15, '14393.2189.amd64fre.rs1_release.180329-1711': 16, '9600.19000.amd64fre.winblue_ltsb.180410-0600': 17, '10586.420.amd64fre.th2_release_sec.160527-1834': 18, '10586.589.x86fre.th2_release.160906-1759': 19, '14393.2189.x86fre.rs1_release.180329-1711': 20, '10240.16384.amd64fre.th1.150709-1700': 21, '16299.402.amd64fre.rs3_release_svc_escrow_im.180415-0955': 22, '10586.306.amd64fre.th2_release_sec.160422-1850': 23, '10240.17443.amd64fre.th1.170602-2340': 24, '10586.1176.amd64fre.th2_release_sec.170913-1848': 25, '14393.321.x86fre.rs1_release_inmarket.161004-2338': 26, '15063.0.x86fre.rs2_release.170317-1834': 27, '10240.16393.amd64fre.th1_st1.150717-1719': 28, '9600.18969.x86fre.winblue_ltsb.180309-0600': 29, '14393.447.amd64fre.rs1_release_inmarket.161102-0100': 30, '10586.103.amd64fre.th2_release.160126-1819': 31, '10586.122.amd64fre.th2_release_inmarket.160222-1549': 32, '14393.1066.amd64fre.rs1_release_sec.170327-1835': 33, '9600.18589.amd64fre.winblue_ltsb.170204-0600': 34, '14393.1715.amd64fre.rs1_release_inmarket.170906-1810': 35, '10586.1176.x86fre.th2_release_sec.170913-1848': 36, '9600.19101.amd64fre.winblue_ltsb_escrow.180718-1800': 37, '14393.2007.x86fre.rs1_release.171231-1800': 38, '14393.1914.amd64fre.rs1_release_inmarket.171117-1700': 39, '10586.162.x86fre.th2_release_sec.160223-1728': 40, '14393.2214.amd64fre.rs1_release_1.180402-1758': 41, '9600.18289.amd64fre.winblue_ltsb.160328-1315': 42, '10240.17946.amd64fre.th1.180806-2045': 43, '7601.23418.amd64fre.win7sp1_ldr.160408-2045': 44, '10240.17354.amd64fre.th1_st1.170327-1827': 45, '10586.0.amd64fre.th2_release.151029-1700': 46, '10586.494.amd64fre.th2_release_sec.160630-1736': 47, '7601.24214.amd64fre.win7sp1_ldr_escrow.180801-1700': 48, '10240.17443.x86fre.th1.170602-2340': 49, '14393.2068.amd64fre.rs1_release.180209-1727': 50, '10586.162.amd64fre.th2_release_sec.160223-1728': 51, '16299.637.amd64fre.rs3_release_svc.180808-1748': 52, '10240.16412.amd64fre.th1.150729-1800': 53, '10240.17113.amd64fre.th1.160906-1755': 54, '7601.23418.x86fre.win7sp1_ldr.160408-2045': 55, '7601.24168.x86fre.win7sp1_ldr.180608-0600': 56, '14393.2214.x86fre.rs1_release_1.180402-1758': 57, '10240.16590.amd64fre.th1_st1.151104-1714': 58, '14393.351.amd64fre.rs1_release_inmarket.161014-1755': 59, '7601.24168.amd64fre.win7sp1_ldr.180608-0600': 60, '14393.1944.x86fre.rs1_release.171129-2100': 61, '14393.1770.amd64fre.rs1_release.170917-1700': 62, '14393.1794.amd64fre.rs1_release.171008-1615': 63, '14393.576.amd64fre.rs1_release_inmarket.161208-2252': 64, '10586.839.amd64fre.th2_release.170303-1605': 65, '14393.2395.amd64fre.rs1_release_inmarket.180714-1932': 66, '14393.2035.amd64fre.rs1_release_inmarket.180110-1910': 67, '9600.18969.amd64fre.winblue_ltsb.180309-0600': 68, '10586.0.x86fre.th2_release.151029-1700': 69, '9600.18505.amd64fre.winblue_ltsb.160930-0600': 70, '14393.1198.amd64fre.rs1_release_sec.170427-1353': 71, '14393.693.x86fre.rs1_release.161220-1747': 72, '14393.1066.x86fre.rs1_release_sec.170327-1835': 73, '14393.1358.amd64fre.rs1_release.170602-2252': 74, '10586.306.x86fre.th2_release_sec.160422-1850': 75, '7601.23915.amd64fre.win7sp1_ldr.170913-0600': 76, '14393.1198.x86fre.rs1_release_sec.170427-1353': 77, '14393.693.amd64fre.rs1_release.161220-1747': 78, '14393.2007.amd64fre.rs1_release.171231-1800': 79, '14393.2430.amd64fre.rs1_release_inmarket_aim.180806-1810': 80, '10586.11.amd64fre.th2_release.151112-1900': 81, '10586.1106.amd64fre.th2_release.170904-1742': 82, '14393.82.amd64fre.rs1_release.160805-1735': 83, '7601.18798.amd64fre.win7sp1_gdr.150316-1654': 84, '10586.494.x86fre.th2_release_sec.160630-1736': 85, '10240.16384.x86fre.th1.150709-1700': 86, '10586.63.amd64fre.th2_release.160104-1513': 87, '16299.637.x86fre.rs3_release_svc.180808-1748': 88, '9600.19035.amd64fre.winblue_ltsb_escrow.180522-1700': 89, '14393.2430.x86fre.rs1_release_inmarket_aim.180806-1810': 90, '10586.962.x86fre.th2_release.170602-2241': 91, '10586.420.x86fre.th2_release_sec.160527-1834': 92, '17744.1001.amd64fre.rs5_release.180818-1845': 93, '10586.672.amd64fre.th2_release_sec.161024-1825': 94, '10586.633.amd64fre.th2_release.161004-1602': 95, '9600.18821.amd64fre.winblue_ltsb.170914-0600': 96, '10586.11.x86fre.th2_release.151112-1900': 97, '7601.24214.x86fre.win7sp1_ldr_escrow.180801-1700': 98, '14393.187.amd64fre.rs1_release_inmarket.160906-1818': 99, '7601.24093.amd64fre.win7sp1_ldr_escrow.180327-2230': 100, '10586.1045.amd64fre.th2_release.170728-1941': 101, '10586.17.x86fre.th2_release.151121-2308': 102, '14393.1480.amd64fre.rs1_release.170706-2004': 103, '14393.1944.amd64fre.rs1_release.171129-2100': 104, '14393.2125.amd64fre.rs1_release.180301-2139': 105, '7601.24150.amd64fre.win7sp1_ldr_escrow.180528-1700': 106, '9600.18895.amd64fre.winblue_ltsb.180101-1800': 107, '17692.1004.amd64fre.rs_prerelease_flt.180625-1624': 108, '10586.212.amd64fre.th2_release_sec.160328-1908': 109, '14393.953.amd64fre.rs1_release_inmarket.170303-1614': 110, '10586.916.amd64fre.th2_release_sec.170427-1350': 111, '7601.24231.amd64fre.win7sp1_ldr.180810-0600': 112, '14393.2248.amd64fre.rs1_release.180427-1804': 113, '10586.3.amd64fre.th2_release_sec.151104-1948': 114, '10586.589.amd64fre.th2_release.160906-1759': 115, '14393.2097.amd64fre.rs1_release_1.180212-1105': 116, '14393.1593.x86fre.rs1_release.170731-1934': 117, '10586.545.x86fre.th2_release.160802-1857': 118, '14393.2485.x86fre.rs1_release.180827-1809': 119, '7601.23807.amd64fre.win7sp1_ldr.170512-0600': 120, '7601.18247.amd64fre.win7sp1_gdr.130828-1532': 121, '10586.672.x86fre.th2_release_sec.161024-1825': 122, '14393.0.x86fre.rs1_release.160715-1616': 123, '9600.18946.amd64fre.winblue_ltsb_escrow.180302-1800': 124, '10586.962.amd64fre.th2_release.170602-2241': 125, '17738.1000.amd64fre.rs5_release.180810-1706': 126, '17751.1.amd64fre.rs5_release.180828-1428': 127, '14393.2125.x86fre.rs1_release.180301-2139': 128, '7601.24150.x86fre.win7sp1_ldr_escrow.180528-1700': 129, '14393.321.amd64fre.rs1_release_inmarket.161004-2338': 130, '10240.16412.x86fre.th1.150729-1800': 131, '14393.1480.x86fre.rs1_release.170706-2004': 132, '7601.24024.x86fre.win7sp1_ldr.180112-0600': 133, '10586.17.amd64fre.th2_release.151121-2308': 134, '10240.17319.amd64fre.th1.170303-1600': 135, '10586.1007.amd64fre.th2_release.170706-2002': 136, '10240.16644.amd64fre.th1.160104-1507': 137, '10240.16463.amd64fre.th1.150819-1946': 138, '7601.23864.amd64fre.win7sp1_ldr.170707-0600': 139, '14393.2068.x86fre.rs1_release.180209-1727': 140, '14393.2457.amd64fre.rs1_release_inmarket.180822-1743': 141, '10586.212.x86fre.th2_release_sec.160328-1908': 142, '10240.17184.amd64fre.th1_st1.161024-1820': 143, '10240.17889.amd64fre.th1_st1.180529-1823': 144, '17755.1.amd64fre.rs5_release.180904-1423': 145, '7601.24094.amd64fre.win7sp1_ldr_escrow.180330-1600': 146, '10240.16841.amd64fre.th1_st1.160408-1853': 147, '10240.17861.amd64fre.th1.180427-1806': 148, '14393.1914.x86fre.rs1_release_inmarket.171117-1700': 149, '14393.447.x86fre.rs1_release_inmarket.161102-0100': 150, '14393.1358.x86fre.rs1_release.170602-2252': 151, '10586.839.x86fre.th2_release.170303-1605': 152, '10240.17071.amd64fre.th1.160802-1852': 153, '10240.17202.amd64fre.th1_st1.161118-1836': 154, '10586.1106.x86fre.th2_release.170904-1742': 155, '10240.16393.x86fre.th1_st1.150717-1719': 156, '9600.18292.amd64fre.winblue_ltsb.160330-1744': 157, '10586.103.x86fre.th2_release.160126-1819': 158, '7601.23539.amd64fre.win7sp1_ldr.160902-0600': 159, '7601.23796.amd64fre.win7sp1_ldr.170427-1518': 160, '9600.19101.x86fre.winblue_ltsb_escrow.180718-1800': 161, '7601.24000.amd64fre.win7sp1_ldr.171231-1547': 162, '14393.2248.x86fre.rs1_release.180427-1804': 163, '14393.953.x86fre.rs1_release_inmarket.170303-1614': 164, '14393.2273.amd64fre.rs1_release_1.180427-1811': 165, '14393.223.amd64fre.rs1_release_inmarket_rim.161002-1903': 166, '9600.18931.amd64fre.winblue_ltsb.180201-0600': 167, '14393.1794.x86fre.rs1_release.171008-1615': 168, '10586.1007.x86fre.th2_release.170706-2002': 169, '7601.24231.x86fre.win7sp1_ldr.180810-0600': 170, '14393.206.amd64fre.rs1_release.160915-0644': 171, '17730.1000.amd64fre.rs5_release.180731-1427': 172, '10586.3.x86fre.th2_release_sec.151104-1948': 173, '17733.1000.amd64fre.rs5_release.180803-1525': 174, '14393.2363.amd64fre.rs1_release.180625-1741': 175, '7601.19160.amd64fre.win7sp1_gdr.160211-0600': 176, '7601.24094.x86fre.win7sp1_ldr_escrow.180330-1600': 177, '10240.17113.x86fre.th1.160906-1755': 178, '14393.206.x86fre.rs1_release.160915-0644': 179, '7601.23569.x86fre.win7sp1_ldr.161007-0600': 180, '7601.23572.x86fre.win7sp1_ldr.161011-0600': 181, '14393.2363.x86fre.rs1_release.180625-1741': 182, '7601.24000.x86fre.win7sp1_ldr.171231-1547': 183, '7601.23915.x86fre.win7sp1_ldr.170913-0600': 184, '14393.2368.x86fre.rs1_release_inmarket_aim.180712-1833': 185, '10240.17976.amd64fre.th1.180905-1120': 186, '17746.1000.amd64fre.rs5_release.180821-1736': 187, '10586.916.x86fre.th2_release_sec.170427-1350': 188, '14393.2457.x86fre.rs1_release_inmarket.180822-1743': 189, '10586.122.x86fre.th2_release_inmarket.160222-1549': 190, '17735.1000.amd64fre.rs5_release.180807-1451': 191, '7601.23564.amd64fre.win7sp1_ldr.160930-0600': 192, '17754.1.amd64fre.rs5_release.180831-1429': 193, '17741.1000.amd64fre.rs5_release.180814-1430': 194, '10586.338.amd64fre.th2_release.160517-1918': 195, '17713.1000.amd64fre.rs5_release.180706-1551': 196, '17740.1000.arm64fre.rs5_release.180813-1429': 197, '7601.24117.x86fre.win7sp1_ldr_escrow.180422-1430': 198, '14393.1770.x86fre.rs1_release.170917-1700': 199, '7601.24117.amd64fre.win7sp1_ldr_escrow.180422-1430': 200, '9600.17630.amd64fre.winblue_r7.150109-2022': 201, '17672.1000.amd64fre.rs_prerelease.180511-1445': 202, '14393.103.x86fre.rs1_release_inmarket.160819-1924': 203, '17758.1.amd64fre.rs5_release.180907-1536': 204, '10240.17709.amd64fre.th1.171130-0900': 205, '9600.19000.x86fre.winblue_ltsb.180410-0600': 206, '14393.2485.amd64fre.rs1_release.180827-1809': 207, '10586.1045.x86fre.th2_release.170728-1941': 208, '17763.1.amd64fre.rs5_release.180914-1434': 209, '7601.24024.amd64fre.win7sp1_ldr.180112-0600': 210, '7601.19045.x86fre.win7sp1_gdr.151019-1254': 211, '9600.18264.amd64fre.winblue_ltsb.160310-0600': 212, '7600.17795.x86fre.win7_gdr.150318-1623': 213, '14393.1715.x86fre.rs1_release_inmarket.170906-1810': 214, '14393.726.x86fre.rs1_release.170112-1758': 215, '17666.1000.amd64fre.rs_prerelease.180504-1501': 216, '17713.1000.x86fre.rs5_release.180706-1551': 217, '9600.19125.x86fre.winblue_ltsb.180812-0703': 218, '10240.17022.amd64fre.th1_st1.160623-1726': 219, '10586.633.x86fre.th2_release.161004-1602': 220, '7601.23677.amd64fre.win7sp1_ldr.170209-0600': 221, '17758.1.x86fre.rs5_release.180907-1536': 222, '9600.18790.amd64fre.winblue_ltsb.170810-1616': 223, '17735.1000.x86fre.rs5_release.180807-1451': 224, '17677.1000.amd64fre.rs_prerelease.180520-0940': 225, '17763.1.x86fre.rs5_release.180914-1434': 226, '14393.2515.amd64fre.rs1_release_1.180830-1044': 227, '14393.351.x86fre.rs1_release_inmarket.161014-1755': 228, '17686.1003.amd64fre.rs_prerelease.180603-1447': 229, '14393.2368.amd64fre.rs1_release_inmarket_aim.180712-1833': 230, '9600.18685.amd64fre.winblue_ltsb.170427-1704': 231, '14393.479.amd64fre.rs1_release.161110-2025': 232, '7601.23889.amd64fre.win7sp1_ldr.170810-1615': 233, '17747.1000.amd64fre.rs5_release_sigma.180822-1727': 234, '7601.23455.amd64fre.win7sp1_ldr.160516-0600': 235, '9600.18946.x86fre.winblue_ltsb_escrow.180302-1800': 236, '10240.17533.amd64fre.th1.170801-1946': 237, '10240.16515.amd64fre.th1.150916-2039': 238, '9600.17238.amd64fre.winblue_gdr.140723-2018': 239, '14314.1944.amd64fre.rs1_release.171194-2100': 240, '10240.17146.x86fre.th1_st1.160929-1748': 241, '9600.17415.amd64fre.winblue_r4.141028-1500': 242, '9600.18758.amd64fre.winblue_ltsb.170707-0600': 243, '7601.18869.x86fre.win7sp1_gdr.150525-0603': 244, '14393.103.amd64fre.rs1_release_inmarket.160819-1924': 245, '10240.17394.amd64fre.th1_st1.170427-1347': 246, '17760.1.amd64fre.rs5_release.180911-1415': 247, '7601.23392.x86fre.win7sp1_ldr.160317-0600': 248, '14393.2312.amd64fre.rs1_release.180607-1919': 249, '10240.17976.x86fre.th1.180905-1120': 250, '9600.18696.amd64fre.winblue_ltsb.170511-1554': 251, '10240.16545.amd64fre.th1.150930-1750': 252, '9600.18438.amd64fre.winblue_ltsb.160812-0914': 253, '9600.18730.amd64fre.winblue_ltsb.170613-0600': 254, '10240.16463.x86fre.th1.150819-1946': 255, '14393.2155.x86fre.rs1_release_1.180305-1842': 256, '10240.17831.amd64fre.th1_st1.180323-1758': 257, '14393.187.x86fre.rs1_release_inmarket.160906-1818': 258, '9600.19035.x86fre.winblue_ltsb_escrow.180522-1700': 259, '17744.1001.arm64fre.rs5_release.180818-1845': 260, '7601.24060.amd64fre.win7sp1_ldr_escrow.180319-1512': 261, '14393.2339.amd64fre.rs1_release_inmarket.180611-1502': 262, '14393.2035.x86fre.rs1_release_inmarket.180110-1910': 263, '9600.18066.amd64fre.winblue_ltsb.150928-1002': 264, '7601.17514.x86fre.win7sp1_rtm.101119-1850': 265, '17759.1.amd64fre.rs5_release.180910-1322': 266, '7601.19135.amd64fre.win7sp1_gdr.160121-1718': 267, '17746.1000.x86fre.rs5_release.180821-1736': 268, '9600.18378.amd64fre.winblue_ltsb.160611-0600': 269, '14393.2155.amd64fre.rs1_release_1.180305-1842': 270, '7601.23392.amd64fre.win7sp1_ldr.160317-0600': 271, '10240.17738.amd64fre.th1.180101-1159': 272, '9600.18895.x86fre.winblue_ltsb.180101-1800': 273, '7601.23539.x86fre.win7sp1_ldr.160902-0600': 274, '14393.479.x86fre.rs1_release.161110-2025': 275, '10240.17797.x86fre.th1_st1.180228-1829': 276, '14393.1532.amd64fre.rs1_release_d.170711-1840': 277, '7601.23714.amd64fre.win7sp1_ldr.170307-1800': 278, '10240.17146.amd64fre.th1_st1.160929-1748': 279, '10586.63.x86fre.th2_release.160104-1513': 280, '10240.17071.x86fre.th1.160802-1852': 281, '9600.18821.x86fre.winblue_ltsb.170914-0600': 282, '10240.16430.amd64fre.th1.150807-2049': 283, '10240.16431.amd64fre.th1.150810-2333': 284, '10240.17918.amd64fre.th1_escrow.180712-1918': 285, '7601.23889.x86fre.win7sp1_ldr.170810-1615': 286, '14393.2097.x86fre.rs1_release_1.180212-1105': 287, '17692.1000.amd64fre.rs_prerelease.180609-1317': 288, '9600.18202.amd64fre.winblue_ltsb.160119-0600': 289, '16299.431.arm64fre.rs3_release_svc_escrow.180502-1908': 290, '17666.1000.x86fre.rs_prerelease.180504-1501': 291, '17754.1.x86fre.rs5_release.180831-1429': 292, '14393.2312.x86fre.rs1_release.180607-1919': 293, '9600.18931.x86fre.winblue_ltsb.180201-0600': 294, '9600.19038.amd64fre.winblue_ltsb_escrow.180608-1416': 295, '7601.19135.x86fre.win7sp1_gdr.160121-1718': 296, '14393.2339.x86fre.rs1_release_inmarket.180611-1502': 297, '7601.18933.amd64fre.win7sp1_gdr.150715-0600': 298, '7601.19160.x86fre.win7sp1_gdr.160211-0600': 299, '10240.17643.x86fre.th1_st1.170918-1824': 300, '7601.18869.amd64fre.win7sp1_gdr.150525-0603': 301, '14393.82.x86fre.rs1_release.160805-1735': 302, '7600.17795.amd64fre.win7_gdr.150318-1623': 303, '9600.18589.x86fre.winblue_ltsb.170204-0600': 304, '17682.1000.amd64fre.rs_prerelease.180525-1616': 305, '7601.23677.x86fre.win7sp1_ldr.170209-0600': 306, '17761.1001.amd64fre.rs5_release_edge.180912-1739': 307, '9600.18505.x86fre.winblue_ltsb.160930-0600': 308, '10240.17770.amd64fre.th1.180209-1728': 309, '10586.456.amd64fre.th2_release.160617-1853': 310, '9600.17085.amd64fre.winblue_gdr.140330-1035': 311, '10240.16387.amd64fre.th1_st1.150711-1429': 312, '10240.17946.x86fre.th1.180806-2045': 313, '10240.17022.x86fre.th1_st1.160623-1726': 314, '10240.17202.x86fre.th1_st1.161118-1836': 315, '7601.19018.x86fre.win7sp1_gdr.150928-1507': 316, '9600.18258.amd64fre.winblue_ltsb.160303-0600': 317, '17704.1000.amd64fre.rs_prerelease.180623-1611': 318, '10240.16841.x86fre.th1_st1.160408-1853': 319, '9600.17936.amd64fre.winblue_ltsb.150715-0840': 320, '7601.18409.x86fre.win7sp1_gdr.140303-2144': 321, '14393.726.amd64fre.rs1_release.170112-1758': 322, '10240.17914.x86fre.th1.180627-1911': 323, '7601.23569.amd64fre.win7sp1_ldr.161007-0600': 324, '17661.1001.amd64fre.rs_prerelease.180428-1349': 325, '7601.18839.x86fre.win7sp1_gdr.150427-0707': 326, '9600.17668.amd64fre.winblue_r8.150127-1500': 327, '10240.17643.amd64fre.th1_st1.170918-1824': 328, '7600.21980.amd64fre.win7_ldr.150318-1623': 329, '17643.1000.amd64fre.rs_prerelease.180405-1509': 330, '18234.1000.amd64fre.rs_prerelease.180831-1440': 331, '7601.23807.x86fre.win7sp1_ldr.170512-0600': 332, '17692.1004.x86fre.rs_prerelease_flt.180625-1624': 333, '7601.17592.amd64fre.win7sp1_gdr.110408-1631': 334, '10240.16724.x86fre.th1_st1.160222-1812': 335, '14393.1537.amd64fre.rs1_release_inmarket.170731-1907': 336, '17618.1000.amd64fre.rs_prerelease.180302-1651': 337, '10240.16431.x86fre.th1.150810-2333': 338, '9600.18289.x86fre.winblue_ltsb.160328-1315': 339, '9600.17476.amd64fre.winblue_r5.141029-1500': 340, '18204.1001.amd64fre.rs_prerelease.180721-1657': 341, '17639.1000.amd64fre.rs_prerelease.180330-1630': 342, '7601.24009.amd64fre.win7sp1_ldr.180107-0421': 343, '7600.16385.x86fre.win7_rtm.090713-1255': 344, 'nan': 345, '9600.18896.amd64fre.winblue_ltsb_escrow.180108-1534': 346, '9600.17736.amd64fre.winblue_r9.150322-1500': 347, '10240.17889.x86fre.th1_st1.180529-1823': 348, '17634.1000.amd64fre.rs_prerelease.180323-1629': 349, '7601.18526.x86fre.win7sp1_gdr.140706-1506': 350, '7600.16792.x86fre.win7_gdr.110408-1633': 351, '7601.23572.amd64fre.win7sp1_ldr.161011-0600': 352, '10240.16430.x86fre.th1.150807-2049': 353, '7601.24059.x86fre.win7sp1_ldr_escrow.180308-1637': 354, '10240.17709.x86fre.th1.171130-0900': 355, '17728.1000.amd64fre.rs5_release.180727-1721': 356, '10586.639.x86fre.th2_release_inmarket.161014-1749': 357, '10240.16724.amd64fre.th1_st1.160222-1812': 358, '7601.18741.amd64fre.win7sp1_gdr.150202-1526': 359, '10240.17797.amd64fre.th1_st1.180228-1829': 360, '7601.17640.amd64fre.win7sp1_gdr.110622-1506': 361, '9600.18438.x86fre.winblue_ltsb.160812-0914': 362, '10240.16590.x86fre.th1_st1.151104-1714': 363, '7601.18741.x86fre.win7sp1_gdr.150202-1526': 364, '7601.19018.amd64fre.win7sp1_gdr.150928-1507': 365, '7601.23391.x86fre.win7sp1_ldr.160316-0600': 366, '10240.16515.x86fre.th1.150916-2039': 367, '10240.17831.x86fre.th1_st1.180323-1758': 368, '18237.1000.amd64fre.rs_prerelease.180907-1621': 369, '9600.18696.x86fre.winblue_ltsb.170511-1554': 370, '18242.1000.amd64fre.rs_prerelease.180914-1435': 371, '10240.17319.x86fre.th1.170303-1600': 372, '7601.24059.amd64fre.win7sp1_ldr_escrow.180308-1637': 373, '7601.23796.x86fre.win7sp1_ldr.170427-1518': 374, '7601.23349.x86fre.win7sp1_ldr.160210-0600': 375, '10240.17918.x86fre.th1_escrow.180712-1918': 376, '14393.2273.x86fre.rs1_release_1.180427-1811': 377, '7601.23564.x86fre.win7sp1_ldr.160930-0600': 378, '9600.17328.amd64fre.winblue_r3.140827-1500': 379, '9600.18758.x86fre.winblue_ltsb.170707-0600': 380, '7601.18247.x86fre.win7sp1_gdr.130828-1532': 381, '7601.21701.x86fre.win7sp1_ldr.110408-1634': 382, '17686.1003.x86fre.rs_prerelease.180603-1447': 383, '17133.1.amd64fre.rs4_release.180323-1312': 384, '14393.2395.x86fre.rs1_release_inmarket.180714-1932': 385, '7601.19110.amd64fre.win7sp1_gdr.151230-0600': 386, '7601.23714.x86fre.win7sp1_ldr.170307-1800': 387, '7600.16792.amd64fre.win7_gdr.110408-1633': 388, '9600.19128.amd64fre.winblue_ltsb_escrow.180813-1030': 389, '17723.1000.amd64fre.rs5_release.180720-1452': 390, '9600.19038.x86fre.winblue_ltsb_escrow.180608-1416': 391, '7601.23864.x86fre.win7sp1_ldr.170707-0600': 392, '10240.17446.amd64fre.th1_escrow.170616-1918': 393, '9600.18730.x86fre.winblue_ltsb.170613-0600': 394, '17692.1000.x86fre.rs_prerelease.180609-1317': 395, '7601.19045.amd64fre.win7sp1_gdr.151019-1254': 396, '17650.1001.x86fre.rs_prerelease.180414-2140': 397, '10586.1177.amd64fre.th2_release_inmarket.171027-1506': 398, '7601.17835.amd64fre.win7sp1_gdr.120503-2030': 399, '17672.1000.x86fre.rs_prerelease.180511-1445': 400, '16299.15.arm64fre.rs3_release.170928-1534': 401, '10240.16644.x86fre.th1.160104-1507': 402, '7601.18229.amd64fre.win7sp1_gdr.130801-1533': 403, '7601.23338.amd64fre.win7sp1_ldr.160121-1716': 404, '14393.729.amd64fre.rs1_release_inmarket_rim.170123-1753': 405, '17744.1001.x86fre.rs5_release.180818-1845': 406, '9600.17041.amd64fre.winblue_gdr.140305-1710': 407, '7601.18839.amd64fre.win7sp1_gdr.150427-0707': 408, '9600.18685.x86fre.winblue_ltsb.170427-1704': 409, '14393.223.x86fre.rs1_release_inmarket_rim.161002-1903': 410, '7601.24236.x86fre.win7sp1_ldr_escrow.180815-1700': 411, '9600.17415.x86fre.winblue_r4.141028-1500': 412, '10586.240.amd64fre.th2_release.160419-1525': 413, '7601.23391.amd64fre.win7sp1_ldr.160316-0600': 414, '9600.18090.amd64fre.winblue_ltsb.151014-0600': 415, '10240.17861.x86fre.th1.180427-1806': 416, '10586.338.x86fre.th2_release.160517-1918': 417, '7601.18939.amd64fre.win7sp1_gdr.150722-0600': 418, '7601.17514.amd64fre.win7sp1_rtm.101119-1850': 419, '7601.23338.x86fre.win7sp1_ldr.160121-1716': 420, '17741.1000.x86fre.rs5_release.180814-1430': 421, '7601.24236.amd64fre.win7sp1_ldr_escrow.180815-1700': 422, '10240.17394.x86fre.th1_st1.170427-1347': 423, '17755.1.x86fre.rs5_release.180904-1423': 424, '7601.18526.amd64fre.win7sp1_gdr.140706-1506': 425, '17738.1000.x86fre.rs5_release.180810-1706': 426, '9600.17933.amd64fre.winblue_ltsb.150713-1132': 427, '9600.18790.x86fre.winblue_ltsb.170810-1616': 428, '9600.18233.amd64fre.winblue_ltsb.160210-0600': 429, '7600.16539.x86fre.win7_gdr.100226-1909': 430, '9600.18423.amd64fre.winblue_ltsb.160723-0600': 431, '9600.18264.x86fre.winblue_ltsb.160310-0600': 432, '17711.1000.amd64fre.rs_prerelease.180703-1539': 433, '9600.17031.amd64fre.winblue_gdr.140221-1952': 434, '7601.18738.x86fre.win7sp1_gdr.150128-1513': 435, '7601.19110.x86fre.win7sp1_gdr.151230-0600': 436, '17650.1001.amd64fre.rs_prerelease.180414-2140': 437, '9600.18146.amd64fre.winblue_ltsb.151121-0600': 438, '17733.1000.x86fre.rs5_release.180803-1525': 439, '9600.17031.x86fre.winblue_gdr.140221-1952': 440, '9600.18185.amd64fre.winblue_ltsb.151230-0600': 441, '17754.1000.amd64fre.rs5_release_sigma.180831-1700': 442, '7601.17944.x86fre.win7sp1_gdr.120830-0333': 443, '17761.1.amd64fre.rs5_release.180912-1340': 444, '9600.17936.x86fre.winblue_ltsb.150715-0840': 445, '7601.23403.amd64fre.win7sp1_ldr.160325-0600': 446, '18219.1000.amd64fre.rs_prerelease.180810-1721': 447, '7601.24060.x86fre.win7sp1_ldr_escrow.180319-1512': 448, '7601.18939.x86fre.win7sp1_gdr.150722-0600': 449, '10240.16545.x86fre.th1.150930-1750': 450, '18240.1000.x86fre.rs_prerelease.180912-1454': 451, '17627.1000.amd64fre.rs_prerelease.180315-1512': 452, '7601.22943.x86fre.win7sp1_ldr.150126-1511': 453, '17751.1.x86fre.rs5_release.180828-1428': 454, '10240.17533.x86fre.th1.170801-1946': 455, '7601.18933.x86fre.win7sp1_gdr.150715-0600': 456, '17711.1000.x86fre.rs_prerelease.180703-1539': 457, '10240.17184.x86fre.th1_st1.161024-1820': 458, '14393.594.amd64fre.rs1_release_inmarket.161213-1754': 459, '17634.1000.x86fre.rs_prerelease.180323-1629': 460, '18234.1000.x86fre.rs_prerelease.180831-1440': 461, '17677.1000.x86fre.rs_prerelease.180520-0940': 462, '17643.1000.x86fre.rs_prerelease.180405-1509': 463, '7601.23313.x86fre.win7sp1_ldr.151230-0600': 464, '7601.24009.x86fre.win7sp1_ldr.180107-0421': 465, '10240.17354.x86fre.th1_st1.170327-1827': 466, '7600.16617.amd64fre.win7_gdr.100618-1621': 467, '7601.18113.x86fre.win7sp1_gdr.130318-1533': 468, '18240.1000.amd64fre.rs_prerelease.180912-1454': 469, '9600.18378.x86fre.winblue_ltsb.160611-0600': 470, '17655.1000.amd64fre.rs_prerelease.180420-1850': 471, '7601.23455.x86fre.win7sp1_ldr.160516-0600': 472, '9600.18340.amd64fre.winblue_ltsb.160513-1153': 473, '14393.2156.amd64fre.rs1_release_inmarket.180321-1733': 474, '17682.1000.x86fre.rs_prerelease.180525-1616': 475, '14393.1532.x86fre.rs1_release_d.170711-1840': 476, '9600.17238.x86fre.winblue_gdr.140723-2018': 477, '9600.18423.x86fre.winblue_ltsb.160723-0600': 478, '7600.16385.amd64fre.win7_rtm.090713-1255': 479, '14393.2515.x86fre.rs1_release_1.180830-1044': 480, '10586.842.amd64fre.th2_release_inmarket.170318-0600': 481, '17760.1.x86fre.rs5_release.180911-1415': 482, '7601.17803.x86fre.win7sp1_gdr.120330-1504': 483, '10240.17738.x86fre.th1.180101-1159': 484, '7601.18923.amd64fre.win7sp1_gdr.150714-1514': 485, '9600.18090.x86fre.winblue_ltsb.151014-0600': 486, '9600.16452.amd64fre.winblue_gdr.131030-1505': 487, '7601.23223.x86fre.win7sp1_ldr.150928-0600': 488, '16299.1003.x86fre.rs3_release_svc_sec.180829-1725': 489, '7601.18700.x86fre.win7sp1_gdr.141211-1742': 490, '17730.1001.amd64fre.rs5_release_sigma_dev.180801-1815': 491, '7601.18923.x86fre.win7sp1_gdr.150714-1514': 492, '16299.0.amd64fre.rs3_release.170922-1354': 493, '7601.18205.amd64fre.win7sp1_gdr.130708-1532': 494, '14393.1613.amd64fre.rs1_release_d.170807-1806': 495, '18214.1000.amd64fre.rs_prerelease.180803-1553': 496, '7601.18798.x86fre.win7sp1_gdr.150316-1654': 497, '10240.17770.x86fre.th1.180209-1728': 498, '7601.18717.amd64fre.win7sp1_gdr.150113-1808': 499, '7601.17803.amd64fre.win7sp1_gdr.120330-1504': 500, '7601.21866.amd64fre.win7sp1_ldr.111123-1509': 501, '9600.18292.x86fre.winblue_ltsb.160330-1744': 502, '7600.17273.amd64fre.win7_gdr.130318-1532': 503, '7601.23313.amd64fre.win7sp1_ldr.151230-0600': 504, '7600.17118.amd64fre.win7_gdr.120830-0334': 505, '17661.1001.x86fre.rs_prerelease.180428-1349': 506, '7601.17592.x86fre.win7sp1_gdr.110408-1631': 507, '18237.1000.x86fre.rs_prerelease.180907-1621': 508, '10586.456.x86fre.th2_release.160617-1853': 509, '9600.18007.amd64fre.winblue_ltsb.150807-0612': 510, '14389.1944.amd64fre.rs1_release.171179-2100': 511, '14393.594.x86fre.rs1_release_inmarket.161213-1754': 512, '7600.16539.amd64fre.win7_gdr.100226-1909': 513, '7601.18147.x86fre.win7sp1_gdr.130505-1534': 514, '10240.17609.amd64fre.th1.170904-1739': 515, '10586.1177.x86fre.th2_release_inmarket.171027-1506': 516, '7601.18700.amd64fre.win7sp1_gdr.141211-1742': 517, '9600.17085.x86fre.winblue_gdr.140330-1035': 518, '7601.17727.amd64fre.win7sp1_gdr.111118-2330': 519, '9600.18146.x86fre.winblue_ltsb.151121-0600': 520, '9600.17630.x86fre.winblue_r7.150109-2022': 521, '7601.23136.x86fre.win7sp1_ldr.150715-0600': 522, '10240.17488.amd64fre.th1.170705-2101': 523, '7600.17273.x86fre.win7_gdr.130318-1532': 524, '7601.17727.x86fre.win7sp1_gdr.111118-2330': 525, '9600.18725.amd64fre.winblue_ltsb.170607-0600': 526, '18242.1000.x86fre.rs_prerelease.180914-1435': 527, '7601.18113.amd64fre.win7sp1_gdr.130318-1533': 528, '9600.17736.x86fre.winblue_r9.150322-1500': 529, '17639.1000.x86fre.rs_prerelease.180330-1630': 530, '18201.1000.amd64fre.rs_prerelease.180717-1400': 531, '9600.18066.x86fre.winblue_ltsb.150928-1002': 532, '10240.16390.x86fre.th1_st1.150714-1601': 533, '16299.64.amd64fre.rs3_release_svc.171024-1647': 534, '7601.23250.amd64fre.win7sp1_ldr.151019-1255': 535, '7601.17944.amd64fre.win7sp1_gdr.120830-0333': 536, '7601.18715.x86fre.win7sp1_gdr.150111-1512': 537, '17655.1000.x86fre.rs_prerelease.180420-1850': 538, '7601.23349.amd64fre.win7sp1_ldr.160210-0600': 539, '17741.1000.arm64fre.rs5_release.180814-1430': 540, '16299.484.amd64fre.rs3_release_svc_escrow.180511-1748': 541, '17604.1000.x86fre.rs_prerelease.180209-1422': 542, '9600.18258.x86fre.winblue_ltsb.160303-0600': 543, '7600.16695.x86fre.win7_gdr.101026-1503': 544, '14393.2423.amd64fre.rs1_release.180718-1836': 545, '17618.1000.x86fre.rs_prerelease.180302-1651': 546, '9600.17328.x86fre.winblue_r3.140827-1500': 547, '7600.16399.x86fre.win7_gdr.090722-1835': 548, '10586.596.amd64fre.th2_release_inmarket.160916-2001': 549, '7601.18717.x86fre.win7sp1_gdr.150113-1808': 550, '17740.1000.amd64fre.rs5_release.180813-1429': 551, '10240.16387.x86fre.th1_st1.150711-1429': 552, '7600.16988.x86fre.win7_gdr.120401-1505': 553, '17751.1.arm64fre.rs5_release.180828-1428': 554, '9600.19128.x86fre.winblue_ltsb_escrow.180813-1030': 555, '14393.2156.x86fre.rs1_release_inmarket.180321-1733': 556, '10240.17974.amd64fre.th1.180827-1810': 557, '7601.17640.x86fre.win7sp1_gdr.110622-1506': 558, '7601.18738.amd64fre.win7sp1_gdr.150128-1513': 559, '9600.18379.x86fre.winblue_ltsb.160614-0600': 560, '9600.17476.x86fre.winblue_r5.141029-1500': 561, '17704.1000.x86fre.rs_prerelease.180623-1611': 562, '14439.1944.amd64fre.rs1_release.171157-2100': 563, '7600.16841.x86fre.win7_gdr.110622-1503': 564, '17627.1000.x86fre.rs_prerelease.180315-1512': 565, '14393.2311.amd64fre.rs1_release.180606-0013': 566, '14298.1944.amd64fre.rs1_release.171132-2100': 567, '17730.1000.x86fre.rs5_release.180731-1427': 568, '7600.20941.x86fre.win7_ldr.110408-1635': 569, '9600.16452.x86fre.winblue_gdr.131030-1505': 570, '7601.17713.x86fre.win7sp1_gdr.111025-1505': 571, '9600.18202.x86fre.winblue_ltsb.160119-0600': 572, '7600.16905.x86fre.win7_gdr.111025-1503': 573, '16299.0.x86fre.rs3_release.170922-1354': 574, '7601.19131.amd64fre.win7sp1_gdr.160116-0600': 575, '14393.1797.amd64fre.rs1_release_inmarket_aim.171027-1738': 576, '10240.16425.amd64fre.th1.150802-1600': 577, '7601.19131.x86fre.win7sp1_gdr.160116-0600': 578, '17723.1000.x86fre.rs5_release.180720-1452': 579, '14393.1670.amd64fre.rs1_release_inmarket.170821-1825': 580, '7601.17105.x86fre.win7sp1_rc.100929-1730': 581, '18219.1000.x86fre.rs_prerelease.180810-1721': 582, '17604.1000.amd64fre.rs_prerelease.180209-1422': 583, '18224.1000.amd64fre.rs_prerelease.180817-1433': 584, '18241.1000.amd64fre.rs_shell.180913-1853': 585, '17689.1000.amd64fre.rs_onecore_stack_per1.180607-1708': 586, '17718.1002.amd64fre.rs5_release_sigma_dev.180717-1700': 587, '17753.1001.amd64fre.rs5_release_edge.180830-1701': 588, '7601.17730.amd64fre.win7sp1_gdr.111123-1510': 589, '7600.16617.x86fre.win7_gdr.100618-1621': 590, '10240.17974.x86fre.th1.180827-1810': 591, '17682.1000.arm64fre.rs_prerelease.180525-1616': 592, '17738.1000.arm64fre.rs5_release.180810-1706': 593, '7600.16695.amd64fre.win7_gdr.101026-1503': 594, '14230.1944.amd64fre.rs1_release.171284-2100': 595, '17728.1000.x86fre.rs5_release.180727-1721': 596, '7600.16481.x86fre.win7_gdr.091207-1941': 597, '17686.1004.amd64fre.rs_onecore_sigma_grfx_dev.180604-1518': 598, '7601.22436.x86fre.win7sp1_ldr.130828-1532': 599, '7601.23403.x86fre.win7sp1_ldr.160325-0600': 600, '16299.695.x86fre.rs3_release_svc.180827-1746': 601, '7600.16841.amd64fre.win7_gdr.110622-1503': 602, '7601.23250.x86fre.win7sp1_ldr.151019-1255': 603, '9600.16384.amd64fre.winblue_rtm.130821-1623': 604, '10240.17609.x86fre.th1.170904-1739': 605, '9600.18185.x86fre.winblue_ltsb.151230-0600': 606, '18204.1001.x86fre.rs_prerelease.180721-1657': 607, '7600.16988.amd64fre.win7_gdr.120401-1505': 608, '7600.17118.x86fre.win7_gdr.120830-0334': 609, '14453.1944.amd64fre.rs1_release.171111-2100': 610, '10586.842.x86fre.th2_release_inmarket.170318-0600': 611, '17747.1000.amd64fre.rs5_release.180822-1441': 612, '17746.1000.arm64fre.rs5_release.180821-1736': 613, '7601.18044.amd64fre.win7sp1_gdr.130104-1431': 614, '18227.1000.arm64fre.rs_onecore_base_cobalt.180822-1700': 615, '14393.1737.amd64fre.rs1_release_inmarket.170914-1249': 616, '7601.17790.x86fre.win7sp1_gdr.120305-1505': 617, '18238.1000.amd64fre.rs_prerelease.180910-1357': 618, '14425.1944.amd64fre.rs1_release.171223-2100': 619, '7601.18205.x86fre.win7sp1_gdr.130708-1532': 620, '17725.1000.arm64fre.rs5_release.180724-1415': 621, '10240.17943.amd64fre.th1.180724-1745': 622, '7600.21980.x86fre.win7_ldr.150318-1623': 623, '7601.22923.amd64fre.win7sp1_ldr.150113-1808': 624, '7601.23072.amd64fre.win7sp1_ldr.150525-0604': 625, '7600.17207.x86fre.win7_gdr.130104-1435': 626, '17694.1000.x86fre.rs_prerelease.180612-1636': 627, '14306.1944.amd64fre.rs1_release.171079-2100': 628, '10586.639.amd64fre.th2_release_inmarket.161014-1749': 629, '17757.1.amd64fre.rs5_release.180906-1403': 630, '7601.18715.amd64fre.win7sp1_gdr.150111-1512': 631, '18233.1000.amd64fre.rs_xbox.180830-1429': 632, '14474.1944.amd64fre.rs1_release.171253-2100': 633, '7601.18044.x86fre.win7sp1_gdr.130104-1431': 634, '9600.17041.x86fre.winblue_gdr.140305-1710': 635, '14206.1944.amd64fre.rs1_release.171177-2100': 636, '17623.1002.amd64fre.rs_prerelease.180310-1600': 637, '17753.1.amd64fre.rs5_release.180830-1437': 638, '14393.1944.amd64fre.rs1_release.171142-2100': 639, '7600.21490.amd64fre.win7_ldr.130318-1533': 640, '17756.1.amd64fre.rs5_release.180905-1436': 641, '14379.1944.amd64fre.rs1_release.171278-2100': 642, '17749.1000.amd64fre.rs5_release.180824-1522': 643, '17734.1000.amd64fre.rs5_release.180806-1423': 644, '17763.1.arm64fre.rs5_release.180914-1434': 645, '16299.691.amd64fre.rs3_release_svc.180813-1858': 646, '17752.1.amd64fre.rs5_release.180829-1740': 647, '9600.18725.x86fre.winblue_ltsb.170607-0600': 648, '18064.1.amd64fre.rs6_prerelease.180710-1809': 649, '9600.18896.x86fre.winblue_ltsb_escrow.180108-1534': 650, '18226.1000.amd64fre.rs_prerelease.180820-1429': 651, '16299.637.arm64fre.rs3_release_svc.180808-1748': 652, '17747.1000.amd64fre.rs5_release_sign.180822-1726': 653, '17747.1000.x86fre.rs5_release.180822-1441': 654, '7601.23153.amd64fre.win7sp1_ldr.150804-0600': 655, '9600.16404.amd64fre.winblue_gdr.130913-2141': 656, '18230.1000.amd64fre.rs_prerelease.180824-1526': 657, '18244.1000.amd64fre.rs_prerelease.180918-1421': 658, '18236.1001.amd64fre.rs_xbox.180906-1100': 659, '16299.431.amd64fre.rs3_release_svc_escrow.180502-1934': 660, '17744.1000.amd64fre.rs5_release.180817-1423': 661, '14466.1944.amd64fre.rs1_release.171244-2100': 662, '17750.1000.amd64fre.rs5_release.180827-1459': 663, '18277.1000.amd64fre.rs_prerelease.181102-1446': 664, '9600.19153.amd64fre.winblue_ltsb.180908-0600': 665, '14393.2608.amd64fre.rs1_release.181024-1742': 666, '10240.18036.amd64fre.th1.181024-1742': 667, '7601.24260.x86fre.win7sp1_ldr.180908-0600': 668, '9600.19179.amd64fre.winblue_ltsb_escrow.181015-1847': 669, '18262.1000.amd64fre.rs_prerelease.181012-1414': 670, '7601.24260.amd64fre.win7sp1_ldr.180908-0600': 671, '18252.1000.amd64fre.rs_prerelease.180928-1410': 672, '14393.2580.amd64fre.rs1_release_inmarket.181009-1745': 673, '14393.2551.amd64fre.rs1_release.181004-1309': 674, '10240.18005.amd64fre.th1.180924-1744': 675, '18272.1000.amd64fre.rs_prerelease.181026-1425': 676, '18252.1000.x86fre.rs_prerelease.180928-1410': 677, '18282.1000.x86fre.rs_prerelease.181109-1613': 678, '18282.1000.amd64fre.rs_prerelease.181109-1613': 679, '9600.19153.x86fre.winblue_ltsb.180908-0600': 680, '14393.2551.x86fre.rs1_release.181004-1309': 681, '7601.24291.x86fre.win7sp1_ldr_escrow.181110-1429': 682, '18247.1001.amd64fre.rs_prerelease.180922-2224': 683, '14393.2608.x86fre.rs1_release.181024-1742': 684, '18267.1001.amd64fre.rs_prerelease.181020-1908': 685, '7601.24291.amd64fre.win7sp1_ldr_escrow.181110-1429': 686, '10240.18005.x86fre.th1.180924-1744': 687, '18277.1000.x86fre.rs_prerelease.181102-1446': 688, '18272.1000.x86fre.rs_prerelease.181026-1425': 689, '9600.19179.x86fre.winblue_ltsb_escrow.181015-1847': 690, '18259.1000.amd64fre.rs_prerelease.181009-1436': 691, '14246.1944.amd64fre.rs1_release.171104-2100': 692, '18285.1000.amd64fre.rs_prerelease.181114-1628': 693, '18262.1000.x86fre.rs_prerelease.181012-1414': 694, '10240.18036.x86fre.th1.181024-1742': 695, '9600.18194.amd64fre.winblue_ltsb.160112-0600': 696, '18267.1001.x86fre.rs_prerelease.181020-1908': 697, '14393.2580.x86fre.rs1_release_inmarket.181009-1745': 698, '18252.1000.arm64fre.rs_prerelease.180928-1410': 699, '18247.1001.x86fre.rs_prerelease.180922-2224': 700, '18280.1000.amd64fre.rs_prerelease.181107-1441': 701, '7601.17591.amd64fre.win7sp1_gdr.110407-1603': 702, '18265.1000.amd64fre.rs_prerelease.181017-1441': 703, '18255.1000.x86fre.rs_prerelease.181003-1454': 704, '18255.1000.amd64fre.rs_prerelease.181003-1454': 705, '7600.20510.amd64fre.win7_ldr.090819-1503': 706, '14393.1797.x86fre.rs1_release_inmarket_aim.171027-1738': 707, '7600.16973.x86fre.win7_gdr.120305-1504': 708, '9600.18233.x86fre.winblue_ltsb.160210-0600': 709, '7601.17790.amd64fre.win7sp1_gdr.120305-1505': 710, '14216.1944.amd64fre.rs1_release.171261-2100': 711, '14436.1944.amd64fre.rs1_release.171173-2100': 712, '18279.1000.amd64fre.rs_onecore_sigma_grfx_dev.181107-1700': 713, '18253.1000.amd64fre.rs_prerelease.181001-1418': 714, '9600.18007.x86fre.winblue_ltsb.150807-0612': 715, '14422.1944.amd64fre.rs1_release.171131-2100': 716, '7600.20941.amd64fre.win7_ldr.110408-1635': 717, '14393.0.amd64fre.rs1_release_srvmedia.160808-1702': 718, '7600.17017.amd64fre.win7_gdr.120503-2030': 719, '18287.1001.amd64fre.rs_prerelease.181117-1527': 720, '18260.1000.amd64fre.rs_prerelease.181010-1554': 721, '18247.1000.x86fre.rs_prerelease.180921-1318': 722, '18259.1000.x86fre.rs_prerelease.181009-1436': 723, '14320.1944.amd64fre.rs1_release.171244-2100': 724, '14393.1613.x86fre.rs1_release_d.170807-1806': 725, '18249.1000.x86fre.rs_prerelease.180925-1413': 726, '7601.22908.x86fre.win7sp1_ldr.141211-1743': 727, '18287.1001.x86fre.rs_prerelease.181117-1527': 728, '14218.1944.amd64fre.rs1_release.171165-2100': 729, '18255.1000.amd64fre.rs_onecore_dep.181003-1700': 730, '14421.1944.amd64fre.rs1_release.171268-2100': 731, '18248.1000.amd64fre.rs_prerelease.180924-1406': 732, '18289.1000.amd64fre.rs_prerelease.181120-1439': 733, '7601.22656.amd64fre.win7sp1_ldr.140417-1532': 734, '16299.811.amd64fre.rs3_release_svc.181008-1831': 735, '14294.1944.amd64fre.rs1_release.171089-2100': 736, '18249.1000.amd64fre.rs_prerelease.180925-1413': 737, '16024.1.amd64fre.rs5_release.180904-1423': 738, '18260.1000.amd64fre.rsmaster.181010-1540': 739, '18256.1000.amd64fre.rs_prerelease.181004-1434': 740, '7601.18711.amd64fre.win7sp1_gdr.150106-1506': 741, '7601.22948.amd64fre.win7sp1_ldr.150202-1521': 742, '14337.1944.amd64fre.rs1_release.171106-2100': 743, '7601.23002.amd64fre.win7sp1_ldr.150316-1651': 744, '14229.1944.amd64fre.rs1_release.171144-2100': 745, '18266.1000.amd64fre.rs_shell.181018-1628': 746, '17704.1002.amd64fre.rs_onecore_sigma_grfx_dev.180625-1838': 747, '18214.1000.x86fre.rs_prerelease.180803-1553': 748, '15063.2.x86fre.rs2_release_svc_d.170531-1743': 749, '7600.20642.amd64fre.win7_ldr.100211-1706': 750, '18270.1000.amd64fre.rs_prerelease.181024-1418': 751, '18269.1000.x86fre.rs_prerelease.181023-1417': 752, '17133.1.x86fre.rs4_release.180323-1312': 753, '14367.1944.amd64fre.rs1_release.171263-2100': 754, '18264.1000.x86fre.rs_prerelease.181016-1426': 755, '18264.1000.amd64fre.rs_prerelease.181016-1426': 756, '10586.461.amd64fre.th2_release.160701-1757': 757, '17653.1000.amd64fre.rs_onecore_sigma_grfx_dev.180419-1700': 758, '7601.22616.amd64fre.win7sp1_ldr.140303-2307': 759, '7601.22379.x86fre.win7sp1_ldr.130707-1535': 760, '7600.16397.x86fre.win7_gdr_staging.090720-1852': 761, '7601.21915.x86fre.win7sp1_ldr.120203-2106': 762, '17134.1*amd64fre.rs4_release.180410-1804': 763, '15063.2.amd64fre.rs2_release.170323-1837': 764, '14372.1944.amd64fre.rs1_release.171205-2100': 765, '14355.1944.amd64fre.rs1_release.171164-2100': 766, '17758.1.arm64fre.rs5_release.180907-1536': 767, '7601.23142.amd64fre.win7sp1_ldr.150722-0600': 768, '14279.1944.amd64fre.rs1_release.171220-2100': 769, '18238.1000.amd64fre.rs_onecore_sigma_grfx_dev.180911-1700': 770, '17760.1.arm64fre.rs5_release.180911-1415': 771, '18279.1001.amd64fre.rs_prerelease.181106-1504': 772, '18289.1000.x86fre.rs_prerelease.181120-1439': 773, '9600.17668.x86fre.winblue_r8.150127-1500': 774, '18252.1001.amd64fre.rs_shell.180929-1715': 775})
    train['SkuEdition'] = train['SkuEdition'].map({'Pro': 0, 'Home': 1, 'Invalid': 2, 'Enterprise LTSB': 3, 'Enterprise': 4, 'Education': 5, 'Cloud': 6, 'Server': 7})
    train['PuaMode'] = train['PuaMode'].map({'nan': 0, 'on': 1, 'audit': 2})
    train['SmartScreen'] = train['SmartScreen'].map({'nan': 0, 'RequireAdmin': 1, 'ExistsNotSet': 2, 'Off': 3, 'Warn': 4, 'Prompt': 5, 'Block': 6, 'off': 7, 'On': 8, 'on': 9, '&#x01;': 10, '&#x02;': 11, 'OFF': 12, 'Promt': 13, '0': 14, 'requireadmin': 15, 'Enabled': 16, 'prompt': 17, 'warn': 18, '00000000': 19, 'requireAdmin': 20, '&#x03;': 21, 'Promprt': 22, 'RequiredAdmin': 23, 'of': 24, 'Deny': 25, 'ON': 26, 'BLOCK': 27})
    train['Census_MDC2FormFactor'] = train['Census_MDC2FormFactor'].map({'Desktop': 0, 'Notebook': 1, 'Detachable': 2, 'PCOther': 3, 'AllInOne': 4, 'Convertible': 5, 'SmallTablet': 6, 'LargeTablet': 7, 'SmallServer': 8, 'LargeServer': 9, 'MediumServer': 10, 'ServerOther': 11, 'IoTOther': 12, 'Other': 13})
    train['Census_DeviceFamily'] = train['Census_DeviceFamily'].map({'Windows.Desktop': 0, 'Windows.Server': 1, 'Windows': 2})
    train['Census_ProcessorClass'] = train['Census_ProcessorClass'].map({'nan': 0, 'mid': 1, 'low': 2, 'high': 3})
    train['Census_PrimaryDiskTypeName'] = train['Census_PrimaryDiskTypeName'].map({'HDD': 0, 'SSD': 1, 'UNKNOWN': 2, 'Unspecified': 3, 'nan': 4})
    train['Census_ChassisTypeName'] = train['Census_ChassisTypeName'].map({'Desktop': 0, 'Notebook': 1, 'MiniTower': 2, 'Portable': 3, 'Detachable': 4, 'Laptop': 5, 'AllinOne': 6, 'LowProfileDesktop': 7, 'SpaceSaving': 8, 'Other': 9, 'Unknown': 10, 'HandHeld': 11, 'UNKNOWN': 12, 'Convertible': 13, 'Tower': 14, 'MainServerChassis': 15, 'LunchBox': 16, 'SubNotebook': 17, 'MiniPC': 18, 'RackMountChassis': 19, 'Tablet': 20, '30': 21, 'StickPC': 22, 'BusExpansionChassis': 23, 'nan': 24, '36': 25, '82': 26, '35': 27, '0': 28, 'Blade': 29, 'IoTGateway': 30, 'MultisystemChassis': 31, 'SealedCasePC': 32, '32': 33, '31': 34, 'ExpansionChassis': 35, 'SubChassis': 36, 'PizzaBox': 37, '88': 38, '25': 39, '28': 40, '44': 41, '127': 42, '76': 43, 'DockingStation': 44, '45': 45, '49': 46, 'CompactPCI': 47, '112': 48, 'BladeEnclosure': 49, '81': 50, 'EmbeddedPC': 51, '39': 52, '83': 53, '120': 54, '64': 55, '93': 56, 'PeripheralChassis': 57})
    train['Census_PowerPlatformRoleName'] = train['Census_PowerPlatformRoleName'].map({'Desktop': 0, 'Mobile': 1, 'Slate': 2, 'Workstation': 3, 'SOHOServer': 4, 'UNKNOWN': 5, 'AppliancePC': 6, 'EnterpriseServer': 7, 'PerformanceServer': 8, 'nan': 9, 'Unspecified': 10})
    train['Census_InternalBatteryType'] = train['Census_InternalBatteryType'].map({'nan': 0, 'lion': 1, 'li-i': 2, '#': 3, 'lip': 4, 'liio': 5, 'vbox': 6, 'li p': 7, 'real': 8, 'unkn': 9, 'pbac': 10, 'li': 11, 'bq20': 12, 'nimh': 13, '\x04lio': 14, 'lgi0': 15, 'lhp0': 16, 'ithi': 17, 'batt': 18, 'lipp': 19, 'lipo': 20, '4cel': 21, 'ram': 22, 'lit': 23, 'a140': 24, 'bad': 25, 'asmb': 26, 'virt': 27, 'ca48': 28, '4ion': 29, 'd': 30, 'a132': 31, 'ÿÿÿÿ': 32, 'cl53': 33, 'lio': 34, 'li-l': 35, '÷ÿóö': 36, 'í\x03-i': 37, '0x0b': 38, 'lgs0': 39, '3ion': 40, 'ots0': 41, 'lai0': 42, 'lilo': 43, 'pa50': 44, 'h4°s': 45, '5nm1': 46, 'li-p': 47, 'lhpo': 48, '0ts0': 49, 'pad0': 50, 'sail': 51, 'p-sn': 52, 'icp3': 53, 'a130': 54, '2337': 55, '\x1f˙˙˙': 56, 'lgl0': 57, 'l\x15': 58, '@i\uf8f5\uf8f5': 59, 'li\x90o': 60, '4lio': 61, 'lp': 62, 'li?': 63, '\x04ion': 64, 'pbso': 65, 'a138': 66, 'li-h': 67, '6ion': 68, '3500': 69, 'h00j': 70, 'li\x10': 71, 'sams': 72, '\x03ip': 73, '8': 74, '#TAB#': 75, 'l\x06&#TAB#': 76, 'liÿÿ': 77, 'lÿÿÿ': 78, '5ion': 79, '45n1': 80, 'fake': 81, '2': 82, 'loÿÿ': 83, '\x90ñ±¿': 84, '\x0c@nn': 85, 'ioon': 86, 'l\x0b?': 87, 'pcs0': 88, 'nion': 89, 'pgd0': 90, 'lgco': 91, '9ion': 92})
    train['Census_OSVersion'] = train['Census_OSVersion'].map({'10.0.17134.165': 0, '10.0.17134.1': 1, '10.0.17134.228': 2, '10.0.17134.191': 3, '10.0.14393.0': 4, '10.0.17134.254': 5, '10.0.16299.431': 6, '10.0.17134.167': 7, '10.0.10586.164': 8, '10.0.17134.285': 9, '10.0.16299.309': 10, '10.0.10240.17914': 11, '10.0.17134.137': 12, '10.0.16299.125': 13, '10.0.17134.112': 14, '10.0.16299.15': 15, '10.0.14393.576': 16, '10.0.15063.1266': 17, '10.0.16299.611': 18, '10.0.15063.850': 19, '10.0.10586.545': 20, '10.0.15063.1206': 21, '10.0.10586.14': 22, '10.0.16299.248': 23, '10.0.14393.1593': 24, '10.0.14393.2189': 25, '10.0.10586.420': 26, '10.0.15063.250': 27, '10.0.10586.589': 28, '10.0.16299.371': 29, '10.0.15063.1324': 30, '10.0.15063.1088': 31, '10.0.10240.16384': 32, '10.0.16299.402': 33, '10.0.15063.1209': 34, '10.0.10586.318': 35, '10.0.16299.547': 36, '10.0.17134.286': 37, '10.0.10240.17443': 38, '10.0.15063.608': 39, '10.0.10586.1176': 40, '10.0.14393.321': 41, '10.0.15063.0': 42, '10.0.10240.16405': 43, '10.0.14393.447': 44, '10.0.15063.413': 45, '10.0.15063.332': 46, '10.0.10586.104': 47, '10.0.10586.122': 48, '10.0.14393.1066': 49, '10.0.15063.726': 50, '10.0.17134.81': 51, '10.0.10586.0': 52, '10.0.14393.1715': 53, '10.0.14393.2007': 54, '10.0.16299.522': 55, '10.0.17134.48': 56, '10.0.15063.1029': 57, '10.0.16299.492': 58, '10.0.14393.1914': 59, '10.0.15063.909': 60, '10.0.16299.192': 61, '10.0.15063.483': 62, '10.0.15063.674': 63, '10.0.14393.2214': 64, '10.0.10586.218': 65, '10.0.10240.17889': 66, '10.0.10240.17354': 67, '10.0.16299.19': 68, '10.0.10586.494': 69, '10.0.15063.966': 70, '10.0.10240.17946': 71, '10.0.14393.2068': 72, '10.0.16299.334': 73, '10.0.16299.665': 74, '10.0.14393.1358': 75, '10.0.10240.16413': 76, '10.0.16299.551': 77, '10.0.17134.319': 78, '10.0.10240.17113': 79, '10.0.15063.1155': 80, '10.0.10240.16601': 81, '10.0.14393.351': 82, '10.0.14393.693': 83, '10.0.16299.64': 84, '10.0.14393.1944': 85, '10.0.16299.579': 86, '10.0.16299.251': 87, '10.0.14393.1770': 88, '10.0.14393.1884': 89, '10.0.10586.873': 90, '10.0.14393.2396': 91, '10.0.15063.786': 92, '10.0.14393.2035': 93, '10.0.14393.1198': 94, '10.0.15063.502': 95, '10.0.15063.877': 96, '10.0.15063.994': 97, '10.0.16299.637': 98, '10.0.16299.461': 99, '10.0.14393.2430': 100, '10.0.10586.839': 101, '10.0.10586.1106': 102, '10.0.14393.82': 103, '10.0.10586.446': 104, '10.0.10586.63': 105, '10.0.10586.29': 106, '10.0.10586.962': 107, '10.0.15063.632': 108, '10.0.16299.98': 109, '10.0.17744.1004': 110, '10.0.15063.13': 111, '10.0.10586.713': 112, '10.0.10586.633': 113, '10.0.16299.214': 114, '10.0.10586.753': 115, '10.0.14393.187': 116, '10.0.14393.51': 117, '10.0.15063.540': 118, '10.0.10586.1045': 119, '10.0.14393.1480': 120, '10.0.10586.36': 121, '10.0.15063.296': 122, '10.0.14393.2125': 123, '10.0.10586.679': 124, '10.0.17692.1004': 125, '10.0.10240.16397': 126, '10.0.16299.666': 127, '10.0.14393.953': 128, '10.0.10586.916': 129, '10.0.15063.1058': 130, '10.0.15063.138': 131, '10.0.10586.71': 132, '10.0.14393.2248': 133, '10.0.17744.1003': 134, '10.0.14393.2395': 135, '10.0.14393.5': 136, '10.0.14393.2097': 137, '10.0.14393.2485': 138, '10.0.14393.105': 139, '10.0.17738.1000': 140, '10.0.15063.1235': 141, '10.0.17751.1': 142, '10.0.16299.696': 143, '10.0.10586.3': 144, '10.0.15063.1112': 145, '10.0.10586.17': 146, '10.0.14393.2155': 147, '10.0.10240.17319': 148, '10.0.10586.1007': 149, '10.0.10240.16683': 150, '10.0.10240.16487': 151, '10.0.14393.2273': 152, '10.0.14393.2457': 153, '10.0.10240.17190': 154, '10.0.17713.1002': 155, '10.0.17755.1': 156, '10.0.15063.447': 157, '10.0.14393.970': 158, '10.0.10240.16942': 159, '10.0.10240.17861': 160, '10.0.10240.17071': 161, '10.0.10240.17202': 162, '10.0.14393.222': 163, '10.0.10240.16854': 164, '10.0.14393.223': 165, '10.0.17741.1000': 166, '10.0.17733.1000': 167, '10.0.14393.2363': 168, '10.0.15063.1182': 169, '10.0.10240.16590': 170, '10.0.14393.2368': 171, '10.0.10240.17976': 172, '10.0.17746.1000': 173, '10.0.17735.1000': 174, '10.0.17754.1': 175, '10.0.15063.729': 176, '10.0.10586.338': 177, '10.0.17758.4': 178, '10.0.14393.67': 179, '10.0.17740.1000': 180, '10.0.17134.83': 181, '10.0.17672.1000': 182, '10.0.17758.1': 183, '10.0.10240.17709': 184, '10.0.15063.1356': 185, '10.0.17763.1': 186, '10.0.10586.11': 187, '10.0.10586.105': 188, '10.0.17134.5': 189, '10.0.14393.969': 190, '10.0.15063.1154': 191, '10.0.14393.726': 192, '10.0.17666.1000': 193, '10.0.10240.16463': 194, '10.0.10240.17024': 195, '10.0.15063.1292': 196, '10.0.17677.1000': 197, '10.0.14393.2515': 198, '10.0.17686.1003': 199, '10.0.14393.479': 200, '10.0.17747.1000': 201, '10.0.16299.194': 202, '10.0.15063.11': 203, '10.0.10240.17533': 204, '10.0.10240.16520': 205, '10.0.14314.228': 206, '10.0.10240.17918': 207, '10.0.10240.17146': 208, '10.0.15063.14': 209, '10.0.10240.17236': 210, '10.0.10240.17394': 211, '10.0.17760.1': 212, '10.0.14393.2312': 213, '10.0.10240.16549': 214, '10.0.17744.1001': 215, '10.0.10240.17831': 216, '10.0.15063.330': 217, '10.0.14393.10': 218, '10.0.17133.1': 219, '10.0.14393.2339': 220, '10.0.17759.1': 221, '10.0.10240.17770': 222, '10.0.10240.17738': 223, '10.0.10240.16393': 224, '10.0.10240.17797': 225, '10.0.14393.1532': 226, '10.0.10240.16430': 227, '10.0.10240.16445': 228, '10.0.10240.16433': 229, '10.0.17692.1000': 230, '10.0.17133.73': 231, '10.0.14393.577': 232, '10.0.10240.17643': 233, '10.0.10586.79': 234, '10.0.15063.936': 235, '10.0.17682.1000': 236, '10.0.17761.1001': 237, '10.0.10586.499': 238, '10.0.16299.165': 239, '10.0.10240.16387': 240, '10.0.17661.1001': 241, '10.0.14393.3': 242, '10.0.10240.17673': 243, '10.0.17643.1000': 244, '10.0.18234.1000': 245, '10.0.14393.729': 246, '10.0.10240.16771': 247, '10.0.14393.1537': 248, '10.0.15063.675': 249, '10.0.18242.1000': 250, '10.0.17639.1000': 251, '10.0.10240.16644': 252, '10.0.14393.1378': 253, '10.0.17634.1000': 254, '10.0.10586.456': 255, '10.0.14986.1001': 256, '10.0.10240.16389': 257, '10.0.10586.639': 258, '10.0.10240.16725': 259, '10.0.10240.16732': 260, '10.0.18237.1000': 261, '10.0.17704.1000': 262, '10.0.10240.17446': 263, '10.0.16275.1000': 264, '10.0.17650.1001': 265, '10.0.14393.594': 266, '10.0.10586.1177': 267, '10.0.15063.448': 268, '10.0.14393.1613': 269, '10.0.10586.240': 270, '10.0.15031.0': 271, '10.0.10586.112': 272, '10.0.14390.0': 273, '10.0.16299.491': 274, '10.0.17723.1000': 275, '10.0.10240.16500': 276, '10.0.14257.1000': 277, '10.0.14295.1005': 278, '10.0.16179.1000': 279, '10.0.14385.0': 280, '10.0.10240.16566': 281, '10.0.16299.201': 282, '10.0.17713.1000': 283, '10.0.17711.1000': 284, '10.0.17754.1000': 285, '10.0.15063.242': 286, '10.0.15042.0': 287, '10.0.17761.1': 288, '10.0.17004.1000': 289, '10.0.14393.103': 290, '10.0.7601.112': 291, '10.0.17728.1000': 292, '10.0.14342.1002': 293, '10.0.15002.1001': 294, '10.0.18240.1000': 295, '10.0.14316.1000': 296, '10.0.17627.1000': 297, '10.0.10586.1540': 298, '10.0.14366.0': 299, '10.0.14295.1004': 300, '10.0.14393.206': 301, '10.0.14946.1000': 302, '10.0.14352.1002': 303, '10.0.17730.1001': 304, '10.0.14905.1000': 305, '10.0.15019.1000': 306, '10.0.16251.1000': 307, '10.0.14372.0': 308, '6.3.9600.19102': 309, '10.0.17655.1000': 310, '10.0.15055.0': 311, '6.1.7601.24241': 312, '10.0.10586.550': 313, '10.0.14393.1230': 314, '10.0.17017.1000': 315, '10.0.14291.1001': 316, '10.0.10586.842': 317, '10.0.14393.2156': 318, '6.2.9200.17617': 319, '10.0.10240.16394': 320, '10.0.18219.1000': 321, '10.0.9600.164': 322, '10.0.14971.1000': 323, '10.0.16299.1003': 324, '10.0.17742.1000': 325, '10.0.16232.1000': 326, '10.0.16299.0': 327, '10.0.15025.1000': 328, '10.0.18204.1001': 329, '10.0.14379.0': 330, '10.0.17063.1000': 331, '10.0.17035.1000': 332, '10.0.14389.19': 333, '10.0.10240.17609': 334, '10.0.14361.0': 335, '10.0.10565.41736': 336, '10.0.14931.1002': 337, '10.0.9200.545': 338, '10.0.17134.111': 339, '10.0.15048.0': 340, '10.0.16294.1': 341, '10.0.11099.1000': 342, '10.0.7601.29': 343, '10.0.14936.1000': 344, '10.0.10240.16399': 345, '6.3.9600.19069': 346, '10.0.18214.1000': 347, '10.0.14931.1000': 348, '10.0.17730.1000': 349, '10.0.17074.1002': 350, '10.0.10240.16391': 351, '10.0.17134.226': 352, '10.0.17623.1002': 353, '10.0.14271.1000': 354, '10.0.14926.1000': 355, '10.0.10240.17488': 356, '10.0.17025.1000': 357, '10.0.16176.1000': 358, '10.0.16299.484': 359, '10.0.17604.1000': 360, '10.0.16241.1001': 361, '10.0.14393.2423': 362, '10.0.17618.1000': 363, '10.0.17128.1': 364, '10.0.10586.601': 365, '10.0.14901.1000': 366, '10.0.16278.1000': 367, '10.0.16199.1000': 368, '10.0.10240.17974': 369, '10.0.14915.1000': 370, '10.0.17127.1': 371, '10.0.14332.1001': 372, '10.0.10576.0': 373, '10.0.14393.2311': 374, '10.0.16193.1001': 375, '10.0.11082.1000': 376, '10.0.14298.1': 377, '6.2.9200.17581': 378, '10.0.14393.1797': 379, '10.0.10240.16425': 380, '6.3.9600.18687': 381, '10.0.14393.1670': 382, '10.0.18224.1000': 383, '10.0.18241.1000': 384, '10.0.17134.281': 385, '10.0.17689.1000': 386, '10.0.17753.1001': 387, '10.0.16281.1000': 388, '10.0.16299.365': 389, '10.0.17093.1000': 390, '10.0.16288.1': 391, '10.0.14230.228': 392, '6.1.7601.24149': 393, '10.0.14393.1737': 394, '10.0.16299.91': 395, '10.0.16215.1000': 396, '10.0.14986.1000': 397, '10.0.14955.1000': 398, '10.0.16299.695': 399, '10.0.17134.284': 400, '10.0.14295.1000': 401, '10.0.17134.283': 402, '10.0.16257.1000': 403, '10.0.17046.1000': 404, '10.0.16299.488': 405, '10.0.14421.191': 406, '10.0.14342.1001': 407, '10.0.14328.1000': 408, '10.0.18227.1000': 409, '10.0.18238.1000': 410, '10.0.14425.191': 411, '10.0.14965.1001': 412, '10.0.14959.1000': 413, '10.0.10240.17943': 414, '10.0.15061.0': 415, '10.0.17107.1000': 416, '10.0.16170.1000': 417, '10.0.17694.1000': 418, '10.0.14306.112': 419, '6.3.9600.18778': 420, '10.0.14300.1000': 421, '10.0.17757.1': 422, '10.0.18233.1000': 423, '10.0.7600.112': 424, '10.0.14474.228': 425, '10.0.16296.0': 426, '10.0.15060.0': 427, '10.0.14206.228': 428, '10.0.17753.1': 429, '10.0.16273.1000': 430, '10.0.14393.112': 431, '10.0.16291.0': 432, '10.0.10586.1417': 433, '10.0.9200.1593': 434, '6.1.7601.24214': 435, '10.0.14951.1000': 436, '10.0.14279.1000': 437, '6.3.9600.18756': 438, '10.0.15063.1149': 439, '10.0.17756.1': 440, '10.0.14942.1000': 441, '10.0.17040.1000': 442, '10.0.14379.1': 443, '10.0.17749.1000': 444, '10.0.14267.1000': 445, '10.0.10240.16498': 446, '10.0.16299.691': 447, '10.0.17752.1': 448, '10.0.16299.109': 449, '6.1.7601.23418': 450, '10.0.18064.228': 451, '10.0.17115.1': 452, '10.0.18226.1000': 453, '10.0.15046.0': 454, '10.0.14383.0': 455, '10.0.16237.1001': 456, '10.0.10586.596': 457, '10.0.10586.1232': 458, '10.0.15063.1322': 459, '10.0.18230.1000': 460, '10.0.18244.1000': 461, '10.0.17083.1000': 462, '10.0.18236.1001': 463, '10.0.17744.1000': 464, '6.2.9200.21703': 465, '10.0.14466.228': 466, '10.0.16299.183': 467, '10.0.17750.1000': 468, '10.0.15063.1387': 469, '10.0.17134.345': 470, '10.0.17763.134': 471, '10.0.16299.785': 472, '10.0.17134.407': 473, '10.0.18282.1000': 474, '10.0.17763.55': 475, '10.0.16299.726': 476, '10.0.17134.320': 477, '10.0.17134.376': 478, '10.0.14393.2608': 479, '10.0.16299.699': 480, '10.0.18277.1006': 481, '10.0.10240.18036': 482, '10.0.18272.1000': 483, '10.0.17763.104': 484, '10.0.17763.107': 485, '10.0.16299.755': 486, '10.0.18262.1000': 487, '10.0.18252.1000': 488, '10.0.15063.1418': 489, '10.0.14393.2580': 490, '10.0.14393.2551': 491, '10.0.10240.18005': 492, '10.0.17763.165': 493, '10.0.18247.1001': 494, '10.0.17763.17': 495, '10.0.18267.1001': 496, '10.0.18277.1000': 497, '6.3.9600.19182': 498, '10.0.18259.1000': 499, '10.0.14246.1': 500, '10.0.18285.1000': 501, '10.0.14393.1794': 502, '10.0.15007.1000': 503, '10.0.15063.1446': 504, '10.0.9600.318': 505, '10.0.18280.1000': 506, '10.0.17074.1000': 507, '10.0.16353.1000': 508, '10.0.18265.1000': 509, '10.0.18255.1000': 510, '6.1.7601.24263': 511, '10.0.15014.1000': 512, '6.1.7601.23934': 513, '10.0.10240.16724': 514, '10.0.17101.1000': 515, '10.0.14282.345': 516, '10.0.14216.248': 517, '10.0.11102.1000': 518, '10.0.14436.286': 519, '10.0.18279.1000': 520, '10.0.18253.1000': 521, '10.0.14367.0': 522, '10.0.17134.373': 523, '10.0.14422.112': 524, '10.0.14251.1000': 525, '6.3.9600.19134': 526, '6.3.9600.19155': 527, '10.0.17763.53': 528, '10.0.18286.1000': 529, '10.0.18287.1001': 530, '10.0.18260.1000': 531, '10.0.18247.1000': 532, '6.1.7601.23539': 533, '10.0.14320.1': 534, '10.0.18249.1000': 535, '10.0.14218.376': 536, '10.0.14421.345': 537, '10.0.18248.1000': 538, '10.0.18289.1000': 539, '10.0.16299.811': 540, '10.0.14294.286': 541, '10.0.15063.1443': 542, '10.0.16299.428': 543, '10.0.14376.0': 544, '10.0.15058.0': 545, '10.0.18256.1000': 546, '10.0.16299.188': 547, '10.0.16251.0': 548, '10.0.10586.1': 549, '10.0.14337.345': 550, '10.0.14229.285': 551, '10.0.18266.1000': 552, '6.3.9600.18202': 553, '10.0.7601.545': 554, '10.0.15063.2': 555, '10.0.18270.1000': 556, '10.0.18269.1000': 557, '10.0.17120.1': 558, '10.0.14367.112': 559, '10.0.18264.1000': 560, '10.0.14232.112': 561, '10.0.9600.1198': 562, '10.0.10586.461': 563, '10.0.17653.1000': 564, '10.0.17110.1000': 565, '10.0.7601.674': 566, '10.0.9600.693': 567, '6.1.7601.18869': 568, '10.0.16299.784': 569, '6.1.7601.24289': 570, '10.0.16299.725': 571, '6.1.7601.18933': 572, '10.0.14301.1': 573, '10.0.14355.191': 574, '10.0.15063.192': 575, '10.0.14279.0': 576, '10.0.16184.1001': 577, '10.0.18279.1001': 578})
    train['Census_OSArchitecture'] = train['Census_OSArchitecture'].map({'amd64': 0, 'arm64': 1, 'x86': 2})
    train['Census_OSBranch'] = train['Census_OSBranch'].map({'rs4_release': 0, 'rs1_release': 1, 'rs3_release_svc_escrow': 2, 'th2_release': 3, 'rs3_release': 4, 'th1_st1': 5, 'rs2_release': 6, 'th1': 7, 'rs3_release_svc_escrow_im': 8, 'th2_release_sec': 9, 'rs5_release': 10, 'rs_prerelease_flt': 11, 'rs_prerelease': 12, 'rs5_release_sigma': 13, 'rs5_release_edge': 14, 'rs5_release_sigma_dev': 15, 'winblue_ltsb_escrow': 16, 'win7sp1_ldr': 17, 'rs1_release_srvmedia': 18, 'win8_gdr': 19, 'rs1_release_sec': 20, 'rs3_release_svc': 21, 'Khmer OS': 22, 'winblue_ltsb': 23, 'rs_shell': 24, 'rs_onecore_stack_per1': 25, 'win7sp1_ldr_escrow': 26, 'rs_onecore_base_cobalt': 27, 'rs1_release_svc': 28, 'rs_xbox': 29, 'rs5_release_sign': 30, 'win8_ldr': 31, 'rs_onecore_sigma_dplat_d7': 32, 'rs_onecore_sigma_grfx_dev': 33, 'rs_onecore_dep': 34, 'rs1_release_inmarket': 35, 'rsmaster': 36, 'rs2_release_svc_d': 37, 'win7sp1_gdr': 38, 'rs_edge': 39})
    train['Census_OSEdition'] = train['Census_OSEdition'].map({'Professional': 0, 'Core': 1, 'CoreSingleLanguage': 2, 'EnterpriseS': 3, 'CoreCountrySpecific': 4, 'EnterpriseSN': 5, 'ProfessionalEducation': 6, 'Enterprise': 7, 'Education': 8, 'Cloud': 9, 'ProfessionalN': 10, 'ServerStandard': 11, 'CoreN': 12, 'ServerStandardEval': 13, 'EducationN': 14, 'ServerDatacenterEval': 15, 'ProfessionalEducationN': 16, 'ServerSolution': 17, 'EnterpriseN': 18, 'ProfessionalWorkstationN': 19, 'ProfessionalWorkstation': 20, 'Ultimate': 21, 'CloudN': 22, 'ProfessionalCountrySpecific': 23, 'ServerRdsh': 24, 'ServerDatacenter': 25, 'HomePremium': 26, 'ProfessionalSingleLanguage': 27, 'ServerDatacenterACor': 28, 'Pro': 29, 'Enterprise 2015 LTSB': 30, 'Home': 31, 'professional': 32, 'EnterpriseG': 33, 'HomeBasic': 34, 'PRO': 35, '00426-OEM-8992662-00006': 36, 'nan': 37, 'Window 10 Enterprise': 38, '#': 39})
    train['Census_OSSkuName'] = train['Census_OSSkuName'].map({'PROFESSIONAL': 0, 'CORE': 1, 'CORE_SINGLELANGUAGE': 2, 'ENTERPRISE_S': 3, 'CORE_COUNTRYSPECIFIC': 4, 'ENTERPRISE_S_N': 5, 'ENTERPRISE': 6, 'EDUCATION': 7, 'CLOUD': 8, 'PROFESSIONAL_N': 9, 'STANDARD_SERVER': 10, 'CORE_N': 11, 'STANDARD_EVALUATION_SERVER': 12, 'EDUCATION_N': 13, 'DATACENTER_EVALUATION_SERVER': 14, 'SB_SOLUTION_SERVER': 15, 'ENTERPRISE_N': 16, 'PRO_WORKSTATION_N': 17, 'PRO_WORKSTATION': 18, 'UNLICENSED': 19, 'ULTIMATE': 20, 'CLOUDN': 21, 'PRO_CHINA': 22, 'SERVERRDSH': 23, 'DATACENTER_SERVER': 24, 'ENTERPRISEG': 25, 'STARTER': 26, 'PRO_FOR_EDUCATION': 27, 'PRO_SINGLE_LANGUAGE': 28, 'UNDEFINED': 29, 'HOME_BASIC': 30, 'HOME_PREMIUM': 31})
    train['Census_OSInstallTypeName'] = train['Census_OSInstallTypeName'].map({'UUPUpgrade': 0, 'IBSClean': 1, 'Update': 2, 'Upgrade': 3, 'Other': 4, 'Clean': 5, 'Reset': 6, 'Refresh': 7, 'CleanPCRefresh': 8})
    train['Census_OSWUAutoUpdateOptionsName'] = train['Census_OSWUAutoUpdateOptionsName'].map({'UNKNOWN': 0, 'FullAuto': 1, 'Notify': 2, 'AutoInstallAndRebootAtMaintenanceTime': 3, 'Off': 4, 'DownloadNotify': 5})
    train['Census_GenuineStateName'] = train['Census_GenuineStateName'].map({'IS_GENUINE': 0, 'OFFLINE': 1, 'INVALID_LICENSE': 2, 'UNKNOWN': 3, 'TAMPERED': 4, 'nan': 5})
    train['Census_ActivationChannel'] = train['Census_ActivationChannel'].map({'Retail': 0, 'OEM:NONSLP': 1, 'OEM:DM': 2, 'Volume:GVLK': 3, 'Volume:MAK': 4, 'Retail:TB:Eval': 5})
    train['Census_FlightRing'] = train['Census_FlightRing'].map({'Retail': 0, 'NOT_SET': 1, 'Unknown': 2, 'Disabled': 3, 'RP': 4, 'WIS': 5, 'WIF': 6, 'OSG': 7, 'Canary': 8, 'Invalid': 9, 'CBCanary': 10})
    print("Encoding end!")
    return train

In [ ]:
%%time
train = pd.read_csv('../input/train.csv', dtype=dtypes,nrows=4000000, usecols=retained_columns)
train = reduce_mem_usage(train)
train = encode(train)
gc.collect()

In [ ]:
train.head()

In [ ]:
y = train['HasDetections']
del train['HasDetections']
gc.collect()

In [ ]:
param = {   
        'boosting_type': 'dart',
        'objective': 'binary',
        'metric': 'auc',
        'nthread': -1,
        'learning_rate': 0.2,
        }
gc.collect()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.3, random_state=42)

lg_train = lgb.Dataset(X_train, label=y_train)
lg_test = lgb.Dataset(X_test, y_test)

In [ ]:
model = lgb.train(param, lg_train, 10000,valid_sets=[lg_test], early_stopping_rounds=500, verbose_eval=100) 

In [ ]:
del train
del y
gc.collect()

In [ ]:
retained_columns.remove('HasDetections')

****We will make an answer file using data loading with chunksize. It takes about 20 minutes****

In [ ]:
%%time
chunksize = 10 ** 6
sub_df = pd.DataFrame(columns = ['HasDetections'])

for chunk in tqdm(pd.read_csv('../input/test.csv',usecols = retained_columns,dtype = dtypes,chunksize=chunksize)):
    tmp = pd.DataFrame(columns = ['HasDetections'])
    chunk = encode(chunk)
    chunk = reduce_mem_usage(chunk)
    pred = model.predict(chunk,num_iteration=model.best_iteration)
    tmp['HasDetections'] = pred
    sub_df = sub_df.append(tmp, ignore_index=True)
    del tmp 
    gc.collect()

In [ ]:
sub_df[:10]

In [ ]:
sbm = pd.read_csv('../input/sample_submission.csv', usecols=['MachineIdentifier'])
sbm['HasDetections'] = sub_df['HasDetections']
sbm.info()

In [ ]:
sbm.to_csv('submission.csv', index=False)